# Many-to-many matching under preference using SAT solving with z3
Sander Vonk, 11868627 UvA <br>
Supervisior: Ronald de Haan

In [1]:
# Note:
###############################################################################################################
# Values between such hashes can and should be changed depending on the desired variable setup
###############################################################################################################

### Reading csv file 

In [2]:
from z3 import *
import pandas as pd
import numpy as np
import timeit

In [3]:
# Reads csv file, sorts it for easier handling and returns it, along with a file to split on submissions
def read_file(file):
    file = pd.read_csv(file)
    file = file.sort_values(by=['Reviewer id', 'Submission number'])
    file_sub = file.sort_values(by=['Submission number', 'Reviewer id'])
    return file, file_sub 

# choose cv file to read
# file, file_sub = read_file("test.csv")
file, file_sub = read_file("data2.1.csv")

file
# file_sub

,Reviewer id,Submission number,Bid
15,0,0,no
16,0,1,no
17,0,2,no
18,0,4,no
19,0,5,no
...,...,...,...
897,30,49,yes
941,30,50,no
942,30,51,no
943,30,52,no


In [4]:
# Checks the number of reviewers and submissions within a file and returns them
def revsub_n(file):
    rev_n = len(file['Reviewer id'].unique())
    sub_n = len(file['Submission number'].unique())
    return rev_n, sub_n

rev_n = revsub_n(file)[0]
sub_n = revsub_n(file)[1]
print(" Number of reviewers: {0} \n Number of submissions: {1}".format(rev_n, sub_n))

 Number of reviewers: 31 
 Number of submissions: 54


### Setting up variables for the file

In [5]:
# reviewers as list
reviewers = []
for r in file['Reviewer id']:
    reviewers.append(r)

rev = []
for i in reviewers:
    rev.append('r' + str(i))
# print(rev)

# reviewers as list for submission splitting
reviewers_s = []
for r in file_sub['Reviewer id']:
    reviewers_s.append(r)
rev_s = []
for i in reviewers_s:
    rev_s.append('r' + str(i))
# print(rev_s)

In [6]:
# submissions as list
submissions = []
for s in file['Submission number']:
    submissions.append(s)
sub = []
for i in submissions:
    sub.append('s' + str(i))
# print(sub)

# submissions as list for submission splitting
submissions_s = []
for s in file_sub['Submission number']:
    submissions_s.append(s)
sub_s = []
for i in submissions_s:
    sub_s.append('s' + str(i))
# print(sub_s)

In [7]:
# list of combinations of reviewers and submissions
var = []
for (r, s) in zip(rev, sub):
    var.append(str(r)+str(s))
# print(var)

# list of combinations of reviewers and submissions for submission splitting
var_s = []
for (r, s) in zip(rev_s, sub_s):
    var_s.append(str(r)+str(s))
# print(var_s)

In [8]:
# variabes per reviewer
rev_vars = []
for s in range(sub_n):
    rev_vars.append(var[s])
# print(rev_vars)

In [9]:
# splits lists of reviewers in lists per reviewer 
rev_split = [var[i:i + sub_n] for i in range(0, len(var), sub_n)]
# print(rev_split)

# adding ,1 after each variable for reviewer rules for Pb
pb_rev_split = []
for i in rev_split:
    pb_rev_split.append(['(' + j + ',1)' for j in i])
# print(pb_rev_split)

# splits lists of submissons in lists per submission
sub_split = [var_s[i:i + rev_n] for i in range(0, len(var_s), rev_n)]
# print(sub_split)

# adding ,1 after each variable for reviewer rules for Pb
pb_sub_split = []
for i in sub_split:
    pb_sub_split.append(['('+ j + ',1)' for j in i])
# print(pb_rev_split)

### Setting up rules for SAT solving 

#### General rules (extra: 
- r1s1, r1s2, etc. = z3.Bools(var)
- scores = [z3.If(r1s1, 2, 0), z3.If(r1s2, 2, 0), etc.]
- rules = [z3.PbLe(([(r1s1,1),(r1s2,1), etc.), etc.))

In [10]:
# Running SAT

s = z3.Solver()
o = z3.Optimize()
score = z3.Int('score')
m_set = z3.Int('m_set')
goal = z3.Int('goal')
# Amount of reviewers per submission and submissions per reviewers

# o.add(rules)
# s.add(rules)

#### Bools(var) rules

In [11]:
# # setting up variables for defining z3 rules
# "'list' of all variables: print(*var, sep=",")" = z3.Bools(var)"
# print(*var, sep=",") = z3.Bools(var)
# print(*var, sep=",")

In [12]:
# variables
r0s0,r0s1,r0s2,r0s4,r0s5,r0s6,r0s7,r0s8,r0s9,r0s10,r0s11,r0s12,r0s13,r0s14,r0s15,r0s16,r0s17,r0s18,r0s19,r0s20,r0s21,r0s22,r0s23,r0s24,r0s25,r0s26,r0s27,r0s28,r0s29,r0s30,r0s31,r0s32,r0s33,r0s34,r0s35,r0s36,r0s37,r0s38,r0s39,r0s40,r0s41,r0s42,r0s43,r0s44,r0s45,r0s46,r0s47,r0s48,r0s49,r0s51,r0s52,r0s53,r1s0,r1s1,r1s2,r1s3,r1s4,r1s5,r1s6,r1s7,r1s8,r1s9,r1s10,r1s11,r1s12,r1s13,r1s14,r1s15,r1s16,r1s17,r1s19,r1s20,r1s21,r1s22,r1s23,r1s24,r1s25,r1s26,r1s27,r1s28,r1s29,r1s30,r1s31,r1s32,r1s33,r1s34,r1s35,r1s36,r1s37,r1s38,r1s39,r1s41,r1s42,r1s43,r1s44,r1s45,r1s46,r1s47,r1s48,r1s49,r1s50,r1s51,r1s52,r1s53,r2s0,r2s1,r2s2,r2s3,r2s4,r2s5,r2s6,r2s7,r2s8,r2s9,r2s10,r2s11,r2s12,r2s13,r2s14,r2s15,r2s16,r2s17,r2s18,r2s19,r2s20,r2s21,r2s22,r2s23,r2s24,r2s25,r2s26,r2s27,r2s28,r2s29,r2s30,r2s31,r2s32,r2s33,r2s34,r2s35,r2s36,r2s37,r2s38,r2s39,r2s40,r2s41,r2s42,r2s43,r2s44,r2s45,r2s46,r2s47,r2s48,r2s49,r2s50,r2s51,r2s52,r2s53,r3s0,r3s1,r3s2,r3s3,r3s4,r3s5,r3s6,r3s7,r3s8,r3s9,r3s10,r3s11,r3s12,r3s13,r3s14,r3s15,r3s16,r3s17,r3s18,r3s19,r3s20,r3s21,r3s22,r3s23,r3s24,r3s25,r3s26,r3s27,r3s28,r3s29,r3s30,r3s31,r3s32,r3s33,r3s34,r3s35,r3s36,r3s37,r3s38,r3s39,r3s40,r3s41,r3s42,r3s43,r3s44,r3s45,r3s46,r3s47,r3s48,r3s49,r3s50,r3s51,r3s52,r3s53,r4s0,r4s1,r4s2,r4s3,r4s4,r4s5,r4s6,r4s7,r4s8,r4s10,r4s11,r4s12,r4s13,r4s14,r4s15,r4s16,r4s17,r4s19,r4s20,r4s21,r4s22,r4s23,r4s24,r4s26,r4s27,r4s28,r4s29,r4s30,r4s31,r4s32,r4s33,r4s34,r4s35,r4s36,r4s37,r4s38,r4s39,r4s40,r4s41,r4s42,r4s43,r4s44,r4s45,r4s46,r4s47,r4s48,r4s49,r4s50,r4s51,r4s52,r4s53,r5s0,r5s1,r5s2,r5s3,r5s4,r5s5,r5s6,r5s7,r5s8,r5s9,r5s10,r5s11,r5s12,r5s13,r5s14,r5s15,r5s16,r5s17,r5s18,r5s19,r5s20,r5s21,r5s22,r5s23,r5s24,r5s25,r5s26,r5s27,r5s28,r5s29,r5s30,r5s31,r5s32,r5s33,r5s34,r5s35,r5s36,r5s37,r5s38,r5s39,r5s40,r5s41,r5s42,r5s43,r5s44,r5s45,r5s46,r5s47,r5s48,r5s49,r5s50,r5s51,r5s52,r5s53,r6s0,r6s1,r6s2,r6s3,r6s4,r6s5,r6s6,r6s7,r6s8,r6s9,r6s10,r6s11,r6s12,r6s13,r6s14,r6s15,r6s16,r6s17,r6s18,r6s19,r6s20,r6s21,r6s22,r6s23,r6s24,r6s25,r6s26,r6s27,r6s28,r6s29,r6s30,r6s31,r6s32,r6s33,r6s34,r6s35,r6s36,r6s37,r6s38,r6s39,r6s40,r6s41,r6s42,r6s43,r6s44,r6s45,r6s46,r6s47,r6s48,r6s49,r6s50,r6s51,r6s52,r6s53,r7s0,r7s1,r7s2,r7s3,r7s4,r7s5,r7s6,r7s7,r7s8,r7s9,r7s10,r7s11,r7s12,r7s13,r7s14,r7s15,r7s16,r7s17,r7s19,r7s20,r7s21,r7s22,r7s23,r7s25,r7s26,r7s27,r7s28,r7s29,r7s30,r7s31,r7s32,r7s33,r7s34,r7s35,r7s36,r7s37,r7s38,r7s39,r7s40,r7s41,r7s42,r7s43,r7s44,r7s45,r7s46,r7s47,r7s48,r7s49,r7s50,r7s51,r7s52,r7s53,r8s0,r8s2,r8s3,r8s4,r8s5,r8s6,r8s7,r8s8,r8s9,r8s10,r8s11,r8s12,r8s13,r8s14,r8s15,r8s16,r8s17,r8s18,r8s19,r8s20,r8s21,r8s22,r8s23,r8s24,r8s25,r8s26,r8s27,r8s28,r8s29,r8s30,r8s31,r8s32,r8s33,r8s34,r8s35,r8s36,r8s37,r8s38,r8s39,r8s40,r8s41,r8s42,r8s43,r8s44,r8s45,r8s46,r8s47,r8s48,r8s49,r8s50,r8s51,r8s52,r8s53,r9s0,r9s1,r9s2,r9s3,r9s4,r9s5,r9s6,r9s7,r9s8,r9s9,r9s10,r9s11,r9s12,r9s13,r9s14,r9s15,r9s16,r9s17,r9s18,r9s19,r9s20,r9s21,r9s22,r9s23,r9s24,r9s25,r9s26,r9s27,r9s28,r9s29,r9s30,r9s31,r9s32,r9s33,r9s34,r9s35,r9s36,r9s37,r9s38,r9s39,r9s40,r9s41,r9s42,r9s43,r9s44,r9s45,r9s46,r9s47,r9s48,r9s49,r9s50,r9s51,r9s52,r9s53,r10s0,r10s1,r10s2,r10s3,r10s4,r10s5,r10s6,r10s7,r10s8,r10s9,r10s10,r10s11,r10s12,r10s13,r10s14,r10s15,r10s16,r10s17,r10s18,r10s19,r10s20,r10s21,r10s22,r10s23,r10s24,r10s25,r10s26,r10s27,r10s28,r10s29,r10s30,r10s31,r10s32,r10s33,r10s34,r10s35,r10s36,r10s37,r10s38,r10s39,r10s40,r10s41,r10s42,r10s43,r10s44,r10s45,r10s46,r10s47,r10s48,r10s49,r10s50,r10s51,r10s52,r10s53,r11s0,r11s1,r11s2,r11s3,r11s4,r11s5,r11s6,r11s7,r11s8,r11s9,r11s10,r11s11,r11s12,r11s13,r11s14,r11s15,r11s16,r11s17,r11s18,r11s19,r11s20,r11s21,r11s22,r11s23,r11s24,r11s25,r11s26,r11s27,r11s28,r11s29,r11s30,r11s31,r11s32,r11s33,r11s34,r11s35,r11s36,r11s37,r11s38,r11s39,r11s40,r11s41,r11s42,r11s43,r11s44,r11s45,r11s46,r11s47,r11s48,r11s49,r11s50,r11s51,r11s52,r11s53,r12s0,r12s1,r12s2,r12s3,r12s4,r12s5,r12s6,r12s8,r12s9,r12s10,r12s11,r12s12,r12s13,r12s14,r12s15,r12s16,r12s17,r12s18,r12s19,r12s20,r12s21,r12s22,r12s23,r12s24,r12s25,r12s26,r12s27,r12s28,r12s30,r12s31,r12s32,r12s33,r12s34,r12s35,r12s36,r12s37,r12s38,r12s39,r12s40,r12s41,r12s42,r12s43,r12s44,r12s45,r12s46,r12s47,r12s48,r12s49,r12s50,r12s51,r12s52,r12s53,r13s0,r13s1,r13s2,r13s3,r13s4,r13s5,r13s6,r13s7,r13s8,r13s9,r13s10,r13s11,r13s12,r13s13,r13s14,r13s15,r13s16,r13s17,r13s18,r13s19,r13s20,r13s21,r13s22,r13s23,r13s24,r13s25,r13s26,r13s27,r13s28,r13s29,r13s30,r13s31,r13s32,r13s33,r13s34,r13s35,r13s36,r13s37,r13s38,r13s39,r13s40,r13s41,r13s42,r13s43,r13s44,r13s45,r13s46,r13s47,r13s48,r13s49,r13s50,r13s51,r13s52,r13s53,r14s0,r14s1,r14s2,r14s3,r14s4,r14s5,r14s6,r14s7,r14s8,r14s9,r14s10,r14s11,r14s12,r14s14,r14s15,r14s16,r14s17,r14s18,r14s19,r14s20,r14s21,r14s22,r14s23,r14s25,r14s26,r14s27,r14s28,r14s29,r14s30,r14s31,r14s32,r14s33,r14s34,r14s35,r14s36,r14s37,r14s38,r14s39,r14s40,r14s41,r14s42,r14s43,r14s44,r14s45,r14s46,r14s47,r14s48,r14s49,r14s50,r14s51,r14s52,r14s53,r15s0,r15s1,r15s2,r15s3,r15s4,r15s5,r15s6,r15s7,r15s8,r15s9,r15s10,r15s11,r15s12,r15s13,r15s14,r15s15,r15s16,r15s17,r15s18,r15s19,r15s20,r15s21,r15s22,r15s23,r15s24,r15s25,r15s26,r15s27,r15s28,r15s29,r15s30,r15s31,r15s32,r15s33,r15s34,r15s35,r15s36,r15s37,r15s38,r15s39,r15s40,r15s41,r15s42,r15s43,r15s44,r15s45,r15s46,r15s47,r15s48,r15s49,r15s50,r15s51,r15s52,r15s53,r16s0,r16s1,r16s2,r16s3,r16s4,r16s5,r16s6,r16s7,r16s8,r16s9,r16s10,r16s11,r16s12,r16s13,r16s14,r16s15,r16s16,r16s17,r16s18,r16s19,r16s20,r16s21,r16s22,r16s23,r16s24,r16s25,r16s26,r16s27,r16s28,r16s29,r16s30,r16s31,r16s32,r16s33,r16s34,r16s35,r16s36,r16s37,r16s38,r16s39,r16s40,r16s41,r16s42,r16s44,r16s45,r16s46,r16s47,r16s48,r16s50,r16s52,r16s53,r17s0,r17s1,r17s2,r17s3,r17s4,r17s5,r17s6,r17s7,r17s8,r17s9,r17s10,r17s11,r17s12,r17s13,r17s14,r17s15,r17s16,r17s17,r17s18,r17s19,r17s20,r17s22,r17s23,r17s24,r17s25,r17s27,r17s28,r17s29,r17s30,r17s31,r17s32,r17s33,r17s34,r17s35,r17s36,r17s37,r17s38,r17s39,r17s40,r17s41,r17s42,r17s43,r17s44,r17s46,r17s47,r17s49,r17s50,r17s51,r17s52,r18s0,r18s1,r18s2,r18s3,r18s4,r18s5,r18s6,r18s7,r18s8,r18s9,r18s10,r18s11,r18s12,r18s13,r18s14,r18s15,r18s16,r18s17,r18s18,r18s19,r18s20,r18s21,r18s22,r18s23,r18s24,r18s25,r18s26,r18s27,r18s28,r18s29,r18s30,r18s31,r18s32,r18s33,r18s34,r18s35,r18s36,r18s37,r18s38,r18s39,r18s40,r18s41,r18s42,r18s43,r18s44,r18s45,r18s46,r18s47,r18s48,r18s49,r18s50,r18s51,r18s52,r18s53,r19s0,r19s1,r19s2,r19s3,r19s4,r19s5,r19s6,r19s7,r19s8,r19s9,r19s10,r19s11,r19s12,r19s13,r19s14,r19s15,r19s16,r19s17,r19s18,r19s19,r19s20,r19s21,r19s22,r19s23,r19s24,r19s25,r19s26,r19s27,r19s28,r19s29,r19s30,r19s31,r19s32,r19s33,r19s34,r19s35,r19s36,r19s37,r19s38,r19s39,r19s40,r19s41,r19s42,r19s43,r19s44,r19s45,r19s46,r19s47,r19s48,r19s49,r19s50,r19s51,r19s52,r19s53,r20s0,r20s1,r20s2,r20s3,r20s4,r20s5,r20s6,r20s7,r20s8,r20s9,r20s10,r20s11,r20s13,r20s14,r20s15,r20s16,r20s17,r20s18,r20s19,r20s20,r20s21,r20s22,r20s23,r20s24,r20s25,r20s26,r20s27,r20s28,r20s29,r20s30,r20s31,r20s32,r20s33,r20s34,r20s35,r20s36,r20s37,r20s38,r20s39,r20s40,r20s41,r20s42,r20s43,r20s44,r20s45,r20s46,r20s47,r20s48,r20s49,r20s50,r20s51,r20s52,r20s53,r21s0,r21s1,r21s2,r21s3,r21s4,r21s5,r21s6,r21s7,r21s8,r21s9,r21s10,r21s11,r21s12,r21s13,r21s14,r21s15,r21s16,r21s17,r21s18,r21s19,r21s20,r21s21,r21s22,r21s23,r21s24,r21s25,r21s26,r21s27,r21s28,r21s29,r21s30,r21s31,r21s32,r21s33,r21s34,r21s35,r21s36,r21s37,r21s38,r21s39,r21s40,r21s41,r21s43,r21s44,r21s45,r21s46,r21s47,r21s48,r21s49,r21s50,r21s51,r21s52,r21s53,r22s0,r22s1,r22s2,r22s3,r22s4,r22s5,r22s6,r22s7,r22s8,r22s9,r22s10,r22s11,r22s12,r22s13,r22s14,r22s15,r22s16,r22s17,r22s18,r22s19,r22s20,r22s21,r22s22,r22s23,r22s24,r22s25,r22s26,r22s27,r22s28,r22s29,r22s30,r22s31,r22s32,r22s33,r22s34,r22s35,r22s36,r22s37,r22s38,r22s39,r22s40,r22s41,r22s42,r22s43,r22s44,r22s45,r22s46,r22s47,r22s48,r22s49,r22s50,r22s51,r22s52,r22s53,r23s0,r23s1,r23s2,r23s3,r23s4,r23s5,r23s6,r23s7,r23s8,r23s9,r23s10,r23s11,r23s12,r23s13,r23s14,r23s15,r23s16,r23s17,r23s19,r23s20,r23s21,r23s22,r23s23,r23s24,r23s25,r23s26,r23s27,r23s28,r23s29,r23s30,r23s31,r23s32,r23s33,r23s34,r23s35,r23s36,r23s37,r23s38,r23s39,r23s41,r23s42,r23s43,r23s44,r23s45,r23s46,r23s47,r23s48,r23s49,r23s50,r23s51,r23s52,r23s53,r24s0,r24s1,r24s2,r24s3,r24s4,r24s5,r24s6,r24s7,r24s8,r24s9,r24s10,r24s11,r24s12,r24s14,r24s15,r24s16,r24s17,r24s18,r24s19,r24s20,r24s21,r24s22,r24s23,r24s24,r24s25,r24s26,r24s27,r24s28,r24s29,r24s30,r24s31,r24s32,r24s33,r24s34,r24s35,r24s36,r24s37,r24s38,r24s39,r24s40,r24s42,r24s44,r24s45,r24s46,r24s47,r24s48,r24s50,r24s52,r24s53,r25s0,r25s1,r25s2,r25s3,r25s4,r25s5,r25s6,r25s7,r25s8,r25s9,r25s10,r25s11,r25s13,r25s14,r25s15,r25s16,r25s17,r25s19,r25s20,r25s21,r25s22,r25s23,r25s24,r25s25,r25s26,r25s27,r25s28,r25s29,r25s30,r25s31,r25s32,r25s33,r25s34,r25s35,r25s36,r25s37,r25s38,r25s39,r25s40,r25s41,r25s42,r25s43,r25s44,r25s45,r25s46,r25s47,r25s48,r25s49,r25s50,r25s51,r25s52,r25s53,r26s0,r26s1,r26s2,r26s3,r26s4,r26s5,r26s6,r26s7,r26s8,r26s9,r26s10,r26s11,r26s12,r26s13,r26s14,r26s15,r26s16,r26s17,r26s18,r26s20,r26s21,r26s22,r26s23,r26s24,r26s25,r26s26,r26s27,r26s28,r26s29,r26s30,r26s31,r26s32,r26s33,r26s34,r26s35,r26s36,r26s37,r26s38,r26s39,r26s40,r26s41,r26s42,r26s43,r26s44,r26s45,r26s46,r26s47,r26s48,r26s49,r26s50,r26s51,r26s52,r26s53,r27s0,r27s1,r27s2,r27s3,r27s4,r27s5,r27s6,r27s7,r27s8,r27s9,r27s10,r27s11,r27s12,r27s13,r27s14,r27s15,r27s16,r27s17,r27s18,r27s19,r27s20,r27s21,r27s22,r27s23,r27s24,r27s25,r27s26,r27s27,r27s28,r27s29,r27s30,r27s31,r27s32,r27s33,r27s34,r27s35,r27s36,r27s37,r27s38,r27s39,r27s40,r27s41,r27s42,r27s43,r27s44,r27s45,r27s46,r27s47,r27s48,r27s49,r27s50,r27s51,r27s52,r27s53,r28s0,r28s1,r28s2,r28s3,r28s4,r28s5,r28s6,r28s7,r28s8,r28s9,r28s10,r28s11,r28s12,r28s13,r28s14,r28s15,r28s16,r28s17,r28s18,r28s19,r28s20,r28s21,r28s23,r28s24,r28s25,r28s26,r28s29,r28s30,r28s32,r28s33,r28s35,r28s36,r28s37,r28s38,r28s39,r28s40,r28s41,r28s42,r28s43,r28s44,r28s45,r28s46,r28s47,r28s48,r28s49,r28s50,r28s51,r28s52,r28s53,r29s0,r29s1,r29s2,r29s3,r29s4,r29s5,r29s6,r29s7,r29s8,r29s9,r29s10,r29s11,r29s12,r29s13,r29s14,r29s15,r29s16,r29s17,r29s18,r29s19,r29s20,r29s21,r29s23,r29s24,r29s25,r29s26,r29s29,r29s30,r29s32,r29s33,r29s35,r29s36,r29s37,r29s38,r29s39,r29s40,r29s41,r29s42,r29s43,r29s44,r29s45,r29s46,r29s47,r29s48,r29s49,r29s50,r29s51,r29s52,r29s53,r30s0,r30s1,r30s2,r30s3,r30s4,r30s5,r30s6,r30s7,r30s8,r30s9,r30s10,r30s11,r30s12,r30s13,r30s14,r30s15,r30s16,r30s17,r30s18,r30s19,r30s20,r30s21,r30s22,r30s23,r30s24,r30s25,r30s26,r30s27,r30s28,r30s29,r30s30,r30s31,r30s32,r30s33,r30s34,r30s35,r30s37,r30s38,r30s39,r30s40,r30s41,r30s42,r30s43,r30s44,r30s45,r30s46,r30s47,r30s48,r30s49,r30s50,r30s51,r30s52,r30s53 = z3.Bools(var)

#### Scores (if())

In [13]:
# # Code for printing the rules corresponding to the bid scores, can be used to create the scores that are set below

# these values decide how many points should be awarded to different bids, so different preferences for a match
###############################################################################################################
yes = 60
maybe = 30
no = 10
###############################################################################################################

# list of all bids, indicated by a nuber below 100 to indicate the preference of a possible match
bids = []
for i, row in file.iterrows():
    if row.iloc[2] == "yes":
        bids.append(yes)
    if row.iloc[2] == "maybe":
        bids.append(maybe)
    if row.iloc[2] == "no":
        bids.append(no)
        
# dictionary of combinations of reviewers and submissions with the preference of the reviewer for the corresponding submission
rs_score = dict(zip(var, bids))

In [14]:
# scores = list with all vars: [z3.If(r?s?, bid, 0), etc.]
ifs = list(zip(var, bids, [0] * len(bids)))
scores = []
for i in ifs:
    scores.append("z3.If{0}".format(i))
for i in scores:
    print(i.replace("'", ""), ',', end=' ')

z3.If(r0s0, 10, 0) , z3.If(r0s1, 10, 0) , z3.If(r0s2, 10, 0) , z3.If(r0s4, 10, 0) , z3.If(r0s5, 10, 0) , z3.If(r0s6, 60, 0) , z3.If(r0s7, 10, 0) , z3.If(r0s8, 10, 0) , z3.If(r0s9, 30, 0) , z3.If(r0s10, 10, 0) , z3.If(r0s11, 10, 0) , z3.If(r0s12, 10, 0) , z3.If(r0s13, 60, 0) , z3.If(r0s14, 10, 0) , z3.If(r0s15, 10, 0) , z3.If(r0s16, 30, 0) , z3.If(r0s17, 30, 0) , z3.If(r0s18, 30, 0) , z3.If(r0s19, 10, 0) , z3.If(r0s20, 10, 0) , z3.If(r0s21, 10, 0) , z3.If(r0s22, 60, 0) , z3.If(r0s23, 10, 0) , z3.If(r0s24, 60, 0) , z3.If(r0s25, 10, 0) , z3.If(r0s26, 10, 0) , z3.If(r0s27, 60, 0) , z3.If(r0s28, 10, 0) , z3.If(r0s29, 10, 0) , z3.If(r0s30, 10, 0) , z3.If(r0s31, 10, 0) , z3.If(r0s32, 30, 0) , z3.If(r0s33, 10, 0) , z3.If(r0s34, 10, 0) , z3.If(r0s35, 10, 0) , z3.If(r0s36, 30, 0) , z3.If(r0s37, 10, 0) , z3.If(r0s38, 10, 0) , z3.If(r0s39, 10, 0) , z3.If(r0s40, 10, 0) , z3.If(r0s41, 10, 0) , z3.If(r0s42, 30, 0) , z3.If(r0s43, 30, 0) , z3.If(r0s44, 10, 0) , z3.If(r0s45, 10, 0) , z3.If(r0s46, 10, 0)

z3.If(r13s7, 10, 0) , z3.If(r13s8, 10, 0) , z3.If(r13s9, 60, 0) , z3.If(r13s10, 10, 0) , z3.If(r13s11, 10, 0) , z3.If(r13s12, 60, 0) , z3.If(r13s13, 10, 0) , z3.If(r13s14, 10, 0) , z3.If(r13s15, 10, 0) , z3.If(r13s16, 10, 0) , z3.If(r13s17, 30, 0) , z3.If(r13s18, 60, 0) , z3.If(r13s19, 10, 0) , z3.If(r13s20, 10, 0) , z3.If(r13s21, 10, 0) , z3.If(r13s22, 10, 0) , z3.If(r13s23, 10, 0) , z3.If(r13s24, 60, 0) , z3.If(r13s25, 60, 0) , z3.If(r13s26, 10, 0) , z3.If(r13s27, 60, 0) , z3.If(r13s28, 10, 0) , z3.If(r13s29, 10, 0) , z3.If(r13s30, 10, 0) , z3.If(r13s31, 10, 0) , z3.If(r13s32, 10, 0) , z3.If(r13s33, 10, 0) , z3.If(r13s34, 10, 0) , z3.If(r13s35, 10, 0) , z3.If(r13s36, 10, 0) , z3.If(r13s37, 10, 0) , z3.If(r13s38, 10, 0) , z3.If(r13s39, 60, 0) , z3.If(r13s40, 30, 0) , z3.If(r13s41, 10, 0) , z3.If(r13s42, 10, 0) , z3.If(r13s43, 10, 0) , z3.If(r13s44, 10, 0) , z3.If(r13s45, 10, 0) , z3.If(r13s46, 10, 0) , z3.If(r13s47, 10, 0) , z3.If(r13s48, 10, 0) , z3.If(r13s49, 10, 0) , z3.If(r13s50, 

 z3.If(r22s41, 10, 0) , z3.If(r22s42, 10, 0) , z3.If(r22s43, 10, 0) , z3.If(r22s44, 10, 0) , z3.If(r22s45, 10, 0) , z3.If(r22s46, 10, 0) , z3.If(r22s47, 10, 0) , z3.If(r22s48, 10, 0) , z3.If(r22s49, 10, 0) , z3.If(r22s50, 10, 0) , z3.If(r22s51, 10, 0) , z3.If(r22s52, 10, 0) , z3.If(r22s53, 10, 0) , z3.If(r23s0, 10, 0) , z3.If(r23s1, 60, 0) , z3.If(r23s2, 10, 0) , z3.If(r23s3, 30, 0) , z3.If(r23s4, 10, 0) , z3.If(r23s5, 10, 0) , z3.If(r23s6, 60, 0) , z3.If(r23s7, 10, 0) , z3.If(r23s8, 10, 0) , z3.If(r23s9, 10, 0) , z3.If(r23s10, 10, 0) , z3.If(r23s11, 10, 0) , z3.If(r23s12, 10, 0) , z3.If(r23s13, 10, 0) , z3.If(r23s14, 60, 0) , z3.If(r23s15, 10, 0) , z3.If(r23s16, 30, 0) , z3.If(r23s17, 60, 0) , z3.If(r23s19, 10, 0) , z3.If(r23s20, 10, 0) , z3.If(r23s21, 10, 0) , z3.If(r23s22, 10, 0) , z3.If(r23s23, 10, 0) , z3.If(r23s24, 10, 0) , z3.If(r23s25, 60, 0) , z3.If(r23s26, 10, 0) , z3.If(r23s27, 10, 0) , z3.If(r23s28, 10, 0) , z3.If(r23s29, 10, 0) , z3.If(r23s30, 60, 0) , z3.If(r23s31, 30, 0)

 z3.If(r30s26, 10, 0) , z3.If(r30s27, 10, 0) , z3.If(r30s28, 10, 0) , z3.If(r30s29, 30, 0) , z3.If(r30s30, 10, 0) , z3.If(r30s31, 10, 0) , z3.If(r30s32, 30, 0) , z3.If(r30s33, 10, 0) , z3.If(r30s34, 10, 0) , z3.If(r30s35, 30, 0) , z3.If(r30s37, 10, 0) , z3.If(r30s38, 10, 0) , z3.If(r30s39, 10, 0) , z3.If(r30s40, 10, 0) , z3.If(r30s41, 10, 0) , z3.If(r30s42, 10, 0) , z3.If(r30s43, 10, 0) , z3.If(r30s44, 10, 0) , z3.If(r30s45, 30, 0) , z3.If(r30s46, 30, 0) , z3.If(r30s47, 10, 0) , z3.If(r30s48, 10, 0) , z3.If(r30s49, 60, 0) , z3.If(r30s50, 10, 0) , z3.If(r30s51, 10, 0) , z3.If(r30s52, 10, 0) , z3.If(r30s53, 10, 0) , 

In [15]:
# These are the default used scores, uncomment them to use them, THEY SHOULD MATCH the above values for yes, maybe and no
# others can be created by changing the yes, maybe and no to the desired values
# then the above code should be uncommmented, ran and used by pasting them into the list with the name scores (scores = [])
###############################################################################################################
# 90/10/0
# scores = [z3.If(r0s0, 0, 0) , z3.If(r0s1, 0, 0) , z3.If(r0s2, 0, 0) , z3.If(r0s4, 0, 0) , z3.If(r0s5, 0, 0) , z3.If(r0s6, 90, 0) , z3.If(r0s7, 0, 0) , z3.If(r0s8, 0, 0) , z3.If(r0s9, 10, 0) , z3.If(r0s10, 0, 0) , z3.If(r0s11, 0, 0) , z3.If(r0s12, 0, 0) , z3.If(r0s13, 90, 0) , z3.If(r0s14, 0, 0) , z3.If(r0s15, 0, 0) , z3.If(r0s16, 10, 0) , z3.If(r0s17, 10, 0) , z3.If(r0s18, 10, 0) , z3.If(r0s19, 0, 0) , z3.If(r0s20, 0, 0) , z3.If(r0s21, 0, 0) , z3.If(r0s22, 90, 0) , z3.If(r0s23, 0, 0) , z3.If(r0s24, 90, 0) , z3.If(r0s25, 0, 0) , z3.If(r0s26, 0, 0) , z3.If(r0s27, 90, 0) , z3.If(r0s28, 0, 0) , z3.If(r0s29, 0, 0) , z3.If(r0s30, 0, 0) , z3.If(r0s31, 0, 0) , z3.If(r0s32, 10, 0) , z3.If(r0s33, 0, 0) , z3.If(r0s34, 0, 0) , z3.If(r0s35, 0, 0) , z3.If(r0s36, 10, 0) , z3.If(r0s37, 0, 0) , z3.If(r0s38, 0, 0) , z3.If(r0s39, 0, 0) , z3.If(r0s40, 0, 0) , z3.If(r0s41, 0, 0) , z3.If(r0s42, 10, 0) , z3.If(r0s43, 10, 0) , z3.If(r0s44, 0, 0) , z3.If(r0s45, 0, 0) , z3.If(r0s46, 0, 0) , z3.If(r0s47, 10, 0) , z3.If(r0s48, 0, 0) , z3.If(r0s49, 0, 0) , z3.If(r0s51, 10, 0) , z3.If(r0s52, 0, 0) , z3.If(r0s53, 0, 0) , z3.If(r1s0, 0, 0) , z3.If(r1s1, 0, 0) , z3.If(r1s2, 0, 0) , z3.If(r1s3, 0, 0) , z3.If(r1s4, 0, 0) , z3.If(r1s5, 0, 0) , z3.If(r1s6, 90, 0) , z3.If(r1s7, 10, 0) , z3.If(r1s8, 0, 0) , z3.If(r1s9, 0, 0) , z3.If(r1s10, 0, 0) , z3.If(r1s11, 0, 0) , z3.If(r1s12, 90, 0) , z3.If(r1s13, 0, 0) , z3.If(r1s14, 10, 0) , z3.If(r1s15, 0, 0) , z3.If(r1s16, 0, 0) , z3.If(r1s17, 90, 0) , z3.If(r1s19, 0, 0) , z3.If(r1s20, 0, 0) , z3.If(r1s21, 0, 0) , z3.If(r1s22, 10, 0) , z3.If(r1s23, 0, 0) , z3.If(r1s24, 90, 0) , z3.If(r1s25, 10, 0) , z3.If(r1s26, 0, 0) , z3.If(r1s27, 90, 0) , z3.If(r1s28, 0, 0) , z3.If(r1s29, 0, 0) , z3.If(r1s30, 10, 0) , z3.If(r1s31, 10, 0) , z3.If(r1s32, 90, 0) , z3.If(r1s33, 0, 0) , z3.If(r1s34, 0, 0) , z3.If(r1s35, 0, 0) , z3.If(r1s36, 10, 0) , z3.If(r1s37, 0, 0) , z3.If(r1s38, 0, 0) , z3.If(r1s39, 0, 0) , z3.If(r1s41, 0, 0) , z3.If(r1s42, 90, 0) , z3.If(r1s43, 10, 0) , z3.If(r1s44, 0, 0) , z3.If(r1s45, 0, 0) , z3.If(r1s46, 0, 0) , z3.If(r1s47, 90, 0) , z3.If(r1s48, 0, 0) , z3.If(r1s49, 10, 0) , z3.If(r1s50, 0, 0) , z3.If(r1s51, 0, 0) , z3.If(r1s52, 0, 0) , z3.If(r1s53, 0, 0) , z3.If(r2s0, 0, 0) , z3.If(r2s1, 10, 0) , z3.If(r2s2, 0, 0) , z3.If(r2s3, 90, 0) , z3.If(r2s4, 0, 0) , z3.If(r2s5, 90, 0) , z3.If(r2s6, 0, 0) , z3.If(r2s7, 90, 0) , z3.If(r2s8, 0, 0) , z3.If(r2s9, 0, 0) , z3.If(r2s10, 0, 0) , z3.If(r2s11, 0, 0) , z3.If(r2s12, 10, 0) , z3.If(r2s13, 0, 0) , z3.If(r2s14, 10, 0) , z3.If(r2s15, 0, 0) , z3.If(r2s16, 0, 0) , z3.If(r2s17, 10, 0) , z3.If(r2s18, 0, 0) , z3.If(r2s19, 0, 0) , z3.If(r2s20, 0, 0) , z3.If(r2s21, 0, 0) , z3.If(r2s22, 0, 0) , z3.If(r2s23, 0, 0) , z3.If(r2s24, 90, 0) , z3.If(r2s25, 0, 0) , z3.If(r2s26, 0, 0) , z3.If(r2s27, 90, 0) , z3.If(r2s28, 0, 0) , z3.If(r2s29, 0, 0) , z3.If(r2s30, 0, 0) , z3.If(r2s31, 0, 0) , z3.If(r2s32, 0, 0) , z3.If(r2s33, 0, 0) , z3.If(r2s34, 0, 0) , z3.If(r2s35, 0, 0) , z3.If(r2s36, 0, 0) , z3.If(r2s37, 0, 0) , z3.If(r2s38, 0, 0) , z3.If(r2s39, 0, 0) , z3.If(r2s40, 90, 0) , z3.If(r2s41, 0, 0) , z3.If(r2s42, 90, 0) , z3.If(r2s43, 10, 0) , z3.If(r2s44, 0, 0) , z3.If(r2s45, 0, 0) , z3.If(r2s46, 0, 0) , z3.If(r2s47, 0, 0) , z3.If(r2s48, 0, 0) , z3.If(r2s49, 0, 0) , z3.If(r2s50, 0, 0) , z3.If(r2s51, 0, 0) , z3.If(r2s52, 0, 0) , z3.If(r2s53, 0, 0) , z3.If(r3s0, 90, 0) , z3.If(r3s1, 0, 0) , z3.If(r3s2, 0, 0) , z3.If(r3s3, 0, 0) , z3.If(r3s4, 0, 0) , z3.If(r3s5, 0, 0) , z3.If(r3s6, 90, 0) , z3.If(r3s7, 0, 0) , z3.If(r3s8, 0, 0) , z3.If(r3s9, 0, 0) , z3.If(r3s10, 0, 0) , z3.If(r3s11, 0, 0) , z3.If(r3s12, 0, 0) , z3.If(r3s13, 90, 0) , z3.If(r3s14, 0, 0) , z3.If(r3s15, 0, 0) , z3.If(r3s16, 0, 0) , z3.If(r3s17, 0, 0) , z3.If(r3s18, 0, 0) , z3.If(r3s19, 0, 0) , z3.If(r3s20, 0, 0) , z3.If(r3s21, 0, 0) , z3.If(r3s22, 0, 0) , z3.If(r3s23, 0, 0) , z3.If(r3s24, 0, 0) , z3.If(r3s25, 0, 0) , z3.If(r3s26, 0, 0) , z3.If(r3s27, 0, 0) , z3.If(r3s28, 0, 0) , z3.If(r3s29, 0, 0) , z3.If(r3s30, 0, 0) , z3.If(r3s31, 0, 0) , z3.If(r3s32, 0, 0) , z3.If(r3s33, 0, 0) , z3.If(r3s34, 0, 0) , z3.If(r3s35, 0, 0) , z3.If(r3s36, 0, 0) , z3.If(r3s37, 90, 0) , z3.If(r3s38, 0, 0) , z3.If(r3s39, 0, 0) , z3.If(r3s40, 90, 0) , z3.If(r3s41, 0, 0) , z3.If(r3s42, 0, 0) , z3.If(r3s43, 0, 0) , z3.If(r3s44, 0, 0) , z3.If(r3s45, 0, 0) , z3.If(r3s46, 0, 0) , z3.If(r3s47, 0, 0) , z3.If(r3s48, 0, 0) , z3.If(r3s49, 0, 0) , z3.If(r3s50, 0, 0) , z3.If(r3s51, 90, 0) , z3.If(r3s52, 0, 0) , z3.If(r3s53, 0, 0) , z3.If(r4s0, 0, 0) , z3.If(r4s1, 0, 0) , z3.If(r4s2, 0, 0) , z3.If(r4s3, 0, 0) , z3.If(r4s4, 0, 0) , z3.If(r4s5, 0, 0) , z3.If(r4s6, 10, 0) , z3.If(r4s7, 0, 0) , z3.If(r4s8, 0, 0) , z3.If(r4s10, 0, 0) , z3.If(r4s11, 0, 0) , z3.If(r4s12, 90, 0) , z3.If(r4s13, 90, 0) , z3.If(r4s14, 0, 0) , z3.If(r4s15, 0, 0) , z3.If(r4s16, 0, 0) , z3.If(r4s17, 90, 0) , z3.If(r4s19, 0, 0) , z3.If(r4s20, 0, 0) , z3.If(r4s21, 0, 0) , z3.If(r4s22, 0, 0) , z3.If(r4s23, 0, 0) , z3.If(r4s24, 90, 0) , z3.If(r4s26, 0, 0) , z3.If(r4s27, 90, 0) , z3.If(r4s28, 0, 0) , z3.If(r4s29, 0, 0) , z3.If(r4s30, 0, 0) , z3.If(r4s31, 0, 0) , z3.If(r4s32, 0, 0) , z3.If(r4s33, 0, 0) , z3.If(r4s34, 0, 0) , z3.If(r4s35, 0, 0) , z3.If(r4s36, 10, 0) , z3.If(r4s37, 0, 0) , z3.If(r4s38, 0, 0) , z3.If(r4s39, 0, 0) , z3.If(r4s40, 10, 0) , z3.If(r4s41, 0, 0) , z3.If(r4s42, 90, 0) , z3.If(r4s43, 0, 0) , z3.If(r4s44, 0, 0) , z3.If(r4s45, 10, 0) , z3.If(r4s46, 10, 0) , z3.If(r4s47, 0, 0) , z3.If(r4s48, 0, 0) , z3.If(r4s49, 0, 0) , z3.If(r4s50, 0, 0) , z3.If(r4s51, 0, 0) , z3.If(r4s52, 0, 0) , z3.If(r4s53, 0, 0) , z3.If(r5s0, 0, 0) , z3.If(r5s1, 0, 0) , z3.If(r5s2, 0, 0) , z3.If(r5s3, 0, 0) , z3.If(r5s4, 0, 0) , z3.If(r5s5, 0, 0) , z3.If(r5s6, 90, 0) , z3.If(r5s7, 0, 0) , z3.If(r5s8, 0, 0) , z3.If(r5s9, 0, 0) , z3.If(r5s10, 0, 0) , z3.If(r5s11, 0, 0) , z3.If(r5s12, 0, 0) , z3.If(r5s13, 90, 0) , z3.If(r5s14, 0, 0) , z3.If(r5s15, 0, 0) , z3.If(r5s16, 0, 0) , z3.If(r5s17, 10, 0) , z3.If(r5s18, 0, 0) , z3.If(r5s19, 0, 0) , z3.If(r5s20, 0, 0) , z3.If(r5s21, 0, 0) , z3.If(r5s22, 0, 0) , z3.If(r5s23, 0, 0) , z3.If(r5s24, 10, 0) , z3.If(r5s25, 10, 0) , z3.If(r5s26, 0, 0) , z3.If(r5s27, 10, 0) , z3.If(r5s28, 0, 0) , z3.If(r5s29, 0, 0) , z3.If(r5s30, 10, 0) , z3.If(r5s31, 0, 0) , z3.If(r5s32, 0, 0) , z3.If(r5s33, 0, 0) , z3.If(r5s34, 0, 0) , z3.If(r5s35, 0, 0) , z3.If(r5s36, 10, 0) , z3.If(r5s37, 0, 0) , z3.If(r5s38, 0, 0) , z3.If(r5s39, 0, 0) , z3.If(r5s40, 10, 0) , z3.If(r5s41, 0, 0) , z3.If(r5s42, 90, 0) , z3.If(r5s43, 0, 0) , z3.If(r5s44, 0, 0) , z3.If(r5s45, 0, 0) , z3.If(r5s46, 0, 0) , z3.If(r5s47, 10, 0) , z3.If(r5s48, 0, 0) , z3.If(r5s49, 0, 0) , z3.If(r5s50, 0, 0) , z3.If(r5s51, 0, 0) , z3.If(r5s52, 0, 0) , z3.If(r5s53, 0, 0) , z3.If(r6s0, 0, 0) , z3.If(r6s1, 0, 0) , z3.If(r6s2, 0, 0) , z3.If(r6s3, 0, 0) , z3.If(r6s4, 0, 0) , z3.If(r6s5, 0, 0) , z3.If(r6s6, 10, 0) , z3.If(r6s7, 0, 0) , z3.If(r6s8, 0, 0) , z3.If(r6s9, 0, 0) , z3.If(r6s10, 0, 0) , z3.If(r6s11, 0, 0) , z3.If(r6s12, 10, 0) , z3.If(r6s13, 0, 0) , z3.If(r6s14, 0, 0) , z3.If(r6s15, 0, 0) , z3.If(r6s16, 0, 0) , z3.If(r6s17, 0, 0) , z3.If(r6s18, 0, 0) , z3.If(r6s19, 0, 0) , z3.If(r6s20, 0, 0) , z3.If(r6s21, 0, 0) , z3.If(r6s22, 0, 0) , z3.If(r6s23, 90, 0) , z3.If(r6s24, 0, 0) , z3.If(r6s25, 0, 0) , z3.If(r6s26, 0, 0) , z3.If(r6s27, 0, 0) , z3.If(r6s28, 0, 0) , z3.If(r6s29, 0, 0) , z3.If(r6s30, 0, 0) , z3.If(r6s31, 0, 0) , z3.If(r6s32, 0, 0) , z3.If(r6s33, 0, 0) , z3.If(r6s34, 0, 0) , z3.If(r6s35, 0, 0) , z3.If(r6s36, 90, 0) , z3.If(r6s37, 0, 0) , z3.If(r6s38, 0, 0) , z3.If(r6s39, 0, 0) , z3.If(r6s40, 10, 0) , z3.If(r6s41, 0, 0) , z3.If(r6s42, 0, 0) , z3.If(r6s43, 0, 0) , z3.If(r6s44, 0, 0) , z3.If(r6s45, 0, 0) , z3.If(r6s46, 0, 0) , z3.If(r6s47, 90, 0) , z3.If(r6s48, 0, 0) , z3.If(r6s49, 10, 0) , z3.If(r6s50, 90, 0) , z3.If(r6s51, 0, 0) , z3.If(r6s52, 0, 0) , z3.If(r6s53, 0, 0) , z3.If(r7s0, 10, 0) , z3.If(r7s1, 0, 0) , z3.If(r7s2, 10, 0) , z3.If(r7s3, 0, 0) , z3.If(r7s4, 0, 0) , z3.If(r7s5, 0, 0) , z3.If(r7s6, 0, 0) , z3.If(r7s7, 0, 0) , z3.If(r7s8, 0, 0) , z3.If(r7s9, 0, 0) , z3.If(r7s10, 0, 0) , z3.If(r7s11, 0, 0) , z3.If(r7s12, 90, 0) , z3.If(r7s13, 10, 0) , z3.If(r7s14, 0, 0) , z3.If(r7s15, 0, 0) , z3.If(r7s16, 0, 0) , z3.If(r7s17, 90, 0) , z3.If(r7s19, 0, 0) , z3.If(r7s20, 0, 0) , z3.If(r7s21, 0, 0) , z3.If(r7s22, 0, 0) , z3.If(r7s23, 0, 0) , z3.If(r7s25, 90, 0) , z3.If(r7s26, 0, 0) , z3.If(r7s27, 10, 0) , z3.If(r7s28, 0, 0) , z3.If(r7s29, 0, 0) , z3.If(r7s30, 10, 0) , z3.If(r7s31, 90, 0) , z3.If(r7s32, 0, 0) , z3.If(r7s33, 0, 0) , z3.If(r7s34, 0, 0) , z3.If(r7s35, 0, 0) , z3.If(r7s36, 0, 0) , z3.If(r7s37, 0, 0) , z3.If(r7s38, 0, 0) , z3.If(r7s39, 0, 0) , z3.If(r7s40, 0, 0) , z3.If(r7s41, 0, 0) , z3.If(r7s42, 90, 0) , z3.If(r7s43, 0, 0) , z3.If(r7s44, 0, 0) , z3.If(r7s45, 0, 0) , z3.If(r7s46, 0, 0) , z3.If(r7s47, 90, 0) , z3.If(r7s48, 0, 0) , z3.If(r7s49, 0, 0) , z3.If(r7s50, 0, 0) , z3.If(r7s51, 0, 0) , z3.If(r7s52, 90, 0) , z3.If(r7s53, 0, 0) , z3.If(r8s0, 0, 0) , z3.If(r8s2, 0, 0) , z3.If(r8s3, 90, 0) , z3.If(r8s4, 90, 0) , z3.If(r8s5, 0, 0) , z3.If(r8s6, 0, 0) , z3.If(r8s7, 90, 0) , z3.If(r8s8, 0, 0) , z3.If(r8s9, 0, 0) , z3.If(r8s10, 0, 0) , z3.If(r8s11, 0, 0) , z3.If(r8s12, 0, 0) , z3.If(r8s13, 0, 0) , z3.If(r8s14, 0, 0) , z3.If(r8s15, 0, 0) , z3.If(r8s16, 90, 0) , z3.If(r8s17, 0, 0) , z3.If(r8s18, 0, 0) , z3.If(r8s19, 90, 0) , z3.If(r8s20, 0, 0) , z3.If(r8s21, 0, 0) , z3.If(r8s22, 0, 0) , z3.If(r8s23, 0, 0) , z3.If(r8s24, 0, 0) , z3.If(r8s25, 0, 0) , z3.If(r8s26, 0, 0) , z3.If(r8s27, 0, 0) , z3.If(r8s28, 0, 0) , z3.If(r8s29, 0, 0) , z3.If(r8s30, 0, 0) , z3.If(r8s31, 0, 0) , z3.If(r8s32, 0, 0) , z3.If(r8s33, 0, 0) , z3.If(r8s34, 0, 0) , z3.If(r8s35, 0, 0) , z3.If(r8s36, 0, 0) , z3.If(r8s37, 0, 0) , z3.If(r8s38, 90, 0) , z3.If(r8s39, 0, 0) , z3.If(r8s40, 0, 0) , z3.If(r8s41, 0, 0) , z3.If(r8s42, 0, 0) , z3.If(r8s43, 0, 0) , z3.If(r8s44, 0, 0) , z3.If(r8s45, 0, 0) , z3.If(r8s46, 0, 0) , z3.If(r8s47, 0, 0) , z3.If(r8s48, 0, 0) , z3.If(r8s49, 0, 0) , z3.If(r8s50, 0, 0) , z3.If(r8s51, 0, 0) , z3.If(r8s52, 0, 0) , z3.If(r8s53, 0, 0) , z3.If(r9s0, 0, 0) , z3.If(r9s1, 90, 0) , z3.If(r9s2, 0, 0) , z3.If(r9s3, 0, 0) , z3.If(r9s4, 0, 0) , z3.If(r9s5, 10, 0) , z3.If(r9s6, 90, 0) , z3.If(r9s7, 0, 0) , z3.If(r9s8, 0, 0) , z3.If(r9s9, 0, 0) , z3.If(r9s10, 0, 0) , z3.If(r9s11, 0, 0) , z3.If(r9s12, 0, 0) , z3.If(r9s13, 90, 0) , z3.If(r9s14, 0, 0) , z3.If(r9s15, 0, 0) , z3.If(r9s16, 0, 0) , z3.If(r9s17, 10, 0) , z3.If(r9s18, 0, 0) , z3.If(r9s19, 0, 0) , z3.If(r9s20, 0, 0) , z3.If(r9s21, 10, 0) , z3.If(r9s22, 0, 0) , z3.If(r9s23, 0, 0) , z3.If(r9s24, 0, 0) , z3.If(r9s25, 10, 0) , z3.If(r9s26, 0, 0) , z3.If(r9s27, 10, 0) , z3.If(r9s28, 0, 0) , z3.If(r9s29, 0, 0) , z3.If(r9s30, 10, 0) , z3.If(r9s31, 0, 0) , z3.If(r9s32, 0, 0) , z3.If(r9s33, 0, 0) , z3.If(r9s34, 0, 0) , z3.If(r9s35, 0, 0) , z3.If(r9s36, 0, 0) , z3.If(r9s37, 0, 0) , z3.If(r9s38, 0, 0) , z3.If(r9s39, 0, 0) , z3.If(r9s40, 10, 0) , z3.If(r9s41, 0, 0) , z3.If(r9s42, 0, 0) , z3.If(r9s43, 0, 0) , z3.If(r9s44, 0, 0) , z3.If(r9s45, 0, 0) , z3.If(r9s46, 0, 0) , z3.If(r9s47, 0, 0) , z3.If(r9s48, 0, 0) , z3.If(r9s49, 0, 0) , z3.If(r9s50, 0, 0) , z3.If(r9s51, 0, 0) , z3.If(r9s52, 90, 0) , z3.If(r9s53, 0, 0) , z3.If(r10s0, 10, 0) , z3.If(r10s1, 0, 0) , z3.If(r10s2, 0, 0) , z3.If(r10s3, 0, 0) , z3.If(r10s4, 0, 0) , z3.If(r10s5, 0, 0) , z3.If(r10s6, 10, 0) , z3.If(r10s7, 10, 0) , z3.If(r10s8, 0, 0) , z3.If(r10s9, 10, 0) , z3.If(r10s10, 0, 0) , z3.If(r10s11, 0, 0) , z3.If(r10s12, 10, 0) , z3.If(r10s13, 10, 0) , z3.If(r10s14, 10, 0) , z3.If(r10s15, 0, 0) , z3.If(r10s16, 10, 0) , z3.If(r10s17, 0, 0) , z3.If(r10s18, 0, 0) , z3.If(r10s19, 0, 0) , z3.If(r10s20, 0, 0) , z3.If(r10s21, 0, 0) , z3.If(r10s22, 10, 0) , z3.If(r10s23, 0, 0) , z3.If(r10s24, 0, 0) , z3.If(r10s25, 0, 0) , z3.If(r10s26, 0, 0) , z3.If(r10s27, 10, 0) , z3.If(r10s28, 0, 0) , z3.If(r10s29, 0, 0) , z3.If(r10s30, 0, 0) , z3.If(r10s31, 0, 0) , z3.If(r10s32, 0, 0) , z3.If(r10s33, 0, 0) , z3.If(r10s34, 0, 0) , z3.If(r10s35, 0, 0) , z3.If(r10s36, 0, 0) , z3.If(r10s37, 0, 0) , z3.If(r10s38, 0, 0) , z3.If(r10s39, 0, 0) , z3.If(r10s40, 10, 0) , z3.If(r10s41, 0, 0) , z3.If(r10s42, 0, 0) , z3.If(r10s43, 0, 0) , z3.If(r10s44, 0, 0) , z3.If(r10s45, 0, 0) , z3.If(r10s46, 0, 0) , z3.If(r10s47, 0, 0) , z3.If(r10s48, 0, 0) , z3.If(r10s49, 0, 0) , z3.If(r10s50, 10, 0) , z3.If(r10s51, 0, 0) , z3.If(r10s52, 0, 0) , z3.If(r10s53, 0, 0) , z3.If(r11s0, 10, 0) , z3.If(r11s1, 0, 0) , z3.If(r11s2, 0, 0) , z3.If(r11s3, 10, 0) , z3.If(r11s4, 0, 0) , z3.If(r11s5, 0, 0) , z3.If(r11s6, 90, 0) , z3.If(r11s7, 0, 0) , z3.If(r11s8, 0, 0) , z3.If(r11s9, 10, 0) , z3.If(r11s10, 0, 0) , z3.If(r11s11, 0, 0) , z3.If(r11s12, 0, 0) , z3.If(r11s13, 10, 0) , z3.If(r11s14, 90, 0) , z3.If(r11s15, 0, 0) , z3.If(r11s16, 0, 0) , z3.If(r11s17, 0, 0) , z3.If(r11s18, 0, 0) , z3.If(r11s19, 0, 0) , z3.If(r11s20, 0, 0) , z3.If(r11s21, 0, 0) , z3.If(r11s22, 0, 0) , z3.If(r11s23, 0, 0) , z3.If(r11s24, 0, 0) , z3.If(r11s25, 0, 0) , z3.If(r11s26, 0, 0) , z3.If(r11s27, 90, 0) , z3.If(r11s28, 0, 0) , z3.If(r11s29, 0, 0) , z3.If(r11s30, 0, 0) , z3.If(r11s31, 0, 0) , z3.If(r11s32, 0, 0) , z3.If(r11s33, 0, 0) , z3.If(r11s34, 0, 0) , z3.If(r11s35, 0, 0) , z3.If(r11s36, 0, 0) , z3.If(r11s37, 0, 0) , z3.If(r11s38, 0, 0) , z3.If(r11s39, 0, 0) , z3.If(r11s40, 0, 0) , z3.If(r11s41, 90, 0) , z3.If(r11s42, 90, 0) , z3.If(r11s43, 0, 0) , z3.If(r11s44, 0, 0) , z3.If(r11s45, 0, 0) , z3.If(r11s46, 0, 0) , z3.If(r11s47, 0, 0) , z3.If(r11s48, 0, 0) , z3.If(r11s49, 0, 0) , z3.If(r11s50, 0, 0) , z3.If(r11s51, 0, 0) , z3.If(r11s52, 0, 0) , z3.If(r11s53, 0, 0) , z3.If(r12s0, 0, 0) , z3.If(r12s1, 10, 0) , z3.If(r12s2, 0, 0) , z3.If(r12s3, 10, 0) , z3.If(r12s4, 0, 0) , z3.If(r12s5, 0, 0) , z3.If(r12s6, 10, 0) , z3.If(r12s8, 0, 0) , z3.If(r12s9, 10, 0) , z3.If(r12s10, 0, 0) , z3.If(r12s11, 0, 0) , z3.If(r12s12, 10, 0) , z3.If(r12s13, 0, 0) , z3.If(r12s14, 0, 0) , z3.If(r12s15, 0, 0) , z3.If(r12s16, 10, 0) , z3.If(r12s17, 10, 0) , z3.If(r12s18, 0, 0) , z3.If(r12s19, 0, 0) , z3.If(r12s20, 0, 0) , z3.If(r12s21, 10, 0) , z3.If(r12s22, 0, 0) , z3.If(r12s23, 0, 0) , z3.If(r12s24, 0, 0) , z3.If(r12s25, 0, 0) , z3.If(r12s26, 0, 0) , z3.If(r12s27, 0, 0) , z3.If(r12s28, 0, 0) , z3.If(r12s30, 0, 0) , z3.If(r12s31, 0, 0) , z3.If(r12s32, 0, 0) , z3.If(r12s33, 0, 0) , z3.If(r12s34, 0, 0) , z3.If(r12s35, 0, 0) , z3.If(r12s36, 0, 0) , z3.If(r12s37, 0, 0) , z3.If(r12s38, 0, 0) , z3.If(r12s39, 0, 0) , z3.If(r12s40, 0, 0) , z3.If(r12s41, 0, 0) , z3.If(r12s42, 0, 0) , z3.If(r12s43, 0, 0) , z3.If(r12s44, 0, 0) , z3.If(r12s45, 0, 0) , z3.If(r12s46, 10, 0) , z3.If(r12s47, 0, 0) , z3.If(r12s48, 0, 0) , z3.If(r12s49, 0, 0) , z3.If(r12s50, 0, 0) , z3.If(r12s51, 0, 0) , z3.If(r12s52, 0, 0) , z3.If(r12s53, 0, 0) , z3.If(r13s0, 90, 0) , z3.If(r13s1, 90, 0) , z3.If(r13s2, 90, 0) , z3.If(r13s3, 0, 0) , z3.If(r13s4, 10, 0) , z3.If(r13s5, 0, 0) , z3.If(r13s6, 0, 0) , z3.If(r13s7, 0, 0) , z3.If(r13s8, 0, 0) , z3.If(r13s9, 90, 0) , z3.If(r13s10, 0, 0) , z3.If(r13s11, 0, 0) , z3.If(r13s12, 90, 0) , z3.If(r13s13, 0, 0) , z3.If(r13s14, 0, 0) , z3.If(r13s15, 0, 0) , z3.If(r13s16, 0, 0) , z3.If(r13s17, 10, 0) , z3.If(r13s18, 90, 0) , z3.If(r13s19, 0, 0) , z3.If(r13s20, 0, 0) , z3.If(r13s21, 0, 0) , z3.If(r13s22, 0, 0) , z3.If(r13s23, 0, 0) , z3.If(r13s24, 90, 0) , z3.If(r13s25, 90, 0) , z3.If(r13s26, 0, 0) , z3.If(r13s27, 90, 0) , z3.If(r13s28, 0, 0) , z3.If(r13s29, 0, 0) , z3.If(r13s30, 0, 0) , z3.If(r13s31, 0, 0) , z3.If(r13s32, 0, 0) , z3.If(r13s33, 0, 0) , z3.If(r13s34, 0, 0) , z3.If(r13s35, 0, 0) , z3.If(r13s36, 0, 0) , z3.If(r13s37, 0, 0) , z3.If(r13s38, 0, 0) , z3.If(r13s39, 90, 0) , z3.If(r13s40, 10, 0) , z3.If(r13s41, 0, 0) , z3.If(r13s42, 0, 0) , z3.If(r13s43, 0, 0) , z3.If(r13s44, 0, 0) , z3.If(r13s45, 0, 0) , z3.If(r13s46, 0, 0) , z3.If(r13s47, 0, 0) , z3.If(r13s48, 0, 0) , z3.If(r13s49, 0, 0) , z3.If(r13s50, 0, 0) , z3.If(r13s51, 0, 0) , z3.If(r13s52, 0, 0) , z3.If(r13s53, 0, 0) , z3.If(r14s0, 0, 0) , z3.If(r14s1, 10, 0) , z3.If(r14s2, 0, 0) , z3.If(r14s3, 0, 0) , z3.If(r14s4, 0, 0) , z3.If(r14s5, 0, 0) , z3.If(r14s6, 90, 0) , z3.If(r14s7, 10, 0) , z3.If(r14s8, 0, 0) , z3.If(r14s9, 10, 0) , z3.If(r14s10, 0, 0) , z3.If(r14s11, 0, 0) , z3.If(r14s12, 0, 0) , z3.If(r14s14, 90, 0) , z3.If(r14s15, 0, 0) , z3.If(r14s16, 0, 0) , z3.If(r14s17, 0, 0) , z3.If(r14s18, 0, 0) , z3.If(r14s19, 0, 0) , z3.If(r14s20, 0, 0) , z3.If(r14s21, 0, 0) , z3.If(r14s22, 0, 0) , z3.If(r14s23, 0, 0) , z3.If(r14s25, 0, 0) , z3.If(r14s26, 0, 0) , z3.If(r14s27, 0, 0) , z3.If(r14s28, 0, 0) , z3.If(r14s29, 0, 0) , z3.If(r14s30, 10, 0) , z3.If(r14s31, 0, 0) , z3.If(r14s32, 0, 0) , z3.If(r14s33, 0, 0) , z3.If(r14s34, 0, 0) , z3.If(r14s35, 0, 0) , z3.If(r14s36, 0, 0) , z3.If(r14s37, 0, 0) , z3.If(r14s38, 0, 0) , z3.If(r14s39, 0, 0) , z3.If(r14s40, 90, 0) , z3.If(r14s41, 0, 0) , z3.If(r14s42, 0, 0) , z3.If(r14s43, 0, 0) , z3.If(r14s44, 0, 0) , z3.If(r14s45, 0, 0) , z3.If(r14s46, 10, 0) , z3.If(r14s47, 0, 0) , z3.If(r14s48, 0, 0) , z3.If(r14s49, 0, 0) , z3.If(r14s50, 0, 0) , z3.If(r14s51, 0, 0) , z3.If(r14s52, 0, 0) , z3.If(r14s53, 0, 0) , z3.If(r15s0, 10, 0) , z3.If(r15s1, 0, 0) , z3.If(r15s2, 0, 0) , z3.If(r15s3, 0, 0) , z3.If(r15s4, 0, 0) , z3.If(r15s5, 0, 0) , z3.If(r15s6, 0, 0) , z3.If(r15s7, 0, 0) , z3.If(r15s8, 0, 0) , z3.If(r15s9, 0, 0) , z3.If(r15s10, 0, 0) , z3.If(r15s11, 0, 0) , z3.If(r15s12, 0, 0) , z3.If(r15s13, 90, 0) , z3.If(r15s14, 0, 0) , z3.If(r15s15, 0, 0) , z3.If(r15s16, 0, 0) , z3.If(r15s17, 0, 0) , z3.If(r15s18, 0, 0) , z3.If(r15s19, 0, 0) , z3.If(r15s20, 0, 0) , z3.If(r15s21, 0, 0) , z3.If(r15s22, 10, 0) , z3.If(r15s23, 0, 0) , z3.If(r15s24, 10, 0) , z3.If(r15s25, 10, 0) , z3.If(r15s26, 0, 0) , z3.If(r15s27, 90, 0) , z3.If(r15s28, 0, 0) , z3.If(r15s29, 0, 0) , z3.If(r15s30, 0, 0) , z3.If(r15s31, 0, 0) , z3.If(r15s32, 0, 0) , z3.If(r15s33, 0, 0) , z3.If(r15s34, 0, 0) , z3.If(r15s35, 0, 0) , z3.If(r15s36, 0, 0) , z3.If(r15s37, 0, 0) , z3.If(r15s38, 0, 0) , z3.If(r15s39, 0, 0) , z3.If(r15s40, 10, 0) , z3.If(r15s41, 0, 0) , z3.If(r15s42, 0, 0) , z3.If(r15s43, 0, 0) , z3.If(r15s44, 0, 0) , z3.If(r15s45, 0, 0) , z3.If(r15s46, 0, 0) , z3.If(r15s47, 0, 0) , z3.If(r15s48, 0, 0) , z3.If(r15s49, 10, 0) , z3.If(r15s50, 0, 0) , z3.If(r15s51, 0, 0) , z3.If(r15s52, 0, 0) , z3.If(r15s53, 0, 0) , z3.If(r16s0, 10, 0) , z3.If(r16s1, 10, 0) , z3.If(r16s2, 0, 0) , z3.If(r16s3, 10, 0) , z3.If(r16s4, 0, 0) , z3.If(r16s5, 10, 0) , z3.If(r16s6, 0, 0) , z3.If(r16s7, 10, 0) , z3.If(r16s8, 0, 0) , z3.If(r16s9, 0, 0) , z3.If(r16s10, 0, 0) , z3.If(r16s11, 90, 0) , z3.If(r16s12, 0, 0) , z3.If(r16s13, 0, 0) , z3.If(r16s14, 0, 0) , z3.If(r16s15, 0, 0) , z3.If(r16s16, 0, 0) , z3.If(r16s17, 0, 0) , z3.If(r16s18, 0, 0) , z3.If(r16s19, 10, 0) , z3.If(r16s20, 0, 0) , z3.If(r16s21, 0, 0) , z3.If(r16s22, 0, 0) , z3.If(r16s23, 0, 0) , z3.If(r16s24, 0, 0) , z3.If(r16s25, 0, 0) , z3.If(r16s26, 0, 0) , z3.If(r16s27, 10, 0) , z3.If(r16s28, 0, 0) , z3.If(r16s29, 0, 0) , z3.If(r16s30, 0, 0) , z3.If(r16s31, 0, 0) , z3.If(r16s32, 0, 0) , z3.If(r16s33, 0, 0) , z3.If(r16s34, 0, 0) , z3.If(r16s35, 0, 0) , z3.If(r16s36, 0, 0) , z3.If(r16s37, 10, 0) , z3.If(r16s38, 0, 0) , z3.If(r16s39, 0, 0) , z3.If(r16s40, 0, 0) , z3.If(r16s41, 0, 0) , z3.If(r16s42, 0, 0) , z3.If(r16s44, 0, 0) , z3.If(r16s45, 0, 0) , z3.If(r16s46, 90, 0) , z3.If(r16s47, 0, 0) , z3.If(r16s48, 0, 0) , z3.If(r16s50, 0, 0) , z3.If(r16s52, 0, 0) , z3.If(r16s53, 0, 0) , z3.If(r17s0, 0, 0) , z3.If(r17s1, 90, 0) , z3.If(r17s2, 0, 0) , z3.If(r17s3, 90, 0) , z3.If(r17s4, 0, 0) , z3.If(r17s5, 0, 0) , z3.If(r17s6, 0, 0) , z3.If(r17s7, 0, 0) , z3.If(r17s8, 0, 0) , z3.If(r17s9, 90, 0) , z3.If(r17s10, 90, 0) , z3.If(r17s11, 0, 0) , z3.If(r17s12, 0, 0) , z3.If(r17s13, 0, 0) , z3.If(r17s14, 0, 0) , z3.If(r17s15, 0, 0) , z3.If(r17s16, 0, 0) , z3.If(r17s17, 0, 0) , z3.If(r17s18, 0, 0) , z3.If(r17s19, 0, 0) , z3.If(r17s20, 0, 0) , z3.If(r17s22, 0, 0) , z3.If(r17s23, 0, 0) , z3.If(r17s24, 0, 0) , z3.If(r17s25, 0, 0) , z3.If(r17s27, 0, 0) , z3.If(r17s28, 0, 0) , z3.If(r17s29, 90, 0) , z3.If(r17s30, 0, 0) , z3.If(r17s31, 0, 0) , z3.If(r17s32, 0, 0) , z3.If(r17s33, 90, 0) , z3.If(r17s34, 0, 0) , z3.If(r17s35, 0, 0) , z3.If(r17s36, 0, 0) , z3.If(r17s37, 0, 0) , z3.If(r17s38, 0, 0) , z3.If(r17s39, 0, 0) , z3.If(r17s40, 0, 0) , z3.If(r17s41, 0, 0) , z3.If(r17s42, 0, 0) , z3.If(r17s43, 0, 0) , z3.If(r17s44, 90, 0) , z3.If(r17s46, 0, 0) , z3.If(r17s47, 0, 0) , z3.If(r17s49, 0, 0) , z3.If(r17s50, 0, 0) , z3.If(r17s51, 0, 0) , z3.If(r17s52, 0, 0) , z3.If(r18s0, 0, 0) , z3.If(r18s1, 0, 0) , z3.If(r18s2, 0, 0) , z3.If(r18s3, 0, 0) , z3.If(r18s4, 0, 0) , z3.If(r18s5, 90, 0) , z3.If(r18s6, 0, 0) , z3.If(r18s7, 0, 0) , z3.If(r18s8, 0, 0) , z3.If(r18s9, 0, 0) , z3.If(r18s10, 0, 0) , z3.If(r18s11, 0, 0) , z3.If(r18s12, 90, 0) , z3.If(r18s13, 0, 0) , z3.If(r18s14, 10, 0) , z3.If(r18s15, 0, 0) , z3.If(r18s16, 0, 0) , z3.If(r18s17, 10, 0) , z3.If(r18s18, 10, 0) , z3.If(r18s19, 0, 0) , z3.If(r18s20, 0, 0) , z3.If(r18s21, 0, 0) , z3.If(r18s22, 90, 0) , z3.If(r18s23, 0, 0) , z3.If(r18s24, 0, 0) , z3.If(r18s25, 0, 0) , z3.If(r18s26, 0, 0) , z3.If(r18s27, 0, 0) , z3.If(r18s28, 0, 0) , z3.If(r18s29, 0, 0) , z3.If(r18s30, 0, 0) , z3.If(r18s31, 0, 0) , z3.If(r18s32, 0, 0) , z3.If(r18s33, 0, 0) , z3.If(r18s34, 0, 0) , z3.If(r18s35, 0, 0) , z3.If(r18s36, 10, 0) , z3.If(r18s37, 0, 0) , z3.If(r18s38, 0, 0) , z3.If(r18s39, 0, 0) , z3.If(r18s40, 0, 0) , z3.If(r18s41, 0, 0) , z3.If(r18s42, 0, 0) , z3.If(r18s43, 0, 0) , z3.If(r18s44, 0, 0) , z3.If(r18s45, 0, 0) , z3.If(r18s46, 0, 0) , z3.If(r18s47, 0, 0) , z3.If(r18s48, 0, 0) , z3.If(r18s49, 0, 0) , z3.If(r18s50, 90, 0) , z3.If(r18s51, 0, 0) , z3.If(r18s52, 0, 0) , z3.If(r18s53, 0, 0) , z3.If(r19s0, 10, 0) , z3.If(r19s1, 0, 0) , z3.If(r19s2, 90, 0) , z3.If(r19s3, 0, 0) , z3.If(r19s4, 0, 0) , z3.If(r19s5, 0, 0) , z3.If(r19s6, 0, 0) , z3.If(r19s7, 0, 0) , z3.If(r19s8, 0, 0) , z3.If(r19s9, 0, 0) , z3.If(r19s10, 0, 0) , z3.If(r19s11, 0, 0) , z3.If(r19s12, 0, 0) , z3.If(r19s13, 10, 0) , z3.If(r19s14, 0, 0) , z3.If(r19s15, 0, 0) , z3.If(r19s16, 10, 0) , z3.If(r19s17, 0, 0) , z3.If(r19s18, 0, 0) , z3.If(r19s19, 0, 0) , z3.If(r19s20, 0, 0) , z3.If(r19s21, 0, 0) , z3.If(r19s22, 0, 0) , z3.If(r19s23, 0, 0) , z3.If(r19s24, 0, 0) , z3.If(r19s25, 0, 0) , z3.If(r19s26, 0, 0) , z3.If(r19s27, 0, 0) , z3.If(r19s28, 0, 0) , z3.If(r19s29, 0, 0) , z3.If(r19s30, 0, 0) , z3.If(r19s31, 0, 0) , z3.If(r19s32, 0, 0) , z3.If(r19s33, 0, 0) , z3.If(r19s34, 0, 0) , z3.If(r19s35, 0, 0) , z3.If(r19s36, 0, 0) , z3.If(r19s37, 0, 0) , z3.If(r19s38, 0, 0) , z3.If(r19s39, 0, 0) , z3.If(r19s40, 0, 0) , z3.If(r19s41, 0, 0) , z3.If(r19s42, 0, 0) , z3.If(r19s43, 0, 0) , z3.If(r19s44, 0, 0) , z3.If(r19s45, 0, 0) , z3.If(r19s46, 0, 0) , z3.If(r19s47, 0, 0) , z3.If(r19s48, 0, 0) , z3.If(r19s49, 0, 0) , z3.If(r19s50, 0, 0) , z3.If(r19s51, 90, 0) , z3.If(r19s52, 0, 0) , z3.If(r19s53, 0, 0) , z3.If(r20s0, 90, 0) , z3.If(r20s1, 0, 0) , z3.If(r20s2, 90, 0) , z3.If(r20s3, 0, 0) , z3.If(r20s4, 0, 0) , z3.If(r20s5, 0, 0) , z3.If(r20s6, 90, 0) , z3.If(r20s7, 0, 0) , z3.If(r20s8, 0, 0) , z3.If(r20s9, 90, 0) , z3.If(r20s10, 0, 0) , z3.If(r20s11, 0, 0) , z3.If(r20s13, 0, 0) , z3.If(r20s14, 0, 0) , z3.If(r20s15, 0, 0) , z3.If(r20s16, 0, 0) , z3.If(r20s17, 90, 0) , z3.If(r20s18, 90, 0) , z3.If(r20s19, 0, 0) , z3.If(r20s20, 0, 0) , z3.If(r20s21, 0, 0) , z3.If(r20s22, 90, 0) , z3.If(r20s23, 0, 0) , z3.If(r20s24, 90, 0) , z3.If(r20s25, 90, 0) , z3.If(r20s26, 0, 0) , z3.If(r20s27, 90, 0) , z3.If(r20s28, 0, 0) , z3.If(r20s29, 0, 0) , z3.If(r20s30, 90, 0) , z3.If(r20s31, 90, 0) , z3.If(r20s32, 0, 0) , z3.If(r20s33, 0, 0) , z3.If(r20s34, 0, 0) , z3.If(r20s35, 0, 0) , z3.If(r20s36, 0, 0) , z3.If(r20s37, 0, 0) , z3.If(r20s38, 0, 0) , z3.If(r20s39, 0, 0) , z3.If(r20s40, 0, 0) , z3.If(r20s41, 0, 0) , z3.If(r20s42, 0, 0) , z3.If(r20s43, 0, 0) , z3.If(r20s44, 0, 0) , z3.If(r20s45, 0, 0) , z3.If(r20s46, 0, 0) , z3.If(r20s47, 0, 0) , z3.If(r20s48, 0, 0) , z3.If(r20s49, 0, 0) , z3.If(r20s50, 0, 0) , z3.If(r20s51, 0, 0) , z3.If(r20s52, 0, 0) , z3.If(r20s53, 0, 0) , z3.If(r21s0, 90, 0) , z3.If(r21s1, 0, 0) , z3.If(r21s2, 0, 0) , z3.If(r21s3, 90, 0) , z3.If(r21s4, 0, 0) , z3.If(r21s5, 0, 0) , z3.If(r21s6, 10, 0) , z3.If(r21s7, 10, 0) , z3.If(r21s8, 0, 0) , z3.If(r21s9, 90, 0) , z3.If(r21s10, 0, 0) , z3.If(r21s11, 0, 0) , z3.If(r21s12, 0, 0) , z3.If(r21s13, 10, 0) , z3.If(r21s14, 0, 0) , z3.If(r21s15, 0, 0) , z3.If(r21s16, 10, 0) , z3.If(r21s17, 0, 0) , z3.If(r21s18, 10, 0) , z3.If(r21s19, 0, 0) , z3.If(r21s20, 0, 0) , z3.If(r21s21, 0, 0) , z3.If(r21s22, 90, 0) , z3.If(r21s23, 0, 0) , z3.If(r21s24, 0, 0) , z3.If(r21s25, 0, 0) , z3.If(r21s26, 0, 0) , z3.If(r21s27, 10, 0) , z3.If(r21s28, 90, 0) , z3.If(r21s29, 0, 0) , z3.If(r21s30, 0, 0) , z3.If(r21s31, 0, 0) , z3.If(r21s32, 0, 0) , z3.If(r21s33, 0, 0) , z3.If(r21s34, 90, 0) , z3.If(r21s35, 0, 0) , z3.If(r21s36, 0, 0) , z3.If(r21s37, 0, 0) , z3.If(r21s38, 0, 0) , z3.If(r21s39, 0, 0) , z3.If(r21s40, 90, 0) , z3.If(r21s41, 0, 0) , z3.If(r21s43, 0, 0) , z3.If(r21s44, 0, 0) , z3.If(r21s45, 0, 0) , z3.If(r21s46, 0, 0) , z3.If(r21s47, 0, 0) , z3.If(r21s48, 0, 0) , z3.If(r21s49, 0, 0) , z3.If(r21s50, 0, 0) , z3.If(r21s51, 0, 0) , z3.If(r21s52, 0, 0) , z3.If(r21s53, 0, 0) , z3.If(r22s0, 0, 0) , z3.If(r22s1, 0, 0) , z3.If(r22s2, 0, 0) , z3.If(r22s3, 90, 0) , z3.If(r22s4, 0, 0) , z3.If(r22s5, 0, 0) , z3.If(r22s6, 0, 0) , z3.If(r22s7, 10, 0) , z3.If(r22s8, 10, 0) , z3.If(r22s9, 0, 0) , z3.If(r22s10, 0, 0) , z3.If(r22s11, 0, 0) , z3.If(r22s12, 10, 0) , z3.If(r22s13, 0, 0) , z3.If(r22s14, 0, 0) , z3.If(r22s15, 0, 0) , z3.If(r22s16, 90, 0) , z3.If(r22s17, 10, 0) , z3.If(r22s18, 0, 0) , z3.If(r22s19, 0, 0) , z3.If(r22s20, 0, 0) , z3.If(r22s21, 0, 0) , z3.If(r22s22, 0, 0) , z3.If(r22s23, 0, 0) , z3.If(r22s24, 0, 0) , z3.If(r22s25, 0, 0) , z3.If(r22s26, 0, 0) , z3.If(r22s27, 0, 0) , z3.If(r22s28, 0, 0) , z3.If(r22s29, 0, 0) , z3.If(r22s30, 0, 0) , z3.If(r22s31, 0, 0) , z3.If(r22s32, 0, 0) , z3.If(r22s33, 0, 0) , z3.If(r22s34, 0, 0) , z3.If(r22s35, 0, 0) , z3.If(r22s36, 0, 0) , z3.If(r22s37, 0, 0) , z3.If(r22s38, 0, 0) , z3.If(r22s39, 0, 0) , z3.If(r22s40, 0, 0) , z3.If(r22s41, 0, 0) , z3.If(r22s42, 0, 0) , z3.If(r22s43, 0, 0) , z3.If(r22s44, 0, 0) , z3.If(r22s45, 0, 0) , z3.If(r22s46, 0, 0) , z3.If(r22s47, 0, 0) , z3.If(r22s48, 0, 0) , z3.If(r22s49, 0, 0) , z3.If(r22s50, 0, 0) , z3.If(r22s51, 0, 0) , z3.If(r22s52, 0, 0) , z3.If(r22s53, 0, 0) , z3.If(r23s0, 0, 0) , z3.If(r23s1, 90, 0) , z3.If(r23s2, 0, 0) , z3.If(r23s3, 10, 0) , z3.If(r23s4, 0, 0) , z3.If(r23s5, 0, 0) , z3.If(r23s6, 90, 0) , z3.If(r23s7, 0, 0) , z3.If(r23s8, 0, 0) , z3.If(r23s9, 0, 0) , z3.If(r23s10, 0, 0) , z3.If(r23s11, 0, 0) , z3.If(r23s12, 0, 0) , z3.If(r23s13, 0, 0) , z3.If(r23s14, 90, 0) , z3.If(r23s15, 0, 0) , z3.If(r23s16, 10, 0) , z3.If(r23s17, 90, 0) , z3.If(r23s19, 0, 0) , z3.If(r23s20, 0, 0) , z3.If(r23s21, 0, 0) , z3.If(r23s22, 0, 0) , z3.If(r23s23, 0, 0) , z3.If(r23s24, 0, 0) , z3.If(r23s25, 90, 0) , z3.If(r23s26, 0, 0) , z3.If(r23s27, 0, 0) , z3.If(r23s28, 0, 0) , z3.If(r23s29, 0, 0) , z3.If(r23s30, 90, 0) , z3.If(r23s31, 10, 0) , z3.If(r23s32, 0, 0) , z3.If(r23s33, 0, 0) , z3.If(r23s34, 0, 0) , z3.If(r23s35, 0, 0) , z3.If(r23s36, 0, 0) , z3.If(r23s37, 90, 0) , z3.If(r23s38, 0, 0) , z3.If(r23s39, 0, 0) , z3.If(r23s41, 0, 0) , z3.If(r23s42, 0, 0) , z3.If(r23s43, 0, 0) , z3.If(r23s44, 0, 0) , z3.If(r23s45, 0, 0) , z3.If(r23s46, 0, 0) , z3.If(r23s47, 0, 0) , z3.If(r23s48, 0, 0) , z3.If(r23s49, 0, 0) , z3.If(r23s50, 0, 0) , z3.If(r23s51, 0, 0) , z3.If(r23s52, 0, 0) , z3.If(r23s53, 0, 0) , z3.If(r24s0, 10, 0) , z3.If(r24s1, 0, 0) , z3.If(r24s2, 0, 0) , z3.If(r24s3, 0, 0) , z3.If(r24s4, 0, 0) , z3.If(r24s5, 0, 0) , z3.If(r24s6, 10, 0) , z3.If(r24s7, 0, 0) , z3.If(r24s8, 0, 0) , z3.If(r24s9, 0, 0) , z3.If(r24s10, 0, 0) , z3.If(r24s11, 0, 0) , z3.If(r24s12, 0, 0) , z3.If(r24s14, 0, 0) , z3.If(r24s15, 90, 0) , z3.If(r24s16, 10, 0) , z3.If(r24s17, 0, 0) , z3.If(r24s18, 0, 0) , z3.If(r24s19, 10, 0) , z3.If(r24s20, 90, 0) , z3.If(r24s21, 0, 0) , z3.If(r24s22, 90, 0) , z3.If(r24s23, 0, 0) , z3.If(r24s24, 0, 0) , z3.If(r24s25, 0, 0) , z3.If(r24s26, 0, 0) , z3.If(r24s27, 10, 0) , z3.If(r24s28, 0, 0) , z3.If(r24s29, 0, 0) , z3.If(r24s30, 0, 0) , z3.If(r24s31, 0, 0) , z3.If(r24s32, 0, 0) , z3.If(r24s33, 0, 0) , z3.If(r24s34, 0, 0) , z3.If(r24s35, 0, 0) , z3.If(r24s36, 0, 0) , z3.If(r24s37, 0, 0) , z3.If(r24s38, 0, 0) , z3.If(r24s39, 0, 0) , z3.If(r24s40, 0, 0) , z3.If(r24s42, 0, 0) , z3.If(r24s44, 0, 0) , z3.If(r24s45, 0, 0) , z3.If(r24s46, 10, 0) , z3.If(r24s47, 0, 0) , z3.If(r24s48, 0, 0) , z3.If(r24s50, 0, 0) , z3.If(r24s52, 0, 0) , z3.If(r24s53, 0, 0) , z3.If(r25s0, 0, 0) , z3.If(r25s1, 0, 0) , z3.If(r25s2, 10, 0) , z3.If(r25s3, 0, 0) , z3.If(r25s4, 0, 0) , z3.If(r25s5, 0, 0) , z3.If(r25s6, 90, 0) , z3.If(r25s7, 10, 0) , z3.If(r25s8, 0, 0) , z3.If(r25s9, 10, 0) , z3.If(r25s10, 0, 0) , z3.If(r25s11, 0, 0) , z3.If(r25s13, 0, 0) , z3.If(r25s14, 0, 0) , z3.If(r25s15, 0, 0) , z3.If(r25s16, 10, 0) , z3.If(r25s17, 90, 0) , z3.If(r25s19, 0, 0) , z3.If(r25s20, 0, 0) , z3.If(r25s21, 0, 0) , z3.If(r25s22, 0, 0) , z3.If(r25s23, 0, 0) , z3.If(r25s24, 0, 0) , z3.If(r25s25, 0, 0) , z3.If(r25s26, 0, 0) , z3.If(r25s27, 90, 0) , z3.If(r25s28, 0, 0) , z3.If(r25s29, 0, 0) , z3.If(r25s30, 10, 0) , z3.If(r25s31, 10, 0) , z3.If(r25s32, 10, 0) , z3.If(r25s33, 0, 0) , z3.If(r25s34, 0, 0) , z3.If(r25s35, 0, 0) , z3.If(r25s36, 10, 0) , z3.If(r25s37, 0, 0) , z3.If(r25s38, 0, 0) , z3.If(r25s39, 0, 0) , z3.If(r25s40, 10, 0) , z3.If(r25s41, 0, 0) , z3.If(r25s42, 10, 0) , z3.If(r25s43, 10, 0) , z3.If(r25s44, 0, 0) , z3.If(r25s45, 0, 0) , z3.If(r25s46, 0, 0) , z3.If(r25s47, 10, 0) , z3.If(r25s48, 0, 0) , z3.If(r25s49, 0, 0) , z3.If(r25s50, 0, 0) , z3.If(r25s51, 0, 0) , z3.If(r25s52, 0, 0) , z3.If(r25s53, 0, 0) , z3.If(r26s0, 90, 0) , z3.If(r26s1, 10, 0) , z3.If(r26s2, 0, 0) , z3.If(r26s3, 90, 0) , z3.If(r26s4, 90, 0) , z3.If(r26s5, 0, 0) , z3.If(r26s6, 0, 0) , z3.If(r26s7, 0, 0) , z3.If(r26s8, 0, 0) , z3.If(r26s9, 0, 0) , z3.If(r26s10, 0, 0) , z3.If(r26s11, 90, 0) , z3.If(r26s12, 0, 0) , z3.If(r26s13, 0, 0) , z3.If(r26s14, 0, 0) , z3.If(r26s15, 0, 0) , z3.If(r26s16, 0, 0) , z3.If(r26s17, 0, 0) , z3.If(r26s18, 0, 0) , z3.If(r26s20, 90, 0) , z3.If(r26s21, 90, 0) , z3.If(r26s22, 0, 0) , z3.If(r26s23, 0, 0) , z3.If(r26s24, 0, 0) , z3.If(r26s25, 0, 0) , z3.If(r26s26, 0, 0) , z3.If(r26s27, 0, 0) , z3.If(r26s28, 0, 0) , z3.If(r26s29, 10, 0) , z3.If(r26s30, 0, 0) , z3.If(r26s31, 0, 0) , z3.If(r26s32, 0, 0) , z3.If(r26s33, 0, 0) , z3.If(r26s34, 0, 0) , z3.If(r26s35, 0, 0) , z3.If(r26s36, 0, 0) , z3.If(r26s37, 0, 0) , z3.If(r26s38, 0, 0) , z3.If(r26s39, 0, 0) , z3.If(r26s40, 0, 0) , z3.If(r26s41, 0, 0) , z3.If(r26s42, 0, 0) , z3.If(r26s43, 90, 0) , z3.If(r26s44, 0, 0) , z3.If(r26s45, 0, 0) , z3.If(r26s46, 0, 0) , z3.If(r26s47, 90, 0) , z3.If(r26s48, 0, 0) , z3.If(r26s49, 90, 0) , z3.If(r26s50, 0, 0) , z3.If(r26s51, 90, 0) , z3.If(r26s52, 0, 0) , z3.If(r26s53, 0, 0) , z3.If(r27s0, 0, 0) , z3.If(r27s1, 90, 0) , z3.If(r27s2, 0, 0) , z3.If(r27s3, 0, 0) , z3.If(r27s4, 0, 0) , z3.If(r27s5, 0, 0) , z3.If(r27s6, 10, 0) , z3.If(r27s7, 0, 0) , z3.If(r27s8, 0, 0) , z3.If(r27s9, 90, 0) , z3.If(r27s10, 0, 0) , z3.If(r27s11, 0, 0) , z3.If(r27s12, 0, 0) , z3.If(r27s13, 0, 0) , z3.If(r27s14, 90, 0) , z3.If(r27s15, 0, 0) , z3.If(r27s16, 0, 0) , z3.If(r27s17, 0, 0) , z3.If(r27s18, 0, 0) , z3.If(r27s19, 0, 0) , z3.If(r27s20, 0, 0) , z3.If(r27s21, 0, 0) , z3.If(r27s22, 0, 0) , z3.If(r27s23, 0, 0) , z3.If(r27s24, 10, 0) , z3.If(r27s25, 0, 0) , z3.If(r27s26, 0, 0) , z3.If(r27s27, 0, 0) , z3.If(r27s28, 0, 0) , z3.If(r27s29, 0, 0) , z3.If(r27s30, 0, 0) , z3.If(r27s31, 0, 0) , z3.If(r27s32, 0, 0) , z3.If(r27s33, 0, 0) , z3.If(r27s34, 0, 0) , z3.If(r27s35, 0, 0) , z3.If(r27s36, 0, 0) , z3.If(r27s37, 90, 0) , z3.If(r27s38, 0, 0) , z3.If(r27s39, 10, 0) , z3.If(r27s40, 0, 0) , z3.If(r27s41, 0, 0) , z3.If(r27s42, 90, 0) , z3.If(r27s43, 10, 0) , z3.If(r27s44, 0, 0) , z3.If(r27s45, 0, 0) , z3.If(r27s46, 0, 0) , z3.If(r27s47, 0, 0) , z3.If(r27s48, 0, 0) , z3.If(r27s49, 0, 0) , z3.If(r27s50, 90, 0) , z3.If(r27s51, 0, 0) , z3.If(r27s52, 0, 0) , z3.If(r27s53, 0, 0) , z3.If(r28s0, 0, 0) , z3.If(r28s1, 10, 0) , z3.If(r28s2, 0, 0) , z3.If(r28s3, 0, 0) , z3.If(r28s4, 0, 0) , z3.If(r28s5, 0, 0) , z3.If(r28s6, 10, 0) , z3.If(r28s7, 0, 0) , z3.If(r28s8, 0, 0) , z3.If(r28s9, 10, 0) , z3.If(r28s10, 0, 0) , z3.If(r28s11, 0, 0) , z3.If(r28s12, 10, 0) , z3.If(r28s13, 90, 0) , z3.If(r28s14, 0, 0) , z3.If(r28s15, 0, 0) , z3.If(r28s16, 0, 0) , z3.If(r28s17, 10, 0) , z3.If(r28s18, 90, 0) , z3.If(r28s19, 0, 0) , z3.If(r28s20, 0, 0) , z3.If(r28s21, 0, 0) , z3.If(r28s23, 0, 0) , z3.If(r28s24, 90, 0) , z3.If(r28s25, 90, 0) , z3.If(r28s26, 0, 0) , z3.If(r28s29, 0, 0) , z3.If(r28s30, 10, 0) , z3.If(r28s32, 0, 0) , z3.If(r28s33, 0, 0) , z3.If(r28s35, 0, 0) , z3.If(r28s36, 0, 0) , z3.If(r28s37, 0, 0) , z3.If(r28s38, 0, 0) , z3.If(r28s39, 0, 0) , z3.If(r28s40, 90, 0) , z3.If(r28s41, 0, 0) , z3.If(r28s42, 10, 0) , z3.If(r28s43, 0, 0) , z3.If(r28s44, 0, 0) , z3.If(r28s45, 0, 0) , z3.If(r28s46, 0, 0) , z3.If(r28s47, 0, 0) , z3.If(r28s48, 0, 0) , z3.If(r28s49, 0, 0) , z3.If(r28s50, 0, 0) , z3.If(r28s51, 0, 0) , z3.If(r28s52, 0, 0) , z3.If(r28s53, 0, 0) , z3.If(r29s0, 0, 0) , z3.If(r29s1, 0, 0) , z3.If(r29s2, 0, 0) , z3.If(r29s3, 0, 0) , z3.If(r29s4, 0, 0) , z3.If(r29s5, 0, 0) , z3.If(r29s6, 90, 0) , z3.If(r29s7, 0, 0) , z3.If(r29s8, 0, 0) , z3.If(r29s9, 0, 0) , z3.If(r29s10, 0, 0) , z3.If(r29s11, 0, 0) , z3.If(r29s12, 90, 0) , z3.If(r29s13, 0, 0) , z3.If(r29s14, 0, 0) , z3.If(r29s15, 0, 0) , z3.If(r29s16, 90, 0) , z3.If(r29s17, 90, 0) , z3.If(r29s18, 0, 0) , z3.If(r29s19, 0, 0) , z3.If(r29s20, 0, 0) , z3.If(r29s21, 0, 0) , z3.If(r29s23, 0, 0) , z3.If(r29s24, 90, 0) , z3.If(r29s25, 90, 0) , z3.If(r29s26, 0, 0) , z3.If(r29s29, 0, 0) , z3.If(r29s30, 90, 0) , z3.If(r29s32, 90, 0) , z3.If(r29s33, 0, 0) , z3.If(r29s35, 0, 0) , z3.If(r29s36, 90, 0) , z3.If(r29s37, 0, 0) , z3.If(r29s38, 0, 0) , z3.If(r29s39, 0, 0) , z3.If(r29s40, 90, 0) , z3.If(r29s41, 0, 0) , z3.If(r29s42, 90, 0) , z3.If(r29s43, 0, 0) , z3.If(r29s44, 0, 0) , z3.If(r29s45, 0, 0) , z3.If(r29s46, 0, 0) , z3.If(r29s47, 0, 0) , z3.If(r29s48, 0, 0) , z3.If(r29s49, 0, 0) , z3.If(r29s50, 0, 0) , z3.If(r29s51, 0, 0) , z3.If(r29s52, 0, 0) , z3.If(r29s53, 0, 0) , z3.If(r30s0, 0, 0) , z3.If(r30s1, 0, 0) , z3.If(r30s2, 0, 0) , z3.If(r30s3, 90, 0) , z3.If(r30s4, 0, 0) , z3.If(r30s5, 10, 0) , z3.If(r30s6, 90, 0) , z3.If(r30s7, 0, 0) , z3.If(r30s8, 0, 0) , z3.If(r30s9, 0, 0) , z3.If(r30s10, 0, 0) , z3.If(r30s11, 10, 0) , z3.If(r30s12, 0, 0) , z3.If(r30s13, 90, 0) , z3.If(r30s14, 0, 0) , z3.If(r30s15, 90, 0) , z3.If(r30s16, 10, 0) , z3.If(r30s17, 0, 0) , z3.If(r30s18, 0, 0) , z3.If(r30s19, 0, 0) , z3.If(r30s20, 0, 0) , z3.If(r30s21, 10, 0) , z3.If(r30s22, 0, 0) , z3.If(r30s23, 0, 0) , z3.If(r30s24, 0, 0) , z3.If(r30s25, 90, 0) , z3.If(r30s26, 0, 0) , z3.If(r30s27, 0, 0) , z3.If(r30s28, 0, 0) , z3.If(r30s29, 10, 0) , z3.If(r30s30, 0, 0) , z3.If(r30s31, 0, 0) , z3.If(r30s32, 10, 0) , z3.If(r30s33, 0, 0) , z3.If(r30s34, 0, 0) , z3.If(r30s35, 10, 0) , z3.If(r30s37, 0, 0) , z3.If(r30s38, 0, 0) , z3.If(r30s39, 0, 0) , z3.If(r30s40, 0, 0) , z3.If(r30s41, 0, 0) , z3.If(r30s42, 0, 0) , z3.If(r30s43, 0, 0) , z3.If(r30s44, 0, 0) , z3.If(r30s45, 10, 0) , z3.If(r30s46, 10, 0) , z3.If(r30s47, 0, 0) , z3.If(r30s48, 0, 0) , z3.If(r30s49, 90, 0) , z3.If(r30s50, 0, 0) , z3.If(r30s51, 0, 0) , z3.If(r30s52, 0, 0) , z3.If(r30s53, 0, 0)]
# 80/20/0
# scores = []
# 70/30/0
# scores = [z3.If(r0s0, 0, 0) , z3.If(r0s1, 0, 0) , z3.If(r0s2, 0, 0) , z3.If(r0s4, 0, 0) , z3.If(r0s5, 0, 0) , z3.If(r0s6, 70, 0) , z3.If(r0s7, 0, 0) , z3.If(r0s8, 0, 0) , z3.If(r0s9, 30, 0) , z3.If(r0s10, 0, 0) , z3.If(r0s11, 0, 0) , z3.If(r0s12, 0, 0) , z3.If(r0s13, 70, 0) , z3.If(r0s14, 0, 0) , z3.If(r0s15, 0, 0) , z3.If(r0s16, 30, 0) , z3.If(r0s17, 30, 0) , z3.If(r0s18, 30, 0) , z3.If(r0s19, 0, 0) , z3.If(r0s20, 0, 0) , z3.If(r0s21, 0, 0) , z3.If(r0s22, 70, 0) , z3.If(r0s23, 0, 0) , z3.If(r0s24, 70, 0) , z3.If(r0s25, 0, 0) , z3.If(r0s26, 0, 0) , z3.If(r0s27, 70, 0) , z3.If(r0s28, 0, 0) , z3.If(r0s29, 0, 0) , z3.If(r0s30, 0, 0) , z3.If(r0s31, 0, 0) , z3.If(r0s32, 30, 0) , z3.If(r0s33, 0, 0) , z3.If(r0s34, 0, 0) , z3.If(r0s35, 0, 0) , z3.If(r0s36, 30, 0) , z3.If(r0s37, 0, 0) , z3.If(r0s38, 0, 0) , z3.If(r0s39, 0, 0) , z3.If(r0s40, 0, 0) , z3.If(r0s41, 0, 0) , z3.If(r0s42, 30, 0) , z3.If(r0s43, 30, 0) , z3.If(r0s44, 0, 0) , z3.If(r0s45, 0, 0) , z3.If(r0s46, 0, 0) , z3.If(r0s47, 30, 0) , z3.If(r0s48, 0, 0) , z3.If(r0s49, 0, 0) , z3.If(r0s51, 30, 0) , z3.If(r0s52, 0, 0) , z3.If(r0s53, 0, 0) , z3.If(r1s0, 0, 0) , z3.If(r1s1, 0, 0) , z3.If(r1s2, 0, 0) , z3.If(r1s3, 0, 0) , z3.If(r1s4, 0, 0) , z3.If(r1s5, 0, 0) , z3.If(r1s6, 70, 0) , z3.If(r1s7, 30, 0) , z3.If(r1s8, 0, 0) , z3.If(r1s9, 0, 0) , z3.If(r1s10, 0, 0) , z3.If(r1s11, 0, 0) , z3.If(r1s12, 70, 0) , z3.If(r1s13, 0, 0) , z3.If(r1s14, 30, 0) , z3.If(r1s15, 0, 0) , z3.If(r1s16, 0, 0) , z3.If(r1s17, 70, 0) , z3.If(r1s19, 0, 0) , z3.If(r1s20, 0, 0) , z3.If(r1s21, 0, 0) , z3.If(r1s22, 30, 0) , z3.If(r1s23, 0, 0) , z3.If(r1s24, 70, 0) , z3.If(r1s25, 30, 0) , z3.If(r1s26, 0, 0) , z3.If(r1s27, 70, 0) , z3.If(r1s28, 0, 0) , z3.If(r1s29, 0, 0) , z3.If(r1s30, 30, 0) , z3.If(r1s31, 30, 0) , z3.If(r1s32, 70, 0) , z3.If(r1s33, 0, 0) , z3.If(r1s34, 0, 0) , z3.If(r1s35, 0, 0) , z3.If(r1s36, 30, 0) , z3.If(r1s37, 0, 0) , z3.If(r1s38, 0, 0) , z3.If(r1s39, 0, 0) , z3.If(r1s41, 0, 0) , z3.If(r1s42, 70, 0) , z3.If(r1s43, 30, 0) , z3.If(r1s44, 0, 0) , z3.If(r1s45, 0, 0) , z3.If(r1s46, 0, 0) , z3.If(r1s47, 70, 0) , z3.If(r1s48, 0, 0) , z3.If(r1s49, 30, 0) , z3.If(r1s50, 0, 0) , z3.If(r1s51, 0, 0) , z3.If(r1s52, 0, 0) , z3.If(r1s53, 0, 0) , z3.If(r2s0, 0, 0) , z3.If(r2s1, 30, 0) , z3.If(r2s2, 0, 0) , z3.If(r2s3, 70, 0) , z3.If(r2s4, 0, 0) , z3.If(r2s5, 70, 0) , z3.If(r2s6, 0, 0) , z3.If(r2s7, 70, 0) , z3.If(r2s8, 0, 0) , z3.If(r2s9, 0, 0) , z3.If(r2s10, 0, 0) , z3.If(r2s11, 0, 0) , z3.If(r2s12, 30, 0) , z3.If(r2s13, 0, 0) , z3.If(r2s14, 30, 0) , z3.If(r2s15, 0, 0) , z3.If(r2s16, 0, 0) , z3.If(r2s17, 30, 0) , z3.If(r2s18, 0, 0) , z3.If(r2s19, 0, 0) , z3.If(r2s20, 0, 0) , z3.If(r2s21, 0, 0) , z3.If(r2s22, 0, 0) , z3.If(r2s23, 0, 0) , z3.If(r2s24, 70, 0) , z3.If(r2s25, 0, 0) , z3.If(r2s26, 0, 0) , z3.If(r2s27, 70, 0) , z3.If(r2s28, 0, 0) , z3.If(r2s29, 0, 0) , z3.If(r2s30, 0, 0) , z3.If(r2s31, 0, 0) , z3.If(r2s32, 0, 0) , z3.If(r2s33, 0, 0) , z3.If(r2s34, 0, 0) , z3.If(r2s35, 0, 0) , z3.If(r2s36, 0, 0) , z3.If(r2s37, 0, 0) , z3.If(r2s38, 0, 0) , z3.If(r2s39, 0, 0) , z3.If(r2s40, 70, 0) , z3.If(r2s41, 0, 0) , z3.If(r2s42, 70, 0) , z3.If(r2s43, 30, 0) , z3.If(r2s44, 0, 0) , z3.If(r2s45, 0, 0) , z3.If(r2s46, 0, 0) , z3.If(r2s47, 0, 0) , z3.If(r2s48, 0, 0) , z3.If(r2s49, 0, 0) , z3.If(r2s50, 0, 0) , z3.If(r2s51, 0, 0) , z3.If(r2s52, 0, 0) , z3.If(r2s53, 0, 0) , z3.If(r3s0, 70, 0) , z3.If(r3s1, 0, 0) , z3.If(r3s2, 0, 0) , z3.If(r3s3, 0, 0) , z3.If(r3s4, 0, 0) , z3.If(r3s5, 0, 0) , z3.If(r3s6, 70, 0) , z3.If(r3s7, 0, 0) , z3.If(r3s8, 0, 0) , z3.If(r3s9, 0, 0) , z3.If(r3s10, 0, 0) , z3.If(r3s11, 0, 0) , z3.If(r3s12, 0, 0) , z3.If(r3s13, 70, 0) , z3.If(r3s14, 0, 0) , z3.If(r3s15, 0, 0) , z3.If(r3s16, 0, 0) , z3.If(r3s17, 0, 0) , z3.If(r3s18, 0, 0) , z3.If(r3s19, 0, 0) , z3.If(r3s20, 0, 0) , z3.If(r3s21, 0, 0) , z3.If(r3s22, 0, 0) , z3.If(r3s23, 0, 0) , z3.If(r3s24, 0, 0) , z3.If(r3s25, 0, 0) , z3.If(r3s26, 0, 0) , z3.If(r3s27, 0, 0) , z3.If(r3s28, 0, 0) , z3.If(r3s29, 0, 0) , z3.If(r3s30, 0, 0) , z3.If(r3s31, 0, 0) , z3.If(r3s32, 0, 0) , z3.If(r3s33, 0, 0) , z3.If(r3s34, 0, 0) , z3.If(r3s35, 0, 0) , z3.If(r3s36, 0, 0) , z3.If(r3s37, 70, 0) , z3.If(r3s38, 0, 0) , z3.If(r3s39, 0, 0) , z3.If(r3s40, 70, 0) , z3.If(r3s41, 0, 0) , z3.If(r3s42, 0, 0) , z3.If(r3s43, 0, 0) , z3.If(r3s44, 0, 0) , z3.If(r3s45, 0, 0) , z3.If(r3s46, 0, 0) , z3.If(r3s47, 0, 0) , z3.If(r3s48, 0, 0) , z3.If(r3s49, 0, 0) , z3.If(r3s50, 0, 0) , z3.If(r3s51, 70, 0) , z3.If(r3s52, 0, 0) , z3.If(r3s53, 0, 0) , z3.If(r4s0, 0, 0) , z3.If(r4s1, 0, 0) , z3.If(r4s2, 0, 0) , z3.If(r4s3, 0, 0) , z3.If(r4s4, 0, 0) , z3.If(r4s5, 0, 0) , z3.If(r4s6, 30, 0) , z3.If(r4s7, 0, 0) , z3.If(r4s8, 0, 0) , z3.If(r4s10, 0, 0) , z3.If(r4s11, 0, 0) , z3.If(r4s12, 70, 0) , z3.If(r4s13, 70, 0) , z3.If(r4s14, 0, 0) , z3.If(r4s15, 0, 0) , z3.If(r4s16, 0, 0) , z3.If(r4s17, 70, 0) , z3.If(r4s19, 0, 0) , z3.If(r4s20, 0, 0) , z3.If(r4s21, 0, 0) , z3.If(r4s22, 0, 0) , z3.If(r4s23, 0, 0) , z3.If(r4s24, 70, 0) , z3.If(r4s26, 0, 0) , z3.If(r4s27, 70, 0) , z3.If(r4s28, 0, 0) , z3.If(r4s29, 0, 0) , z3.If(r4s30, 0, 0) , z3.If(r4s31, 0, 0) , z3.If(r4s32, 0, 0) , z3.If(r4s33, 0, 0) , z3.If(r4s34, 0, 0) , z3.If(r4s35, 0, 0) , z3.If(r4s36, 30, 0) , z3.If(r4s37, 0, 0) , z3.If(r4s38, 0, 0) , z3.If(r4s39, 0, 0) , z3.If(r4s40, 30, 0) , z3.If(r4s41, 0, 0) , z3.If(r4s42, 70, 0) , z3.If(r4s43, 0, 0) , z3.If(r4s44, 0, 0) , z3.If(r4s45, 30, 0) , z3.If(r4s46, 30, 0) , z3.If(r4s47, 0, 0) , z3.If(r4s48, 0, 0) , z3.If(r4s49, 0, 0) , z3.If(r4s50, 0, 0) , z3.If(r4s51, 0, 0) , z3.If(r4s52, 0, 0) , z3.If(r4s53, 0, 0) , z3.If(r5s0, 0, 0) , z3.If(r5s1, 0, 0) , z3.If(r5s2, 0, 0) , z3.If(r5s3, 0, 0) , z3.If(r5s4, 0, 0) , z3.If(r5s5, 0, 0) , z3.If(r5s6, 70, 0) , z3.If(r5s7, 0, 0) , z3.If(r5s8, 0, 0) , z3.If(r5s9, 0, 0) , z3.If(r5s10, 0, 0) , z3.If(r5s11, 0, 0) , z3.If(r5s12, 0, 0) , z3.If(r5s13, 70, 0) , z3.If(r5s14, 0, 0) , z3.If(r5s15, 0, 0) , z3.If(r5s16, 0, 0) , z3.If(r5s17, 30, 0) , z3.If(r5s18, 0, 0) , z3.If(r5s19, 0, 0) , z3.If(r5s20, 0, 0) , z3.If(r5s21, 0, 0) , z3.If(r5s22, 0, 0) , z3.If(r5s23, 0, 0) , z3.If(r5s24, 30, 0) , z3.If(r5s25, 30, 0) , z3.If(r5s26, 0, 0) , z3.If(r5s27, 30, 0) , z3.If(r5s28, 0, 0) , z3.If(r5s29, 0, 0) , z3.If(r5s30, 30, 0) , z3.If(r5s31, 0, 0) , z3.If(r5s32, 0, 0) , z3.If(r5s33, 0, 0) , z3.If(r5s34, 0, 0) , z3.If(r5s35, 0, 0) , z3.If(r5s36, 30, 0) , z3.If(r5s37, 0, 0) , z3.If(r5s38, 0, 0) , z3.If(r5s39, 0, 0) , z3.If(r5s40, 30, 0) , z3.If(r5s41, 0, 0) , z3.If(r5s42, 70, 0) , z3.If(r5s43, 0, 0) , z3.If(r5s44, 0, 0) , z3.If(r5s45, 0, 0) , z3.If(r5s46, 0, 0) , z3.If(r5s47, 30, 0) , z3.If(r5s48, 0, 0) , z3.If(r5s49, 0, 0) , z3.If(r5s50, 0, 0) , z3.If(r5s51, 0, 0) , z3.If(r5s52, 0, 0) , z3.If(r5s53, 0, 0) , z3.If(r6s0, 0, 0) , z3.If(r6s1, 0, 0) , z3.If(r6s2, 0, 0) , z3.If(r6s3, 0, 0) , z3.If(r6s4, 0, 0) , z3.If(r6s5, 0, 0) , z3.If(r6s6, 30, 0) , z3.If(r6s7, 0, 0) , z3.If(r6s8, 0, 0) , z3.If(r6s9, 0, 0) , z3.If(r6s10, 0, 0) , z3.If(r6s11, 0, 0) , z3.If(r6s12, 30, 0) , z3.If(r6s13, 0, 0) , z3.If(r6s14, 0, 0) , z3.If(r6s15, 0, 0) , z3.If(r6s16, 0, 0) , z3.If(r6s17, 0, 0) , z3.If(r6s18, 0, 0) , z3.If(r6s19, 0, 0) , z3.If(r6s20, 0, 0) , z3.If(r6s21, 0, 0) , z3.If(r6s22, 0, 0) , z3.If(r6s23, 70, 0) , z3.If(r6s24, 0, 0) , z3.If(r6s25, 0, 0) , z3.If(r6s26, 0, 0) , z3.If(r6s27, 0, 0) , z3.If(r6s28, 0, 0) , z3.If(r6s29, 0, 0) , z3.If(r6s30, 0, 0) , z3.If(r6s31, 0, 0) , z3.If(r6s32, 0, 0) , z3.If(r6s33, 0, 0) , z3.If(r6s34, 0, 0) , z3.If(r6s35, 0, 0) , z3.If(r6s36, 70, 0) , z3.If(r6s37, 0, 0) , z3.If(r6s38, 0, 0) , z3.If(r6s39, 0, 0) , z3.If(r6s40, 30, 0) , z3.If(r6s41, 0, 0) , z3.If(r6s42, 0, 0) , z3.If(r6s43, 0, 0) , z3.If(r6s44, 0, 0) , z3.If(r6s45, 0, 0) , z3.If(r6s46, 0, 0) , z3.If(r6s47, 70, 0) , z3.If(r6s48, 0, 0) , z3.If(r6s49, 30, 0) , z3.If(r6s50, 70, 0) , z3.If(r6s51, 0, 0) , z3.If(r6s52, 0, 0) , z3.If(r6s53, 0, 0) , z3.If(r7s0, 30, 0) , z3.If(r7s1, 0, 0) , z3.If(r7s2, 30, 0) , z3.If(r7s3, 0, 0) , z3.If(r7s4, 0, 0) , z3.If(r7s5, 0, 0) , z3.If(r7s6, 0, 0) , z3.If(r7s7, 0, 0) , z3.If(r7s8, 0, 0) , z3.If(r7s9, 0, 0) , z3.If(r7s10, 0, 0) , z3.If(r7s11, 0, 0) , z3.If(r7s12, 70, 0) , z3.If(r7s13, 30, 0) , z3.If(r7s14, 0, 0) , z3.If(r7s15, 0, 0) , z3.If(r7s16, 0, 0) , z3.If(r7s17, 70, 0) , z3.If(r7s19, 0, 0) , z3.If(r7s20, 0, 0) , z3.If(r7s21, 0, 0) , z3.If(r7s22, 0, 0) , z3.If(r7s23, 0, 0) , z3.If(r7s25, 70, 0) , z3.If(r7s26, 0, 0) , z3.If(r7s27, 30, 0) , z3.If(r7s28, 0, 0) , z3.If(r7s29, 0, 0) , z3.If(r7s30, 30, 0) , z3.If(r7s31, 70, 0) , z3.If(r7s32, 0, 0) , z3.If(r7s33, 0, 0) , z3.If(r7s34, 0, 0) , z3.If(r7s35, 0, 0) , z3.If(r7s36, 0, 0) , z3.If(r7s37, 0, 0) , z3.If(r7s38, 0, 0) , z3.If(r7s39, 0, 0) , z3.If(r7s40, 0, 0) , z3.If(r7s41, 0, 0) , z3.If(r7s42, 70, 0) , z3.If(r7s43, 0, 0) , z3.If(r7s44, 0, 0) , z3.If(r7s45, 0, 0) , z3.If(r7s46, 0, 0) , z3.If(r7s47, 70, 0) , z3.If(r7s48, 0, 0) , z3.If(r7s49, 0, 0) , z3.If(r7s50, 0, 0) , z3.If(r7s51, 0, 0) , z3.If(r7s52, 70, 0) , z3.If(r7s53, 0, 0) , z3.If(r8s0, 0, 0) , z3.If(r8s2, 0, 0) , z3.If(r8s3, 70, 0) , z3.If(r8s4, 70, 0) , z3.If(r8s5, 0, 0) , z3.If(r8s6, 0, 0) , z3.If(r8s7, 70, 0) , z3.If(r8s8, 0, 0) , z3.If(r8s9, 0, 0) , z3.If(r8s10, 0, 0) , z3.If(r8s11, 0, 0) , z3.If(r8s12, 0, 0) , z3.If(r8s13, 0, 0) , z3.If(r8s14, 0, 0) , z3.If(r8s15, 0, 0) , z3.If(r8s16, 70, 0) , z3.If(r8s17, 0, 0) , z3.If(r8s18, 0, 0) , z3.If(r8s19, 70, 0) , z3.If(r8s20, 0, 0) , z3.If(r8s21, 0, 0) , z3.If(r8s22, 0, 0) , z3.If(r8s23, 0, 0) , z3.If(r8s24, 0, 0) , z3.If(r8s25, 0, 0) , z3.If(r8s26, 0, 0) , z3.If(r8s27, 0, 0) , z3.If(r8s28, 0, 0) , z3.If(r8s29, 0, 0) , z3.If(r8s30, 0, 0) , z3.If(r8s31, 0, 0) , z3.If(r8s32, 0, 0) , z3.If(r8s33, 0, 0) , z3.If(r8s34, 0, 0) , z3.If(r8s35, 0, 0) , z3.If(r8s36, 0, 0) , z3.If(r8s37, 0, 0) , z3.If(r8s38, 70, 0) , z3.If(r8s39, 0, 0) , z3.If(r8s40, 0, 0) , z3.If(r8s41, 0, 0) , z3.If(r8s42, 0, 0) , z3.If(r8s43, 0, 0) , z3.If(r8s44, 0, 0) , z3.If(r8s45, 0, 0) , z3.If(r8s46, 0, 0) , z3.If(r8s47, 0, 0) , z3.If(r8s48, 0, 0) , z3.If(r8s49, 0, 0) , z3.If(r8s50, 0, 0) , z3.If(r8s51, 0, 0) , z3.If(r8s52, 0, 0) , z3.If(r8s53, 0, 0) , z3.If(r9s0, 0, 0) , z3.If(r9s1, 70, 0) , z3.If(r9s2, 0, 0) , z3.If(r9s3, 0, 0) , z3.If(r9s4, 0, 0) , z3.If(r9s5, 30, 0) , z3.If(r9s6, 70, 0) , z3.If(r9s7, 0, 0) , z3.If(r9s8, 0, 0) , z3.If(r9s9, 0, 0) , z3.If(r9s10, 0, 0) , z3.If(r9s11, 0, 0) , z3.If(r9s12, 0, 0) , z3.If(r9s13, 70, 0) , z3.If(r9s14, 0, 0) , z3.If(r9s15, 0, 0) , z3.If(r9s16, 0, 0) , z3.If(r9s17, 30, 0) , z3.If(r9s18, 0, 0) , z3.If(r9s19, 0, 0) , z3.If(r9s20, 0, 0) , z3.If(r9s21, 30, 0) , z3.If(r9s22, 0, 0) , z3.If(r9s23, 0, 0) , z3.If(r9s24, 0, 0) , z3.If(r9s25, 30, 0) , z3.If(r9s26, 0, 0) , z3.If(r9s27, 30, 0) , z3.If(r9s28, 0, 0) , z3.If(r9s29, 0, 0) , z3.If(r9s30, 30, 0) , z3.If(r9s31, 0, 0) , z3.If(r9s32, 0, 0) , z3.If(r9s33, 0, 0) , z3.If(r9s34, 0, 0) , z3.If(r9s35, 0, 0) , z3.If(r9s36, 0, 0) , z3.If(r9s37, 0, 0) , z3.If(r9s38, 0, 0) , z3.If(r9s39, 0, 0) , z3.If(r9s40, 30, 0) , z3.If(r9s41, 0, 0) , z3.If(r9s42, 0, 0) , z3.If(r9s43, 0, 0) , z3.If(r9s44, 0, 0) , z3.If(r9s45, 0, 0) , z3.If(r9s46, 0, 0) , z3.If(r9s47, 0, 0) , z3.If(r9s48, 0, 0) , z3.If(r9s49, 0, 0) , z3.If(r9s50, 0, 0) , z3.If(r9s51, 0, 0) , z3.If(r9s52, 70, 0) , z3.If(r9s53, 0, 0) , z3.If(r10s0, 30, 0) , z3.If(r10s1, 0, 0) , z3.If(r10s2, 0, 0) , z3.If(r10s3, 0, 0) , z3.If(r10s4, 0, 0) , z3.If(r10s5, 0, 0) , z3.If(r10s6, 30, 0) , z3.If(r10s7, 30, 0) , z3.If(r10s8, 0, 0) , z3.If(r10s9, 30, 0) , z3.If(r10s10, 0, 0) , z3.If(r10s11, 0, 0) , z3.If(r10s12, 30, 0) , z3.If(r10s13, 30, 0) , z3.If(r10s14, 30, 0) , z3.If(r10s15, 0, 0) , z3.If(r10s16, 30, 0) , z3.If(r10s17, 0, 0) , z3.If(r10s18, 0, 0) , z3.If(r10s19, 0, 0) , z3.If(r10s20, 0, 0) , z3.If(r10s21, 0, 0) , z3.If(r10s22, 30, 0) , z3.If(r10s23, 0, 0) , z3.If(r10s24, 0, 0) , z3.If(r10s25, 0, 0) , z3.If(r10s26, 0, 0) , z3.If(r10s27, 30, 0) , z3.If(r10s28, 0, 0) , z3.If(r10s29, 0, 0) , z3.If(r10s30, 0, 0) , z3.If(r10s31, 0, 0) , z3.If(r10s32, 0, 0) , z3.If(r10s33, 0, 0) , z3.If(r10s34, 0, 0) , z3.If(r10s35, 0, 0) , z3.If(r10s36, 0, 0) , z3.If(r10s37, 0, 0) , z3.If(r10s38, 0, 0) , z3.If(r10s39, 0, 0) , z3.If(r10s40, 30, 0) , z3.If(r10s41, 0, 0) , z3.If(r10s42, 0, 0) , z3.If(r10s43, 0, 0) , z3.If(r10s44, 0, 0) , z3.If(r10s45, 0, 0) , z3.If(r10s46, 0, 0) , z3.If(r10s47, 0, 0) , z3.If(r10s48, 0, 0) , z3.If(r10s49, 0, 0) , z3.If(r10s50, 30, 0) , z3.If(r10s51, 0, 0) , z3.If(r10s52, 0, 0) , z3.If(r10s53, 0, 0) , z3.If(r11s0, 30, 0) , z3.If(r11s1, 0, 0) , z3.If(r11s2, 0, 0) , z3.If(r11s3, 30, 0) , z3.If(r11s4, 0, 0) , z3.If(r11s5, 0, 0) , z3.If(r11s6, 70, 0) , z3.If(r11s7, 0, 0) , z3.If(r11s8, 0, 0) , z3.If(r11s9, 30, 0) , z3.If(r11s10, 0, 0) , z3.If(r11s11, 0, 0) , z3.If(r11s12, 0, 0) , z3.If(r11s13, 30, 0) , z3.If(r11s14, 70, 0) , z3.If(r11s15, 0, 0) , z3.If(r11s16, 0, 0) , z3.If(r11s17, 0, 0) , z3.If(r11s18, 0, 0) , z3.If(r11s19, 0, 0) , z3.If(r11s20, 0, 0) , z3.If(r11s21, 0, 0) , z3.If(r11s22, 0, 0) , z3.If(r11s23, 0, 0) , z3.If(r11s24, 0, 0) , z3.If(r11s25, 0, 0) , z3.If(r11s26, 0, 0) , z3.If(r11s27, 70, 0) , z3.If(r11s28, 0, 0) , z3.If(r11s29, 0, 0) , z3.If(r11s30, 0, 0) , z3.If(r11s31, 0, 0) , z3.If(r11s32, 0, 0) , z3.If(r11s33, 0, 0) , z3.If(r11s34, 0, 0) , z3.If(r11s35, 0, 0) , z3.If(r11s36, 0, 0) , z3.If(r11s37, 0, 0) , z3.If(r11s38, 0, 0) , z3.If(r11s39, 0, 0) , z3.If(r11s40, 0, 0) , z3.If(r11s41, 70, 0) , z3.If(r11s42, 70, 0) , z3.If(r11s43, 0, 0) , z3.If(r11s44, 0, 0) , z3.If(r11s45, 0, 0) , z3.If(r11s46, 0, 0) , z3.If(r11s47, 0, 0) , z3.If(r11s48, 0, 0) , z3.If(r11s49, 0, 0) , z3.If(r11s50, 0, 0) , z3.If(r11s51, 0, 0) , z3.If(r11s52, 0, 0) , z3.If(r11s53, 0, 0) , z3.If(r12s0, 0, 0) , z3.If(r12s1, 30, 0) , z3.If(r12s2, 0, 0) , z3.If(r12s3, 30, 0) , z3.If(r12s4, 0, 0) , z3.If(r12s5, 0, 0) , z3.If(r12s6, 30, 0) , z3.If(r12s8, 0, 0) , z3.If(r12s9, 30, 0) , z3.If(r12s10, 0, 0) , z3.If(r12s11, 0, 0) , z3.If(r12s12, 30, 0) , z3.If(r12s13, 0, 0) , z3.If(r12s14, 0, 0) , z3.If(r12s15, 0, 0) , z3.If(r12s16, 30, 0) , z3.If(r12s17, 30, 0) , z3.If(r12s18, 0, 0) , z3.If(r12s19, 0, 0) , z3.If(r12s20, 0, 0) , z3.If(r12s21, 30, 0) , z3.If(r12s22, 0, 0) , z3.If(r12s23, 0, 0) , z3.If(r12s24, 0, 0) , z3.If(r12s25, 0, 0) , z3.If(r12s26, 0, 0) , z3.If(r12s27, 0, 0) , z3.If(r12s28, 0, 0) , z3.If(r12s30, 0, 0) , z3.If(r12s31, 0, 0) , z3.If(r12s32, 0, 0) , z3.If(r12s33, 0, 0) , z3.If(r12s34, 0, 0) , z3.If(r12s35, 0, 0) , z3.If(r12s36, 0, 0) , z3.If(r12s37, 0, 0) , z3.If(r12s38, 0, 0) , z3.If(r12s39, 0, 0) , z3.If(r12s40, 0, 0) , z3.If(r12s41, 0, 0) , z3.If(r12s42, 0, 0) , z3.If(r12s43, 0, 0) , z3.If(r12s44, 0, 0) , z3.If(r12s45, 0, 0) , z3.If(r12s46, 30, 0) , z3.If(r12s47, 0, 0) , z3.If(r12s48, 0, 0) , z3.If(r12s49, 0, 0) , z3.If(r12s50, 0, 0) , z3.If(r12s51, 0, 0) , z3.If(r12s52, 0, 0) , z3.If(r12s53, 0, 0) , z3.If(r13s0, 70, 0) , z3.If(r13s1, 70, 0) , z3.If(r13s2, 70, 0) , z3.If(r13s3, 0, 0) , z3.If(r13s4, 30, 0) , z3.If(r13s5, 0, 0) , z3.If(r13s6, 0, 0) , z3.If(r13s7, 0, 0) , z3.If(r13s8, 0, 0) , z3.If(r13s9, 70, 0) , z3.If(r13s10, 0, 0) , z3.If(r13s11, 0, 0) , z3.If(r13s12, 70, 0) , z3.If(r13s13, 0, 0) , z3.If(r13s14, 0, 0) , z3.If(r13s15, 0, 0) , z3.If(r13s16, 0, 0) , z3.If(r13s17, 30, 0) , z3.If(r13s18, 70, 0) , z3.If(r13s19, 0, 0) , z3.If(r13s20, 0, 0) , z3.If(r13s21, 0, 0) , z3.If(r13s22, 0, 0) , z3.If(r13s23, 0, 0) , z3.If(r13s24, 70, 0) , z3.If(r13s25, 70, 0) , z3.If(r13s26, 0, 0) , z3.If(r13s27, 70, 0) , z3.If(r13s28, 0, 0) , z3.If(r13s29, 0, 0) , z3.If(r13s30, 0, 0) , z3.If(r13s31, 0, 0) , z3.If(r13s32, 0, 0) , z3.If(r13s33, 0, 0) , z3.If(r13s34, 0, 0) , z3.If(r13s35, 0, 0) , z3.If(r13s36, 0, 0) , z3.If(r13s37, 0, 0) , z3.If(r13s38, 0, 0) , z3.If(r13s39, 70, 0) , z3.If(r13s40, 30, 0) , z3.If(r13s41, 0, 0) , z3.If(r13s42, 0, 0) , z3.If(r13s43, 0, 0) , z3.If(r13s44, 0, 0) , z3.If(r13s45, 0, 0) , z3.If(r13s46, 0, 0) , z3.If(r13s47, 0, 0) , z3.If(r13s48, 0, 0) , z3.If(r13s49, 0, 0) , z3.If(r13s50, 0, 0) , z3.If(r13s51, 0, 0) , z3.If(r13s52, 0, 0) , z3.If(r13s53, 0, 0) , z3.If(r14s0, 0, 0) , z3.If(r14s1, 30, 0) , z3.If(r14s2, 0, 0) , z3.If(r14s3, 0, 0) , z3.If(r14s4, 0, 0) , z3.If(r14s5, 0, 0) , z3.If(r14s6, 70, 0) , z3.If(r14s7, 30, 0) , z3.If(r14s8, 0, 0) , z3.If(r14s9, 30, 0) , z3.If(r14s10, 0, 0) , z3.If(r14s11, 0, 0) , z3.If(r14s12, 0, 0) , z3.If(r14s14, 70, 0) , z3.If(r14s15, 0, 0) , z3.If(r14s16, 0, 0) , z3.If(r14s17, 0, 0) , z3.If(r14s18, 0, 0) , z3.If(r14s19, 0, 0) , z3.If(r14s20, 0, 0) , z3.If(r14s21, 0, 0) , z3.If(r14s22, 0, 0) , z3.If(r14s23, 0, 0) , z3.If(r14s25, 0, 0) , z3.If(r14s26, 0, 0) , z3.If(r14s27, 0, 0) , z3.If(r14s28, 0, 0) , z3.If(r14s29, 0, 0) , z3.If(r14s30, 30, 0) , z3.If(r14s31, 0, 0) , z3.If(r14s32, 0, 0) , z3.If(r14s33, 0, 0) , z3.If(r14s34, 0, 0) , z3.If(r14s35, 0, 0) , z3.If(r14s36, 0, 0) , z3.If(r14s37, 0, 0) , z3.If(r14s38, 0, 0) , z3.If(r14s39, 0, 0) , z3.If(r14s40, 70, 0) , z3.If(r14s41, 0, 0) , z3.If(r14s42, 0, 0) , z3.If(r14s43, 0, 0) , z3.If(r14s44, 0, 0) , z3.If(r14s45, 0, 0) , z3.If(r14s46, 30, 0) , z3.If(r14s47, 0, 0) , z3.If(r14s48, 0, 0) , z3.If(r14s49, 0, 0) , z3.If(r14s50, 0, 0) , z3.If(r14s51, 0, 0) , z3.If(r14s52, 0, 0) , z3.If(r14s53, 0, 0) , z3.If(r15s0, 30, 0) , z3.If(r15s1, 0, 0) , z3.If(r15s2, 0, 0) , z3.If(r15s3, 0, 0) , z3.If(r15s4, 0, 0) , z3.If(r15s5, 0, 0) , z3.If(r15s6, 0, 0) , z3.If(r15s7, 0, 0) , z3.If(r15s8, 0, 0) , z3.If(r15s9, 0, 0) , z3.If(r15s10, 0, 0) , z3.If(r15s11, 0, 0) , z3.If(r15s12, 0, 0) , z3.If(r15s13, 70, 0) , z3.If(r15s14, 0, 0) , z3.If(r15s15, 0, 0) , z3.If(r15s16, 0, 0) , z3.If(r15s17, 0, 0) , z3.If(r15s18, 0, 0) , z3.If(r15s19, 0, 0) , z3.If(r15s20, 0, 0) , z3.If(r15s21, 0, 0) , z3.If(r15s22, 30, 0) , z3.If(r15s23, 0, 0) , z3.If(r15s24, 30, 0) , z3.If(r15s25, 30, 0) , z3.If(r15s26, 0, 0) , z3.If(r15s27, 70, 0) , z3.If(r15s28, 0, 0) , z3.If(r15s29, 0, 0) , z3.If(r15s30, 0, 0) , z3.If(r15s31, 0, 0) , z3.If(r15s32, 0, 0) , z3.If(r15s33, 0, 0) , z3.If(r15s34, 0, 0) , z3.If(r15s35, 0, 0) , z3.If(r15s36, 0, 0) , z3.If(r15s37, 0, 0) , z3.If(r15s38, 0, 0) , z3.If(r15s39, 0, 0) , z3.If(r15s40, 30, 0) , z3.If(r15s41, 0, 0) , z3.If(r15s42, 0, 0) , z3.If(r15s43, 0, 0) , z3.If(r15s44, 0, 0) , z3.If(r15s45, 0, 0) , z3.If(r15s46, 0, 0) , z3.If(r15s47, 0, 0) , z3.If(r15s48, 0, 0) , z3.If(r15s49, 30, 0) , z3.If(r15s50, 0, 0) , z3.If(r15s51, 0, 0) , z3.If(r15s52, 0, 0) , z3.If(r15s53, 0, 0) , z3.If(r16s0, 30, 0) , z3.If(r16s1, 30, 0) , z3.If(r16s2, 0, 0) , z3.If(r16s3, 30, 0) , z3.If(r16s4, 0, 0) , z3.If(r16s5, 30, 0) , z3.If(r16s6, 0, 0) , z3.If(r16s7, 30, 0) , z3.If(r16s8, 0, 0) , z3.If(r16s9, 0, 0) , z3.If(r16s10, 0, 0) , z3.If(r16s11, 70, 0) , z3.If(r16s12, 0, 0) , z3.If(r16s13, 0, 0) , z3.If(r16s14, 0, 0) , z3.If(r16s15, 0, 0) , z3.If(r16s16, 0, 0) , z3.If(r16s17, 0, 0) , z3.If(r16s18, 0, 0) , z3.If(r16s19, 30, 0) , z3.If(r16s20, 0, 0) , z3.If(r16s21, 0, 0) , z3.If(r16s22, 0, 0) , z3.If(r16s23, 0, 0) , z3.If(r16s24, 0, 0) , z3.If(r16s25, 0, 0) , z3.If(r16s26, 0, 0) , z3.If(r16s27, 30, 0) , z3.If(r16s28, 0, 0) , z3.If(r16s29, 0, 0) , z3.If(r16s30, 0, 0) , z3.If(r16s31, 0, 0) , z3.If(r16s32, 0, 0) , z3.If(r16s33, 0, 0) , z3.If(r16s34, 0, 0) , z3.If(r16s35, 0, 0) , z3.If(r16s36, 0, 0) , z3.If(r16s37, 30, 0) , z3.If(r16s38, 0, 0) , z3.If(r16s39, 0, 0) , z3.If(r16s40, 0, 0) , z3.If(r16s41, 0, 0) , z3.If(r16s42, 0, 0) , z3.If(r16s44, 0, 0) , z3.If(r16s45, 0, 0) , z3.If(r16s46, 70, 0) , z3.If(r16s47, 0, 0) , z3.If(r16s48, 0, 0) , z3.If(r16s50, 0, 0) , z3.If(r16s52, 0, 0) , z3.If(r16s53, 0, 0) , z3.If(r17s0, 0, 0) , z3.If(r17s1, 70, 0) , z3.If(r17s2, 0, 0) , z3.If(r17s3, 70, 0) , z3.If(r17s4, 0, 0) , z3.If(r17s5, 0, 0) , z3.If(r17s6, 0, 0) , z3.If(r17s7, 0, 0) , z3.If(r17s8, 0, 0) , z3.If(r17s9, 70, 0) , z3.If(r17s10, 70, 0) , z3.If(r17s11, 0, 0) , z3.If(r17s12, 0, 0) , z3.If(r17s13, 0, 0) , z3.If(r17s14, 0, 0) , z3.If(r17s15, 0, 0) , z3.If(r17s16, 0, 0) , z3.If(r17s17, 0, 0) , z3.If(r17s18, 0, 0) , z3.If(r17s19, 0, 0) , z3.If(r17s20, 0, 0) , z3.If(r17s22, 0, 0) , z3.If(r17s23, 0, 0) , z3.If(r17s24, 0, 0) , z3.If(r17s25, 0, 0) , z3.If(r17s27, 0, 0) , z3.If(r17s28, 0, 0) , z3.If(r17s29, 70, 0) , z3.If(r17s30, 0, 0) , z3.If(r17s31, 0, 0) , z3.If(r17s32, 0, 0) , z3.If(r17s33, 70, 0) , z3.If(r17s34, 0, 0) , z3.If(r17s35, 0, 0) , z3.If(r17s36, 0, 0) , z3.If(r17s37, 0, 0) , z3.If(r17s38, 0, 0) , z3.If(r17s39, 0, 0) , z3.If(r17s40, 0, 0) , z3.If(r17s41, 0, 0) , z3.If(r17s42, 0, 0) , z3.If(r17s43, 0, 0) , z3.If(r17s44, 70, 0) , z3.If(r17s46, 0, 0) , z3.If(r17s47, 0, 0) , z3.If(r17s49, 0, 0) , z3.If(r17s50, 0, 0) , z3.If(r17s51, 0, 0) , z3.If(r17s52, 0, 0) , z3.If(r18s0, 0, 0) , z3.If(r18s1, 0, 0) , z3.If(r18s2, 0, 0) , z3.If(r18s3, 0, 0) , z3.If(r18s4, 0, 0) , z3.If(r18s5, 70, 0) , z3.If(r18s6, 0, 0) , z3.If(r18s7, 0, 0) , z3.If(r18s8, 0, 0) , z3.If(r18s9, 0, 0) , z3.If(r18s10, 0, 0) , z3.If(r18s11, 0, 0) , z3.If(r18s12, 70, 0) , z3.If(r18s13, 0, 0) , z3.If(r18s14, 30, 0) , z3.If(r18s15, 0, 0) , z3.If(r18s16, 0, 0) , z3.If(r18s17, 30, 0) , z3.If(r18s18, 30, 0) , z3.If(r18s19, 0, 0) , z3.If(r18s20, 0, 0) , z3.If(r18s21, 0, 0) , z3.If(r18s22, 70, 0) , z3.If(r18s23, 0, 0) , z3.If(r18s24, 0, 0) , z3.If(r18s25, 0, 0) , z3.If(r18s26, 0, 0) , z3.If(r18s27, 0, 0) , z3.If(r18s28, 0, 0) , z3.If(r18s29, 0, 0) , z3.If(r18s30, 0, 0) , z3.If(r18s31, 0, 0) , z3.If(r18s32, 0, 0) , z3.If(r18s33, 0, 0) , z3.If(r18s34, 0, 0) , z3.If(r18s35, 0, 0) , z3.If(r18s36, 30, 0) , z3.If(r18s37, 0, 0) , z3.If(r18s38, 0, 0) , z3.If(r18s39, 0, 0) , z3.If(r18s40, 0, 0) , z3.If(r18s41, 0, 0) , z3.If(r18s42, 0, 0) , z3.If(r18s43, 0, 0) , z3.If(r18s44, 0, 0) , z3.If(r18s45, 0, 0) , z3.If(r18s46, 0, 0) , z3.If(r18s47, 0, 0) , z3.If(r18s48, 0, 0) , z3.If(r18s49, 0, 0) , z3.If(r18s50, 70, 0) , z3.If(r18s51, 0, 0) , z3.If(r18s52, 0, 0) , z3.If(r18s53, 0, 0) , z3.If(r19s0, 30, 0) , z3.If(r19s1, 0, 0) , z3.If(r19s2, 70, 0) , z3.If(r19s3, 0, 0) , z3.If(r19s4, 0, 0) , z3.If(r19s5, 0, 0) , z3.If(r19s6, 0, 0) , z3.If(r19s7, 0, 0) , z3.If(r19s8, 0, 0) , z3.If(r19s9, 0, 0) , z3.If(r19s10, 0, 0) , z3.If(r19s11, 0, 0) , z3.If(r19s12, 0, 0) , z3.If(r19s13, 30, 0) , z3.If(r19s14, 0, 0) , z3.If(r19s15, 0, 0) , z3.If(r19s16, 30, 0) , z3.If(r19s17, 0, 0) , z3.If(r19s18, 0, 0) , z3.If(r19s19, 0, 0) , z3.If(r19s20, 0, 0) , z3.If(r19s21, 0, 0) , z3.If(r19s22, 0, 0) , z3.If(r19s23, 0, 0) , z3.If(r19s24, 0, 0) , z3.If(r19s25, 0, 0) , z3.If(r19s26, 0, 0) , z3.If(r19s27, 0, 0) , z3.If(r19s28, 0, 0) , z3.If(r19s29, 0, 0) , z3.If(r19s30, 0, 0) , z3.If(r19s31, 0, 0) , z3.If(r19s32, 0, 0) , z3.If(r19s33, 0, 0) , z3.If(r19s34, 0, 0) , z3.If(r19s35, 0, 0) , z3.If(r19s36, 0, 0) , z3.If(r19s37, 0, 0) , z3.If(r19s38, 0, 0) , z3.If(r19s39, 0, 0) , z3.If(r19s40, 0, 0) , z3.If(r19s41, 0, 0) , z3.If(r19s42, 0, 0) , z3.If(r19s43, 0, 0) , z3.If(r19s44, 0, 0) , z3.If(r19s45, 0, 0) , z3.If(r19s46, 0, 0) , z3.If(r19s47, 0, 0) , z3.If(r19s48, 0, 0) , z3.If(r19s49, 0, 0) , z3.If(r19s50, 0, 0) , z3.If(r19s51, 70, 0) , z3.If(r19s52, 0, 0) , z3.If(r19s53, 0, 0) , z3.If(r20s0, 70, 0) , z3.If(r20s1, 0, 0) , z3.If(r20s2, 70, 0) , z3.If(r20s3, 0, 0) , z3.If(r20s4, 0, 0) , z3.If(r20s5, 0, 0) , z3.If(r20s6, 70, 0) , z3.If(r20s7, 0, 0) , z3.If(r20s8, 0, 0) , z3.If(r20s9, 70, 0) , z3.If(r20s10, 0, 0) , z3.If(r20s11, 0, 0) , z3.If(r20s13, 0, 0) , z3.If(r20s14, 0, 0) , z3.If(r20s15, 0, 0) , z3.If(r20s16, 0, 0) , z3.If(r20s17, 70, 0) , z3.If(r20s18, 70, 0) , z3.If(r20s19, 0, 0) , z3.If(r20s20, 0, 0) , z3.If(r20s21, 0, 0) , z3.If(r20s22, 70, 0) , z3.If(r20s23, 0, 0) , z3.If(r20s24, 70, 0) , z3.If(r20s25, 70, 0) , z3.If(r20s26, 0, 0) , z3.If(r20s27, 70, 0) , z3.If(r20s28, 0, 0) , z3.If(r20s29, 0, 0) , z3.If(r20s30, 70, 0) , z3.If(r20s31, 70, 0) , z3.If(r20s32, 0, 0) , z3.If(r20s33, 0, 0) , z3.If(r20s34, 0, 0) , z3.If(r20s35, 0, 0) , z3.If(r20s36, 0, 0) , z3.If(r20s37, 0, 0) , z3.If(r20s38, 0, 0) , z3.If(r20s39, 0, 0) , z3.If(r20s40, 0, 0) , z3.If(r20s41, 0, 0) , z3.If(r20s42, 0, 0) , z3.If(r20s43, 0, 0) , z3.If(r20s44, 0, 0) , z3.If(r20s45, 0, 0) , z3.If(r20s46, 0, 0) , z3.If(r20s47, 0, 0) , z3.If(r20s48, 0, 0) , z3.If(r20s49, 0, 0) , z3.If(r20s50, 0, 0) , z3.If(r20s51, 0, 0) , z3.If(r20s52, 0, 0) , z3.If(r20s53, 0, 0) , z3.If(r21s0, 70, 0) , z3.If(r21s1, 0, 0) , z3.If(r21s2, 0, 0) , z3.If(r21s3, 70, 0) , z3.If(r21s4, 0, 0) , z3.If(r21s5, 0, 0) , z3.If(r21s6, 30, 0) , z3.If(r21s7, 30, 0) , z3.If(r21s8, 0, 0) , z3.If(r21s9, 70, 0) , z3.If(r21s10, 0, 0) , z3.If(r21s11, 0, 0) , z3.If(r21s12, 0, 0) , z3.If(r21s13, 30, 0) , z3.If(r21s14, 0, 0) , z3.If(r21s15, 0, 0) , z3.If(r21s16, 30, 0) , z3.If(r21s17, 0, 0) , z3.If(r21s18, 30, 0) , z3.If(r21s19, 0, 0) , z3.If(r21s20, 0, 0) , z3.If(r21s21, 0, 0) , z3.If(r21s22, 70, 0) , z3.If(r21s23, 0, 0) , z3.If(r21s24, 0, 0) , z3.If(r21s25, 0, 0) , z3.If(r21s26, 0, 0) , z3.If(r21s27, 30, 0) , z3.If(r21s28, 70, 0) , z3.If(r21s29, 0, 0) , z3.If(r21s30, 0, 0) , z3.If(r21s31, 0, 0) , z3.If(r21s32, 0, 0) , z3.If(r21s33, 0, 0) , z3.If(r21s34, 70, 0) , z3.If(r21s35, 0, 0) , z3.If(r21s36, 0, 0) , z3.If(r21s37, 0, 0) , z3.If(r21s38, 0, 0) , z3.If(r21s39, 0, 0) , z3.If(r21s40, 70, 0) , z3.If(r21s41, 0, 0) , z3.If(r21s43, 0, 0) , z3.If(r21s44, 0, 0) , z3.If(r21s45, 0, 0) , z3.If(r21s46, 0, 0) , z3.If(r21s47, 0, 0) , z3.If(r21s48, 0, 0) , z3.If(r21s49, 0, 0) , z3.If(r21s50, 0, 0) , z3.If(r21s51, 0, 0) , z3.If(r21s52, 0, 0) , z3.If(r21s53, 0, 0) , z3.If(r22s0, 0, 0) , z3.If(r22s1, 0, 0) , z3.If(r22s2, 0, 0) , z3.If(r22s3, 70, 0) , z3.If(r22s4, 0, 0) , z3.If(r22s5, 0, 0) , z3.If(r22s6, 0, 0) , z3.If(r22s7, 30, 0) , z3.If(r22s8, 30, 0) , z3.If(r22s9, 0, 0) , z3.If(r22s10, 0, 0) , z3.If(r22s11, 0, 0) , z3.If(r22s12, 30, 0) , z3.If(r22s13, 0, 0) , z3.If(r22s14, 0, 0) , z3.If(r22s15, 0, 0) , z3.If(r22s16, 70, 0) , z3.If(r22s17, 30, 0) , z3.If(r22s18, 0, 0) , z3.If(r22s19, 0, 0) , z3.If(r22s20, 0, 0) , z3.If(r22s21, 0, 0) , z3.If(r22s22, 0, 0) , z3.If(r22s23, 0, 0) , z3.If(r22s24, 0, 0) , z3.If(r22s25, 0, 0) , z3.If(r22s26, 0, 0) , z3.If(r22s27, 0, 0) , z3.If(r22s28, 0, 0) , z3.If(r22s29, 0, 0) , z3.If(r22s30, 0, 0) , z3.If(r22s31, 0, 0) , z3.If(r22s32, 0, 0) , z3.If(r22s33, 0, 0) , z3.If(r22s34, 0, 0) , z3.If(r22s35, 0, 0) , z3.If(r22s36, 0, 0) , z3.If(r22s37, 0, 0) , z3.If(r22s38, 0, 0) , z3.If(r22s39, 0, 0) , z3.If(r22s40, 0, 0) , z3.If(r22s41, 0, 0) , z3.If(r22s42, 0, 0) , z3.If(r22s43, 0, 0) , z3.If(r22s44, 0, 0) , z3.If(r22s45, 0, 0) , z3.If(r22s46, 0, 0) , z3.If(r22s47, 0, 0) , z3.If(r22s48, 0, 0) , z3.If(r22s49, 0, 0) , z3.If(r22s50, 0, 0) , z3.If(r22s51, 0, 0) , z3.If(r22s52, 0, 0) , z3.If(r22s53, 0, 0) , z3.If(r23s0, 0, 0) , z3.If(r23s1, 70, 0) , z3.If(r23s2, 0, 0) , z3.If(r23s3, 30, 0) , z3.If(r23s4, 0, 0) , z3.If(r23s5, 0, 0) , z3.If(r23s6, 70, 0) , z3.If(r23s7, 0, 0) , z3.If(r23s8, 0, 0) , z3.If(r23s9, 0, 0) , z3.If(r23s10, 0, 0) , z3.If(r23s11, 0, 0) , z3.If(r23s12, 0, 0) , z3.If(r23s13, 0, 0) , z3.If(r23s14, 70, 0) , z3.If(r23s15, 0, 0) , z3.If(r23s16, 30, 0) , z3.If(r23s17, 70, 0) , z3.If(r23s19, 0, 0) , z3.If(r23s20, 0, 0) , z3.If(r23s21, 0, 0) , z3.If(r23s22, 0, 0) , z3.If(r23s23, 0, 0) , z3.If(r23s24, 0, 0) , z3.If(r23s25, 70, 0) , z3.If(r23s26, 0, 0) , z3.If(r23s27, 0, 0) , z3.If(r23s28, 0, 0) , z3.If(r23s29, 0, 0) , z3.If(r23s30, 70, 0) , z3.If(r23s31, 30, 0) , z3.If(r23s32, 0, 0) , z3.If(r23s33, 0, 0) , z3.If(r23s34, 0, 0) , z3.If(r23s35, 0, 0) , z3.If(r23s36, 0, 0) , z3.If(r23s37, 70, 0) , z3.If(r23s38, 0, 0) , z3.If(r23s39, 0, 0) , z3.If(r23s41, 0, 0) , z3.If(r23s42, 0, 0) , z3.If(r23s43, 0, 0) , z3.If(r23s44, 0, 0) , z3.If(r23s45, 0, 0) , z3.If(r23s46, 0, 0) , z3.If(r23s47, 0, 0) , z3.If(r23s48, 0, 0) , z3.If(r23s49, 0, 0) , z3.If(r23s50, 0, 0) , z3.If(r23s51, 0, 0) , z3.If(r23s52, 0, 0) , z3.If(r23s53, 0, 0) , z3.If(r24s0, 30, 0) , z3.If(r24s1, 0, 0) , z3.If(r24s2, 0, 0) , z3.If(r24s3, 0, 0) , z3.If(r24s4, 0, 0) , z3.If(r24s5, 0, 0) , z3.If(r24s6, 30, 0) , z3.If(r24s7, 0, 0) , z3.If(r24s8, 0, 0) , z3.If(r24s9, 0, 0) , z3.If(r24s10, 0, 0) , z3.If(r24s11, 0, 0) , z3.If(r24s12, 0, 0) , z3.If(r24s14, 0, 0) , z3.If(r24s15, 70, 0) , z3.If(r24s16, 30, 0) , z3.If(r24s17, 0, 0) , z3.If(r24s18, 0, 0) , z3.If(r24s19, 30, 0) , z3.If(r24s20, 70, 0) , z3.If(r24s21, 0, 0) , z3.If(r24s22, 70, 0) , z3.If(r24s23, 0, 0) , z3.If(r24s24, 0, 0) , z3.If(r24s25, 0, 0) , z3.If(r24s26, 0, 0) , z3.If(r24s27, 30, 0) , z3.If(r24s28, 0, 0) , z3.If(r24s29, 0, 0) , z3.If(r24s30, 0, 0) , z3.If(r24s31, 0, 0) , z3.If(r24s32, 0, 0) , z3.If(r24s33, 0, 0) , z3.If(r24s34, 0, 0) , z3.If(r24s35, 0, 0) , z3.If(r24s36, 0, 0) , z3.If(r24s37, 0, 0) , z3.If(r24s38, 0, 0) , z3.If(r24s39, 0, 0) , z3.If(r24s40, 0, 0) , z3.If(r24s42, 0, 0) , z3.If(r24s44, 0, 0) , z3.If(r24s45, 0, 0) , z3.If(r24s46, 30, 0) , z3.If(r24s47, 0, 0) , z3.If(r24s48, 0, 0) , z3.If(r24s50, 0, 0) , z3.If(r24s52, 0, 0) , z3.If(r24s53, 0, 0) , z3.If(r25s0, 0, 0) , z3.If(r25s1, 0, 0) , z3.If(r25s2, 30, 0) , z3.If(r25s3, 0, 0) , z3.If(r25s4, 0, 0) , z3.If(r25s5, 0, 0) , z3.If(r25s6, 70, 0) , z3.If(r25s7, 30, 0) , z3.If(r25s8, 0, 0) , z3.If(r25s9, 30, 0) , z3.If(r25s10, 0, 0) , z3.If(r25s11, 0, 0) , z3.If(r25s13, 0, 0) , z3.If(r25s14, 0, 0) , z3.If(r25s15, 0, 0) , z3.If(r25s16, 30, 0) , z3.If(r25s17, 70, 0) , z3.If(r25s19, 0, 0) , z3.If(r25s20, 0, 0) , z3.If(r25s21, 0, 0) , z3.If(r25s22, 0, 0) , z3.If(r25s23, 0, 0) , z3.If(r25s24, 0, 0) , z3.If(r25s25, 0, 0) , z3.If(r25s26, 0, 0) , z3.If(r25s27, 70, 0) , z3.If(r25s28, 0, 0) , z3.If(r25s29, 0, 0) , z3.If(r25s30, 30, 0) , z3.If(r25s31, 30, 0) , z3.If(r25s32, 30, 0) , z3.If(r25s33, 0, 0) , z3.If(r25s34, 0, 0) , z3.If(r25s35, 0, 0) , z3.If(r25s36, 30, 0) , z3.If(r25s37, 0, 0) , z3.If(r25s38, 0, 0) , z3.If(r25s39, 0, 0) , z3.If(r25s40, 30, 0) , z3.If(r25s41, 0, 0) , z3.If(r25s42, 30, 0) , z3.If(r25s43, 30, 0) , z3.If(r25s44, 0, 0) , z3.If(r25s45, 0, 0) , z3.If(r25s46, 0, 0) , z3.If(r25s47, 30, 0) , z3.If(r25s48, 0, 0) , z3.If(r25s49, 0, 0) , z3.If(r25s50, 0, 0) , z3.If(r25s51, 0, 0) , z3.If(r25s52, 0, 0) , z3.If(r25s53, 0, 0) , z3.If(r26s0, 70, 0) , z3.If(r26s1, 30, 0) , z3.If(r26s2, 0, 0) , z3.If(r26s3, 70, 0) , z3.If(r26s4, 70, 0) , z3.If(r26s5, 0, 0) , z3.If(r26s6, 0, 0) , z3.If(r26s7, 0, 0) , z3.If(r26s8, 0, 0) , z3.If(r26s9, 0, 0) , z3.If(r26s10, 0, 0) , z3.If(r26s11, 70, 0) , z3.If(r26s12, 0, 0) , z3.If(r26s13, 0, 0) , z3.If(r26s14, 0, 0) , z3.If(r26s15, 0, 0) , z3.If(r26s16, 0, 0) , z3.If(r26s17, 0, 0) , z3.If(r26s18, 0, 0) , z3.If(r26s20, 70, 0) , z3.If(r26s21, 70, 0) , z3.If(r26s22, 0, 0) , z3.If(r26s23, 0, 0) , z3.If(r26s24, 0, 0) , z3.If(r26s25, 0, 0) , z3.If(r26s26, 0, 0) , z3.If(r26s27, 0, 0) , z3.If(r26s28, 0, 0) , z3.If(r26s29, 30, 0) , z3.If(r26s30, 0, 0) , z3.If(r26s31, 0, 0) , z3.If(r26s32, 0, 0) , z3.If(r26s33, 0, 0) , z3.If(r26s34, 0, 0) , z3.If(r26s35, 0, 0) , z3.If(r26s36, 0, 0) , z3.If(r26s37, 0, 0) , z3.If(r26s38, 0, 0) , z3.If(r26s39, 0, 0) , z3.If(r26s40, 0, 0) , z3.If(r26s41, 0, 0) , z3.If(r26s42, 0, 0) , z3.If(r26s43, 70, 0) , z3.If(r26s44, 0, 0) , z3.If(r26s45, 0, 0) , z3.If(r26s46, 0, 0) , z3.If(r26s47, 70, 0) , z3.If(r26s48, 0, 0) , z3.If(r26s49, 70, 0) , z3.If(r26s50, 0, 0) , z3.If(r26s51, 70, 0) , z3.If(r26s52, 0, 0) , z3.If(r26s53, 0, 0) , z3.If(r27s0, 0, 0) , z3.If(r27s1, 70, 0) , z3.If(r27s2, 0, 0) , z3.If(r27s3, 0, 0) , z3.If(r27s4, 0, 0) , z3.If(r27s5, 0, 0) , z3.If(r27s6, 30, 0) , z3.If(r27s7, 0, 0) , z3.If(r27s8, 0, 0) , z3.If(r27s9, 70, 0) , z3.If(r27s10, 0, 0) , z3.If(r27s11, 0, 0) , z3.If(r27s12, 0, 0) , z3.If(r27s13, 0, 0) , z3.If(r27s14, 70, 0) , z3.If(r27s15, 0, 0) , z3.If(r27s16, 0, 0) , z3.If(r27s17, 0, 0) , z3.If(r27s18, 0, 0) , z3.If(r27s19, 0, 0) , z3.If(r27s20, 0, 0) , z3.If(r27s21, 0, 0) , z3.If(r27s22, 0, 0) , z3.If(r27s23, 0, 0) , z3.If(r27s24, 30, 0) , z3.If(r27s25, 0, 0) , z3.If(r27s26, 0, 0) , z3.If(r27s27, 0, 0) , z3.If(r27s28, 0, 0) , z3.If(r27s29, 0, 0) , z3.If(r27s30, 0, 0) , z3.If(r27s31, 0, 0) , z3.If(r27s32, 0, 0) , z3.If(r27s33, 0, 0) , z3.If(r27s34, 0, 0) , z3.If(r27s35, 0, 0) , z3.If(r27s36, 0, 0) , z3.If(r27s37, 70, 0) , z3.If(r27s38, 0, 0) , z3.If(r27s39, 30, 0) , z3.If(r27s40, 0, 0) , z3.If(r27s41, 0, 0) , z3.If(r27s42, 70, 0) , z3.If(r27s43, 30, 0) , z3.If(r27s44, 0, 0) , z3.If(r27s45, 0, 0) , z3.If(r27s46, 0, 0) , z3.If(r27s47, 0, 0) , z3.If(r27s48, 0, 0) , z3.If(r27s49, 0, 0) , z3.If(r27s50, 70, 0) , z3.If(r27s51, 0, 0) , z3.If(r27s52, 0, 0) , z3.If(r27s53, 0, 0) , z3.If(r28s0, 0, 0) , z3.If(r28s1, 30, 0) , z3.If(r28s2, 0, 0) , z3.If(r28s3, 0, 0) , z3.If(r28s4, 0, 0) , z3.If(r28s5, 0, 0) , z3.If(r28s6, 30, 0) , z3.If(r28s7, 0, 0) , z3.If(r28s8, 0, 0) , z3.If(r28s9, 30, 0) , z3.If(r28s10, 0, 0) , z3.If(r28s11, 0, 0) , z3.If(r28s12, 30, 0) , z3.If(r28s13, 70, 0) , z3.If(r28s14, 0, 0) , z3.If(r28s15, 0, 0) , z3.If(r28s16, 0, 0) , z3.If(r28s17, 30, 0) , z3.If(r28s18, 70, 0) , z3.If(r28s19, 0, 0) , z3.If(r28s20, 0, 0) , z3.If(r28s21, 0, 0) , z3.If(r28s23, 0, 0) , z3.If(r28s24, 70, 0) , z3.If(r28s25, 70, 0) , z3.If(r28s26, 0, 0) , z3.If(r28s29, 0, 0) , z3.If(r28s30, 30, 0) , z3.If(r28s32, 0, 0) , z3.If(r28s33, 0, 0) , z3.If(r28s35, 0, 0) , z3.If(r28s36, 0, 0) , z3.If(r28s37, 0, 0) , z3.If(r28s38, 0, 0) , z3.If(r28s39, 0, 0) , z3.If(r28s40, 70, 0) , z3.If(r28s41, 0, 0) , z3.If(r28s42, 30, 0) , z3.If(r28s43, 0, 0) , z3.If(r28s44, 0, 0) , z3.If(r28s45, 0, 0) , z3.If(r28s46, 0, 0) , z3.If(r28s47, 0, 0) , z3.If(r28s48, 0, 0) , z3.If(r28s49, 0, 0) , z3.If(r28s50, 0, 0) , z3.If(r28s51, 0, 0) , z3.If(r28s52, 0, 0) , z3.If(r28s53, 0, 0) , z3.If(r29s0, 0, 0) , z3.If(r29s1, 0, 0) , z3.If(r29s2, 0, 0) , z3.If(r29s3, 0, 0) , z3.If(r29s4, 0, 0) , z3.If(r29s5, 0, 0) , z3.If(r29s6, 70, 0) , z3.If(r29s7, 0, 0) , z3.If(r29s8, 0, 0) , z3.If(r29s9, 0, 0) , z3.If(r29s10, 0, 0) , z3.If(r29s11, 0, 0) , z3.If(r29s12, 70, 0) , z3.If(r29s13, 0, 0) , z3.If(r29s14, 0, 0) , z3.If(r29s15, 0, 0) , z3.If(r29s16, 70, 0) , z3.If(r29s17, 70, 0) , z3.If(r29s18, 0, 0) , z3.If(r29s19, 0, 0) , z3.If(r29s20, 0, 0) , z3.If(r29s21, 0, 0) , z3.If(r29s23, 0, 0) , z3.If(r29s24, 70, 0) , z3.If(r29s25, 70, 0) , z3.If(r29s26, 0, 0) , z3.If(r29s29, 0, 0) , z3.If(r29s30, 70, 0) , z3.If(r29s32, 70, 0) , z3.If(r29s33, 0, 0) , z3.If(r29s35, 0, 0) , z3.If(r29s36, 70, 0) , z3.If(r29s37, 0, 0) , z3.If(r29s38, 0, 0) , z3.If(r29s39, 0, 0) , z3.If(r29s40, 70, 0) , z3.If(r29s41, 0, 0) , z3.If(r29s42, 70, 0) , z3.If(r29s43, 0, 0) , z3.If(r29s44, 0, 0) , z3.If(r29s45, 0, 0) , z3.If(r29s46, 0, 0) , z3.If(r29s47, 0, 0) , z3.If(r29s48, 0, 0) , z3.If(r29s49, 0, 0) , z3.If(r29s50, 0, 0) , z3.If(r29s51, 0, 0) , z3.If(r29s52, 0, 0) , z3.If(r29s53, 0, 0) , z3.If(r30s0, 0, 0) , z3.If(r30s1, 0, 0) , z3.If(r30s2, 0, 0) , z3.If(r30s3, 70, 0) , z3.If(r30s4, 0, 0) , z3.If(r30s5, 30, 0) , z3.If(r30s6, 70, 0) , z3.If(r30s7, 0, 0) , z3.If(r30s8, 0, 0) , z3.If(r30s9, 0, 0) , z3.If(r30s10, 0, 0) , z3.If(r30s11, 30, 0) , z3.If(r30s12, 0, 0) , z3.If(r30s13, 70, 0) , z3.If(r30s14, 0, 0) , z3.If(r30s15, 70, 0) , z3.If(r30s16, 30, 0) , z3.If(r30s17, 0, 0) , z3.If(r30s18, 0, 0) , z3.If(r30s19, 0, 0) , z3.If(r30s20, 0, 0) , z3.If(r30s21, 30, 0) , z3.If(r30s22, 0, 0) , z3.If(r30s23, 0, 0) , z3.If(r30s24, 0, 0) , z3.If(r30s25, 70, 0) , z3.If(r30s26, 0, 0) , z3.If(r30s27, 0, 0) , z3.If(r30s28, 0, 0) , z3.If(r30s29, 30, 0) , z3.If(r30s30, 0, 0) , z3.If(r30s31, 0, 0) , z3.If(r30s32, 30, 0) , z3.If(r30s33, 0, 0) , z3.If(r30s34, 0, 0) , z3.If(r30s35, 30, 0) , z3.If(r30s37, 0, 0) , z3.If(r30s38, 0, 0) , z3.If(r30s39, 0, 0) , z3.If(r30s40, 0, 0) , z3.If(r30s41, 0, 0) , z3.If(r30s42, 0, 0) , z3.If(r30s43, 0, 0) , z3.If(r30s44, 0, 0) , z3.If(r30s45, 30, 0) , z3.If(r30s46, 30, 0) , z3.If(r30s47, 0, 0) , z3.If(r30s48, 0, 0) , z3.If(r30s49, 70, 0) , z3.If(r30s50, 0, 0) , z3.If(r30s51, 0, 0) , z3.If(r30s52, 0, 0) , z3.If(r30s53, 0, 0)]
# 60/40/0
# scores = []
###############################################################################################################
# other setting
# scores = []

# 60/30/10
scores = [z3.If(r0s0, 10, 0) , z3.If(r0s1, 10, 0) , z3.If(r0s2, 10, 0) , z3.If(r0s4, 10, 0) , z3.If(r0s5, 10, 0) , z3.If(r0s6, 60, 0) , z3.If(r0s7, 10, 0) , z3.If(r0s8, 10, 0) , z3.If(r0s9, 30, 0) , z3.If(r0s10, 10, 0) , z3.If(r0s11, 10, 0) , z3.If(r0s12, 10, 0) , z3.If(r0s13, 60, 0) , z3.If(r0s14, 10, 0) , z3.If(r0s15, 10, 0) , z3.If(r0s16, 30, 0) , z3.If(r0s17, 30, 0) , z3.If(r0s18, 30, 0) , z3.If(r0s19, 10, 0) , z3.If(r0s20, 10, 0) , z3.If(r0s21, 10, 0) , z3.If(r0s22, 60, 0) , z3.If(r0s23, 10, 0) , z3.If(r0s24, 60, 0) , z3.If(r0s25, 10, 0) , z3.If(r0s26, 10, 0) , z3.If(r0s27, 60, 0) , z3.If(r0s28, 10, 0) , z3.If(r0s29, 10, 0) , z3.If(r0s30, 10, 0) , z3.If(r0s31, 10, 0) , z3.If(r0s32, 30, 0) , z3.If(r0s33, 10, 0) , z3.If(r0s34, 10, 0) , z3.If(r0s35, 10, 0) , z3.If(r0s36, 30, 0) , z3.If(r0s37, 10, 0) , z3.If(r0s38, 10, 0) , z3.If(r0s39, 10, 0) , z3.If(r0s40, 10, 0) , z3.If(r0s41, 10, 0) , z3.If(r0s42, 30, 0) , z3.If(r0s43, 30, 0) , z3.If(r0s44, 10, 0) , z3.If(r0s45, 10, 0) , z3.If(r0s46, 10, 0) , z3.If(r0s47, 30, 0) , z3.If(r0s48, 10, 0) , z3.If(r0s49, 10, 0) , z3.If(r0s51, 30, 0) , z3.If(r0s52, 10, 0) , z3.If(r0s53, 10, 0) , z3.If(r1s0, 10, 0) , z3.If(r1s1, 10, 0) , z3.If(r1s2, 10, 0) , z3.If(r1s3, 10, 0) , z3.If(r1s4, 10, 0) , z3.If(r1s5, 10, 0) , z3.If(r1s6, 60, 0) , z3.If(r1s7, 30, 0) , z3.If(r1s8, 10, 0) , z3.If(r1s9, 10, 0) , z3.If(r1s10, 10, 0) , z3.If(r1s11, 10, 0) , z3.If(r1s12, 60, 0) , z3.If(r1s13, 10, 0) , z3.If(r1s14, 30, 0) , z3.If(r1s15, 10, 0) , z3.If(r1s16, 10, 0) , z3.If(r1s17, 60, 0) , z3.If(r1s19, 10, 0) , z3.If(r1s20, 10, 0) , z3.If(r1s21, 10, 0) , z3.If(r1s22, 30, 0) , z3.If(r1s23, 10, 0) , z3.If(r1s24, 60, 0) , z3.If(r1s25, 30, 0) , z3.If(r1s26, 10, 0) , z3.If(r1s27, 60, 0) , z3.If(r1s28, 10, 0) , z3.If(r1s29, 10, 0) , z3.If(r1s30, 30, 0) , z3.If(r1s31, 30, 0) , z3.If(r1s32, 60, 0) , z3.If(r1s33, 10, 0) , z3.If(r1s34, 10, 0) , z3.If(r1s35, 10, 0) , z3.If(r1s36, 30, 0) , z3.If(r1s37, 10, 0) , z3.If(r1s38, 10, 0) , z3.If(r1s39, 10, 0) , z3.If(r1s41, 10, 0) , z3.If(r1s42, 60, 0) , z3.If(r1s43, 30, 0) , z3.If(r1s44, 10, 0) , z3.If(r1s45, 10, 0) , z3.If(r1s46, 10, 0) , z3.If(r1s47, 60, 0) , z3.If(r1s48, 10, 0) , z3.If(r1s49, 30, 0) , z3.If(r1s50, 10, 0) , z3.If(r1s51, 10, 0) , z3.If(r1s52, 10, 0) , z3.If(r1s53, 10, 0) , z3.If(r2s0, 10, 0) , z3.If(r2s1, 30, 0) , z3.If(r2s2, 10, 0) , z3.If(r2s3, 60, 0) , z3.If(r2s4, 10, 0) , z3.If(r2s5, 60, 0) , z3.If(r2s6, 10, 0) , z3.If(r2s7, 60, 0) , z3.If(r2s8, 10, 0) , z3.If(r2s9, 10, 0) , z3.If(r2s10, 10, 0) , z3.If(r2s11, 10, 0) , z3.If(r2s12, 30, 0) , z3.If(r2s13, 10, 0) , z3.If(r2s14, 30, 0) , z3.If(r2s15, 10, 0) , z3.If(r2s16, 10, 0) , z3.If(r2s17, 30, 0) , z3.If(r2s18, 10, 0) , z3.If(r2s19, 10, 0) , z3.If(r2s20, 10, 0) , z3.If(r2s21, 10, 0) , z3.If(r2s22, 10, 0) , z3.If(r2s23, 10, 0) , z3.If(r2s24, 60, 0) , z3.If(r2s25, 10, 0) , z3.If(r2s26, 10, 0) , z3.If(r2s27, 60, 0) , z3.If(r2s28, 10, 0) , z3.If(r2s29, 10, 0) , z3.If(r2s30, 10, 0) , z3.If(r2s31, 10, 0) , z3.If(r2s32, 10, 0) , z3.If(r2s33, 10, 0) , z3.If(r2s34, 10, 0) , z3.If(r2s35, 10, 0) , z3.If(r2s36, 10, 0) , z3.If(r2s37, 10, 0) , z3.If(r2s38, 10, 0) , z3.If(r2s39, 10, 0) , z3.If(r2s40, 60, 0) , z3.If(r2s41, 10, 0) , z3.If(r2s42, 60, 0) , z3.If(r2s43, 30, 0) , z3.If(r2s44, 10, 0) , z3.If(r2s45, 10, 0) , z3.If(r2s46, 10, 0) , z3.If(r2s47, 10, 0) , z3.If(r2s48, 10, 0) , z3.If(r2s49, 10, 0) , z3.If(r2s50, 10, 0) , z3.If(r2s51, 10, 0) , z3.If(r2s52, 10, 0) , z3.If(r2s53, 10, 0) , z3.If(r3s0, 60, 0) , z3.If(r3s1, 10, 0) , z3.If(r3s2, 10, 0) , z3.If(r3s3, 10, 0) , z3.If(r3s4, 10, 0) , z3.If(r3s5, 10, 0) , z3.If(r3s6, 60, 0) , z3.If(r3s7, 10, 0) , z3.If(r3s8, 10, 0) , z3.If(r3s9, 10, 0) , z3.If(r3s10, 10, 0) , z3.If(r3s11, 10, 0) , z3.If(r3s12, 10, 0) , z3.If(r3s13, 60, 0) , z3.If(r3s14, 10, 0) , z3.If(r3s15, 10, 0) , z3.If(r3s16, 10, 0) , z3.If(r3s17, 10, 0) , z3.If(r3s18, 10, 0) , z3.If(r3s19, 10, 0) , z3.If(r3s20, 10, 0) , z3.If(r3s21, 10, 0) , z3.If(r3s22, 10, 0) , z3.If(r3s23, 10, 0) , z3.If(r3s24, 10, 0) , z3.If(r3s25, 10, 0) , z3.If(r3s26, 10, 0) , z3.If(r3s27, 10, 0) , z3.If(r3s28, 10, 0) , z3.If(r3s29, 10, 0) , z3.If(r3s30, 10, 0) , z3.If(r3s31, 10, 0) , z3.If(r3s32, 10, 0) , z3.If(r3s33, 10, 0) , z3.If(r3s34, 10, 0) , z3.If(r3s35, 10, 0) , z3.If(r3s36, 10, 0) , z3.If(r3s37, 60, 0) , z3.If(r3s38, 10, 0) , z3.If(r3s39, 10, 0) , z3.If(r3s40, 60, 0) , z3.If(r3s41, 10, 0) , z3.If(r3s42, 10, 0) , z3.If(r3s43, 10, 0) , z3.If(r3s44, 10, 0) , z3.If(r3s45, 10, 0) , z3.If(r3s46, 10, 0) , z3.If(r3s47, 10, 0) , z3.If(r3s48, 10, 0) , z3.If(r3s49, 10, 0) , z3.If(r3s50, 10, 0) , z3.If(r3s51, 60, 0) , z3.If(r3s52, 10, 0) , z3.If(r3s53, 10, 0) , z3.If(r4s0, 10, 0) , z3.If(r4s1, 10, 0) , z3.If(r4s2, 10, 0) , z3.If(r4s3, 10, 0) , z3.If(r4s4, 10, 0) , z3.If(r4s5, 10, 0) , z3.If(r4s6, 30, 0) , z3.If(r4s7, 10, 0) , z3.If(r4s8, 10, 0) , z3.If(r4s10, 10, 0) , z3.If(r4s11, 10, 0) , z3.If(r4s12, 60, 0) , z3.If(r4s13, 60, 0) , z3.If(r4s14, 10, 0) , z3.If(r4s15, 10, 0) , z3.If(r4s16, 10, 0) , z3.If(r4s17, 60, 0) , z3.If(r4s19, 10, 0) , z3.If(r4s20, 10, 0) , z3.If(r4s21, 10, 0) , z3.If(r4s22, 10, 0) , z3.If(r4s23, 10, 0) , z3.If(r4s24, 60, 0) , z3.If(r4s26, 10, 0) , z3.If(r4s27, 60, 0) , z3.If(r4s28, 10, 0) , z3.If(r4s29, 10, 0) , z3.If(r4s30, 10, 0) , z3.If(r4s31, 10, 0) , z3.If(r4s32, 10, 0) , z3.If(r4s33, 10, 0) , z3.If(r4s34, 10, 0) , z3.If(r4s35, 10, 0) , z3.If(r4s36, 30, 0) , z3.If(r4s37, 10, 0) , z3.If(r4s38, 10, 0) , z3.If(r4s39, 10, 0) , z3.If(r4s40, 30, 0) , z3.If(r4s41, 10, 0) , z3.If(r4s42, 60, 0) , z3.If(r4s43, 10, 0) , z3.If(r4s44, 10, 0) , z3.If(r4s45, 30, 0) , z3.If(r4s46, 30, 0) , z3.If(r4s47, 10, 0) , z3.If(r4s48, 10, 0) , z3.If(r4s49, 10, 0) , z3.If(r4s50, 10, 0) , z3.If(r4s51, 10, 0) , z3.If(r4s52, 10, 0) , z3.If(r4s53, 10, 0) , z3.If(r5s0, 10, 0) , z3.If(r5s1, 10, 0) , z3.If(r5s2, 10, 0) , z3.If(r5s3, 10, 0) , z3.If(r5s4, 10, 0) , z3.If(r5s5, 10, 0) , z3.If(r5s6, 60, 0) , z3.If(r5s7, 10, 0) , z3.If(r5s8, 10, 0) , z3.If(r5s9, 10, 0) , z3.If(r5s10, 10, 0) , z3.If(r5s11, 10, 0) , z3.If(r5s12, 10, 0) , z3.If(r5s13, 60, 0) , z3.If(r5s14, 10, 0) , z3.If(r5s15, 10, 0) , z3.If(r5s16, 10, 0) , z3.If(r5s17, 30, 0) , z3.If(r5s18, 10, 0) , z3.If(r5s19, 10, 0) , z3.If(r5s20, 10, 0) , z3.If(r5s21, 10, 0) , z3.If(r5s22, 10, 0) , z3.If(r5s23, 10, 0) , z3.If(r5s24, 30, 0) , z3.If(r5s25, 30, 0) , z3.If(r5s26, 10, 0) , z3.If(r5s27, 30, 0) , z3.If(r5s28, 10, 0) , z3.If(r5s29, 10, 0) , z3.If(r5s30, 30, 0) , z3.If(r5s31, 10, 0) , z3.If(r5s32, 10, 0) , z3.If(r5s33, 10, 0) , z3.If(r5s34, 10, 0) , z3.If(r5s35, 10, 0) , z3.If(r5s36, 30, 0) , z3.If(r5s37, 10, 0) , z3.If(r5s38, 10, 0) , z3.If(r5s39, 10, 0) , z3.If(r5s40, 30, 0) , z3.If(r5s41, 10, 0) , z3.If(r5s42, 60, 0) , z3.If(r5s43, 10, 0) , z3.If(r5s44, 10, 0) , z3.If(r5s45, 10, 0) , z3.If(r5s46, 10, 0) , z3.If(r5s47, 30, 0) , z3.If(r5s48, 10, 0) , z3.If(r5s49, 10, 0) , z3.If(r5s50, 10, 0) , z3.If(r5s51, 10, 0) , z3.If(r5s52, 10, 0) , z3.If(r5s53, 10, 0) , z3.If(r6s0, 10, 0) , z3.If(r6s1, 10, 0) , z3.If(r6s2, 10, 0) , z3.If(r6s3, 10, 0) , z3.If(r6s4, 10, 0) , z3.If(r6s5, 10, 0) , z3.If(r6s6, 30, 0) , z3.If(r6s7, 10, 0) , z3.If(r6s8, 10, 0) , z3.If(r6s9, 10, 0) , z3.If(r6s10, 10, 0) , z3.If(r6s11, 10, 0) , z3.If(r6s12, 30, 0) , z3.If(r6s13, 10, 0) , z3.If(r6s14, 10, 0) , z3.If(r6s15, 10, 0) , z3.If(r6s16, 10, 0) , z3.If(r6s17, 10, 0) , z3.If(r6s18, 10, 0) , z3.If(r6s19, 10, 0) , z3.If(r6s20, 10, 0) , z3.If(r6s21, 10, 0) , z3.If(r6s22, 10, 0) , z3.If(r6s23, 60, 0) , z3.If(r6s24, 10, 0) , z3.If(r6s25, 10, 0) , z3.If(r6s26, 10, 0) , z3.If(r6s27, 10, 0) , z3.If(r6s28, 10, 0) , z3.If(r6s29, 10, 0) , z3.If(r6s30, 10, 0) , z3.If(r6s31, 10, 0) , z3.If(r6s32, 10, 0) , z3.If(r6s33, 10, 0) , z3.If(r6s34, 10, 0) , z3.If(r6s35, 10, 0) , z3.If(r6s36, 60, 0) , z3.If(r6s37, 10, 0) , z3.If(r6s38, 10, 0) , z3.If(r6s39, 10, 0) , z3.If(r6s40, 30, 0) , z3.If(r6s41, 10, 0) , z3.If(r6s42, 10, 0) , z3.If(r6s43, 10, 0) , z3.If(r6s44, 10, 0) , z3.If(r6s45, 10, 0) , z3.If(r6s46, 10, 0) , z3.If(r6s47, 60, 0) , z3.If(r6s48, 10, 0) , z3.If(r6s49, 30, 0) , z3.If(r6s50, 60, 0) , z3.If(r6s51, 10, 0) , z3.If(r6s52, 10, 0) , z3.If(r6s53, 10, 0) , z3.If(r7s0, 30, 0) , z3.If(r7s1, 10, 0) , z3.If(r7s2, 30, 0) , z3.If(r7s3, 10, 0) , z3.If(r7s4, 10, 0) , z3.If(r7s5, 10, 0) , z3.If(r7s6, 10, 0) , z3.If(r7s7, 10, 0) , z3.If(r7s8, 10, 0) , z3.If(r7s9, 10, 0) , z3.If(r7s10, 10, 0) , z3.If(r7s11, 10, 0) , z3.If(r7s12, 60, 0) , z3.If(r7s13, 30, 0) , z3.If(r7s14, 10, 0) , z3.If(r7s15, 10, 0) , z3.If(r7s16, 10, 0) , z3.If(r7s17, 60, 0) , z3.If(r7s19, 10, 0) , z3.If(r7s20, 10, 0) , z3.If(r7s21, 10, 0) , z3.If(r7s22, 10, 0) , z3.If(r7s23, 10, 0) , z3.If(r7s25, 60, 0) , z3.If(r7s26, 10, 0) , z3.If(r7s27, 30, 0) , z3.If(r7s28, 10, 0) , z3.If(r7s29, 10, 0) , z3.If(r7s30, 30, 0) , z3.If(r7s31, 60, 0) , z3.If(r7s32, 10, 0) , z3.If(r7s33, 10, 0) , z3.If(r7s34, 10, 0) , z3.If(r7s35, 10, 0) , z3.If(r7s36, 10, 0) , z3.If(r7s37, 10, 0) , z3.If(r7s38, 10, 0) , z3.If(r7s39, 10, 0) , z3.If(r7s40, 10, 0) , z3.If(r7s41, 10, 0) , z3.If(r7s42, 60, 0) , z3.If(r7s43, 10, 0) , z3.If(r7s44, 10, 0) , z3.If(r7s45, 10, 0) , z3.If(r7s46, 10, 0) , z3.If(r7s47, 60, 0) , z3.If(r7s48, 10, 0) , z3.If(r7s49, 10, 0) , z3.If(r7s50, 10, 0) , z3.If(r7s51, 10, 0) , z3.If(r7s52, 60, 0) , z3.If(r7s53, 10, 0) , z3.If(r8s0, 10, 0) , z3.If(r8s2, 10, 0) , z3.If(r8s3, 60, 0) , z3.If(r8s4, 60, 0) , z3.If(r8s5, 10, 0) , z3.If(r8s6, 10, 0) , z3.If(r8s7, 60, 0) , z3.If(r8s8, 10, 0) , z3.If(r8s9, 10, 0) , z3.If(r8s10, 10, 0) , z3.If(r8s11, 10, 0) , z3.If(r8s12, 10, 0) , z3.If(r8s13, 10, 0) , z3.If(r8s14, 10, 0) , z3.If(r8s15, 10, 0) , z3.If(r8s16, 60, 0) , z3.If(r8s17, 10, 0) , z3.If(r8s18, 10, 0) , z3.If(r8s19, 60, 0) , z3.If(r8s20, 10, 0) , z3.If(r8s21, 10, 0) , z3.If(r8s22, 10, 0) , z3.If(r8s23, 10, 0) , z3.If(r8s24, 10, 0) , z3.If(r8s25, 10, 0) , z3.If(r8s26, 10, 0) , z3.If(r8s27, 10, 0) , z3.If(r8s28, 10, 0) , z3.If(r8s29, 10, 0) , z3.If(r8s30, 10, 0) , z3.If(r8s31, 10, 0) , z3.If(r8s32, 10, 0) , z3.If(r8s33, 10, 0) , z3.If(r8s34, 10, 0) , z3.If(r8s35, 10, 0) , z3.If(r8s36, 10, 0) , z3.If(r8s37, 10, 0) , z3.If(r8s38, 60, 0) , z3.If(r8s39, 10, 0) , z3.If(r8s40, 10, 0) , z3.If(r8s41, 10, 0) , z3.If(r8s42, 10, 0) , z3.If(r8s43, 10, 0) , z3.If(r8s44, 10, 0) , z3.If(r8s45, 10, 0) , z3.If(r8s46, 10, 0) , z3.If(r8s47, 10, 0) , z3.If(r8s48, 10, 0) , z3.If(r8s49, 10, 0) , z3.If(r8s50, 10, 0) , z3.If(r8s51, 10, 0) , z3.If(r8s52, 10, 0) , z3.If(r8s53, 10, 0) , z3.If(r9s0, 10, 0) , z3.If(r9s1, 60, 0) , z3.If(r9s2, 10, 0) , z3.If(r9s3, 10, 0) , z3.If(r9s4, 10, 0) , z3.If(r9s5, 30, 0) , z3.If(r9s6, 60, 0) , z3.If(r9s7, 10, 0) , z3.If(r9s8, 10, 0) , z3.If(r9s9, 10, 0) , z3.If(r9s10, 10, 0) , z3.If(r9s11, 10, 0) , z3.If(r9s12, 10, 0) , z3.If(r9s13, 60, 0) , z3.If(r9s14, 10, 0) , z3.If(r9s15, 10, 0) , z3.If(r9s16, 10, 0) , z3.If(r9s17, 30, 0) , z3.If(r9s18, 10, 0) , z3.If(r9s19, 10, 0) , z3.If(r9s20, 10, 0) , z3.If(r9s21, 30, 0) , z3.If(r9s22, 10, 0) , z3.If(r9s23, 10, 0) , z3.If(r9s24, 10, 0) , z3.If(r9s25, 30, 0) , z3.If(r9s26, 10, 0) , z3.If(r9s27, 30, 0) , z3.If(r9s28, 10, 0) , z3.If(r9s29, 10, 0) , z3.If(r9s30, 30, 0) , z3.If(r9s31, 10, 0) , z3.If(r9s32, 10, 0) , z3.If(r9s33, 10, 0) , z3.If(r9s34, 10, 0) , z3.If(r9s35, 10, 0) , z3.If(r9s36, 10, 0) , z3.If(r9s37, 10, 0) , z3.If(r9s38, 10, 0) , z3.If(r9s39, 10, 0) , z3.If(r9s40, 30, 0) , z3.If(r9s41, 10, 0) , z3.If(r9s42, 10, 0) , z3.If(r9s43, 10, 0) , z3.If(r9s44, 10, 0) , z3.If(r9s45, 10, 0) , z3.If(r9s46, 10, 0) , z3.If(r9s47, 10, 0) , z3.If(r9s48, 10, 0) , z3.If(r9s49, 10, 0) , z3.If(r9s50, 10, 0) , z3.If(r9s51, 10, 0) , z3.If(r9s52, 60, 0) , z3.If(r9s53, 10, 0) , z3.If(r10s0, 30, 0) , z3.If(r10s1, 10, 0) , z3.If(r10s2, 10, 0) , z3.If(r10s3, 10, 0) , z3.If(r10s4, 10, 0) , z3.If(r10s5, 10, 0) , z3.If(r10s6, 30, 0) , z3.If(r10s7, 30, 0) , z3.If(r10s8, 10, 0) , z3.If(r10s9, 30, 0) , z3.If(r10s10, 10, 0) , z3.If(r10s11, 10, 0) , z3.If(r10s12, 30, 0) , z3.If(r10s13, 30, 0) , z3.If(r10s14, 30, 0) , z3.If(r10s15, 10, 0) , z3.If(r10s16, 30, 0) , z3.If(r10s17, 10, 0) , z3.If(r10s18, 10, 0) , z3.If(r10s19, 10, 0) , z3.If(r10s20, 10, 0) , z3.If(r10s21, 10, 0) , z3.If(r10s22, 30, 0) , z3.If(r10s23, 10, 0) , z3.If(r10s24, 10, 0) , z3.If(r10s25, 10, 0) , z3.If(r10s26, 10, 0) , z3.If(r10s27, 30, 0) , z3.If(r10s28, 10, 0) , z3.If(r10s29, 10, 0) , z3.If(r10s30, 10, 0) , z3.If(r10s31, 10, 0) , z3.If(r10s32, 10, 0) , z3.If(r10s33, 10, 0) , z3.If(r10s34, 10, 0) , z3.If(r10s35, 10, 0) , z3.If(r10s36, 10, 0) , z3.If(r10s37, 10, 0) , z3.If(r10s38, 10, 0) , z3.If(r10s39, 10, 0) , z3.If(r10s40, 30, 0) , z3.If(r10s41, 10, 0) , z3.If(r10s42, 10, 0) , z3.If(r10s43, 10, 0) , z3.If(r10s44, 10, 0) , z3.If(r10s45, 10, 0) , z3.If(r10s46, 10, 0) , z3.If(r10s47, 10, 0) , z3.If(r10s48, 10, 0) , z3.If(r10s49, 10, 0) , z3.If(r10s50, 30, 0) , z3.If(r10s51, 10, 0) , z3.If(r10s52, 10, 0) , z3.If(r10s53, 10, 0) , z3.If(r11s0, 30, 0) , z3.If(r11s1, 10, 0) , z3.If(r11s2, 10, 0) , z3.If(r11s3, 30, 0) , z3.If(r11s4, 10, 0) , z3.If(r11s5, 10, 0) , z3.If(r11s6, 60, 0) , z3.If(r11s7, 10, 0) , z3.If(r11s8, 10, 0) , z3.If(r11s9, 30, 0) , z3.If(r11s10, 10, 0) , z3.If(r11s11, 10, 0) , z3.If(r11s12, 10, 0) , z3.If(r11s13, 30, 0) , z3.If(r11s14, 60, 0) , z3.If(r11s15, 10, 0) , z3.If(r11s16, 10, 0) , z3.If(r11s17, 10, 0) , z3.If(r11s18, 10, 0) , z3.If(r11s19, 10, 0) , z3.If(r11s20, 10, 0) , z3.If(r11s21, 10, 0) , z3.If(r11s22, 10, 0) , z3.If(r11s23, 10, 0) , z3.If(r11s24, 10, 0) , z3.If(r11s25, 10, 0) , z3.If(r11s26, 10, 0) , z3.If(r11s27, 60, 0) , z3.If(r11s28, 10, 0) , z3.If(r11s29, 10, 0) , z3.If(r11s30, 10, 0) , z3.If(r11s31, 10, 0) , z3.If(r11s32, 10, 0) , z3.If(r11s33, 10, 0) , z3.If(r11s34, 10, 0) , z3.If(r11s35, 10, 0) , z3.If(r11s36, 10, 0) , z3.If(r11s37, 10, 0) , z3.If(r11s38, 10, 0) , z3.If(r11s39, 10, 0) , z3.If(r11s40, 10, 0) , z3.If(r11s41, 60, 0) , z3.If(r11s42, 60, 0) , z3.If(r11s43, 10, 0) , z3.If(r11s44, 10, 0) , z3.If(r11s45, 10, 0) , z3.If(r11s46, 10, 0) , z3.If(r11s47, 10, 0) , z3.If(r11s48, 10, 0) , z3.If(r11s49, 10, 0) , z3.If(r11s50, 10, 0) , z3.If(r11s51, 10, 0) , z3.If(r11s52, 10, 0) , z3.If(r11s53, 10, 0) , z3.If(r12s0, 10, 0) , z3.If(r12s1, 30, 0) , z3.If(r12s2, 10, 0) , z3.If(r12s3, 30, 0) , z3.If(r12s4, 10, 0) , z3.If(r12s5, 10, 0) , z3.If(r12s6, 30, 0) , z3.If(r12s8, 10, 0) , z3.If(r12s9, 30, 0) , z3.If(r12s10, 10, 0) , z3.If(r12s11, 10, 0) , z3.If(r12s12, 30, 0) , z3.If(r12s13, 10, 0) , z3.If(r12s14, 10, 0) , z3.If(r12s15, 10, 0) , z3.If(r12s16, 30, 0) , z3.If(r12s17, 30, 0) , z3.If(r12s18, 10, 0) , z3.If(r12s19, 10, 0) , z3.If(r12s20, 10, 0) , z3.If(r12s21, 30, 0) , z3.If(r12s22, 10, 0) , z3.If(r12s23, 10, 0) , z3.If(r12s24, 10, 0) , z3.If(r12s25, 10, 0) , z3.If(r12s26, 10, 0) , z3.If(r12s27, 10, 0) , z3.If(r12s28, 10, 0) , z3.If(r12s30, 10, 0) , z3.If(r12s31, 10, 0) , z3.If(r12s32, 10, 0) , z3.If(r12s33, 10, 0) , z3.If(r12s34, 10, 0) , z3.If(r12s35, 10, 0) , z3.If(r12s36, 10, 0) , z3.If(r12s37, 10, 0) , z3.If(r12s38, 10, 0) , z3.If(r12s39, 10, 0) , z3.If(r12s40, 10, 0) , z3.If(r12s41, 10, 0) , z3.If(r12s42, 10, 0) , z3.If(r12s43, 10, 0) , z3.If(r12s44, 10, 0) , z3.If(r12s45, 10, 0) , z3.If(r12s46, 30, 0) , z3.If(r12s47, 10, 0) , z3.If(r12s48, 10, 0) , z3.If(r12s49, 10, 0) , z3.If(r12s50, 10, 0) , z3.If(r12s51, 10, 0) , z3.If(r12s52, 10, 0) , z3.If(r12s53, 10, 0) , z3.If(r13s0, 60, 0) , z3.If(r13s1, 60, 0) , z3.If(r13s2, 60, 0) , z3.If(r13s3, 10, 0) , z3.If(r13s4, 30, 0) , z3.If(r13s5, 10, 0) , z3.If(r13s6, 10, 0) , z3.If(r13s7, 10, 0) , z3.If(r13s8, 10, 0) , z3.If(r13s9, 60, 0) , z3.If(r13s10, 10, 0) , z3.If(r13s11, 10, 0) , z3.If(r13s12, 60, 0) , z3.If(r13s13, 10, 0) , z3.If(r13s14, 10, 0) , z3.If(r13s15, 10, 0) , z3.If(r13s16, 10, 0) , z3.If(r13s17, 30, 0) , z3.If(r13s18, 60, 0) , z3.If(r13s19, 10, 0) , z3.If(r13s20, 10, 0) , z3.If(r13s21, 10, 0) , z3.If(r13s22, 10, 0) , z3.If(r13s23, 10, 0) , z3.If(r13s24, 60, 0) , z3.If(r13s25, 60, 0) , z3.If(r13s26, 10, 0) , z3.If(r13s27, 60, 0) , z3.If(r13s28, 10, 0) , z3.If(r13s29, 10, 0) , z3.If(r13s30, 10, 0) , z3.If(r13s31, 10, 0) , z3.If(r13s32, 10, 0) , z3.If(r13s33, 10, 0) , z3.If(r13s34, 10, 0) , z3.If(r13s35, 10, 0) , z3.If(r13s36, 10, 0) , z3.If(r13s37, 10, 0) , z3.If(r13s38, 10, 0) , z3.If(r13s39, 60, 0) , z3.If(r13s40, 30, 0) , z3.If(r13s41, 10, 0) , z3.If(r13s42, 10, 0) , z3.If(r13s43, 10, 0) , z3.If(r13s44, 10, 0) , z3.If(r13s45, 10, 0) , z3.If(r13s46, 10, 0) , z3.If(r13s47, 10, 0) , z3.If(r13s48, 10, 0) , z3.If(r13s49, 10, 0) , z3.If(r13s50, 10, 0) , z3.If(r13s51, 10, 0) , z3.If(r13s52, 10, 0) , z3.If(r13s53, 10, 0) , z3.If(r14s0, 10, 0) , z3.If(r14s1, 30, 0) , z3.If(r14s2, 10, 0) , z3.If(r14s3, 10, 0) , z3.If(r14s4, 10, 0) , z3.If(r14s5, 10, 0) , z3.If(r14s6, 60, 0) , z3.If(r14s7, 30, 0) , z3.If(r14s8, 10, 0) , z3.If(r14s9, 30, 0) , z3.If(r14s10, 10, 0) , z3.If(r14s11, 10, 0) , z3.If(r14s12, 10, 0) , z3.If(r14s14, 60, 0) , z3.If(r14s15, 10, 0) , z3.If(r14s16, 10, 0) , z3.If(r14s17, 10, 0) , z3.If(r14s18, 10, 0) , z3.If(r14s19, 10, 0) , z3.If(r14s20, 10, 0) , z3.If(r14s21, 10, 0) , z3.If(r14s22, 10, 0) , z3.If(r14s23, 10, 0) , z3.If(r14s25, 10, 0) , z3.If(r14s26, 10, 0) , z3.If(r14s27, 10, 0) , z3.If(r14s28, 10, 0) , z3.If(r14s29, 10, 0) , z3.If(r14s30, 30, 0) , z3.If(r14s31, 10, 0) , z3.If(r14s32, 10, 0) , z3.If(r14s33, 10, 0) , z3.If(r14s34, 10, 0) , z3.If(r14s35, 10, 0) , z3.If(r14s36, 10, 0) , z3.If(r14s37, 10, 0) , z3.If(r14s38, 10, 0) , z3.If(r14s39, 10, 0) , z3.If(r14s40, 60, 0) , z3.If(r14s41, 10, 0) , z3.If(r14s42, 10, 0) , z3.If(r14s43, 10, 0) , z3.If(r14s44, 10, 0) , z3.If(r14s45, 10, 0) , z3.If(r14s46, 30, 0) , z3.If(r14s47, 10, 0) , z3.If(r14s48, 10, 0) , z3.If(r14s49, 10, 0) , z3.If(r14s50, 10, 0) , z3.If(r14s51, 10, 0) , z3.If(r14s52, 10, 0) , z3.If(r14s53, 10, 0) , z3.If(r15s0, 30, 0) , z3.If(r15s1, 10, 0) , z3.If(r15s2, 10, 0) , z3.If(r15s3, 10, 0) , z3.If(r15s4, 10, 0) , z3.If(r15s5, 10, 0) , z3.If(r15s6, 10, 0) , z3.If(r15s7, 10, 0) , z3.If(r15s8, 10, 0) , z3.If(r15s9, 10, 0) , z3.If(r15s10, 10, 0) , z3.If(r15s11, 10, 0) , z3.If(r15s12, 10, 0) , z3.If(r15s13, 60, 0) , z3.If(r15s14, 10, 0) , z3.If(r15s15, 10, 0) , z3.If(r15s16, 10, 0) , z3.If(r15s17, 10, 0) , 
z3.If(r15s18, 10, 0) , z3.If(r15s19, 10, 0) , z3.If(r15s20, 10, 0) , z3.If(r15s21, 10, 0) , z3.If(r15s22, 30, 0) , z3.If(r15s23, 10, 0) , z3.If(r15s24, 30, 0) , z3.If(r15s25, 30, 0) , z3.If(r15s26, 10, 0) , z3.If(r15s27, 60, 0) , z3.If(r15s28, 10, 0) , z3.If(r15s29, 10, 0) , z3.If(r15s30, 10, 0) , z3.If(r15s31, 10, 0) , z3.If(r15s32, 10, 0) , z3.If(r15s33, 10, 0) , z3.If(r15s34, 10, 0) , z3.If(r15s35, 10, 0) , z3.If(r15s36, 10, 0) , z3.If(r15s37, 10, 0) , z3.If(r15s38, 10, 0) , z3.If(r15s39, 10, 0) , z3.If(r15s40, 30, 0) , z3.If(r15s41, 10, 0) , z3.If(r15s42, 10, 0) , z3.If(r15s43, 10, 0) , z3.If(r15s44, 10, 0) , z3.If(r15s45, 10, 0) , z3.If(r15s46, 10, 0) , z3.If(r15s47, 10, 0) , z3.If(r15s48, 10, 0) , z3.If(r15s49, 30, 0) , z3.If(r15s50, 10, 0) , z3.If(r15s51, 10, 0) , z3.If(r15s52, 10, 0) , z3.If(r15s53, 10, 0) , z3.If(r16s0, 30, 0) , z3.If(r16s1, 30, 0) , z3.If(r16s2, 10, 0) , z3.If(r16s3, 30, 0) , z3.If(r16s4, 10, 0) , z3.If(r16s5, 30, 0) , z3.If(r16s6, 10, 0) , z3.If(r16s7, 30, 0) , z3.If(r16s8, 10, 0) , z3.If(r16s9, 10, 0) , z3.If(r16s10, 10, 0) , z3.If(r16s11, 60, 0) , z3.If(r16s12, 10, 0) , z3.If(r16s13, 10, 0) , z3.If(r16s14, 10, 0) , z3.If(r16s15, 10, 0) , z3.If(r16s16, 10, 0) , z3.If(r16s17, 10, 0) , z3.If(r16s18, 10, 0) , z3.If(r16s19, 30, 0) , z3.If(r16s20, 10, 0) , z3.If(r16s21, 10, 0) , z3.If(r16s22, 10, 0) , z3.If(r16s23, 10, 0) , z3.If(r16s24, 10, 0) , z3.If(r16s25, 10, 0) , z3.If(r16s26, 10, 0) , z3.If(r16s27, 30, 0) , z3.If(r16s28, 10, 0) , z3.If(r16s29, 10, 0) , z3.If(r16s30, 10, 0) , z3.If(r16s31, 10, 0) , z3.If(r16s32, 10, 0) , z3.If(r16s33, 10, 0) , z3.If(r16s34, 10, 0) , z3.If(r16s35, 10, 0) , z3.If(r16s36, 10, 0) , z3.If(r16s37, 30, 0) , z3.If(r16s38, 10, 0) , z3.If(r16s39, 10, 0) , z3.If(r16s40, 10, 0) , z3.If(r16s41, 10, 0) , z3.If(r16s42, 10, 0) , z3.If(r16s44, 10, 0) , z3.If(r16s45, 10, 0) , z3.If(r16s46, 60, 0) , z3.If(r16s47, 10, 0) , z3.If(r16s48, 10, 0) , z3.If(r16s50, 10, 0) , z3.If(r16s52, 10, 0) , z3.If(r16s53, 10, 0) , z3.If(r17s0, 10, 0) , z3.If(r17s1, 60, 0) , z3.If(r17s2, 10, 0) , z3.If(r17s3, 60, 0) , z3.If(r17s4, 10, 0) , z3.If(r17s5, 10, 0) , z3.If(r17s6, 10, 0) , z3.If(r17s7, 10, 0) , z3.If(r17s8, 10, 0) , z3.If(r17s9, 60, 0) , z3.If(r17s10, 60, 0) , z3.If(r17s11, 10, 0) , z3.If(r17s12, 10, 0) , z3.If(r17s13, 10, 0) , z3.If(r17s14, 10, 0) , z3.If(r17s15, 10, 0) , z3.If(r17s16, 10, 0) , z3.If(r17s17, 10, 0) , z3.If(r17s18, 10, 0) , z3.If(r17s19, 10, 0) , z3.If(r17s20, 10, 0) , z3.If(r17s22, 10, 0) , z3.If(r17s23, 10, 0) , z3.If(r17s24, 10, 0) , z3.If(r17s25, 10, 0) , z3.If(r17s27, 10, 0) , z3.If(r17s28, 10, 0) , z3.If(r17s29, 60, 0) , z3.If(r17s30, 10, 0) , z3.If(r17s31, 10, 0) , z3.If(r17s32, 10, 0) , z3.If(r17s33, 60, 0) , z3.If(r17s34, 10, 0) , z3.If(r17s35, 10, 0) , z3.If(r17s36, 10, 0) , z3.If(r17s37, 10, 0) , z3.If(r17s38, 10, 0) , z3.If(r17s39, 10, 0) , z3.If(r17s40, 10, 0) , z3.If(r17s41, 10, 0) , z3.If(r17s42, 10, 0) , z3.If(r17s43, 10, 0) , z3.If(r17s44, 60, 0) , z3.If(r17s46, 10, 0) , z3.If(r17s47, 10, 0) , z3.If(r17s49, 10, 0) , z3.If(r17s50, 10, 0) , z3.If(r17s51, 10, 0) , z3.If(r17s52, 10, 0) , z3.If(r18s0, 10, 0) , z3.If(r18s1, 10, 0) , z3.If(r18s2, 10, 0) , z3.If(r18s3, 10, 0) , z3.If(r18s4, 10, 0) , z3.If(r18s5, 60, 0) , z3.If(r18s6, 10, 0) , z3.If(r18s7, 10, 0) , z3.If(r18s8, 10, 0) , z3.If(r18s9, 10, 0) , z3.If(r18s10, 10, 0) , z3.If(r18s11, 10, 0) , z3.If(r18s12, 60, 0) , z3.If(r18s13, 10, 0) , z3.If(r18s14, 30, 0) , z3.If(r18s15, 10, 0) , z3.If(r18s16, 10, 0) , z3.If(r18s17, 30, 0) , z3.If(r18s18, 30, 0) , z3.If(r18s19, 10, 0) , z3.If(r18s20, 10, 0) , z3.If(r18s21, 10, 0) , z3.If(r18s22, 60, 0) , z3.If(r18s23, 10, 0) , z3.If(r18s24, 10, 0) , z3.If(r18s25, 10, 0) , z3.If(r18s26, 10, 0) , z3.If(r18s27, 10, 0) , z3.If(r18s28, 10, 0) , z3.If(r18s29, 10, 0) , z3.If(r18s30, 10, 0) , z3.If(r18s31, 10, 0) , z3.If(r18s32, 10, 0) , z3.If(r18s33, 10, 0) , z3.If(r18s34, 10, 0) , z3.If(r18s35, 10, 0) , z3.If(r18s36, 30, 0) , z3.If(r18s37, 10, 0) , z3.If(r18s38, 10, 0) , z3.If(r18s39, 10, 0) , z3.If(r18s40, 10, 0) , z3.If(r18s41, 10, 0) , z3.If(r18s42, 10, 0) , z3.If(r18s43, 10, 0) , z3.If(r18s44, 10, 0) , z3.If(r18s45, 10, 0) , z3.If(r18s46, 10, 0) , z3.If(r18s47, 10, 0) , z3.If(r18s48, 10, 0) , z3.If(r18s49, 10, 0) , z3.If(r18s50, 60, 0) , z3.If(r18s51, 10, 0) , z3.If(r18s52, 10, 0) , z3.If(r18s53, 10, 0) , z3.If(r19s0, 30, 0) , z3.If(r19s1, 10, 0) , z3.If(r19s2, 60, 0) , z3.If(r19s3, 10, 0) , z3.If(r19s4, 10, 0) , z3.If(r19s5, 10, 0) , z3.If(r19s6, 10, 0) , z3.If(r19s7, 10, 0) , z3.If(r19s8, 10, 0) , z3.If(r19s9, 10, 0) , z3.If(r19s10, 10, 0) , z3.If(r19s11, 10, 0) , z3.If(r19s12, 10, 0) , z3.If(r19s13, 30, 0) , z3.If(r19s14, 10, 0) , z3.If(r19s15, 10, 0) , z3.If(r19s16, 30, 0) , z3.If(r19s17, 10, 0) , z3.If(r19s18, 10, 0) , z3.If(r19s19, 10, 0) , z3.If(r19s20, 10, 0) , z3.If(r19s21, 10, 0) , z3.If(r19s22, 10, 0) , z3.If(r19s23, 10, 0) , z3.If(r19s24, 10, 0) , z3.If(r19s25, 10, 0) , z3.If(r19s26, 10, 0) , z3.If(r19s27, 10, 0) , z3.If(r19s28, 10, 0) , z3.If(r19s29, 10, 0) , z3.If(r19s30, 10, 0) , z3.If(r19s31, 10, 0) , z3.If(r19s32, 10, 0) , z3.If(r19s33, 10, 0) , z3.If(r19s34, 10, 0) , z3.If(r19s35, 10, 0) , z3.If(r19s36, 10, 0) , z3.If(r19s37, 10, 0) , z3.If(r19s38, 10, 0) , z3.If(r19s39, 10, 0) , z3.If(r19s40, 10, 0) , z3.If(r19s41, 10, 0) , z3.If(r19s42, 10, 0) , z3.If(r19s43, 10, 0) , z3.If(r19s44, 10, 0) , z3.If(r19s45, 10, 0) , z3.If(r19s46, 10, 0) , z3.If(r19s47, 10, 0) , z3.If(r19s48, 10, 0) , z3.If(r19s49, 10, 0) , z3.If(r19s50, 10, 0) , z3.If(r19s51, 60, 0) , z3.If(r19s52, 10, 0) , z3.If(r19s53, 10, 0) , z3.If(r20s0, 60, 0) , z3.If(r20s1, 10, 0) , z3.If(r20s2, 60, 0) , z3.If(r20s3, 10, 0) , z3.If(r20s4, 10, 0) , z3.If(r20s5, 10, 0) , z3.If(r20s6, 60, 0) , z3.If(r20s7, 10, 0) , z3.If(r20s8, 10, 0) , z3.If(r20s9, 60, 0) , z3.If(r20s10, 10, 0) , z3.If(r20s11, 10, 0) , z3.If(r20s13, 10, 0) , z3.If(r20s14, 10, 0) , z3.If(r20s15, 10, 0) , z3.If(r20s16, 10, 0) , z3.If(r20s17, 60, 0) , z3.If(r20s18, 60, 0) , z3.If(r20s19, 10, 0) , z3.If(r20s20, 10, 0) , z3.If(r20s21, 10, 0) , z3.If(r20s22, 60, 0) , z3.If(r20s23, 10, 0) , z3.If(r20s24, 60, 0) , z3.If(r20s25, 60, 0) , z3.If(r20s26, 10, 0) , z3.If(r20s27, 60, 0) , z3.If(r20s28, 10, 0) , z3.If(r20s29, 10, 0) , z3.If(r20s30, 60, 0) , z3.If(r20s31, 60, 0) , z3.If(r20s32, 10, 0) , z3.If(r20s33, 10, 0) , z3.If(r20s34, 10, 0) , z3.If(r20s35, 10, 0) , z3.If(r20s36, 10, 0) , z3.If(r20s37, 10, 0) , z3.If(r20s38, 10, 0) , z3.If(r20s39, 10, 0) , z3.If(r20s40, 10, 0) , z3.If(r20s41, 10, 0) , z3.If(r20s42, 10, 0) , z3.If(r20s43, 10, 0) , z3.If(r20s44, 10, 0) , z3.If(r20s45, 10, 0) , z3.If(r20s46, 10, 0) , z3.If(r20s47, 10, 0) , z3.If(r20s48, 10, 0) , z3.If(r20s49, 10, 0) , z3.If(r20s50, 10, 0) , z3.If(r20s51, 10, 0) , z3.If(r20s52, 10, 0) , z3.If(r20s53, 10, 0) , z3.If(r21s0, 60, 0) , z3.If(r21s1, 10, 0) , z3.If(r21s2, 10, 0) , z3.If(r21s3, 60, 0) , z3.If(r21s4, 10, 0) , z3.If(r21s5, 10, 0) , z3.If(r21s6, 30, 0) , z3.If(r21s7, 30, 0) , z3.If(r21s8, 10, 0) , z3.If(r21s9, 60, 0) , z3.If(r21s10, 10, 0) , z3.If(r21s11, 10, 0) , z3.If(r21s12, 10, 0) , z3.If(r21s13, 30, 0) , z3.If(r21s14, 10, 0) , z3.If(r21s15, 10, 0) , z3.If(r21s16, 30, 0) , z3.If(r21s17, 10, 0) , z3.If(r21s18, 30, 0) , z3.If(r21s19, 10, 0) , z3.If(r21s20, 10, 0) , z3.If(r21s21, 10, 0) , z3.If(r21s22, 60, 0) , z3.If(r21s23, 10, 0) , z3.If(r21s24, 10, 0) , z3.If(r21s25, 10, 0) , z3.If(r21s26, 10, 0) , z3.If(r21s27, 30, 0) , z3.If(r21s28, 60, 0) , z3.If(r21s29, 10, 0) , z3.If(r21s30, 10, 0) , z3.If(r21s31, 10, 0) , z3.If(r21s32, 10, 0) , z3.If(r21s33, 10, 0) , z3.If(r21s34, 60, 0) , z3.If(r21s35, 10, 0) , z3.If(r21s36, 10, 0) , z3.If(r21s37, 10, 0) , z3.If(r21s38, 10, 0) , z3.If(r21s39, 10, 0) , z3.If(r21s40, 60, 0) , z3.If(r21s41, 10, 0) , z3.If(r21s43, 10, 0) , z3.If(r21s44, 10, 0) , z3.If(r21s45, 10, 0) , z3.If(r21s46, 10, 0) , z3.If(r21s47, 10, 0) , z3.If(r21s48, 10, 0) , z3.If(r21s49, 10, 0) , z3.If(r21s50, 10, 0) , z3.If(r21s51, 10, 0) , z3.If(r21s52, 10, 0) , z3.If(r21s53, 10, 0) , z3.If(r22s0, 10, 0) , z3.If(r22s1, 10, 0) , z3.If(r22s2, 10, 0) , z3.If(r22s3, 60, 0) , z3.If(r22s4, 10, 0) , z3.If(r22s5, 10, 0) , z3.If(r22s6, 10, 0) , z3.If(r22s7, 30, 0) , z3.If(r22s8, 30, 0) , z3.If(r22s9, 10, 0) , z3.If(r22s10, 10, 0) , z3.If(r22s11, 10, 0) , z3.If(r22s12, 30, 0) , z3.If(r22s13, 10, 0) , z3.If(r22s14, 10, 0) , z3.If(r22s15, 10, 0) , z3.If(r22s16, 60, 0) , z3.If(r22s17, 30, 0) , z3.If(r22s18, 10, 0) , z3.If(r22s19, 10, 0) , z3.If(r22s20, 10, 0) , z3.If(r22s21, 10, 0) , z3.If(r22s22, 10, 0) , z3.If(r22s23, 10, 0) , z3.If(r22s24, 10, 0) , z3.If(r22s25, 10, 0) , z3.If(r22s26, 10, 0) , z3.If(r22s27, 10, 0) , z3.If(r22s28, 10, 0) , z3.If(r22s29, 10, 0) , z3.If(r22s30, 10, 0) , z3.If(r22s31, 10, 0) , z3.If(r22s32, 10, 0) , z3.If(r22s33, 10, 0) , z3.If(r22s34, 10, 0) , z3.If(r22s35, 10, 0) , z3.If(r22s36, 10, 0) , z3.If(r22s37, 10, 0) , z3.If(r22s38, 10, 0) , z3.If(r22s39, 10, 0) , z3.If(r22s40, 10, 0) , z3.If(r22s41, 10, 0) , z3.If(r22s42, 10, 0) , z3.If(r22s43, 10, 0) , z3.If(r22s44, 10, 0) , z3.If(r22s45, 10, 0) , z3.If(r22s46, 10, 0) , z3.If(r22s47, 10, 0) , z3.If(r22s48, 10, 0) , z3.If(r22s49, 10, 0) , z3.If(r22s50, 10, 0) , z3.If(r22s51, 10, 0) , z3.If(r22s52, 10, 0) , z3.If(r22s53, 10, 0) , z3.If(r23s0, 10, 0) , z3.If(r23s1, 60, 0) , z3.If(r23s2, 10, 0) , z3.If(r23s3, 30, 0) , z3.If(r23s4, 10, 0) , z3.If(r23s5, 10, 0) , z3.If(r23s6, 60, 0) , z3.If(r23s7, 10, 0) , z3.If(r23s8, 10, 0) , z3.If(r23s9, 10, 0) , z3.If(r23s10, 10, 0) , z3.If(r23s11, 10, 0) , z3.If(r23s12, 10, 0) , z3.If(r23s13, 10, 0) , z3.If(r23s14, 60, 0) , z3.If(r23s15, 10, 0) , z3.If(r23s16, 30, 0) , z3.If(r23s17, 60, 0) , z3.If(r23s19, 10, 0) , z3.If(r23s20, 10, 0) , z3.If(r23s21, 10, 0) , z3.If(r23s22, 10, 0) , z3.If(r23s23, 10, 0) , z3.If(r23s24, 10, 0) , z3.If(r23s25, 60, 0) , z3.If(r23s26, 10, 0) , z3.If(r23s27, 10, 0) , z3.If(r23s28, 10, 0) , z3.If(r23s29, 10, 0) , z3.If(r23s30, 60, 0) , z3.If(r23s31, 30, 0) , z3.If(r23s32, 10, 0) , z3.If(r23s33, 10, 0) , z3.If(r23s34, 10, 0) , z3.If(r23s35, 10, 0) , z3.If(r23s36, 10, 0) , z3.If(r23s37, 60, 0) , z3.If(r23s38, 10, 0) , z3.If(r23s39, 10, 0) , z3.If(r23s41, 10, 0) , z3.If(r23s42, 10, 0) , z3.If(r23s43, 10, 0) , z3.If(r23s44, 10, 0) , z3.If(r23s45, 10, 0) , z3.If(r23s46, 10, 0) , z3.If(r23s47, 10, 0) , z3.If(r23s48, 10, 0) , z3.If(r23s49, 10, 0) , z3.If(r23s50, 10, 0) , z3.If(r23s51, 10, 0) , z3.If(r23s52, 10, 0) , z3.If(r23s53, 10, 0) , z3.If(r24s0, 30, 0) , z3.If(r24s1, 10, 0) , z3.If(r24s2, 10, 0) , z3.If(r24s3, 10, 0) , z3.If(r24s4, 10, 0) , z3.If(r24s5, 10, 0) , z3.If(r24s6, 30, 0) , z3.If(r24s7, 10, 0) , z3.If(r24s8, 10, 0) , z3.If(r24s9, 10, 0) , z3.If(r24s10, 10, 0) , z3.If(r24s11, 10, 0) , z3.If(r24s12, 10, 0) , z3.If(r24s14, 10, 0) , z3.If(r24s15, 60, 0) , z3.If(r24s16, 30, 0) , z3.If(r24s17, 10, 0) , z3.If(r24s18, 10, 0) , z3.If(r24s19, 30, 0) , z3.If(r24s20, 60, 0) , z3.If(r24s21, 10, 0) , z3.If(r24s22, 60, 0) , z3.If(r24s23, 10, 0) , z3.If(r24s24, 10, 0) , z3.If(r24s25, 10, 0) , z3.If(r24s26, 10, 0) , z3.If(r24s27, 30, 0) , z3.If(r24s28, 10, 0) , z3.If(r24s29, 10, 0) , z3.If(r24s30, 10, 0) , z3.If(r24s31, 10, 0) , z3.If(r24s32, 10, 0) , z3.If(r24s33, 10, 0) , z3.If(r24s34, 10, 0) , z3.If(r24s35, 10, 0) , z3.If(r24s36, 10, 0) , z3.If(r24s37, 10, 0) , z3.If(r24s38, 10, 0) , z3.If(r24s39, 10, 0) , z3.If(r24s40, 10, 0) , z3.If(r24s42, 10, 0) , z3.If(r24s44, 10, 0) , z3.If(r24s45, 10, 0) , z3.If(r24s46, 30, 0) , z3.If(r24s47, 10, 0) , z3.If(r24s48, 10, 0) , z3.If(r24s50, 10, 0) , z3.If(r24s52, 10, 0) , z3.If(r24s53, 10, 0) , z3.If(r25s0, 10, 0) , z3.If(r25s1, 10, 0) , z3.If(r25s2, 30, 0) , z3.If(r25s3, 10, 0) , z3.If(r25s4, 10, 0) , z3.If(r25s5, 10, 0) , z3.If(r25s6, 60, 0) , z3.If(r25s7, 30, 0) , z3.If(r25s8, 10, 0) , z3.If(r25s9, 30, 0) , z3.If(r25s10, 10, 0) , z3.If(r25s11, 10, 0) , z3.If(r25s13, 10, 0) , z3.If(r25s14, 10, 0) , z3.If(r25s15, 10, 0) , z3.If(r25s16, 30, 0) , z3.If(r25s17, 60, 0) , z3.If(r25s19, 10, 0) , z3.If(r25s20, 10, 0) , z3.If(r25s21, 10, 0) , z3.If(r25s22, 10, 0) , z3.If(r25s23, 10, 0) , z3.If(r25s24, 10, 0) , z3.If(r25s25, 10, 0) , z3.If(r25s26, 10, 0) , z3.If(r25s27, 60, 0) , z3.If(r25s28, 10, 0) , z3.If(r25s29, 10, 0) , z3.If(r25s30, 30, 0) , z3.If(r25s31, 30, 0) , z3.If(r25s32, 30, 0) , z3.If(r25s33, 10, 0) , z3.If(r25s34, 10, 0) , z3.If(r25s35, 10, 0) , z3.If(r25s36, 30, 0) , z3.If(r25s37, 10, 0) , z3.If(r25s38, 10, 0) , z3.If(r25s39, 10, 0) , z3.If(r25s40, 30, 0) , z3.If(r25s41, 10, 0) , z3.If(r25s42, 30, 0) , z3.If(r25s43, 30, 0) , z3.If(r25s44, 10, 0) , z3.If(r25s45, 10, 0) , z3.If(r25s46, 10, 0) , z3.If(r25s47, 30, 0) , z3.If(r25s48, 10, 0) , z3.If(r25s49, 10, 0) , z3.If(r25s50, 10, 0) , z3.If(r25s51, 10, 0) , z3.If(r25s52, 10, 0) , z3.If(r25s53, 10, 0) , z3.If(r26s0, 60, 0) , z3.If(r26s1, 30, 0) , z3.If(r26s2, 10, 0) , z3.If(r26s3, 60, 0) , z3.If(r26s4, 60, 0) , z3.If(r26s5, 10, 0) , z3.If(r26s6, 10, 0) , z3.If(r26s7, 10, 0) , z3.If(r26s8, 10, 0) , z3.If(r26s9, 10, 0) , z3.If(r26s10, 10, 0) , z3.If(r26s11, 60, 0) , z3.If(r26s12, 10, 0) , z3.If(r26s13, 10, 0) , z3.If(r26s14, 10, 0) , z3.If(r26s15, 10, 0) , z3.If(r26s16, 10, 0) , z3.If(r26s17, 10, 0) , z3.If(r26s18, 10, 0) , z3.If(r26s20, 60, 0) , z3.If(r26s21, 60, 0) , z3.If(r26s22, 10, 0) , z3.If(r26s23, 10, 0) , z3.If(r26s24, 10, 0) , z3.If(r26s25, 10, 0) , z3.If(r26s26, 10, 0) , z3.If(r26s27, 10, 0) , z3.If(r26s28, 10, 0) , z3.If(r26s29, 30, 0) , z3.If(r26s30, 10, 0) , z3.If(r26s31, 10, 0) , z3.If(r26s32, 10, 0) , z3.If(r26s33, 10, 0) , z3.If(r26s34, 10, 0) , z3.If(r26s35, 10, 0) , z3.If(r26s36, 10, 0) , z3.If(r26s37, 10, 0) , z3.If(r26s38, 10, 0) , z3.If(r26s39, 10, 0) , z3.If(r26s40, 10, 0) , z3.If(r26s41, 10, 0) , z3.If(r26s42, 10, 0) , z3.If(r26s43, 60, 0) , z3.If(r26s44, 10, 0) , z3.If(r26s45, 10, 0) , z3.If(r26s46, 10, 0) , z3.If(r26s47, 60, 0) , z3.If(r26s48, 10, 0) , z3.If(r26s49, 60, 0) , z3.If(r26s50, 10, 0) , z3.If(r26s51, 60, 0) , z3.If(r26s52, 10, 0) , z3.If(r26s53, 10, 0) , z3.If(r27s0, 10, 0) , z3.If(r27s1, 60, 0) , z3.If(r27s2, 10, 0) , z3.If(r27s3, 10, 0) , z3.If(r27s4, 10, 0) , z3.If(r27s5, 10, 0) , z3.If(r27s6, 30, 0) , z3.If(r27s7, 10, 0) , z3.If(r27s8, 10, 0) , z3.If(r27s9, 60, 0) , z3.If(r27s10, 10, 0) , z3.If(r27s11, 10, 0) , z3.If(r27s12, 10, 0) , z3.If(r27s13, 10, 0) , z3.If(r27s14, 60, 0) , z3.If(r27s15, 10, 0) , z3.If(r27s16, 10, 0) , z3.If(r27s17, 10, 0) , z3.If(r27s18, 10, 0) , z3.If(r27s19, 10, 0) , z3.If(r27s20, 10, 0) , z3.If(r27s21, 10, 0) , z3.If(r27s22, 10, 0) , z3.If(r27s23, 10, 0) , z3.If(r27s24, 30, 0) , z3.If(r27s25, 10, 0) , z3.If(r27s26, 10, 0) , z3.If(r27s27, 10, 0) , z3.If(r27s28, 10, 0) , z3.If(r27s29, 10, 0) , z3.If(r27s30, 10, 0) , z3.If(r27s31, 10, 0) , z3.If(r27s32, 10, 0) , z3.If(r27s33, 10, 0) , z3.If(r27s34, 10, 0) , z3.If(r27s35, 10, 0) , z3.If(r27s36, 10, 0) , z3.If(r27s37, 60, 0) , z3.If(r27s38, 10, 0) , z3.If(r27s39, 30, 0) , z3.If(r27s40, 10, 0) , z3.If(r27s41, 10, 0) , z3.If(r27s42, 60, 0) , z3.If(r27s43, 30, 0) , z3.If(r27s44, 10, 0) , z3.If(r27s45, 10, 0) , z3.If(r27s46, 10, 0) , z3.If(r27s47, 10, 0) , z3.If(r27s48, 10, 0) , z3.If(r27s49, 10, 0) , z3.If(r27s50, 60, 0) , z3.If(r27s51, 10, 0) , z3.If(r27s52, 10, 0) , z3.If(r27s53, 10, 0) , z3.If(r28s0, 10, 0) , z3.If(r28s1, 30, 0) , z3.If(r28s2, 10, 0) , z3.If(r28s3, 10, 0) , z3.If(r28s4, 10, 0) , z3.If(r28s5, 10, 0) , z3.If(r28s6, 30, 0) , z3.If(r28s7, 10, 0) , z3.If(r28s8, 10, 0) , z3.If(r28s9, 30, 0) , z3.If(r28s10, 10, 0) , z3.If(r28s11, 10, 0) , z3.If(r28s12, 30, 0) , z3.If(r28s13, 60, 0) , z3.If(r28s14, 10, 0) , z3.If(r28s15, 10, 0) , z3.If(r28s16, 10, 0) , z3.If(r28s17, 30, 0) , z3.If(r28s18, 60, 0) , z3.If(r28s19, 10, 0) , z3.If(r28s20, 10, 0) , z3.If(r28s21, 10, 0) , z3.If(r28s23, 10, 0) , z3.If(r28s24, 60, 0) , z3.If(r28s25, 60, 0) , z3.If(r28s26, 10, 0) , z3.If(r28s29, 10, 0) , z3.If(r28s30, 30, 0) , z3.If(r28s32, 10, 0) , z3.If(r28s33, 10, 0) , z3.If(r28s35, 10, 0) , z3.If(r28s36, 10, 0) , z3.If(r28s37, 10, 0) , z3.If(r28s38, 10, 0) , z3.If(r28s39, 10, 0) , z3.If(r28s40, 60, 0) , z3.If(r28s41, 10, 0) , z3.If(r28s42, 30, 0) , z3.If(r28s43, 10, 0) , z3.If(r28s44, 10, 0) , z3.If(r28s45, 10, 0) , z3.If(r28s46, 10, 0) , z3.If(r28s47, 10, 0) , z3.If(r28s48, 10, 0) , z3.If(r28s49, 10, 0) , z3.If(r28s50, 10, 0) , z3.If(r28s51, 10, 0) , z3.If(r28s52, 10, 0) , z3.If(r28s53, 10, 0) , z3.If(r29s0, 10, 0) , z3.If(r29s1, 10, 0) , z3.If(r29s2, 10, 0) , z3.If(r29s3, 10, 0) , z3.If(r29s4, 10, 0) , z3.If(r29s5, 10, 0) , z3.If(r29s6, 60, 0) , z3.If(r29s7, 10, 0) , z3.If(r29s8, 10, 0) , z3.If(r29s9, 10, 0) , z3.If(r29s10, 10, 0) , z3.If(r29s11, 10, 0) , z3.If(r29s12, 60, 0) , z3.If(r29s13, 10, 0) , z3.If(r29s14, 10, 0) , z3.If(r29s15, 10, 0) , z3.If(r29s16, 60, 0) , z3.If(r29s17, 60, 0) , z3.If(r29s18, 10, 0) , z3.If(r29s19, 10, 0) , z3.If(r29s20, 10, 0) , z3.If(r29s21, 10, 0) , z3.If(r29s23, 10, 0) , z3.If(r29s24, 60, 0) , z3.If(r29s25, 60, 0) , z3.If(r29s26, 10, 0) , z3.If(r29s29, 10, 0) , z3.If(r29s30, 60, 0) , z3.If(r29s32, 60, 0) , z3.If(r29s33, 10, 0) , z3.If(r29s35, 10, 0) , z3.If(r29s36, 60, 0) , z3.If(r29s37, 10, 0) , z3.If(r29s38, 10, 0) , z3.If(r29s39, 10, 0) , z3.If(r29s40, 60, 0) , z3.If(r29s41, 10, 0) , z3.If(r29s42, 60, 0) , z3.If(r29s43, 10, 0) , z3.If(r29s44, 10, 0) , z3.If(r29s45, 10, 0) , z3.If(r29s46, 10, 0) , z3.If(r29s47, 10, 0) , z3.If(r29s48, 10, 0) , z3.If(r29s49, 10, 0) , z3.If(r29s50, 10, 0) , z3.If(r29s51, 10, 0) , z3.If(r29s52, 10, 0) , z3.If(r29s53, 10, 0) , z3.If(r30s0, 10, 0) , z3.If(r30s1, 10, 0) , z3.If(r30s2, 10, 0) , z3.If(r30s3, 60, 0) , z3.If(r30s4, 10, 0) , z3.If(r30s5, 30, 0) , z3.If(r30s6, 60, 0) , z3.If(r30s7, 10, 0) , z3.If(r30s8, 10, 0) , z3.If(r30s9, 10, 0) , z3.If(r30s10, 10, 0) , z3.If(r30s11, 30, 0) , z3.If(r30s12, 10, 0) , z3.If(r30s13, 60, 0) , z3.If(r30s14, 10, 0) , z3.If(r30s15, 60, 0) , z3.If(r30s16, 30, 0) , z3.If(r30s17, 10, 0) , z3.If(r30s18, 10, 0) , z3.If(r30s19, 10, 0) , z3.If(r30s20, 10, 0) , z3.If(r30s21, 30, 0) , z3.If(r30s22, 10, 0) , z3.If(r30s23, 10, 0) , z3.If(r30s24, 10, 0) , z3.If(r30s25, 60, 0) , z3.If(r30s26, 10, 0) , z3.If(r30s27, 10, 0) , z3.If(r30s28, 10, 0) , z3.If(r30s29, 30, 0) , z3.If(r30s30, 10, 0) , z3.If(r30s31, 10, 0) , z3.If(r30s32, 30, 0) , z3.If(r30s33, 10, 0) , z3.If(r30s34, 10, 0) , z3.If(r30s35, 30, 0) , z3.If(r30s37, 10, 0) , z3.If(r30s38, 10, 0) , z3.If(r30s39, 10, 0) , z3.If(r30s40, 10, 0) , z3.If(r30s41, 10, 0) , z3.If(r30s42, 10, 0) , z3.If(r30s43, 10, 0) , z3.If(r30s44, 10, 0) , z3.If(r30s45, 30, 0) , z3.If(r30s46, 30, 0) , z3.If(r30s47, 10, 0) , z3.If(r30s48, 10, 0) , z3.If(r30s49, 60, 0) , z3.If(r30s50, 10, 0) , z3.If(r30s51, 10, 0) , z3.If(r30s52, 10, 0) , z3.If(r30s53, 10, 0)]

# 95/4/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 95, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 4, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 95, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 4, 0) , z3.If(r0s17, 4, 0) , z3.If(r0s18, 4, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 95, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 95, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 95, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 4, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 4, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 4, 0) , z3.If(r0s43, 4, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 4, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 4, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 95, 0) , z3.If(r1s7, 4, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 95, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 4, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 95, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 4, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 95, 0) , z3.If(r1s25, 4, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 95, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 4, 0) , z3.If(r1s31, 4, 0) , z3.If(r1s32, 95, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 4, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 95, 0) , z3.If(r1s43, 4, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 95, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 4, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 4, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 95, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 95, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 95, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 4, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 4, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 4, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 95, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 95, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 95, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 95, 0) , z3.If(r2s43, 4, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 95, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 95, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 95, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 95, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 95, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 95, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 4, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 95, 0) , z3.If(r4s13, 95, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 95, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 95, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 95, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 4, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 4, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 95, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 4, 0) , z3.If(r4s46, 4, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 95, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 95, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 4, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 4, 0) , z3.If(r5s25, 4, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 4, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 4, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 4, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 4, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 95, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 4, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 4, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 4, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 95, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 95, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 4, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 95, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 4, 0) , z3.If(r6s50, 95, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 4, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 4, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 95, 0) , z3.If(r7s13, 4, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 95, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 95, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 4, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 4, 0) , z3.If(r7s31, 95, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 95, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 95, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 95, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 95, 0) , z3.If(r8s4, 95, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 95, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 95, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 95, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 95, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 95, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 4, 0) , z3.If(r9s6, 95, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 95, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 4, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 4, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 4, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 4, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 4, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 4, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 95, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 4, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 4, 0) , z3.If(r10s7, 4, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 4, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 4, 0) , z3.If(r10s13, 4, 0) , z3.If(r10s14, 4, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 4, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 4, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 4, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 4, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 4, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 4, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 4, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 95, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 4, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 4, 0) , z3.If(r11s14, 95, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 95, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 95, 0) , z3.If(r11s42, 95, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 4, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 4, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 4, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 4, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 4, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 4, 0) , z3.If(r12s17, 4, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 4, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 4, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 95, 0) , z3.If(r13s1, 95, 0) , z3.If(r13s2, 95, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 4, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 95, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 95, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 4, 0) , z3.If(r13s18, 95, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 95, 0) , z3.If(r13s25, 95, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 95, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 95, 0) , z3.If(r13s40, 4, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 4, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 95, 0) , z3.If(r14s7, 4, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 4, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 95, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 4, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 95, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 4, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 4, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 95, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 4, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 4, 0) , z3.If(r15s25, 4, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 95, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 4, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 4, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 4, 0) , z3.If(r16s1, 4, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 4, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 4, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 4, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 95, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 4, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 4, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 4, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 95, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 95, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 95, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 95, 0) , z3.If(r17s10, 95, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 95, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 95, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 95, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 95, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 95, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 4, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 4, 0) , z3.If(r18s18, 4, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 95, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 4, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 95, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 4, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 95, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 4, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 4, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 95, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 95, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 95, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 95, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 95, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 95, 0) , z3.If(r20s18, 95, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 95, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 95, 0) , z3.If(r20s25, 95, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 95, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 95, 0) , z3.If(r20s31, 95, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 95, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 95, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 4, 0) , z3.If(r21s7, 4, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 95, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 4, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 4, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 4, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 95, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 4, 0) , z3.If(r21s28, 95, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 95, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 95, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 95, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 4, 0) , z3.If(r22s8, 4, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 4, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 95, 0) , z3.If(r22s17, 4, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 95, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 4, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 95, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 95, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 4, 0) , z3.If(r23s17, 95, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 95, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 95, 0) , z3.If(r23s31, 4, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 95, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 4, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 4, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 95, 0) , z3.If(r24s16, 4, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 4, 0) , z3.If(r24s20, 95, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 95, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 4, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 4, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 4, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 95, 0) , z3.If(r25s7, 4, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 4, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 4, 0) , z3.If(r25s17, 95, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 95, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 4, 0) , z3.If(r25s31, 4, 0) , z3.If(r25s32, 4, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 4, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 4, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 4, 0) , z3.If(r25s43, 4, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 4, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 95, 0) , z3.If(r26s1, 4, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 95, 0) , z3.If(r26s4, 95, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 95, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 95, 0) , z3.If(r26s21, 95, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 4, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 95, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 95, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 95, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 95, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 95, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 4, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 95, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 95, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 4, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 95, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 4, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 95, 0) , z3.If(r27s43, 4, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 95, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 4, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 4, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 4, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 4, 0) , z3.If(r28s13, 95, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 4, 0) , z3.If(r28s18, 95, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 95, 0) , z3.If(r28s25, 95, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 4, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 95, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 4, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 95, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 95, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 95, 0) , z3.If(r29s17, 95, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 95, 0) , z3.If(r29s25, 95, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 95, 0) , z3.If(r29s32, 95, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 95, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 95, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 95, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 95, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 4, 0) , z3.If(r30s6, 95, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 4, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 95, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 95, 0) , z3.If(r30s16, 4, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 4, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 95, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 4, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 4, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 4, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 4, 0) , z3.If(r30s46, 4, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 95, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]
# 90/9/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 90, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 9, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 90, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 9, 0) , z3.If(r0s17, 9, 0) , z3.If(r0s18, 9, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 90, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 90, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 90, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 9, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 9, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 9, 0) , z3.If(r0s43, 9, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 9, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 9, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 90, 0) , z3.If(r1s7, 9, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 90, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 9, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 90, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 9, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 90, 0) , z3.If(r1s25, 9, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 90, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 9, 0) , z3.If(r1s31, 9, 0) , z3.If(r1s32, 90, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 9, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 90, 0) , z3.If(r1s43, 9, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 90, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 9, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 9, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 90, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 90, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 90, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 9, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 9, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 9, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 90, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 90, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 90, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 90, 0) , z3.If(r2s43, 9, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 90, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 90, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 90, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 90, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 90, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 90, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 9, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 90, 0) , z3.If(r4s13, 90, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 90, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 90, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 90, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 9, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 9, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 90, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 9, 0) , z3.If(r4s46, 9, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 90, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 90, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 9, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 9, 0) , z3.If(r5s25, 9, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 9, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 9, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 9, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 9, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 90, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 9, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 9, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 9, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 90, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 90, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 9, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 90, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 9, 0) , z3.If(r6s50, 90, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 9, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 9, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 90, 0) , z3.If(r7s13, 9, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 90, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 90, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 9, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 9, 0) , z3.If(r7s31, 90, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 90, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 90, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 90, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 90, 0) , z3.If(r8s4, 90, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 90, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 90, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 90, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 90, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 90, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 9, 0) , z3.If(r9s6, 90, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 90, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 9, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 9, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 9, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 9, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 9, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 9, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 90, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 9, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 9, 0) , z3.If(r10s7, 9, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 9, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 9, 0) , z3.If(r10s13, 9, 0) , z3.If(r10s14, 9, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 9, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 9, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 9, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 9, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 9, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 9, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 9, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 90, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 9, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 9, 0) , z3.If(r11s14, 90, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 90, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 90, 0) , z3.If(r11s42, 90, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 9, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 9, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 9, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 9, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 9, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 9, 0) , z3.If(r12s17, 9, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 9, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 9, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 90, 0) , z3.If(r13s1, 90, 0) , z3.If(r13s2, 90, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 9, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 90, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 90, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 9, 0) , z3.If(r13s18, 90, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 90, 0) , z3.If(r13s25, 90, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 90, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 90, 0) , z3.If(r13s40, 9, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 9, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 90, 0) , z3.If(r14s7, 9, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 9, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 90, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 9, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 90, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 9, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 9, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 90, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 9, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 9, 0) , z3.If(r15s25, 9, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 90, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 9, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 9, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 9, 0) , z3.If(r16s1, 9, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 9, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 9, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 9, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 90, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 9, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 9, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 9, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 90, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 90, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 90, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 90, 0) , z3.If(r17s10, 90, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 90, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 90, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 90, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 90, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 90, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 9, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 9, 0) , z3.If(r18s18, 9, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 90, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 9, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 90, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 9, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 90, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 9, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 9, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 90, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 90, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 90, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 90, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 90, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 90, 0) , z3.If(r20s18, 90, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 90, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 90, 0) , z3.If(r20s25, 90, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 90, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 90, 0) , z3.If(r20s31, 90, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 90, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 90, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 9, 0) , z3.If(r21s7, 9, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 90, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 9, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 9, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 9, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 90, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 9, 0) , z3.If(r21s28, 90, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 90, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 90, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 90, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 9, 0) , z3.If(r22s8, 9, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 9, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 90, 0) , z3.If(r22s17, 9, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 90, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 9, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 90, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 90, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 9, 0) , z3.If(r23s17, 90, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 90, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 90, 0) , z3.If(r23s31, 9, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 90, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 9, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 9, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 90, 0) , z3.If(r24s16, 9, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 9, 0) , z3.If(r24s20, 90, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 90, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 9, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 9, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 9, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 90, 0) , z3.If(r25s7, 9, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 9, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 9, 0) , z3.If(r25s17, 90, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 90, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 9, 0) , z3.If(r25s31, 9, 0) , z3.If(r25s32, 9, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 9, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 9, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 9, 0) , z3.If(r25s43, 9, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 9, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 90, 0) , z3.If(r26s1, 9, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 90, 0) , z3.If(r26s4, 90, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 90, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 90, 0) , z3.If(r26s21, 90, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 9, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 90, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 90, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 90, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 90, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 90, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 9, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 90, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 90, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 9, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 90, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 9, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 90, 0) , z3.If(r27s43, 9, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 90, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 9, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 9, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 9, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 9, 0) , z3.If(r28s13, 90, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 9, 0) , z3.If(r28s18, 90, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 90, 0) , z3.If(r28s25, 90, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 9, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 90, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 9, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 90, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 90, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 90, 0) , z3.If(r29s17, 90, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 90, 0) , z3.If(r29s25, 90, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 90, 0) , z3.If(r29s32, 90, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 90, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 90, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 90, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 90, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 9, 0) , z3.If(r30s6, 90, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 9, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 90, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 90, 0) , z3.If(r30s16, 9, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 9, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 90, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 9, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 9, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 9, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 9, 0) , z3.If(r30s46, 9, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 90, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]
# 70/20/10
# scores = [z3.If(r0s0, 10, 0) , z3.If(r0s1, 10, 0) , z3.If(r0s2, 10, 0) , z3.If(r0s4, 10, 0) , z3.If(r0s5, 10, 0) , z3.If(r0s6, 70, 0) , z3.If(r0s7, 10, 0) , z3.If(r0s8, 10, 0) , z3.If(r0s9, 20, 0) , z3.If(r0s10, 10, 0) , z3.If(r0s11, 10, 0) , z3.If(r0s12, 10, 0) , z3.If(r0s13, 70, 0) , z3.If(r0s14, 10, 0) , z3.If(r0s15, 10, 0) , z3.If(r0s16, 20, 0) , z3.If(r0s17, 20, 0) , z3.If(r0s18, 20, 0) , z3.If(r0s19, 10, 0) , z3.If(r0s20, 10, 0) , z3.If(r0s21, 10, 0) , z3.If(r0s22, 70, 0) , z3.If(r0s23, 10, 0) , z3.If(r0s24, 70, 0) , z3.If(r0s25, 10, 0) , z3.If(r0s26, 10, 0) , z3.If(r0s27, 70, 0) , z3.If(r0s28, 10, 0) , z3.If(r0s29, 10, 0) , z3.If(r0s30, 10, 0) , z3.If(r0s31, 10, 0) , z3.If(r0s32, 20, 0) , z3.If(r0s33, 10, 0) , z3.If(r0s34, 10, 0) , z3.If(r0s35, 10, 0) , z3.If(r0s36, 20, 0) , z3.If(r0s37, 10, 0) , z3.If(r0s38, 10, 0) , z3.If(r0s39, 10, 0) , z3.If(r0s40, 10, 0) , z3.If(r0s41, 10, 0) , z3.If(r0s42, 20, 0) , z3.If(r0s43, 20, 0) , z3.If(r0s44, 10, 0) , z3.If(r0s45, 10, 0) , z3.If(r0s46, 10, 0) , z3.If(r0s47, 20, 0) , z3.If(r0s48, 10, 0) , z3.If(r0s49, 10, 0) , z3.If(r0s51, 20, 0) , z3.If(r0s52, 10, 0) , z3.If(r0s53, 10, 0) , z3.If(r1s0, 10, 0) , z3.If(r1s1, 10, 0) , z3.If(r1s2, 10, 0) , z3.If(r1s3, 10, 0) , z3.If(r1s4, 10, 0) , z3.If(r1s5, 10, 0) , z3.If(r1s6, 70, 0) , z3.If(r1s7, 20, 0) , z3.If(r1s8, 10, 0) , z3.If(r1s9, 10, 0) , z3.If(r1s10, 10, 0) , z3.If(r1s11, 10, 0) , z3.If(r1s12, 70, 0) , z3.If(r1s13, 10, 0) , z3.If(r1s14, 20, 0) , z3.If(r1s15, 10, 0) , z3.If(r1s16, 10, 0) , z3.If(r1s17, 70, 0) , z3.If(r1s19, 10, 0) , z3.If(r1s20, 10, 0) , z3.If(r1s21, 10, 0) , z3.If(r1s22, 20, 0) , z3.If(r1s23, 10, 0) , z3.If(r1s24, 70, 0) , z3.If(r1s25, 20, 0) , z3.If(r1s26, 10, 0) , z3.If(r1s27, 70, 0) , z3.If(r1s28, 10, 0) , z3.If(r1s29, 10, 0) , z3.If(r1s30, 20, 0) , z3.If(r1s31, 20, 0) , z3.If(r1s32, 70, 0) , z3.If(r1s33, 10, 0) , z3.If(r1s34, 10, 0) , z3.If(r1s35, 10, 0) , z3.If(r1s36, 20, 0) , z3.If(r1s37, 10, 0) , z3.If(r1s38, 10, 0) , z3.If(r1s39, 10, 0) , z3.If(r1s41, 10, 0) , z3.If(r1s42, 70, 0) , z3.If(r1s43, 20, 0) , z3.If(r1s44, 10, 0) , z3.If(r1s45, 10, 0) , z3.If(r1s46, 10, 0) , z3.If(r1s47, 70, 0) , z3.If(r1s48, 10, 0) , z3.If(r1s49, 20, 0) , z3.If(r1s50, 10, 0) , z3.If(r1s51, 10, 0) , z3.If(r1s52, 10, 0) , z3.If(r1s53, 10, 0) , z3.If(r2s0, 10, 0) , z3.If(r2s1, 20, 0) , z3.If(r2s2, 10, 0) , z3.If(r2s3, 70, 0) , z3.If(r2s4, 10, 0) , z3.If(r2s5, 70, 0) , z3.If(r2s6, 10, 0) , z3.If(r2s7, 70, 0) , z3.If(r2s8, 10, 0) , z3.If(r2s9, 10, 0) , z3.If(r2s10, 10, 0) , z3.If(r2s11, 10, 0) , z3.If(r2s12, 20, 0) , z3.If(r2s13, 10, 0) , z3.If(r2s14, 20, 0) , z3.If(r2s15, 10, 0) , z3.If(r2s16, 10, 0) , z3.If(r2s17, 20, 0) , z3.If(r2s18, 10, 0) , z3.If(r2s19, 10, 0) , z3.If(r2s20, 10, 0) , z3.If(r2s21, 10, 0) , z3.If(r2s22, 10, 0) , z3.If(r2s23, 10, 0) , z3.If(r2s24, 70, 0) , z3.If(r2s25, 10, 0) , z3.If(r2s26, 10, 0) , z3.If(r2s27, 70, 0) , z3.If(r2s28, 10, 0) , z3.If(r2s29, 10, 0) , z3.If(r2s30, 10, 0) , z3.If(r2s31, 10, 0) , z3.If(r2s32, 10, 0) , z3.If(r2s33, 10, 0) , z3.If(r2s34, 10, 0) , z3.If(r2s35, 10, 0) , z3.If(r2s36, 10, 0) , z3.If(r2s37, 10, 0) , z3.If(r2s38, 10, 0) , z3.If(r2s39, 10, 0) , z3.If(r2s40, 70, 0) , z3.If(r2s41, 10, 0) , z3.If(r2s42, 70, 0) , z3.If(r2s43, 20, 0) , z3.If(r2s44, 10, 0) , z3.If(r2s45, 10, 0) , z3.If(r2s46, 10, 0) , z3.If(r2s47, 10, 0) , z3.If(r2s48, 10, 0) , z3.If(r2s49, 10, 0) , z3.If(r2s50, 10, 0) , z3.If(r2s51, 10, 0) , z3.If(r2s52, 10, 0) , z3.If(r2s53, 10, 0) , z3.If(r3s0, 70, 0) , z3.If(r3s1, 10, 0) , z3.If(r3s2, 10, 0) , z3.If(r3s3, 10, 0) , z3.If(r3s4, 10, 0) , z3.If(r3s5, 10, 0) , z3.If(r3s6, 70, 0) , z3.If(r3s7, 10, 0) , z3.If(r3s8, 10, 0) , z3.If(r3s9, 10, 0) , z3.If(r3s10, 10, 0) , z3.If(r3s11, 10, 0) , z3.If(r3s12, 10, 0) , z3.If(r3s13, 70, 0) , z3.If(r3s14, 10, 0) , z3.If(r3s15, 10, 0) , z3.If(r3s16, 10, 0) , z3.If(r3s17, 10, 0) , z3.If(r3s18, 10, 0) , z3.If(r3s19, 10, 0) , z3.If(r3s20, 10, 0) , z3.If(r3s21, 10, 0) , z3.If(r3s22, 10, 0) , z3.If(r3s23, 10, 0) , z3.If(r3s24, 10, 0) , z3.If(r3s25, 10, 0) , z3.If(r3s26, 10, 0) , z3.If(r3s27, 10, 0) , z3.If(r3s28, 10, 0) , z3.If(r3s29, 10, 0) , z3.If(r3s30, 10, 0) , z3.If(r3s31, 10, 0) , z3.If(r3s32, 10, 0) , z3.If(r3s33, 10, 0) , z3.If(r3s34, 10, 0) , z3.If(r3s35, 10, 0) , z3.If(r3s36, 10, 0) , z3.If(r3s37, 70, 0) , z3.If(r3s38, 10, 0) , z3.If(r3s39, 10, 0) , z3.If(r3s40, 70, 0) , z3.If(r3s41, 10, 0) , z3.If(r3s42, 10, 0) , z3.If(r3s43, 10, 0) , z3.If(r3s44, 10, 0) , z3.If(r3s45, 10, 0) , z3.If(r3s46, 10, 0) , z3.If(r3s47, 10, 0) , z3.If(r3s48, 10, 0) , z3.If(r3s49, 10, 0) , z3.If(r3s50, 10, 0) , z3.If(r3s51, 70, 0) , z3.If(r3s52, 10, 0) , z3.If(r3s53, 10, 0) , z3.If(r4s0, 10, 0) , z3.If(r4s1, 10, 0) , z3.If(r4s2, 10, 0) , z3.If(r4s3, 10, 0) , z3.If(r4s4, 10, 0) , z3.If(r4s5, 10, 0) , z3.If(r4s6, 20, 0) , z3.If(r4s7, 10, 0) , z3.If(r4s8, 10, 0) , z3.If(r4s10, 10, 0) , z3.If(r4s11, 10, 0) , z3.If(r4s12, 70, 0) , z3.If(r4s13, 70, 0) , z3.If(r4s14, 10, 0) , z3.If(r4s15, 10, 0) , z3.If(r4s16, 10, 0) , z3.If(r4s17, 70, 0) , z3.If(r4s19, 10, 0) , z3.If(r4s20, 10, 0) , z3.If(r4s21, 10, 0) , z3.If(r4s22, 10, 0) , z3.If(r4s23, 10, 0) , z3.If(r4s24, 70, 0) , z3.If(r4s26, 10, 0) , z3.If(r4s27, 70, 0) , z3.If(r4s28, 10, 0) , z3.If(r4s29, 10, 0) , z3.If(r4s30, 10, 0) , z3.If(r4s31, 10, 0) , z3.If(r4s32, 10, 0) , z3.If(r4s33, 10, 0) , z3.If(r4s34, 10, 0) , z3.If(r4s35, 10, 0) , z3.If(r4s36, 20, 0) , z3.If(r4s37, 10, 0) , z3.If(r4s38, 10, 0) , z3.If(r4s39, 10, 0) , z3.If(r4s40, 20, 0) , z3.If(r4s41, 10, 0) , z3.If(r4s42, 70, 0) , z3.If(r4s43, 10, 0) , z3.If(r4s44, 10, 0) , z3.If(r4s45, 20, 0) , z3.If(r4s46, 20, 0) , z3.If(r4s47, 10, 0) , z3.If(r4s48, 10, 0) , z3.If(r4s49, 10, 0) , z3.If(r4s50, 10, 0) , z3.If(r4s51, 10, 0) , z3.If(r4s52, 10, 0) , z3.If(r4s53, 10, 0) , z3.If(r5s0, 10, 0) , z3.If(r5s1, 10, 0) , z3.If(r5s2, 10, 0) , z3.If(r5s3, 10, 0) , z3.If(r5s4, 10, 0) , z3.If(r5s5, 10, 0) , z3.If(r5s6, 70, 0) , z3.If(r5s7, 10, 0) , z3.If(r5s8, 10, 0) , z3.If(r5s9, 10, 0) , z3.If(r5s10, 10, 0) , z3.If(r5s11, 10, 0) , z3.If(r5s12, 10, 0) , z3.If(r5s13, 70, 0) , z3.If(r5s14, 10, 0) , z3.If(r5s15, 10, 0) , z3.If(r5s16, 10, 0) , z3.If(r5s17, 20, 0) , z3.If(r5s18, 10, 0) , z3.If(r5s19, 10, 0) , z3.If(r5s20, 10, 0) , z3.If(r5s21, 10, 0) , z3.If(r5s22, 10, 0) , z3.If(r5s23, 10, 0) , z3.If(r5s24, 20, 0) , z3.If(r5s25, 20, 0) , z3.If(r5s26, 10, 0) , z3.If(r5s27, 20, 0) , z3.If(r5s28, 10, 0) , z3.If(r5s29, 10, 0) , z3.If(r5s30, 20, 0) , z3.If(r5s31, 10, 0) , z3.If(r5s32, 10, 0) , z3.If(r5s33, 10, 0) , z3.If(r5s34, 10, 0) , z3.If(r5s35, 10, 0) , z3.If(r5s36, 20, 0) , z3.If(r5s37, 10, 0) , z3.If(r5s38, 10, 0) , z3.If(r5s39, 10, 0) , z3.If(r5s40, 20, 0) , z3.If(r5s41, 10, 0) , z3.If(r5s42, 70, 0) , z3.If(r5s43, 10, 0) , z3.If(r5s44, 10, 0) , z3.If(r5s45, 10, 0) , z3.If(r5s46, 10, 0) , z3.If(r5s47, 20, 0) , z3.If(r5s48, 10, 0) , z3.If(r5s49, 10, 0) , z3.If(r5s50, 10, 0) , z3.If(r5s51, 10, 0) , z3.If(r5s52, 10, 0) , z3.If(r5s53, 10, 0) , z3.If(r6s0, 10, 0) , z3.If(r6s1, 10, 0) , z3.If(r6s2, 10, 0) , z3.If(r6s3, 10, 0) , z3.If(r6s4, 10, 0) , z3.If(r6s5, 10, 0) , z3.If(r6s6, 20, 0) , z3.If(r6s7, 10, 0) , z3.If(r6s8, 10, 0) , z3.If(r6s9, 10, 0) , z3.If(r6s10, 10, 0) , z3.If(r6s11, 10, 0) , z3.If(r6s12, 20, 0) , z3.If(r6s13, 10, 0) , z3.If(r6s14, 10, 0) , z3.If(r6s15, 10, 0) , z3.If(r6s16, 10, 0) , z3.If(r6s17, 10, 0) , z3.If(r6s18, 10, 0) , z3.If(r6s19, 10, 0) , z3.If(r6s20, 10, 0) , z3.If(r6s21, 10, 0) , z3.If(r6s22, 10, 0) , z3.If(r6s23, 70, 0) , z3.If(r6s24, 10, 0) , z3.If(r6s25, 10, 0) , z3.If(r6s26, 10, 0) , z3.If(r6s27, 10, 0) , z3.If(r6s28, 10, 0) , z3.If(r6s29, 10, 0) , z3.If(r6s30, 10, 0) , z3.If(r6s31, 10, 0) , z3.If(r6s32, 10, 0) , z3.If(r6s33, 10, 0) , z3.If(r6s34, 10, 0) , z3.If(r6s35, 10, 0) , z3.If(r6s36, 70, 0) , z3.If(r6s37, 10, 0) , z3.If(r6s38, 10, 0) , z3.If(r6s39, 10, 0) , z3.If(r6s40, 20, 0) , z3.If(r6s41, 10, 0) , z3.If(r6s42, 10, 0) , z3.If(r6s43, 10, 0) , z3.If(r6s44, 10, 0) , z3.If(r6s45, 10, 0) , z3.If(r6s46, 10, 0) , z3.If(r6s47, 70, 0) , z3.If(r6s48, 10, 0) , z3.If(r6s49, 20, 0) , z3.If(r6s50, 70, 0) , z3.If(r6s51, 10, 0) , z3.If(r6s52, 10, 0) , z3.If(r6s53, 10, 0) , z3.If(r7s0, 20, 0) , z3.If(r7s1, 10, 0) , z3.If(r7s2, 20, 0) , z3.If(r7s3, 10, 0) , z3.If(r7s4, 10, 0) , z3.If(r7s5, 10, 0) , z3.If(r7s6, 10, 0) , z3.If(r7s7, 10, 0) , z3.If(r7s8, 10, 0) , z3.If(r7s9, 10, 0) , z3.If(r7s10, 10, 0) , z3.If(r7s11, 10, 0) , z3.If(r7s12, 70, 0) , z3.If(r7s13, 20, 0) , z3.If(r7s14, 10, 0) , z3.If(r7s15, 10, 0) , z3.If(r7s16, 10, 0) , z3.If(r7s17, 70, 0) , z3.If(r7s19, 10, 0) , z3.If(r7s20, 10, 0) , z3.If(r7s21, 10, 0) , z3.If(r7s22, 10, 0) , z3.If(r7s23, 10, 0) , z3.If(r7s25, 70, 0) , z3.If(r7s26, 10, 0) , z3.If(r7s27, 20, 0) , z3.If(r7s28, 10, 0) , z3.If(r7s29, 10, 0) , z3.If(r7s30, 20, 0) , z3.If(r7s31, 70, 0) , z3.If(r7s32, 10, 0) , z3.If(r7s33, 10, 0) , z3.If(r7s34, 10, 0) , z3.If(r7s35, 10, 0) , z3.If(r7s36, 10, 0) , z3.If(r7s37, 10, 0) , z3.If(r7s38, 10, 0) , z3.If(r7s39, 10, 0) , z3.If(r7s40, 10, 0) , z3.If(r7s41, 10, 0) , z3.If(r7s42, 70, 0) , z3.If(r7s43, 10, 0) , z3.If(r7s44, 10, 0) , z3.If(r7s45, 10, 0) , z3.If(r7s46, 10, 0) , z3.If(r7s47, 70, 0) , z3.If(r7s48, 10, 0) , z3.If(r7s49, 10, 0) , z3.If(r7s50, 10, 0) , z3.If(r7s51, 10, 0) , z3.If(r7s52, 70, 0) , z3.If(r7s53, 10, 0) , z3.If(r8s0, 10, 0) , z3.If(r8s2, 10, 0) , z3.If(r8s3, 70, 0) , z3.If(r8s4, 70, 0) , z3.If(r8s5, 10, 0) , z3.If(r8s6, 10, 0) , z3.If(r8s7, 70, 0) , z3.If(r8s8, 10, 0) , z3.If(r8s9, 10, 0) , z3.If(r8s10, 10, 0) , z3.If(r8s11, 10, 0) , z3.If(r8s12, 10, 0) , z3.If(r8s13, 10, 0) , z3.If(r8s14, 10, 0) , z3.If(r8s15, 10, 0) , z3.If(r8s16, 70, 0) , z3.If(r8s17, 10, 0) , z3.If(r8s18, 10, 0) , z3.If(r8s19, 70, 0) , z3.If(r8s20, 10, 0) , z3.If(r8s21, 10, 0) , z3.If(r8s22, 10, 0) , z3.If(r8s23, 10, 0) , z3.If(r8s24, 10, 0) , z3.If(r8s25, 10, 0) , z3.If(r8s26, 10, 0) , z3.If(r8s27, 10, 0) , z3.If(r8s28, 10, 0) , z3.If(r8s29, 10, 0) , z3.If(r8s30, 10, 0) , z3.If(r8s31, 10, 0) , z3.If(r8s32, 10, 0) , z3.If(r8s33, 10, 0) , z3.If(r8s34, 10, 0) , z3.If(r8s35, 10, 0) , z3.If(r8s36, 10, 0) , z3.If(r8s37, 10, 0) , z3.If(r8s38, 70, 0) , z3.If(r8s39, 10, 0) , z3.If(r8s40, 10, 0) , z3.If(r8s41, 10, 0) , z3.If(r8s42, 10, 0) , z3.If(r8s43, 10, 0) , z3.If(r8s44, 10, 0) , z3.If(r8s45, 10, 0) , z3.If(r8s46, 10, 0) , z3.If(r8s47, 10, 0) , z3.If(r8s48, 10, 0) , z3.If(r8s49, 10, 0) , z3.If(r8s50, 10, 0) , z3.If(r8s51, 10, 0) , z3.If(r8s52, 10, 0) , z3.If(r8s53, 10, 0) , z3.If(r9s0, 10, 0) , z3.If(r9s1, 70, 0) , z3.If(r9s2, 10, 0) , z3.If(r9s3, 10, 0) , z3.If(r9s4, 10, 0) , z3.If(r9s5, 20, 0) , z3.If(r9s6, 70, 0) , z3.If(r9s7, 10, 0) , z3.If(r9s8, 10, 0) , z3.If(r9s9, 10, 0) , z3.If(r9s10, 10, 0) , z3.If(r9s11, 10, 0) , z3.If(r9s12, 10, 0) , z3.If(r9s13, 70, 0) , z3.If(r9s14, 10, 0) , z3.If(r9s15, 10, 0) , z3.If(r9s16, 10, 0) , z3.If(r9s17, 20, 0) , z3.If(r9s18, 10, 0) , z3.If(r9s19, 10, 0) , z3.If(r9s20, 10, 0) , z3.If(r9s21, 20, 0) , z3.If(r9s22, 10, 0) , z3.If(r9s23, 10, 0) , z3.If(r9s24, 10, 0) , z3.If(r9s25, 20, 0) , z3.If(r9s26, 10, 0) , z3.If(r9s27, 20, 0) , z3.If(r9s28, 10, 0) , z3.If(r9s29, 10, 0) , z3.If(r9s30, 20, 0) , z3.If(r9s31, 10, 0) , z3.If(r9s32, 10, 0) , z3.If(r9s33, 10, 0) , z3.If(r9s34, 10, 0) , z3.If(r9s35, 10, 0) , z3.If(r9s36, 10, 0) , z3.If(r9s37, 10, 0) , z3.If(r9s38, 10, 0) , z3.If(r9s39, 10, 0) , z3.If(r9s40, 20, 0) , z3.If(r9s41, 10, 0) , z3.If(r9s42, 10, 0) , z3.If(r9s43, 10, 0) , z3.If(r9s44, 10, 0) , z3.If(r9s45, 10, 0) , z3.If(r9s46, 10, 0) , z3.If(r9s47, 10, 0) , z3.If(r9s48, 10, 0) , z3.If(r9s49, 10, 0) , z3.If(r9s50, 10, 0) , z3.If(r9s51, 10, 0) , z3.If(r9s52, 70, 0) , z3.If(r9s53, 10, 0) , z3.If(r10s0, 20, 0) , z3.If(r10s1, 10, 0) , z3.If(r10s2, 10, 0) , z3.If(r10s3, 10, 0) , z3.If(r10s4, 10, 0) , z3.If(r10s5, 10, 0) , z3.If(r10s6, 20, 0) , z3.If(r10s7, 20, 0) , z3.If(r10s8, 10, 0) , z3.If(r10s9, 20, 0) , z3.If(r10s10, 10, 0) , z3.If(r10s11, 10, 0) , z3.If(r10s12, 20, 0) , z3.If(r10s13, 20, 0) , z3.If(r10s14, 20, 0) , z3.If(r10s15, 10, 0) , z3.If(r10s16, 20, 0) , z3.If(r10s17, 10, 0) , z3.If(r10s18, 10, 0) , z3.If(r10s19, 10, 0) , z3.If(r10s20, 10, 0) , z3.If(r10s21, 10, 0) , z3.If(r10s22, 20, 0) , z3.If(r10s23, 10, 0) , z3.If(r10s24, 10, 0) , z3.If(r10s25, 10, 0) , z3.If(r10s26, 10, 0) , z3.If(r10s27, 20, 0) , z3.If(r10s28, 10, 0) , z3.If(r10s29, 10, 0) , z3.If(r10s30, 10, 0) , z3.If(r10s31, 10, 0) , z3.If(r10s32, 10, 0) , z3.If(r10s33, 10, 0) , z3.If(r10s34, 10, 0) , z3.If(r10s35, 10, 0) , z3.If(r10s36, 10, 0) , z3.If(r10s37, 10, 0) , z3.If(r10s38, 10, 0) , z3.If(r10s39, 10, 0) , z3.If(r10s40, 20, 0) , z3.If(r10s41, 10, 0) , z3.If(r10s42, 10, 0) , z3.If(r10s43, 10, 0) , z3.If(r10s44, 10, 0) , z3.If(r10s45, 10, 0) , z3.If(r10s46, 10, 0) , z3.If(r10s47, 10, 0) , z3.If(r10s48, 10, 0) , z3.If(r10s49, 10, 0) , z3.If(r10s50, 20, 0) , z3.If(r10s51, 10, 0) , z3.If(r10s52, 10, 0) , z3.If(r10s53, 10, 0) , z3.If(r11s0, 20, 0) , z3.If(r11s1, 10, 0) , z3.If(r11s2, 10, 0) , z3.If(r11s3, 20, 0) , z3.If(r11s4, 10, 0) , z3.If(r11s5, 10, 0) , z3.If(r11s6, 70, 0) , z3.If(r11s7, 10, 0) , z3.If(r11s8, 10, 0) , z3.If(r11s9, 20, 0) , z3.If(r11s10, 10, 0) , z3.If(r11s11, 10, 0) , z3.If(r11s12, 10, 0) , z3.If(r11s13, 20, 0) , z3.If(r11s14, 70, 0) , z3.If(r11s15, 10, 0) , z3.If(r11s16, 10, 0) , z3.If(r11s17, 10, 0) , z3.If(r11s18, 10, 0) , z3.If(r11s19, 10, 0) , z3.If(r11s20, 10, 0) , z3.If(r11s21, 10, 0) , z3.If(r11s22, 10, 0) , z3.If(r11s23, 10, 0) , z3.If(r11s24, 10, 0) , z3.If(r11s25, 10, 0) , z3.If(r11s26, 10, 0) , z3.If(r11s27, 70, 0) , z3.If(r11s28, 10, 0) , z3.If(r11s29, 10, 0) , z3.If(r11s30, 10, 0) , z3.If(r11s31, 10, 0) , z3.If(r11s32, 10, 0) , z3.If(r11s33, 10, 0) , z3.If(r11s34, 10, 0) , z3.If(r11s35, 10, 0) , z3.If(r11s36, 10, 0) , z3.If(r11s37, 10, 0) , z3.If(r11s38, 10, 0) , z3.If(r11s39, 10, 0) , z3.If(r11s40, 10, 0) , z3.If(r11s41, 70, 0) , z3.If(r11s42, 70, 0) , z3.If(r11s43, 10, 0) , z3.If(r11s44, 10, 0) , z3.If(r11s45, 10, 0) , z3.If(r11s46, 10, 0) , z3.If(r11s47, 10, 0) , z3.If(r11s48, 10, 0) , z3.If(r11s49, 10, 0) , z3.If(r11s50, 10, 0) , z3.If(r11s51, 10, 0) , z3.If(r11s52, 10, 0) , z3.If(r11s53, 10, 0) , z3.If(r12s0, 10, 0) , z3.If(r12s1, 20, 0) , z3.If(r12s2, 10, 0) , z3.If(r12s3, 20, 0) , z3.If(r12s4, 10, 0) , z3.If(r12s5, 10, 0) , z3.If(r12s6, 20, 0) , z3.If(r12s8, 10, 0) , z3.If(r12s9, 20, 0) , z3.If(r12s10, 10, 0) , z3.If(r12s11, 10, 0) , z3.If(r12s12, 20, 0) , z3.If(r12s13, 10, 0) , z3.If(r12s14, 10, 0) , z3.If(r12s15, 10, 0) , z3.If(r12s16, 20, 0) , z3.If(r12s17, 20, 0) , z3.If(r12s18, 10, 0) , z3.If(r12s19, 10, 0) , z3.If(r12s20, 10, 0) , z3.If(r12s21, 20, 0) , z3.If(r12s22, 10, 0) , z3.If(r12s23, 10, 0) , z3.If(r12s24, 10, 0) , z3.If(r12s25, 10, 0) , z3.If(r12s26, 10, 0) , z3.If(r12s27, 10, 0) , z3.If(r12s28, 10, 0) , z3.If(r12s30, 10, 0) , z3.If(r12s31, 10, 0) , z3.If(r12s32, 10, 0) , z3.If(r12s33, 10, 0) , z3.If(r12s34, 10, 0) , z3.If(r12s35, 10, 0) , z3.If(r12s36, 10, 0) , z3.If(r12s37, 10, 0) , z3.If(r12s38, 10, 0) , z3.If(r12s39, 10, 0) , z3.If(r12s40, 10, 0) , z3.If(r12s41, 10, 0) , z3.If(r12s42, 10, 0) , z3.If(r12s43, 10, 0) , z3.If(r12s44, 10, 0) , z3.If(r12s45, 10, 0) , z3.If(r12s46, 20, 0) , z3.If(r12s47, 10, 0) , z3.If(r12s48, 10, 0) , z3.If(r12s49, 10, 0) , z3.If(r12s50, 10, 0) , z3.If(r12s51, 10, 0) , z3.If(r12s52, 10, 0) , z3.If(r12s53, 10, 0) , z3.If(r13s0, 70, 0) , z3.If(r13s1, 70, 0) , z3.If(r13s2, 70, 0) , z3.If(r13s3, 10, 0) , z3.If(r13s4, 20, 0) , z3.If(r13s5, 10, 0) , z3.If(r13s6, 10, 0) , z3.If(r13s7, 10, 0) , z3.If(r13s8, 10, 0) , z3.If(r13s9, 70, 0) , z3.If(r13s10, 10, 0) , z3.If(r13s11, 10, 0) , z3.If(r13s12, 70, 0) , z3.If(r13s13, 10, 0) , z3.If(r13s14, 10, 0) , z3.If(r13s15, 10, 0) , z3.If(r13s16, 10, 0) , z3.If(r13s17, 20, 0) , z3.If(r13s18, 70, 0) , z3.If(r13s19, 10, 0) , z3.If(r13s20, 10, 0) , z3.If(r13s21, 10, 0) , z3.If(r13s22, 10, 0) , z3.If(r13s23, 10, 0) , z3.If(r13s24, 70, 0) , z3.If(r13s25, 70, 0) , z3.If(r13s26, 10, 0) , z3.If(r13s27, 70, 0) , z3.If(r13s28, 10, 0) , z3.If(r13s29, 10, 0) , z3.If(r13s30, 10, 0) , z3.If(r13s31, 10, 0) , z3.If(r13s32, 10, 0) , z3.If(r13s33, 10, 0) , z3.If(r13s34, 10, 0) , z3.If(r13s35, 10, 0) , z3.If(r13s36, 10, 0) , z3.If(r13s37, 10, 0) , z3.If(r13s38, 10, 0) , z3.If(r13s39, 70, 0) , z3.If(r13s40, 20, 0) , z3.If(r13s41, 10, 0) , z3.If(r13s42, 10, 0) , z3.If(r13s43, 10, 0) , z3.If(r13s44, 10, 0) , z3.If(r13s45, 10, 0) , z3.If(r13s46, 10, 0) , z3.If(r13s47, 10, 0) , z3.If(r13s48, 10, 0) , z3.If(r13s49, 10, 0) , z3.If(r13s50, 10, 0) , z3.If(r13s51, 10, 0) , z3.If(r13s52, 10, 0) , z3.If(r13s53, 10, 0) , z3.If(r14s0, 10, 0) , z3.If(r14s1, 20, 0) , z3.If(r14s2, 10, 0) , z3.If(r14s3, 10, 0) , z3.If(r14s4, 10, 0) , z3.If(r14s5, 10, 0) , z3.If(r14s6, 70, 0) , z3.If(r14s7, 20, 0) , z3.If(r14s8, 10, 0) , z3.If(r14s9, 20, 0) , z3.If(r14s10, 10, 0) , z3.If(r14s11, 10, 0) , z3.If(r14s12, 10, 0) , z3.If(r14s14, 70, 0) , z3.If(r14s15, 10, 0) , z3.If(r14s16, 10, 0) , z3.If(r14s17, 10, 0) , z3.If(r14s18, 10, 0) , z3.If(r14s19, 10, 0) , z3.If(r14s20, 10, 0) , z3.If(r14s21, 10, 0) , z3.If(r14s22, 10, 0) , z3.If(r14s23, 10, 0) , z3.If(r14s25, 10, 0) , z3.If(r14s26, 10, 0) , z3.If(r14s27, 10, 0) , z3.If(r14s28, 10, 0) , z3.If(r14s29, 10, 0) , z3.If(r14s30, 20, 0) , z3.If(r14s31, 10, 0) , z3.If(r14s32, 10, 0) , z3.If(r14s33, 10, 0) , z3.If(r14s34, 10, 0) , z3.If(r14s35, 10, 0) , z3.If(r14s36, 10, 0) , z3.If(r14s37, 10, 0) , z3.If(r14s38, 10, 0) , z3.If(r14s39, 10, 0) , z3.If(r14s40, 70, 0) , z3.If(r14s41, 10, 0) , z3.If(r14s42, 10, 0) , z3.If(r14s43, 10, 0) , z3.If(r14s44, 10, 0) , z3.If(r14s45, 10, 0) , z3.If(r14s46, 20, 0) , z3.If(r14s47, 10, 0) , z3.If(r14s48, 10, 0) , z3.If(r14s49, 10, 0) , z3.If(r14s50, 10, 0) , z3.If(r14s51, 10, 0) , z3.If(r14s52, 10, 0) , z3.If(r14s53, 10, 0) , z3.If(r15s0, 20, 0) , z3.If(r15s1, 10, 0) , z3.If(r15s2, 10, 0) , z3.If(r15s3, 10, 0) , z3.If(r15s4, 10, 0) , z3.If(r15s5, 10, 0) , z3.If(r15s6, 10, 0) , z3.If(r15s7, 10, 0) , z3.If(r15s8, 10, 0) , z3.If(r15s9, 10, 0) , z3.If(r15s10, 10, 0) , z3.If(r15s11, 10, 0) , z3.If(r15s12, 10, 0) , z3.If(r15s13, 70, 0) , z3.If(r15s14, 10, 0) , z3.If(r15s15, 10, 0) , z3.If(r15s16, 10, 0) , z3.If(r15s17, 10, 0) , z3.If(r15s18, 10, 0) , z3.If(r15s19, 10, 0) , z3.If(r15s20, 10, 0) , z3.If(r15s21, 10, 0) , z3.If(r15s22, 20, 0) , z3.If(r15s23, 10, 0) , z3.If(r15s24, 20, 0) , z3.If(r15s25, 20, 0) , z3.If(r15s26, 10, 0) , z3.If(r15s27, 70, 0) , z3.If(r15s28, 10, 0) , z3.If(r15s29, 10, 0) , z3.If(r15s30, 10, 0) , z3.If(r15s31, 10, 0) , z3.If(r15s32, 10, 0) , z3.If(r15s33, 10, 0) , z3.If(r15s34, 10, 0) , z3.If(r15s35, 10, 0) , z3.If(r15s36, 10, 0) , z3.If(r15s37, 10, 0) , z3.If(r15s38, 10, 0) , z3.If(r15s39, 10, 0) , z3.If(r15s40, 20, 0) , z3.If(r15s41, 10, 0) , z3.If(r15s42, 10, 0) , z3.If(r15s43, 10, 0) , z3.If(r15s44, 10, 0) , z3.If(r15s45, 10, 0) , z3.If(r15s46, 10, 0) , z3.If(r15s47, 10, 0) , z3.If(r15s48, 10, 0) , z3.If(r15s49, 20, 0) , z3.If(r15s50, 10, 0) , z3.If(r15s51, 10, 0) , z3.If(r15s52, 10, 0) , z3.If(r15s53, 10, 0) , z3.If(r16s0, 20, 0) , z3.If(r16s1, 20, 0) , z3.If(r16s2, 10, 0) , z3.If(r16s3, 20, 0) , z3.If(r16s4, 10, 0) , z3.If(r16s5, 20, 0) , z3.If(r16s6, 10, 0) , z3.If(r16s7, 20, 0) , z3.If(r16s8, 10, 0) , z3.If(r16s9, 10, 0) , z3.If(r16s10, 10, 0) , z3.If(r16s11, 70, 0) , z3.If(r16s12, 10, 0) , z3.If(r16s13, 10, 0) , z3.If(r16s14, 10, 0) , z3.If(r16s15, 10, 0) , z3.If(r16s16, 10, 0) , z3.If(r16s17, 10, 0) , z3.If(r16s18, 10, 0) , z3.If(r16s19, 20, 0) , z3.If(r16s20, 10, 0) , z3.If(r16s21, 10, 0) , z3.If(r16s22, 10, 0) , z3.If(r16s23, 10, 0) , z3.If(r16s24, 10, 0) , z3.If(r16s25, 10, 0) , z3.If(r16s26, 10, 0) , z3.If(r16s27, 20, 0) , z3.If(r16s28, 10, 0) , z3.If(r16s29, 10, 0) , z3.If(r16s30, 10, 0) , z3.If(r16s31, 10, 0) , z3.If(r16s32, 10, 0) , z3.If(r16s33, 10, 0) , z3.If(r16s34, 10, 0) , z3.If(r16s35, 10, 0) , z3.If(r16s36, 10, 0) , z3.If(r16s37, 20, 0) , z3.If(r16s38, 10, 0) , z3.If(r16s39, 10, 0) , z3.If(r16s40, 10, 0) , z3.If(r16s41, 10, 0) , z3.If(r16s42, 10, 0) , z3.If(r16s44, 10, 0) , z3.If(r16s45, 10, 0) , z3.If(r16s46, 70, 0) , z3.If(r16s47, 10, 0) , z3.If(r16s48, 10, 0) , z3.If(r16s50, 10, 0) , z3.If(r16s52, 10, 0) , z3.If(r16s53, 10, 0) , z3.If(r17s0, 10, 0) , z3.If(r17s1, 70, 0) , z3.If(r17s2, 10, 0) , z3.If(r17s3, 70, 0) , z3.If(r17s4, 10, 0) , z3.If(r17s5, 10, 0) , z3.If(r17s6, 10, 0) , z3.If(r17s7, 10, 0) , z3.If(r17s8, 10, 0) , z3.If(r17s9, 70, 0) , z3.If(r17s10, 70, 0) , z3.If(r17s11, 10, 0) , z3.If(r17s12, 10, 0) , z3.If(r17s13, 10, 0) , z3.If(r17s14, 10, 0) , z3.If(r17s15, 10, 0) , z3.If(r17s16, 10, 0) , z3.If(r17s17, 10, 0) , z3.If(r17s18, 10, 0) , z3.If(r17s19, 10, 0) , z3.If(r17s20, 10, 0) , z3.If(r17s22, 10, 0) , z3.If(r17s23, 10, 0) , z3.If(r17s24, 10, 0) , z3.If(r17s25, 10, 0) , z3.If(r17s27, 10, 0) , z3.If(r17s28, 10, 0) , z3.If(r17s29, 70, 0) , z3.If(r17s30, 10, 0) , z3.If(r17s31, 10, 0) , z3.If(r17s32, 10, 0) , z3.If(r17s33, 70, 0) , z3.If(r17s34, 10, 0) , z3.If(r17s35, 10, 0) , z3.If(r17s36, 10, 0) , z3.If(r17s37, 10, 0) , z3.If(r17s38, 10, 0) , z3.If(r17s39, 10, 0) , z3.If(r17s40, 10, 0) , z3.If(r17s41, 10, 0) , z3.If(r17s42, 10, 0) , z3.If(r17s43, 10, 0) , z3.If(r17s44, 70, 0) , z3.If(r17s46, 10, 0) , z3.If(r17s47, 10, 0) , z3.If(r17s49, 10, 0) , z3.If(r17s50, 10, 0) , z3.If(r17s51, 10, 0) , z3.If(r17s52, 10, 0) , z3.If(r18s0, 10, 0) , z3.If(r18s1, 10, 0) , z3.If(r18s2, 10, 0) , z3.If(r18s3, 10, 0) , z3.If(r18s4, 10, 0) , z3.If(r18s5, 70, 0) , z3.If(r18s6, 10, 0) , z3.If(r18s7, 10, 0) , z3.If(r18s8, 10, 0) , z3.If(r18s9, 10, 0) , z3.If(r18s10, 10, 0) , z3.If(r18s11, 10, 0) , z3.If(r18s12, 70, 0) , z3.If(r18s13, 10, 0) , z3.If(r18s14, 20, 0) , z3.If(r18s15, 10, 0) , z3.If(r18s16, 10, 0) , z3.If(r18s17, 20, 0) , z3.If(r18s18, 20, 0) , z3.If(r18s19, 10, 0) , z3.If(r18s20, 10, 0) , z3.If(r18s21, 10, 0) , z3.If(r18s22, 70, 0) , z3.If(r18s23, 10, 0) , z3.If(r18s24, 10, 0) , z3.If(r18s25, 10, 0) , z3.If(r18s26, 10, 0) , z3.If(r18s27, 10, 0) , z3.If(r18s28, 10, 0) , z3.If(r18s29, 10, 0) , z3.If(r18s30, 10, 0) , z3.If(r18s31, 10, 0) , z3.If(r18s32, 10, 0) , z3.If(r18s33, 10, 0) , z3.If(r18s34, 10, 0) , z3.If(r18s35, 10, 0) , z3.If(r18s36, 20, 0) , z3.If(r18s37, 10, 0) , z3.If(r18s38, 10, 0) , z3.If(r18s39, 10, 0) , z3.If(r18s40, 10, 0) , z3.If(r18s41, 10, 0) , z3.If(r18s42, 10, 0) , z3.If(r18s43, 10, 0) , z3.If(r18s44, 10, 0) , z3.If(r18s45, 10, 0) , z3.If(r18s46, 10, 0) , z3.If(r18s47, 10, 0) , z3.If(r18s48, 10, 0) , z3.If(r18s49, 10, 0) , z3.If(r18s50, 70, 0) , z3.If(r18s51, 10, 0) , z3.If(r18s52, 10, 0) , z3.If(r18s53, 10, 0) , z3.If(r19s0, 20, 0) , z3.If(r19s1, 10, 0) , z3.If(r19s2, 70, 0) , z3.If(r19s3, 10, 0) , z3.If(r19s4, 10, 0) , z3.If(r19s5, 10, 0) , z3.If(r19s6, 10, 0) , z3.If(r19s7, 10, 0) , z3.If(r19s8, 10, 0) , z3.If(r19s9, 10, 0) , z3.If(r19s10, 10, 0) , z3.If(r19s11, 10, 0) , z3.If(r19s12, 10, 0) , z3.If(r19s13, 20, 0) , z3.If(r19s14, 10, 0) , z3.If(r19s15, 10, 0) , z3.If(r19s16, 20, 0) , z3.If(r19s17, 10, 0) , z3.If(r19s18, 10, 0) , z3.If(r19s19, 10, 0) , z3.If(r19s20, 10, 0) , z3.If(r19s21, 10, 0) , z3.If(r19s22, 10, 0) , z3.If(r19s23, 10, 0) , z3.If(r19s24, 10, 0) , z3.If(r19s25, 10, 0) , z3.If(r19s26, 10, 0) , z3.If(r19s27, 10, 0) , z3.If(r19s28, 10, 0) , z3.If(r19s29, 10, 0) , z3.If(r19s30, 10, 0) , z3.If(r19s31, 10, 0) , z3.If(r19s32, 10, 0) , z3.If(r19s33, 10, 0) , z3.If(r19s34, 10, 0) , z3.If(r19s35, 10, 0) , z3.If(r19s36, 10, 0) , z3.If(r19s37, 10, 0) , z3.If(r19s38, 10, 0) , z3.If(r19s39, 10, 0) , z3.If(r19s40, 10, 0) , z3.If(r19s41, 10, 0) , z3.If(r19s42, 10, 0) , z3.If(r19s43, 10, 0) , z3.If(r19s44, 10, 0) , z3.If(r19s45, 10, 0) , z3.If(r19s46, 10, 0) , z3.If(r19s47, 10, 0) , z3.If(r19s48, 10, 0) , z3.If(r19s49, 10, 0) , z3.If(r19s50, 10, 0) , z3.If(r19s51, 70, 0) , z3.If(r19s52, 10, 0) , z3.If(r19s53, 10, 0) , z3.If(r20s0, 70, 0) , z3.If(r20s1, 10, 0) , z3.If(r20s2, 70, 0) , z3.If(r20s3, 10, 0) , z3.If(r20s4, 10, 0) , z3.If(r20s5, 10, 0) , z3.If(r20s6, 70, 0) , z3.If(r20s7, 10, 0) , z3.If(r20s8, 10, 0) , z3.If(r20s9, 70, 0) , z3.If(r20s10, 10, 0) , z3.If(r20s11, 10, 0) , z3.If(r20s13, 10, 0) , z3.If(r20s14, 10, 0) , z3.If(r20s15, 10, 0) , z3.If(r20s16, 10, 0) , z3.If(r20s17, 70, 0) , z3.If(r20s18, 70, 0) , z3.If(r20s19, 10, 0) , z3.If(r20s20, 10, 0) , z3.If(r20s21, 10, 0) , z3.If(r20s22, 70, 0) , z3.If(r20s23, 10, 0) , z3.If(r20s24, 70, 0) , z3.If(r20s25, 70, 0) , z3.If(r20s26, 10, 0) , z3.If(r20s27, 70, 0) , z3.If(r20s28, 10, 0) , z3.If(r20s29, 10, 0) , z3.If(r20s30, 70, 0) , z3.If(r20s31, 70, 0) , z3.If(r20s32, 10, 0) , z3.If(r20s33, 10, 0) , z3.If(r20s34, 10, 0) , z3.If(r20s35, 10, 0) , z3.If(r20s36, 10, 0) , z3.If(r20s37, 10, 0) , z3.If(r20s38, 10, 0) , z3.If(r20s39, 10, 0) , z3.If(r20s40, 10, 0) , z3.If(r20s41, 10, 0) , z3.If(r20s42, 10, 0) , z3.If(r20s43, 10, 0) , z3.If(r20s44, 10, 0) , z3.If(r20s45, 10, 0) , z3.If(r20s46, 10, 0) , z3.If(r20s47, 10, 0) , z3.If(r20s48, 10, 0) , z3.If(r20s49, 10, 0) , z3.If(r20s50, 10, 0) , z3.If(r20s51, 10, 0) , z3.If(r20s52, 10, 0) , z3.If(r20s53, 10, 0) , z3.If(r21s0, 70, 0) , z3.If(r21s1, 10, 0) , z3.If(r21s2, 10, 0) , z3.If(r21s3, 70, 0) , z3.If(r21s4, 10, 0) , z3.If(r21s5, 10, 0) , z3.If(r21s6, 20, 0) , z3.If(r21s7, 20, 0) , z3.If(r21s8, 10, 0) , z3.If(r21s9, 70, 0) , z3.If(r21s10, 10, 0) , z3.If(r21s11, 10, 0) , z3.If(r21s12, 10, 0) , z3.If(r21s13, 20, 0) , z3.If(r21s14, 10, 0) , z3.If(r21s15, 10, 0) , z3.If(r21s16, 20, 0) , z3.If(r21s17, 10, 0) , z3.If(r21s18, 20, 0) , z3.If(r21s19, 10, 0) , z3.If(r21s20, 10, 0) , z3.If(r21s21, 10, 0) , z3.If(r21s22, 70, 0) , z3.If(r21s23, 10, 0) , z3.If(r21s24, 10, 0) , z3.If(r21s25, 10, 0) , z3.If(r21s26, 10, 0) , z3.If(r21s27, 20, 0) , z3.If(r21s28, 70, 0) , z3.If(r21s29, 10, 0) , z3.If(r21s30, 10, 0) , z3.If(r21s31, 10, 0) , z3.If(r21s32, 10, 0) , z3.If(r21s33, 10, 0) , z3.If(r21s34, 70, 0) , z3.If(r21s35, 10, 0) , z3.If(r21s36, 10, 0) , z3.If(r21s37, 10, 0) , z3.If(r21s38, 10, 0) , z3.If(r21s39, 10, 0) , z3.If(r21s40, 70, 0) , z3.If(r21s41, 10, 0) , z3.If(r21s43, 10, 0) , z3.If(r21s44, 10, 0) , z3.If(r21s45, 10, 0) , z3.If(r21s46, 10, 0) , z3.If(r21s47, 10, 0) , z3.If(r21s48, 10, 0) , z3.If(r21s49, 10, 0) , z3.If(r21s50, 10, 0) , z3.If(r21s51, 10, 0) , z3.If(r21s52, 10, 0) , z3.If(r21s53, 10, 0) , z3.If(r22s0, 10, 0) , z3.If(r22s1, 10, 0) , z3.If(r22s2, 10, 0) , z3.If(r22s3, 70, 0) , z3.If(r22s4, 10, 0) , z3.If(r22s5, 10, 0) , z3.If(r22s6, 10, 0) , z3.If(r22s7, 20, 0) , z3.If(r22s8, 20, 0) , z3.If(r22s9, 10, 0) , z3.If(r22s10, 10, 0) , z3.If(r22s11, 10, 0) , z3.If(r22s12, 20, 0) , z3.If(r22s13, 10, 0) , z3.If(r22s14, 10, 0) , z3.If(r22s15, 10, 0) , z3.If(r22s16, 70, 0) , z3.If(r22s17, 20, 0) , z3.If(r22s18, 10, 0) , z3.If(r22s19, 10, 0) , z3.If(r22s20, 10, 0) , z3.If(r22s21, 10, 0) , z3.If(r22s22, 10, 0) , z3.If(r22s23, 10, 0) , z3.If(r22s24, 10, 0) , z3.If(r22s25, 10, 0) , z3.If(r22s26, 10, 0) , z3.If(r22s27, 10, 0) , z3.If(r22s28, 10, 0) , z3.If(r22s29, 10, 0) , z3.If(r22s30, 10, 0) , z3.If(r22s31, 10, 0) , z3.If(r22s32, 10, 0) , z3.If(r22s33, 10, 0) , z3.If(r22s34, 10, 0) , z3.If(r22s35, 10, 0) , z3.If(r22s36, 10, 0) , z3.If(r22s37, 10, 0) , z3.If(r22s38, 10, 0) , z3.If(r22s39, 10, 0) , z3.If(r22s40, 10, 0) , z3.If(r22s41, 10, 0) , z3.If(r22s42, 10, 0) , z3.If(r22s43, 10, 0) , z3.If(r22s44, 10, 0) , z3.If(r22s45, 10, 0) , z3.If(r22s46, 10, 0) , z3.If(r22s47, 10, 0) , z3.If(r22s48, 10, 0) , z3.If(r22s49, 10, 0) , z3.If(r22s50, 10, 0) , z3.If(r22s51, 10, 0) , z3.If(r22s52, 10, 0) , z3.If(r22s53, 10, 0) , z3.If(r23s0, 10, 0) , z3.If(r23s1, 70, 0) , z3.If(r23s2, 10, 0) , z3.If(r23s3, 20, 0) , z3.If(r23s4, 10, 0) , z3.If(r23s5, 10, 0) , z3.If(r23s6, 70, 0) , z3.If(r23s7, 10, 0) , z3.If(r23s8, 10, 0) , z3.If(r23s9, 10, 0) , z3.If(r23s10, 10, 0) , z3.If(r23s11, 10, 0) , z3.If(r23s12, 10, 0) , z3.If(r23s13, 10, 0) , z3.If(r23s14, 70, 0) , z3.If(r23s15, 10, 0) , z3.If(r23s16, 20, 0) , z3.If(r23s17, 70, 0) , z3.If(r23s19, 10, 0) , z3.If(r23s20, 10, 0) , z3.If(r23s21, 10, 0) , z3.If(r23s22, 10, 0) , z3.If(r23s23, 10, 0) , z3.If(r23s24, 10, 0) , z3.If(r23s25, 70, 0) , z3.If(r23s26, 10, 0) , z3.If(r23s27, 10, 0) , z3.If(r23s28, 10, 0) , z3.If(r23s29, 10, 0) , z3.If(r23s30, 70, 0) , z3.If(r23s31, 20, 0) , z3.If(r23s32, 10, 0) , z3.If(r23s33, 10, 0) , z3.If(r23s34, 10, 0) , z3.If(r23s35, 10, 0) , z3.If(r23s36, 10, 0) , z3.If(r23s37, 70, 0) , z3.If(r23s38, 10, 0) , z3.If(r23s39, 10, 0) , z3.If(r23s41, 10, 0) , z3.If(r23s42, 10, 0) , z3.If(r23s43, 10, 0) , z3.If(r23s44, 10, 0) , z3.If(r23s45, 10, 0) , z3.If(r23s46, 10, 0) , z3.If(r23s47, 10, 0) , z3.If(r23s48, 10, 0) , z3.If(r23s49, 10, 0) , z3.If(r23s50, 10, 0) , z3.If(r23s51, 10, 0) , z3.If(r23s52, 10, 0) , z3.If(r23s53, 10, 0) , z3.If(r24s0, 20, 0) , z3.If(r24s1, 10, 0) , z3.If(r24s2, 10, 0) , z3.If(r24s3, 10, 0) , z3.If(r24s4, 10, 0) , z3.If(r24s5, 10, 0) , z3.If(r24s6, 20, 0) , z3.If(r24s7, 10, 0) , z3.If(r24s8, 10, 0) , z3.If(r24s9, 10, 0) , z3.If(r24s10, 10, 0) , z3.If(r24s11, 10, 0) , z3.If(r24s12, 10, 0) , z3.If(r24s14, 10, 0) , z3.If(r24s15, 70, 0) , z3.If(r24s16, 20, 0) , z3.If(r24s17, 10, 0) , z3.If(r24s18, 10, 0) , z3.If(r24s19, 20, 0) , z3.If(r24s20, 70, 0) , z3.If(r24s21, 10, 0) , z3.If(r24s22, 70, 0) , z3.If(r24s23, 10, 0) , z3.If(r24s24, 10, 0) , z3.If(r24s25, 10, 0) , z3.If(r24s26, 10, 0) , z3.If(r24s27, 20, 0) , z3.If(r24s28, 10, 0) , z3.If(r24s29, 10, 0) , z3.If(r24s30, 10, 0) , z3.If(r24s31, 10, 0) , z3.If(r24s32, 10, 0) , z3.If(r24s33, 10, 0) , z3.If(r24s34, 10, 0) , z3.If(r24s35, 10, 0) , z3.If(r24s36, 10, 0) , z3.If(r24s37, 10, 0) , z3.If(r24s38, 10, 0) , z3.If(r24s39, 10, 0) , z3.If(r24s40, 10, 0) , z3.If(r24s42, 10, 0) , z3.If(r24s44, 10, 0) , z3.If(r24s45, 10, 0) , z3.If(r24s46, 20, 0) , z3.If(r24s47, 10, 0) , z3.If(r24s48, 10, 0) , z3.If(r24s50, 10, 0) , z3.If(r24s52, 10, 0) , z3.If(r24s53, 10, 0) , z3.If(r25s0, 10, 0) , z3.If(r25s1, 10, 0) , z3.If(r25s2, 20, 0) , z3.If(r25s3, 10, 0) , z3.If(r25s4, 10, 0) , z3.If(r25s5, 10, 0) , z3.If(r25s6, 70, 0) , z3.If(r25s7, 20, 0) , z3.If(r25s8, 10, 0) , z3.If(r25s9, 20, 0) , z3.If(r25s10, 10, 0) , z3.If(r25s11, 10, 0) , z3.If(r25s13, 10, 0) , z3.If(r25s14, 10, 0) , z3.If(r25s15, 10, 0) , z3.If(r25s16, 20, 0) , z3.If(r25s17, 70, 0) , z3.If(r25s19, 10, 0) , z3.If(r25s20, 10, 0) , z3.If(r25s21, 10, 0) , z3.If(r25s22, 10, 0) , z3.If(r25s23, 10, 0) , z3.If(r25s24, 10, 0) , z3.If(r25s25, 10, 0) , z3.If(r25s26, 10, 0) , z3.If(r25s27, 70, 0) , z3.If(r25s28, 10, 0) , z3.If(r25s29, 10, 0) , z3.If(r25s30, 20, 0) , z3.If(r25s31, 20, 0) , z3.If(r25s32, 20, 0) , z3.If(r25s33, 10, 0) , z3.If(r25s34, 10, 0) , z3.If(r25s35, 10, 0) , z3.If(r25s36, 20, 0) , z3.If(r25s37, 10, 0) , z3.If(r25s38, 10, 0) , z3.If(r25s39, 10, 0) , z3.If(r25s40, 20, 0) , z3.If(r25s41, 10, 0) , z3.If(r25s42, 20, 0) , z3.If(r25s43, 20, 0) , z3.If(r25s44, 10, 0) , z3.If(r25s45, 10, 0) , z3.If(r25s46, 10, 0) , z3.If(r25s47, 20, 0) , z3.If(r25s48, 10, 0) , z3.If(r25s49, 10, 0) , z3.If(r25s50, 10, 0) , z3.If(r25s51, 10, 0) , z3.If(r25s52, 10, 0) , z3.If(r25s53, 10, 0) , z3.If(r26s0, 70, 0) , z3.If(r26s1, 20, 0) , z3.If(r26s2, 10, 0) , z3.If(r26s3, 70, 0) , z3.If(r26s4, 70, 0) , z3.If(r26s5, 10, 0) , z3.If(r26s6, 10, 0) , z3.If(r26s7, 10, 0) , z3.If(r26s8, 10, 0) , z3.If(r26s9, 10, 0) , z3.If(r26s10, 10, 0) , z3.If(r26s11, 70, 0) , z3.If(r26s12, 10, 0) , z3.If(r26s13, 10, 0) , z3.If(r26s14, 10, 0) , z3.If(r26s15, 10, 0) , z3.If(r26s16, 10, 0) , z3.If(r26s17, 10, 0) , z3.If(r26s18, 10, 0) , z3.If(r26s20, 70, 0) , z3.If(r26s21, 70, 0) , z3.If(r26s22, 10, 0) , z3.If(r26s23, 10, 0) , z3.If(r26s24, 10, 0) , z3.If(r26s25, 10, 0) , z3.If(r26s26, 10, 0) , z3.If(r26s27, 10, 0) , z3.If(r26s28, 10, 0) , z3.If(r26s29, 20, 0) , z3.If(r26s30, 10, 0) , z3.If(r26s31, 10, 0) , z3.If(r26s32, 10, 0) , z3.If(r26s33, 10, 0) , z3.If(r26s34, 10, 0) , z3.If(r26s35, 10, 0) , z3.If(r26s36, 10, 0) , z3.If(r26s37, 10, 0) , z3.If(r26s38, 10, 0) , z3.If(r26s39, 10, 0) , z3.If(r26s40, 10, 0) , z3.If(r26s41, 10, 0) , z3.If(r26s42, 10, 0) , z3.If(r26s43, 70, 0) , z3.If(r26s44, 10, 0) , z3.If(r26s45, 10, 0) , z3.If(r26s46, 10, 0) , z3.If(r26s47, 70, 0) , z3.If(r26s48, 10, 0) , z3.If(r26s49, 70, 0) , z3.If(r26s50, 10, 0) , z3.If(r26s51, 70, 0) , z3.If(r26s52, 10, 0) , z3.If(r26s53, 10, 0) , z3.If(r27s0, 10, 0) , z3.If(r27s1, 70, 0) , z3.If(r27s2, 10, 0) , z3.If(r27s3, 10, 0) , z3.If(r27s4, 10, 0) , z3.If(r27s5, 10, 0) , z3.If(r27s6, 20, 0) , z3.If(r27s7, 10, 0) , z3.If(r27s8, 10, 0) , z3.If(r27s9, 70, 0) , z3.If(r27s10, 10, 0) , z3.If(r27s11, 10, 0) , z3.If(r27s12, 10, 0) , z3.If(r27s13, 10, 0) , z3.If(r27s14, 70, 0) , z3.If(r27s15, 10, 0) , z3.If(r27s16, 10, 0) , z3.If(r27s17, 10, 0) , z3.If(r27s18, 10, 0) , z3.If(r27s19, 10, 0) , z3.If(r27s20, 10, 0) , z3.If(r27s21, 10, 0) , z3.If(r27s22, 10, 0) , z3.If(r27s23, 10, 0) , z3.If(r27s24, 20, 0) , z3.If(r27s25, 10, 0) , z3.If(r27s26, 10, 0) , z3.If(r27s27, 10, 0) , z3.If(r27s28, 10, 0) , z3.If(r27s29, 10, 0) , z3.If(r27s30, 10, 0) , z3.If(r27s31, 10, 0) , z3.If(r27s32, 10, 0) , z3.If(r27s33, 10, 0) , z3.If(r27s34, 10, 0) , z3.If(r27s35, 10, 0) , z3.If(r27s36, 10, 0) , z3.If(r27s37, 70, 0) , z3.If(r27s38, 10, 0) , z3.If(r27s39, 20, 0) , z3.If(r27s40, 10, 0) , z3.If(r27s41, 10, 0) , z3.If(r27s42, 70, 0) , z3.If(r27s43, 20, 0) , z3.If(r27s44, 10, 0) , z3.If(r27s45, 10, 0) , z3.If(r27s46, 10, 0) , z3.If(r27s47, 10, 0) , z3.If(r27s48, 10, 0) , z3.If(r27s49, 10, 0) , z3.If(r27s50, 70, 0) , z3.If(r27s51, 10, 0) , z3.If(r27s52, 10, 0) , z3.If(r27s53, 10, 0) , z3.If(r28s0, 10, 0) , z3.If(r28s1, 20, 0) , z3.If(r28s2, 10, 0) , z3.If(r28s3, 10, 0) , z3.If(r28s4, 10, 0) , z3.If(r28s5, 10, 0) , z3.If(r28s6, 20, 0) , z3.If(r28s7, 10, 0) , z3.If(r28s8, 10, 0) , z3.If(r28s9, 20, 0) , z3.If(r28s10, 10, 0) , z3.If(r28s11, 10, 0) , z3.If(r28s12, 20, 0) , z3.If(r28s13, 70, 0) , z3.If(r28s14, 10, 0) , z3.If(r28s15, 10, 0) , z3.If(r28s16, 10, 0) , z3.If(r28s17, 20, 0) , z3.If(r28s18, 70, 0) , z3.If(r28s19, 10, 0) , z3.If(r28s20, 10, 0) , z3.If(r28s21, 10, 0) , z3.If(r28s23, 10, 0) , z3.If(r28s24, 70, 0) , z3.If(r28s25, 70, 0) , z3.If(r28s26, 10, 0) , z3.If(r28s29, 10, 0) , z3.If(r28s30, 20, 0) , z3.If(r28s32, 10, 0) , z3.If(r28s33, 10, 0) , z3.If(r28s35, 10, 0) , z3.If(r28s36, 10, 0) , z3.If(r28s37, 10, 0) , z3.If(r28s38, 10, 0) , z3.If(r28s39, 10, 0) , z3.If(r28s40, 70, 0) , z3.If(r28s41, 10, 0) , z3.If(r28s42, 20, 0) , z3.If(r28s43, 10, 0) , z3.If(r28s44, 10, 0) , z3.If(r28s45, 10, 0) , z3.If(r28s46, 10, 0) , z3.If(r28s47, 10, 0) , z3.If(r28s48, 10, 0) , z3.If(r28s49, 10, 0) , z3.If(r28s50, 10, 0) , z3.If(r28s51, 10, 0) , z3.If(r28s52, 10, 0) , z3.If(r28s53, 10, 0) , z3.If(r29s0, 10, 0) , z3.If(r29s1, 10, 0) , z3.If(r29s2, 10, 0) , z3.If(r29s3, 10, 0) , z3.If(r29s4, 10, 0) , z3.If(r29s5, 10, 0) , z3.If(r29s6, 70, 0) , z3.If(r29s7, 10, 0) , z3.If(r29s8, 10, 0) , z3.If(r29s9, 10, 0) , z3.If(r29s10, 10, 0) , z3.If(r29s11, 10, 0) , z3.If(r29s12, 70, 0) , z3.If(r29s13, 10, 0) , z3.If(r29s14, 10, 0) , z3.If(r29s15, 10, 0) , z3.If(r29s16, 70, 0) , z3.If(r29s17, 70, 0) , z3.If(r29s18, 10, 0) , z3.If(r29s19, 10, 0) , z3.If(r29s20, 10, 0) , z3.If(r29s21, 10, 0) , z3.If(r29s23, 10, 0) , z3.If(r29s24, 70, 0) , z3.If(r29s25, 70, 0) , z3.If(r29s26, 10, 0) , z3.If(r29s29, 10, 0) , z3.If(r29s30, 70, 0) , z3.If(r29s32, 70, 0) , z3.If(r29s33, 10, 0) , z3.If(r29s35, 10, 0) , z3.If(r29s36, 70, 0) , z3.If(r29s37, 10, 0) , z3.If(r29s38, 10, 0) , z3.If(r29s39, 10, 0) , z3.If(r29s40, 70, 0) , z3.If(r29s41, 10, 0) , z3.If(r29s42, 70, 0) , z3.If(r29s43, 10, 0) , z3.If(r29s44, 10, 0) , z3.If(r29s45, 10, 0) , z3.If(r29s46, 10, 0) , z3.If(r29s47, 10, 0) , z3.If(r29s48, 10, 0) , z3.If(r29s49, 10, 0) , z3.If(r29s50, 10, 0) , z3.If(r29s51, 10, 0) , z3.If(r29s52, 10, 0) , z3.If(r29s53, 10, 0) , z3.If(r30s0, 10, 0) , z3.If(r30s1, 10, 0) , z3.If(r30s2, 10, 0) , z3.If(r30s3, 70, 0) , z3.If(r30s4, 10, 0) , z3.If(r30s5, 20, 0) , z3.If(r30s6, 70, 0) , z3.If(r30s7, 10, 0) , z3.If(r30s8, 10, 0) , z3.If(r30s9, 10, 0) , z3.If(r30s10, 10, 0) , z3.If(r30s11, 20, 0) , z3.If(r30s12, 10, 0) , z3.If(r30s13, 70, 0) , z3.If(r30s14, 10, 0) , z3.If(r30s15, 70, 0) , z3.If(r30s16, 20, 0) , z3.If(r30s17, 10, 0) , z3.If(r30s18, 10, 0) , z3.If(r30s19, 10, 0) , z3.If(r30s20, 10, 0) , z3.If(r30s21, 20, 0) , z3.If(r30s22, 10, 0) , z3.If(r30s23, 10, 0) , z3.If(r30s24, 10, 0) , z3.If(r30s25, 70, 0) , z3.If(r30s26, 10, 0) , z3.If(r30s27, 10, 0) , z3.If(r30s28, 10, 0) , z3.If(r30s29, 20, 0) , z3.If(r30s30, 10, 0) , z3.If(r30s31, 10, 0) , z3.If(r30s32, 20, 0) , z3.If(r30s33, 10, 0) , z3.If(r30s34, 10, 0) , z3.If(r30s35, 20, 0) , z3.If(r30s37, 10, 0) , z3.If(r30s38, 10, 0) , z3.If(r30s39, 10, 0) , z3.If(r30s40, 10, 0) , z3.If(r30s41, 10, 0) , z3.If(r30s42, 10, 0) , z3.If(r30s43, 10, 0) , z3.If(r30s44, 10, 0) , z3.If(r30s45, 20, 0) , z3.If(r30s46, 20, 0) , z3.If(r30s47, 10, 0) , z3.If(r30s48, 10, 0) , z3.If(r30s49, 70, 0) , z3.If(r30s50, 10, 0) , z3.If(r30s51, 10, 0) , z3.If(r30s52, 10, 0) , z3.If(r30s53, 10, 0)]
# 75/20/5
# scores = [z3.If(r0s0, 5, 0) , z3.If(r0s1, 5, 0) , z3.If(r0s2, 5, 0) , z3.If(r0s4, 5, 0) , z3.If(r0s5, 5, 0) , z3.If(r0s6, 75, 0) , z3.If(r0s7, 5, 0) , z3.If(r0s8, 5, 0) , z3.If(r0s9, 20, 0) , z3.If(r0s10, 5, 0) , z3.If(r0s11, 5, 0) , z3.If(r0s12, 5, 0) , z3.If(r0s13, 75, 0) , z3.If(r0s14, 5, 0) , z3.If(r0s15, 5, 0) , z3.If(r0s16, 20, 0) , z3.If(r0s17, 20, 0) , z3.If(r0s18, 20, 0) , z3.If(r0s19, 5, 0) , z3.If(r0s20, 5, 0) , z3.If(r0s21, 5, 0) , z3.If(r0s22, 75, 0) , z3.If(r0s23, 5, 0) , z3.If(r0s24, 75, 0) , z3.If(r0s25, 5, 0) , z3.If(r0s26, 5, 0) , z3.If(r0s27, 75, 0) , z3.If(r0s28, 5, 0) , z3.If(r0s29, 5, 0) , z3.If(r0s30, 5, 0) , z3.If(r0s31, 5, 0) , z3.If(r0s32, 20, 0) , z3.If(r0s33, 5, 0) , z3.If(r0s34, 5, 0) , z3.If(r0s35, 5, 0) , z3.If(r0s36, 20, 0) , z3.If(r0s37, 5, 0) , z3.If(r0s38, 5, 0) , z3.If(r0s39, 5, 0) , z3.If(r0s40, 5, 0) , z3.If(r0s41, 5, 0) , z3.If(r0s42, 20, 0) , z3.If(r0s43, 20, 0) , z3.If(r0s44, 5, 0) , z3.If(r0s45, 5, 0) , z3.If(r0s46, 5, 0) , z3.If(r0s47, 20, 0) , z3.If(r0s48, 5, 0) , z3.If(r0s49, 5, 0) , z3.If(r0s51, 20, 0) , z3.If(r0s52, 5, 0) , z3.If(r0s53, 5, 0) , z3.If(r1s0, 5, 0) , z3.If(r1s1, 5, 0) , z3.If(r1s2, 5, 0) , z3.If(r1s3, 5, 0) , z3.If(r1s4, 5, 0) , z3.If(r1s5, 5, 0) , z3.If(r1s6, 75, 0) , z3.If(r1s7, 20, 0) , z3.If(r1s8, 5, 0) , z3.If(r1s9, 5, 0) , z3.If(r1s10, 5, 0) , z3.If(r1s11, 5, 0) , z3.If(r1s12, 75, 0) , z3.If(r1s13, 5, 0) , z3.If(r1s14, 20, 0) , z3.If(r1s15, 5, 0) , z3.If(r1s16, 5, 0) , z3.If(r1s17, 75, 0) , z3.If(r1s19, 5, 0) , z3.If(r1s20, 5, 0) , z3.If(r1s21, 5, 0) , z3.If(r1s22, 20, 0) , z3.If(r1s23, 5, 0) , z3.If(r1s24, 75, 0) , z3.If(r1s25, 20, 0) , z3.If(r1s26, 5, 0) , z3.If(r1s27, 75, 0) , z3.If(r1s28, 5, 0) , z3.If(r1s29, 5, 0) , z3.If(r1s30, 20, 0) , z3.If(r1s31, 20, 0) , z3.If(r1s32, 75, 0) , z3.If(r1s33, 5, 0) , z3.If(r1s34, 5, 0) , z3.If(r1s35, 5, 0) , z3.If(r1s36, 20, 0) , z3.If(r1s37, 5, 0) , z3.If(r1s38, 5, 0) , z3.If(r1s39, 5, 0) , z3.If(r1s41, 5, 0) , z3.If(r1s42, 75, 0) , z3.If(r1s43, 20, 0) , z3.If(r1s44, 5, 0) , z3.If(r1s45, 5, 0) , z3.If(r1s46, 5, 0) , z3.If(r1s47, 75, 0) , z3.If(r1s48, 5, 0) , z3.If(r1s49, 20, 0) , z3.If(r1s50, 5, 0) , z3.If(r1s51, 5, 0) , z3.If(r1s52, 5, 0) , z3.If(r1s53, 5, 0) , z3.If(r2s0, 5, 0) , z3.If(r2s1, 20, 0) , z3.If(r2s2, 5, 0) , z3.If(r2s3, 75, 0) , z3.If(r2s4, 5, 0) , z3.If(r2s5, 75, 0) , z3.If(r2s6, 5, 0) , z3.If(r2s7, 75, 0) , z3.If(r2s8, 5, 0) , z3.If(r2s9, 5, 0) , z3.If(r2s10, 5, 0) , z3.If(r2s11, 5, 0) , z3.If(r2s12, 20, 0) , z3.If(r2s13, 5, 0) , z3.If(r2s14, 20, 0) , z3.If(r2s15, 5, 0) , z3.If(r2s16, 5, 0) , z3.If(r2s17, 20, 0) , z3.If(r2s18, 5, 0) , z3.If(r2s19, 5, 0) , z3.If(r2s20, 5, 0) , z3.If(r2s21, 5, 0) , z3.If(r2s22, 5, 0) , z3.If(r2s23, 5, 0) , z3.If(r2s24, 75, 0) , z3.If(r2s25, 5, 0) , z3.If(r2s26, 5, 0) , z3.If(r2s27, 75, 0) , z3.If(r2s28, 5, 0) , z3.If(r2s29, 5, 0) , z3.If(r2s30, 5, 0) , z3.If(r2s31, 5, 0) , z3.If(r2s32, 5, 0) , z3.If(r2s33, 5, 0) , z3.If(r2s34, 5, 0) , z3.If(r2s35, 5, 0) , z3.If(r2s36, 5, 0) , z3.If(r2s37, 5, 0) , z3.If(r2s38, 5, 0) , z3.If(r2s39, 5, 0) , z3.If(r2s40, 75, 0) , z3.If(r2s41, 5, 0) , z3.If(r2s42, 75, 0) , z3.If(r2s43, 20, 0) , z3.If(r2s44, 5, 0) , z3.If(r2s45, 5, 0) , z3.If(r2s46, 5, 0) , z3.If(r2s47, 5, 0) , z3.If(r2s48, 5, 0) , z3.If(r2s49, 5, 0) , z3.If(r2s50, 5, 0) , z3.If(r2s51, 5, 0) , z3.If(r2s52, 5, 0) , z3.If(r2s53, 5, 0) , z3.If(r3s0, 75, 0) , z3.If(r3s1, 5, 0) , z3.If(r3s2, 5, 0) , z3.If(r3s3, 5, 0) , z3.If(r3s4, 5, 0) , z3.If(r3s5, 5, 0) , z3.If(r3s6, 75, 0) , z3.If(r3s7, 5, 0) , z3.If(r3s8, 5, 0) , z3.If(r3s9, 5, 0) , z3.If(r3s10, 5, 0) , z3.If(r3s11, 5, 0) , z3.If(r3s12, 5, 0) , z3.If(r3s13, 75, 0) , z3.If(r3s14, 5, 0) , z3.If(r3s15, 5, 0) , z3.If(r3s16, 5, 0) , z3.If(r3s17, 5, 0) , z3.If(r3s18, 5, 0) , z3.If(r3s19, 5, 0) , z3.If(r3s20, 5, 0) , z3.If(r3s21, 5, 0) , z3.If(r3s22, 5, 0) , z3.If(r3s23, 5, 0) , z3.If(r3s24, 5, 0) , z3.If(r3s25, 5, 0) , z3.If(r3s26, 5, 0) , z3.If(r3s27, 5, 0) , z3.If(r3s28, 5, 0) , z3.If(r3s29, 5, 0) , z3.If(r3s30, 5, 0) , z3.If(r3s31, 5, 0) , z3.If(r3s32, 5, 0) , z3.If(r3s33, 5, 0) , z3.If(r3s34, 5, 0) , z3.If(r3s35, 5, 0) , z3.If(r3s36, 5, 0) , z3.If(r3s37, 75, 0) , z3.If(r3s38, 5, 0) , z3.If(r3s39, 5, 0) , z3.If(r3s40, 75, 0) , z3.If(r3s41, 5, 0) , z3.If(r3s42, 5, 0) , z3.If(r3s43, 5, 0) , z3.If(r3s44, 5, 0) , z3.If(r3s45, 5, 0) , z3.If(r3s46, 5, 0) , z3.If(r3s47, 5, 0) , z3.If(r3s48, 5, 0) , z3.If(r3s49, 5, 0) , z3.If(r3s50, 5, 0) , z3.If(r3s51, 75, 0) , z3.If(r3s52, 5, 0) , z3.If(r3s53, 5, 0) , z3.If(r4s0, 5, 0) , z3.If(r4s1, 5, 0) , z3.If(r4s2, 5, 0) , z3.If(r4s3, 5, 0) , z3.If(r4s4, 5, 0) , z3.If(r4s5, 5, 0) , z3.If(r4s6, 20, 0) , z3.If(r4s7, 5, 0) , z3.If(r4s8, 5, 0) , z3.If(r4s10, 5, 0) , z3.If(r4s11, 5, 0) , z3.If(r4s12, 75, 0) , z3.If(r4s13, 75, 0) , z3.If(r4s14, 5, 0) , z3.If(r4s15, 5, 0) , z3.If(r4s16, 5, 0) , z3.If(r4s17, 75, 0) , z3.If(r4s19, 5, 0) , z3.If(r4s20, 5, 0) , z3.If(r4s21, 5, 0) , z3.If(r4s22, 5, 0) , z3.If(r4s23, 5, 0) , z3.If(r4s24, 75, 0) , z3.If(r4s26, 5, 0) , z3.If(r4s27, 75, 0) , z3.If(r4s28, 5, 0) , z3.If(r4s29, 5, 0) , z3.If(r4s30, 5, 0) , z3.If(r4s31, 5, 0) , z3.If(r4s32, 5, 0) , z3.If(r4s33, 5, 0) , z3.If(r4s34, 5, 0) , z3.If(r4s35, 5, 0) , z3.If(r4s36, 20, 0) , z3.If(r4s37, 5, 0) , z3.If(r4s38, 5, 0) , z3.If(r4s39, 5, 0) , z3.If(r4s40, 20, 0) , z3.If(r4s41, 5, 0) , z3.If(r4s42, 75, 0) , z3.If(r4s43, 5, 0) , z3.If(r4s44, 5, 0) , z3.If(r4s45, 20, 0) , z3.If(r4s46, 20, 0) , z3.If(r4s47, 5, 0) , z3.If(r4s48, 5, 0) , z3.If(r4s49, 5, 0) , z3.If(r4s50, 5, 0) , z3.If(r4s51, 5, 0) , z3.If(r4s52, 5, 0) , z3.If(r4s53, 5, 0) , z3.If(r5s0, 5, 0) , z3.If(r5s1, 5, 0) , z3.If(r5s2, 5, 0) , z3.If(r5s3, 5, 0) , z3.If(r5s4, 5, 0) , z3.If(r5s5, 5, 0) , z3.If(r5s6, 75, 0) , z3.If(r5s7, 5, 0) , z3.If(r5s8, 5, 0) , z3.If(r5s9, 5, 0) , z3.If(r5s10, 5, 0) , z3.If(r5s11, 5, 0) , z3.If(r5s12, 5, 0) , z3.If(r5s13, 75, 0) , z3.If(r5s14, 5, 0) , z3.If(r5s15, 5, 0) , z3.If(r5s16, 5, 0) , z3.If(r5s17, 20, 0) , z3.If(r5s18, 5, 0) , z3.If(r5s19, 5, 0) , z3.If(r5s20, 5, 0) , z3.If(r5s21, 5, 0) , z3.If(r5s22, 5, 0) , z3.If(r5s23, 5, 0) , z3.If(r5s24, 20, 0) , z3.If(r5s25, 20, 0) , z3.If(r5s26, 5, 0) , z3.If(r5s27, 20, 0) , z3.If(r5s28, 5, 0) , z3.If(r5s29, 5, 0) , z3.If(r5s30, 20, 0) , z3.If(r5s31, 5, 0) , z3.If(r5s32, 5, 0) , z3.If(r5s33, 5, 0) , z3.If(r5s34, 5, 0) , z3.If(r5s35, 5, 0) , z3.If(r5s36, 20, 0) , z3.If(r5s37, 5, 0) , z3.If(r5s38, 5, 0) , z3.If(r5s39, 5, 0) , z3.If(r5s40, 20, 0) , z3.If(r5s41, 5, 0) , z3.If(r5s42, 75, 0) , z3.If(r5s43, 5, 0) , z3.If(r5s44, 5, 0) , z3.If(r5s45, 5, 0) , z3.If(r5s46, 5, 0) , z3.If(r5s47, 20, 0) , z3.If(r5s48, 5, 0) , z3.If(r5s49, 5, 0) , z3.If(r5s50, 5, 0) , z3.If(r5s51, 5, 0) , z3.If(r5s52, 5, 0) , z3.If(r5s53, 5, 0) , z3.If(r6s0, 5, 0) , z3.If(r6s1, 5, 0) , z3.If(r6s2, 5, 0) , z3.If(r6s3, 5, 0) , z3.If(r6s4, 5, 0) , z3.If(r6s5, 5, 0) , z3.If(r6s6, 20, 0) , z3.If(r6s7, 5, 0) , z3.If(r6s8, 5, 0) , z3.If(r6s9, 5, 0) , z3.If(r6s10, 5, 0) , z3.If(r6s11, 5, 0) , z3.If(r6s12, 20, 0) , z3.If(r6s13, 5, 0) , z3.If(r6s14, 5, 0) , z3.If(r6s15, 5, 0) , z3.If(r6s16, 5, 0) , z3.If(r6s17, 5, 0) , z3.If(r6s18, 5, 0) , z3.If(r6s19, 5, 0) , z3.If(r6s20, 5, 0) , z3.If(r6s21, 5, 0) , z3.If(r6s22, 5, 0) , z3.If(r6s23, 75, 0) , z3.If(r6s24, 5, 0) , z3.If(r6s25, 5, 0) , z3.If(r6s26, 5, 0) , z3.If(r6s27, 5, 0) , z3.If(r6s28, 5, 0) , z3.If(r6s29, 5, 0) , z3.If(r6s30, 5, 0) , z3.If(r6s31, 5, 0) , z3.If(r6s32, 5, 0) , z3.If(r6s33, 5, 0) , z3.If(r6s34, 5, 0) , z3.If(r6s35, 5, 0) , z3.If(r6s36, 75, 0) , z3.If(r6s37, 5, 0) , z3.If(r6s38, 5, 0) , z3.If(r6s39, 5, 0) , z3.If(r6s40, 20, 0) , z3.If(r6s41, 5, 0) , z3.If(r6s42, 5, 0) , z3.If(r6s43, 5, 0) , z3.If(r6s44, 5, 0) , z3.If(r6s45, 5, 0) , z3.If(r6s46, 5, 0) , z3.If(r6s47, 75, 0) , z3.If(r6s48, 5, 0) , z3.If(r6s49, 20, 0) , z3.If(r6s50, 75, 0) , z3.If(r6s51, 5, 0) , z3.If(r6s52, 5, 0) , z3.If(r6s53, 5, 0) , z3.If(r7s0, 20, 0) , z3.If(r7s1, 5, 0) , z3.If(r7s2, 20, 0) , z3.If(r7s3, 5, 0) , z3.If(r7s4, 5, 0) , z3.If(r7s5, 5, 0) , z3.If(r7s6, 5, 0) , z3.If(r7s7, 5, 0) , z3.If(r7s8, 5, 0) , z3.If(r7s9, 5, 0) , z3.If(r7s10, 5, 0) , z3.If(r7s11, 5, 0) , z3.If(r7s12, 75, 0) , z3.If(r7s13, 20, 0) , z3.If(r7s14, 5, 0) , z3.If(r7s15, 5, 0) , z3.If(r7s16, 5, 0) , z3.If(r7s17, 75, 0) , z3.If(r7s19, 5, 0) , z3.If(r7s20, 5, 0) , z3.If(r7s21, 5, 0) , z3.If(r7s22, 5, 0) , z3.If(r7s23, 5, 0) , z3.If(r7s25, 75, 0) , z3.If(r7s26, 5, 0) , z3.If(r7s27, 20, 0) , z3.If(r7s28, 5, 0) , z3.If(r7s29, 5, 0) , z3.If(r7s30, 20, 0) , z3.If(r7s31, 75, 0) , z3.If(r7s32, 5, 0) , z3.If(r7s33, 5, 0) , z3.If(r7s34, 5, 0) , z3.If(r7s35, 5, 0) , z3.If(r7s36, 5, 0) , z3.If(r7s37, 5, 0) , z3.If(r7s38, 5, 0) , z3.If(r7s39, 5, 0) , z3.If(r7s40, 5, 0) , z3.If(r7s41, 5, 0) , z3.If(r7s42, 75, 0) , z3.If(r7s43, 5, 0) , z3.If(r7s44, 5, 0) , z3.If(r7s45, 5, 0) , z3.If(r7s46, 5, 0) , z3.If(r7s47, 75, 0) , z3.If(r7s48, 5, 0) , z3.If(r7s49, 5, 0) , z3.If(r7s50, 5, 0) , z3.If(r7s51, 5, 0) , z3.If(r7s52, 75, 0) , z3.If(r7s53, 5, 0) , z3.If(r8s0, 5, 0) , z3.If(r8s2, 5, 0) , z3.If(r8s3, 75, 0) , z3.If(r8s4, 75, 0) , z3.If(r8s5, 5, 0) , z3.If(r8s6, 5, 0) , z3.If(r8s7, 75, 0) , z3.If(r8s8, 5, 0) , z3.If(r8s9, 5, 0) , z3.If(r8s10, 5, 0) , z3.If(r8s11, 5, 0) , z3.If(r8s12, 5, 0) , z3.If(r8s13, 5, 0) , z3.If(r8s14, 5, 0) , z3.If(r8s15, 5, 0) , z3.If(r8s16, 75, 0) , z3.If(r8s17, 5, 0) , z3.If(r8s18, 5, 0) , z3.If(r8s19, 75, 0) , z3.If(r8s20, 5, 0) , z3.If(r8s21, 5, 0) , z3.If(r8s22, 5, 0) , z3.If(r8s23, 5, 0) , z3.If(r8s24, 5, 0) , z3.If(r8s25, 5, 0) , z3.If(r8s26, 5, 0) , z3.If(r8s27, 5, 0) , z3.If(r8s28, 5, 0) , z3.If(r8s29, 5, 0) , z3.If(r8s30, 5, 0) , z3.If(r8s31, 5, 0) , z3.If(r8s32, 5, 0) , z3.If(r8s33, 5, 0) , z3.If(r8s34, 5, 0) , z3.If(r8s35, 5, 0) , z3.If(r8s36, 5, 0) , z3.If(r8s37, 5, 0) , z3.If(r8s38, 75, 0) , z3.If(r8s39, 5, 0) , z3.If(r8s40, 5, 0) , z3.If(r8s41, 5, 0) , z3.If(r8s42, 5, 0) , z3.If(r8s43, 5, 0) , z3.If(r8s44, 5, 0) , z3.If(r8s45, 5, 0) , z3.If(r8s46, 5, 0) , z3.If(r8s47, 5, 0) , z3.If(r8s48, 5, 0) , z3.If(r8s49, 5, 0) , z3.If(r8s50, 5, 0) , z3.If(r8s51, 5, 0) , z3.If(r8s52, 5, 0) , z3.If(r8s53, 5, 0) , z3.If(r9s0, 5, 0) , z3.If(r9s1, 75, 0) , z3.If(r9s2, 5, 0) , z3.If(r9s3, 5, 0) , z3.If(r9s4, 5, 0) , z3.If(r9s5, 20, 0) , z3.If(r9s6, 75, 0) , z3.If(r9s7, 5, 0) , z3.If(r9s8, 5, 0) , z3.If(r9s9, 5, 0) , z3.If(r9s10, 5, 0) , z3.If(r9s11, 5, 0) , z3.If(r9s12, 5, 0) , z3.If(r9s13, 75, 0) , z3.If(r9s14, 5, 0) , z3.If(r9s15, 5, 0) , z3.If(r9s16, 5, 0) , z3.If(r9s17, 20, 0) , z3.If(r9s18, 5, 0) , z3.If(r9s19, 5, 0) , z3.If(r9s20, 5, 0) , z3.If(r9s21, 20, 0) , z3.If(r9s22, 5, 0) , z3.If(r9s23, 5, 0) , z3.If(r9s24, 5, 0) , z3.If(r9s25, 20, 0) , z3.If(r9s26, 5, 0) , z3.If(r9s27, 20, 0) , z3.If(r9s28, 5, 0) , z3.If(r9s29, 5, 0) , z3.If(r9s30, 20, 0) , z3.If(r9s31, 5, 0) , z3.If(r9s32, 5, 0) , z3.If(r9s33, 5, 0) , z3.If(r9s34, 5, 0) , z3.If(r9s35, 5, 0) , z3.If(r9s36, 5, 0) , z3.If(r9s37, 5, 0) , z3.If(r9s38, 5, 0) , z3.If(r9s39, 5, 0) , z3.If(r9s40, 20, 0) , z3.If(r9s41, 5, 0) , z3.If(r9s42, 5, 0) , z3.If(r9s43, 5, 0) , z3.If(r9s44, 5, 0) , z3.If(r9s45, 5, 0) , z3.If(r9s46, 5, 0) , z3.If(r9s47, 5, 0) , z3.If(r9s48, 5, 0) , z3.If(r9s49, 5, 0) , z3.If(r9s50, 5, 0) , z3.If(r9s51, 5, 0) , z3.If(r9s52, 75, 0) , z3.If(r9s53, 5, 0) , z3.If(r10s0, 20, 0) , z3.If(r10s1, 5, 0) , z3.If(r10s2, 5, 0) , z3.If(r10s3, 5, 0) , z3.If(r10s4, 5, 0) , z3.If(r10s5, 5, 0) , z3.If(r10s6, 20, 0) , z3.If(r10s7, 20, 0) , z3.If(r10s8, 5, 0) , z3.If(r10s9, 20, 0) , z3.If(r10s10, 5, 0) , z3.If(r10s11, 5, 0) , z3.If(r10s12, 20, 0) , z3.If(r10s13, 20, 0) , z3.If(r10s14, 20, 0) , z3.If(r10s15, 5, 0) , z3.If(r10s16, 20, 0) , z3.If(r10s17, 5, 0) , z3.If(r10s18, 5, 0) , z3.If(r10s19, 5, 0) , z3.If(r10s20, 5, 0) , z3.If(r10s21, 5, 0) , z3.If(r10s22, 20, 0) , z3.If(r10s23, 5, 0) , z3.If(r10s24, 5, 0) , z3.If(r10s25, 5, 0) , z3.If(r10s26, 5, 0) , z3.If(r10s27, 20, 0) , z3.If(r10s28, 5, 0) , z3.If(r10s29, 5, 0) , z3.If(r10s30, 5, 0) , z3.If(r10s31, 5, 0) , z3.If(r10s32, 5, 0) , z3.If(r10s33, 5, 0) , z3.If(r10s34, 5, 0) , z3.If(r10s35, 5, 0) , z3.If(r10s36, 5, 0) , z3.If(r10s37, 5, 0) , z3.If(r10s38, 5, 0) , z3.If(r10s39, 5, 0) , z3.If(r10s40, 20, 0) , z3.If(r10s41, 5, 0) , z3.If(r10s42, 5, 0) , z3.If(r10s43, 5, 0) , z3.If(r10s44, 5, 0) , z3.If(r10s45, 5, 0) , z3.If(r10s46, 5, 0) , z3.If(r10s47, 5, 0) , z3.If(r10s48, 5, 0) , z3.If(r10s49, 5, 0) , z3.If(r10s50, 20, 0) , z3.If(r10s51, 5, 0) , z3.If(r10s52, 5, 0) , z3.If(r10s53, 5, 0) , z3.If(r11s0, 20, 0) , z3.If(r11s1, 5, 0) , z3.If(r11s2, 5, 0) , z3.If(r11s3, 20, 0) , z3.If(r11s4, 5, 0) , z3.If(r11s5, 5, 0) , z3.If(r11s6, 75, 0) , z3.If(r11s7, 5, 0) , z3.If(r11s8, 5, 0) , z3.If(r11s9, 20, 0) , z3.If(r11s10, 5, 0) , z3.If(r11s11, 5, 0) , z3.If(r11s12, 5, 0) , z3.If(r11s13, 20, 0) , z3.If(r11s14, 75, 0) , z3.If(r11s15, 5, 0) , z3.If(r11s16, 5, 0) , z3.If(r11s17, 5, 0) , z3.If(r11s18, 5, 0) , z3.If(r11s19, 5, 0) , z3.If(r11s20, 5, 0) , z3.If(r11s21, 5, 0) , z3.If(r11s22, 5, 0) , z3.If(r11s23, 5, 0) , z3.If(r11s24, 5, 0) , z3.If(r11s25, 5, 0) , z3.If(r11s26, 5, 0) , z3.If(r11s27, 75, 0) , z3.If(r11s28, 5, 0) , z3.If(r11s29, 5, 0) , z3.If(r11s30, 5, 0) , z3.If(r11s31, 5, 0) , z3.If(r11s32, 5, 0) , z3.If(r11s33, 5, 0) , z3.If(r11s34, 5, 0) , z3.If(r11s35, 5, 0) , z3.If(r11s36, 5, 0) , z3.If(r11s37, 5, 0) , z3.If(r11s38, 5, 0) , z3.If(r11s39, 5, 0) , z3.If(r11s40, 5, 0) , z3.If(r11s41, 75, 0) , z3.If(r11s42, 75, 0) , z3.If(r11s43, 5, 0) , z3.If(r11s44, 5, 0) , z3.If(r11s45, 5, 0) , z3.If(r11s46, 5, 0) , z3.If(r11s47, 5, 0) , z3.If(r11s48, 5, 0) , z3.If(r11s49, 5, 0) , z3.If(r11s50, 5, 0) , z3.If(r11s51, 5, 0) , z3.If(r11s52, 5, 0) , z3.If(r11s53, 5, 0) , z3.If(r12s0, 5, 0) , z3.If(r12s1, 20, 0) , z3.If(r12s2, 5, 0) , z3.If(r12s3, 20, 0) , z3.If(r12s4, 5, 0) , z3.If(r12s5, 5, 0) , z3.If(r12s6, 20, 0) , z3.If(r12s8, 5, 0) , z3.If(r12s9, 20, 0) , z3.If(r12s10, 5, 0) , z3.If(r12s11, 5, 0) , z3.If(r12s12, 20, 0) , z3.If(r12s13, 5, 0) , z3.If(r12s14, 5, 0) , z3.If(r12s15, 5, 0) , z3.If(r12s16, 20, 0) , z3.If(r12s17, 20, 0) , z3.If(r12s18, 5, 0) , z3.If(r12s19, 5, 0) , z3.If(r12s20, 5, 0) , z3.If(r12s21, 20, 0) , z3.If(r12s22, 5, 0) , z3.If(r12s23, 5, 0) , z3.If(r12s24, 5, 0) , z3.If(r12s25, 5, 0) , z3.If(r12s26, 5, 0) , z3.If(r12s27, 5, 0) , z3.If(r12s28, 5, 0) , z3.If(r12s30, 5, 0) , z3.If(r12s31, 5, 0) , z3.If(r12s32, 5, 0) , z3.If(r12s33, 5, 0) , z3.If(r12s34, 5, 0) , z3.If(r12s35, 5, 0) , z3.If(r12s36, 5, 0) , z3.If(r12s37, 5, 0) , z3.If(r12s38, 5, 0) , z3.If(r12s39, 5, 0) , z3.If(r12s40, 5, 0) , z3.If(r12s41, 5, 0) , z3.If(r12s42, 5, 0) , z3.If(r12s43, 5, 0) , z3.If(r12s44, 5, 0) , z3.If(r12s45, 5, 0) , z3.If(r12s46, 20, 0) , z3.If(r12s47, 5, 0) , z3.If(r12s48, 5, 0) , z3.If(r12s49, 5, 0) , z3.If(r12s50, 5, 0) , z3.If(r12s51, 5, 0) , z3.If(r12s52, 5, 0) , z3.If(r12s53, 5, 0) , z3.If(r13s0, 75, 0) , z3.If(r13s1, 75, 0) , z3.If(r13s2, 75, 0) , z3.If(r13s3, 5, 0) , z3.If(r13s4, 20, 0) , z3.If(r13s5, 5, 0) , z3.If(r13s6, 5, 0) , z3.If(r13s7, 5, 0) , z3.If(r13s8, 5, 0) , z3.If(r13s9, 75, 0) , z3.If(r13s10, 5, 0) , z3.If(r13s11, 5, 0) , z3.If(r13s12, 75, 0) , z3.If(r13s13, 5, 0) , z3.If(r13s14, 5, 0) , z3.If(r13s15, 5, 0) , z3.If(r13s16, 5, 0) , z3.If(r13s17, 20, 0) , z3.If(r13s18, 75, 0) , z3.If(r13s19, 5, 0) , z3.If(r13s20, 5, 0) , z3.If(r13s21, 5, 0) , z3.If(r13s22, 5, 0) , z3.If(r13s23, 5, 0) , z3.If(r13s24, 75, 0) , z3.If(r13s25, 75, 0) , z3.If(r13s26, 5, 0) , z3.If(r13s27, 75, 0) , z3.If(r13s28, 5, 0) , z3.If(r13s29, 5, 0) , z3.If(r13s30, 5, 0) , z3.If(r13s31, 5, 0) , z3.If(r13s32, 5, 0) , z3.If(r13s33, 5, 0) , z3.If(r13s34, 5, 0) , z3.If(r13s35, 5, 0) , z3.If(r13s36, 5, 0) , z3.If(r13s37, 5, 0) , z3.If(r13s38, 5, 0) , z3.If(r13s39, 75, 0) , z3.If(r13s40, 20, 0) , z3.If(r13s41, 5, 0) , z3.If(r13s42, 5, 0) , z3.If(r13s43, 5, 0) , z3.If(r13s44, 5, 0) , z3.If(r13s45, 5, 0) , z3.If(r13s46, 5, 0) , z3.If(r13s47, 5, 0) , z3.If(r13s48, 5, 0) , z3.If(r13s49, 5, 0) , z3.If(r13s50, 5, 0) , z3.If(r13s51, 5, 0) , z3.If(r13s52, 5, 0) , z3.If(r13s53, 5, 0) , z3.If(r14s0, 5, 0) , z3.If(r14s1, 20, 0) , z3.If(r14s2, 5, 0) , z3.If(r14s3, 5, 0) , z3.If(r14s4, 5, 0) , z3.If(r14s5, 5, 0) , z3.If(r14s6, 75, 0) , z3.If(r14s7, 20, 0) , z3.If(r14s8, 5, 0) , z3.If(r14s9, 20, 0) , z3.If(r14s10, 5, 0) , z3.If(r14s11, 5, 0) , z3.If(r14s12, 5, 0) , z3.If(r14s14, 75, 0) , z3.If(r14s15, 5, 0) , z3.If(r14s16, 5, 0) , z3.If(r14s17, 5, 0) , z3.If(r14s18, 5, 0) , z3.If(r14s19, 5, 0) , z3.If(r14s20, 5, 0) , z3.If(r14s21, 5, 0) , z3.If(r14s22, 5, 0) , z3.If(r14s23, 5, 0) , z3.If(r14s25, 5, 0) , z3.If(r14s26, 5, 0) , z3.If(r14s27, 5, 0) , z3.If(r14s28, 5, 0) , z3.If(r14s29, 5, 0) , z3.If(r14s30, 20, 0) , z3.If(r14s31, 5, 0) , z3.If(r14s32, 5, 0) , z3.If(r14s33, 5, 0) , z3.If(r14s34, 5, 0) , z3.If(r14s35, 5, 0) , z3.If(r14s36, 5, 0) , z3.If(r14s37, 5, 0) , z3.If(r14s38, 5, 0) , z3.If(r14s39, 5, 0) , z3.If(r14s40, 75, 0) , z3.If(r14s41, 5, 0) , z3.If(r14s42, 5, 0) , z3.If(r14s43, 5, 0) , z3.If(r14s44, 5, 0) , z3.If(r14s45, 5, 0) , z3.If(r14s46, 20, 0) , z3.If(r14s47, 5, 0) , z3.If(r14s48, 5, 0) , z3.If(r14s49, 5, 0) , z3.If(r14s50, 5, 0) , z3.If(r14s51, 5, 0) , z3.If(r14s52, 5, 0) , z3.If(r14s53, 5, 0) , z3.If(r15s0, 20, 0) , z3.If(r15s1, 5, 0) , z3.If(r15s2, 5, 0) , z3.If(r15s3, 5, 0) , z3.If(r15s4, 5, 0) , z3.If(r15s5, 5, 0) , z3.If(r15s6, 5, 0) , z3.If(r15s7, 5, 0) , z3.If(r15s8, 5, 0) , z3.If(r15s9, 5, 0) , z3.If(r15s10, 5, 0) , z3.If(r15s11, 5, 0) , z3.If(r15s12, 5, 0) , z3.If(r15s13, 75, 0) , z3.If(r15s14, 5, 0) , z3.If(r15s15, 5, 0) , z3.If(r15s16, 5, 0) , z3.If(r15s17, 5, 0) , z3.If(r15s18, 5, 0) , z3.If(r15s19, 5, 0) , z3.If(r15s20, 5, 0) , z3.If(r15s21, 5, 0) , z3.If(r15s22, 20, 0) , z3.If(r15s23, 5, 0) , z3.If(r15s24, 20, 0) , z3.If(r15s25, 20, 0) , z3.If(r15s26, 5, 0) , z3.If(r15s27, 75, 0) , z3.If(r15s28, 5, 0) , z3.If(r15s29, 5, 0) , z3.If(r15s30, 5, 0) , z3.If(r15s31, 5, 0) , z3.If(r15s32, 5, 0) , z3.If(r15s33, 5, 0) , z3.If(r15s34, 5, 0) , z3.If(r15s35, 5, 0) , z3.If(r15s36, 5, 0) , z3.If(r15s37, 5, 0) , z3.If(r15s38, 5, 0) , z3.If(r15s39, 5, 0) , z3.If(r15s40, 20, 0) , z3.If(r15s41, 5, 0) , z3.If(r15s42, 5, 0) , z3.If(r15s43, 5, 0) , z3.If(r15s44, 5, 0) , z3.If(r15s45, 5, 0) , z3.If(r15s46, 5, 0) , z3.If(r15s47, 5, 0) , z3.If(r15s48, 5, 0) , z3.If(r15s49, 20, 0) , z3.If(r15s50, 5, 0) , z3.If(r15s51, 5, 0) , z3.If(r15s52, 5, 0) , z3.If(r15s53, 5, 0) , z3.If(r16s0, 20, 0) , z3.If(r16s1, 20, 0) , z3.If(r16s2, 5, 0) , z3.If(r16s3, 20, 0) , z3.If(r16s4, 5, 0) , z3.If(r16s5, 20, 0) , z3.If(r16s6, 5, 0) , z3.If(r16s7, 20, 0) , z3.If(r16s8, 5, 0) , z3.If(r16s9, 5, 0) , z3.If(r16s10, 5, 0) , z3.If(r16s11, 75, 0) , z3.If(r16s12, 5, 0) , z3.If(r16s13, 5, 0) , z3.If(r16s14, 5, 0) , z3.If(r16s15, 5, 0) , z3.If(r16s16, 5, 0) , z3.If(r16s17, 5, 0) , z3.If(r16s18, 5, 0) , z3.If(r16s19, 20, 0) , z3.If(r16s20, 5, 0) , z3.If(r16s21, 5, 0) , z3.If(r16s22, 5, 0) , z3.If(r16s23, 5, 0) , z3.If(r16s24, 5, 0) , z3.If(r16s25, 5, 0) , z3.If(r16s26, 5, 0) , z3.If(r16s27, 20, 0) , z3.If(r16s28, 5, 0) , z3.If(r16s29, 5, 0) , z3.If(r16s30, 5, 0) , z3.If(r16s31, 5, 0) , z3.If(r16s32, 5, 0) , z3.If(r16s33, 5, 0) , z3.If(r16s34, 5, 0) , z3.If(r16s35, 5, 0) , z3.If(r16s36, 5, 0) , z3.If(r16s37, 20, 0) , z3.If(r16s38, 5, 0) , z3.If(r16s39, 5, 0) , z3.If(r16s40, 5, 0) , z3.If(r16s41, 5, 0) , z3.If(r16s42, 5, 0) , z3.If(r16s44, 5, 0) , z3.If(r16s45, 5, 0) , z3.If(r16s46, 75, 0) , z3.If(r16s47, 5, 0) , z3.If(r16s48, 5, 0) , z3.If(r16s50, 5, 0) , z3.If(r16s52, 5, 0) , z3.If(r16s53, 5, 0) , z3.If(r17s0, 5, 0) , z3.If(r17s1, 75, 0) , z3.If(r17s2, 5, 0) , z3.If(r17s3, 75, 0) , z3.If(r17s4, 5, 0) , z3.If(r17s5, 5, 0) , z3.If(r17s6, 5, 0) , z3.If(r17s7, 5, 0) , z3.If(r17s8, 5, 0) , z3.If(r17s9, 75, 0) , z3.If(r17s10, 75, 0) , z3.If(r17s11, 5, 0) , z3.If(r17s12, 5, 0) , z3.If(r17s13, 5, 0) , z3.If(r17s14, 5, 0) , z3.If(r17s15, 5, 0) , z3.If(r17s16, 5, 0) , z3.If(r17s17, 5, 0) , z3.If(r17s18, 5, 0) , z3.If(r17s19, 5, 0) , z3.If(r17s20, 5, 0) , z3.If(r17s22, 5, 0) , z3.If(r17s23, 5, 0) , z3.If(r17s24, 5, 0) , z3.If(r17s25, 5, 0) , z3.If(r17s27, 5, 0) , z3.If(r17s28, 5, 0) , z3.If(r17s29, 75, 0) , z3.If(r17s30, 5, 0) , z3.If(r17s31, 5, 0) , z3.If(r17s32, 5, 0) , z3.If(r17s33, 75, 0) , z3.If(r17s34, 5, 0) , z3.If(r17s35, 5, 0) , z3.If(r17s36, 5, 0) , z3.If(r17s37, 5, 0) , z3.If(r17s38, 5, 0) , z3.If(r17s39, 5, 0) , z3.If(r17s40, 5, 0) , z3.If(r17s41, 5, 0) , z3.If(r17s42, 5, 0) , z3.If(r17s43, 5, 0) , z3.If(r17s44, 75, 0) , z3.If(r17s46, 5, 0) , z3.If(r17s47, 5, 0) , z3.If(r17s49, 5, 0) , z3.If(r17s50, 5, 0) , z3.If(r17s51, 5, 0) , z3.If(r17s52, 5, 0) , z3.If(r18s0, 5, 0) , z3.If(r18s1, 5, 0) , z3.If(r18s2, 5, 0) , z3.If(r18s3, 5, 0) , z3.If(r18s4, 5, 0) , z3.If(r18s5, 75, 0) , z3.If(r18s6, 5, 0) , z3.If(r18s7, 5, 0) , z3.If(r18s8, 5, 0) , z3.If(r18s9, 5, 0) , z3.If(r18s10, 5, 0) , z3.If(r18s11, 5, 0) , z3.If(r18s12, 75, 0) , z3.If(r18s13, 5, 0) , z3.If(r18s14, 20, 0) , z3.If(r18s15, 5, 0) , z3.If(r18s16, 5, 0) , z3.If(r18s17, 20, 0) , z3.If(r18s18, 20, 0) , z3.If(r18s19, 5, 0) , z3.If(r18s20, 5, 0) , z3.If(r18s21, 5, 0) , z3.If(r18s22, 75, 0) , z3.If(r18s23, 5, 0) , z3.If(r18s24, 5, 0) , z3.If(r18s25, 5, 0) , z3.If(r18s26, 5, 0) , z3.If(r18s27, 5, 0) , z3.If(r18s28, 5, 0) , z3.If(r18s29, 5, 0) , z3.If(r18s30, 5, 0) , z3.If(r18s31, 5, 0) , z3.If(r18s32, 5, 0) , z3.If(r18s33, 5, 0) , z3.If(r18s34, 5, 0) , z3.If(r18s35, 5, 0) , z3.If(r18s36, 20, 0) , z3.If(r18s37, 5, 0) , z3.If(r18s38, 5, 0) , z3.If(r18s39, 5, 0) , z3.If(r18s40, 5, 0) , z3.If(r18s41, 5, 0) , z3.If(r18s42, 5, 0) , z3.If(r18s43, 5, 0) , z3.If(r18s44, 5, 0) , z3.If(r18s45, 5, 0) , z3.If(r18s46, 5, 0) , z3.If(r18s47, 5, 0) , z3.If(r18s48, 5, 0) , z3.If(r18s49, 5, 0) , z3.If(r18s50, 75, 0) , z3.If(r18s51, 5, 0) , z3.If(r18s52, 5, 0) , z3.If(r18s53, 5, 0) , z3.If(r19s0, 20, 0) , z3.If(r19s1, 5, 0) , z3.If(r19s2, 75, 0) , z3.If(r19s3, 5, 0) , z3.If(r19s4, 5, 0) , z3.If(r19s5, 5, 0) , z3.If(r19s6, 5, 0) , z3.If(r19s7, 5, 0) , z3.If(r19s8, 5, 0) , z3.If(r19s9, 5, 0) , z3.If(r19s10, 5, 0) , z3.If(r19s11, 5, 0) , z3.If(r19s12, 5, 0) , z3.If(r19s13, 20, 0) , z3.If(r19s14, 5, 0) , z3.If(r19s15, 5, 0) , z3.If(r19s16, 20, 0) , z3.If(r19s17, 5, 0) , z3.If(r19s18, 5, 0) , z3.If(r19s19, 5, 0) , z3.If(r19s20, 5, 0) , z3.If(r19s21, 5, 0) , z3.If(r19s22, 5, 0) , z3.If(r19s23, 5, 0) , z3.If(r19s24, 5, 0) , z3.If(r19s25, 5, 0) , z3.If(r19s26, 5, 0) , z3.If(r19s27, 5, 0) , z3.If(r19s28, 5, 0) , z3.If(r19s29, 5, 0) , z3.If(r19s30, 5, 0) , z3.If(r19s31, 5, 0) , z3.If(r19s32, 5, 0) , z3.If(r19s33, 5, 0) , z3.If(r19s34, 5, 0) , z3.If(r19s35, 5, 0) , z3.If(r19s36, 5, 0) , z3.If(r19s37, 5, 0) , z3.If(r19s38, 5, 0) , z3.If(r19s39, 5, 0) , z3.If(r19s40, 5, 0) , z3.If(r19s41, 5, 0) , z3.If(r19s42, 5, 0) , z3.If(r19s43, 5, 0) , z3.If(r19s44, 5, 0) , z3.If(r19s45, 5, 0) , z3.If(r19s46, 5, 0) , z3.If(r19s47, 5, 0) , z3.If(r19s48, 5, 0) , z3.If(r19s49, 5, 0) , z3.If(r19s50, 5, 0) , z3.If(r19s51, 75, 0) , z3.If(r19s52, 5, 0) , z3.If(r19s53, 5, 0) , z3.If(r20s0, 75, 0) , z3.If(r20s1, 5, 0) , z3.If(r20s2, 75, 0) , z3.If(r20s3, 5, 0) , z3.If(r20s4, 5, 0) , z3.If(r20s5, 5, 0) , z3.If(r20s6, 75, 0) , z3.If(r20s7, 5, 0) , z3.If(r20s8, 5, 0) , z3.If(r20s9, 75, 0) , z3.If(r20s10, 5, 0) , z3.If(r20s11, 5, 0) , z3.If(r20s13, 5, 0) , z3.If(r20s14, 5, 0) , z3.If(r20s15, 5, 0) , z3.If(r20s16, 5, 0) , z3.If(r20s17, 75, 0) , z3.If(r20s18, 75, 0) , z3.If(r20s19, 5, 0) , z3.If(r20s20, 5, 0) , z3.If(r20s21, 5, 0) , z3.If(r20s22, 75, 0) , z3.If(r20s23, 5, 0) , z3.If(r20s24, 75, 0) , z3.If(r20s25, 75, 0) , z3.If(r20s26, 5, 0) , z3.If(r20s27, 75, 0) , z3.If(r20s28, 5, 0) , z3.If(r20s29, 5, 0) , z3.If(r20s30, 75, 0) , z3.If(r20s31, 75, 0) , z3.If(r20s32, 5, 0) , z3.If(r20s33, 5, 0) , z3.If(r20s34, 5, 0) , z3.If(r20s35, 5, 0) , z3.If(r20s36, 5, 0) , z3.If(r20s37, 5, 0) , z3.If(r20s38, 5, 0) , z3.If(r20s39, 5, 0) , z3.If(r20s40, 5, 0) , z3.If(r20s41, 5, 0) , z3.If(r20s42, 5, 0) , z3.If(r20s43, 5, 0) , z3.If(r20s44, 5, 0) , z3.If(r20s45, 5, 0) , z3.If(r20s46, 5, 0) , z3.If(r20s47, 5, 0) , z3.If(r20s48, 5, 0) , z3.If(r20s49, 5, 0) , z3.If(r20s50, 5, 0) , z3.If(r20s51, 5, 0) , z3.If(r20s52, 5, 0) , z3.If(r20s53, 5, 0) , z3.If(r21s0, 75, 0) , z3.If(r21s1, 5, 0) , z3.If(r21s2, 5, 0) , z3.If(r21s3, 75, 0) , z3.If(r21s4, 5, 0) , z3.If(r21s5, 5, 0) , z3.If(r21s6, 20, 0) , z3.If(r21s7, 20, 0) , z3.If(r21s8, 5, 0) , z3.If(r21s9, 75, 0) , z3.If(r21s10, 5, 0) , z3.If(r21s11, 5, 0) , z3.If(r21s12, 5, 0) , z3.If(r21s13, 20, 0) , z3.If(r21s14, 5, 0) , z3.If(r21s15, 5, 0) , z3.If(r21s16, 20, 0) , z3.If(r21s17, 5, 0) , z3.If(r21s18, 20, 0) , z3.If(r21s19, 5, 0) , z3.If(r21s20, 5, 0) , z3.If(r21s21, 5, 0) , z3.If(r21s22, 75, 0) , z3.If(r21s23, 5, 0) , z3.If(r21s24, 5, 0) , z3.If(r21s25, 5, 0) , z3.If(r21s26, 5, 0) , z3.If(r21s27, 20, 0) , z3.If(r21s28, 75, 0) , z3.If(r21s29, 5, 0) , z3.If(r21s30, 5, 0) , z3.If(r21s31, 5, 0) , z3.If(r21s32, 5, 0) , z3.If(r21s33, 5, 0) , z3.If(r21s34, 75, 0) , z3.If(r21s35, 5, 0) , z3.If(r21s36, 5, 0) , z3.If(r21s37, 5, 0) , z3.If(r21s38, 5, 0) , z3.If(r21s39, 5, 0) , z3.If(r21s40, 75, 0) , z3.If(r21s41, 5, 0) , z3.If(r21s43, 5, 0) , z3.If(r21s44, 5, 0) , z3.If(r21s45, 5, 0) , z3.If(r21s46, 5, 0) , z3.If(r21s47, 5, 0) , z3.If(r21s48, 5, 0) , z3.If(r21s49, 5, 0) , z3.If(r21s50, 5, 0) , z3.If(r21s51, 5, 0) , z3.If(r21s52, 5, 0) , z3.If(r21s53, 5, 0) , z3.If(r22s0, 5, 0) , z3.If(r22s1, 5, 0) , z3.If(r22s2, 5, 0) , z3.If(r22s3, 75, 0) , z3.If(r22s4, 5, 0) , z3.If(r22s5, 5, 0) , z3.If(r22s6, 5, 0) , z3.If(r22s7, 20, 0) , z3.If(r22s8, 20, 0) , z3.If(r22s9, 5, 0) , z3.If(r22s10, 5, 0) , z3.If(r22s11, 5, 0) , z3.If(r22s12, 20, 0) , z3.If(r22s13, 5, 0) , z3.If(r22s14, 5, 0) , z3.If(r22s15, 5, 0) , z3.If(r22s16, 75, 0) , z3.If(r22s17, 20, 0) , z3.If(r22s18, 5, 0) , z3.If(r22s19, 5, 0) , z3.If(r22s20, 5, 0) , z3.If(r22s21, 5, 0) , z3.If(r22s22, 5, 0) , z3.If(r22s23, 5, 0) , z3.If(r22s24, 5, 0) , z3.If(r22s25, 5, 0) , z3.If(r22s26, 5, 0) , z3.If(r22s27, 5, 0) , z3.If(r22s28, 5, 0) , z3.If(r22s29, 5, 0) , z3.If(r22s30, 5, 0) , z3.If(r22s31, 5, 0) , z3.If(r22s32, 5, 0) , z3.If(r22s33, 5, 0) , z3.If(r22s34, 5, 0) , z3.If(r22s35, 5, 0) , z3.If(r22s36, 5, 0) , z3.If(r22s37, 5, 0) , z3.If(r22s38, 5, 0) , z3.If(r22s39, 5, 0) , z3.If(r22s40, 5, 0) , z3.If(r22s41, 5, 0) , z3.If(r22s42, 5, 0) , z3.If(r22s43, 5, 0) , z3.If(r22s44, 5, 0) , z3.If(r22s45, 5, 0) , z3.If(r22s46, 5, 0) , z3.If(r22s47, 5, 0) , z3.If(r22s48, 5, 0) , z3.If(r22s49, 5, 0) , z3.If(r22s50, 5, 0) , z3.If(r22s51, 5, 0) , z3.If(r22s52, 5, 0) , z3.If(r22s53, 5, 0) , z3.If(r23s0, 5, 0) , z3.If(r23s1, 75, 0) , z3.If(r23s2, 5, 0) , z3.If(r23s3, 20, 0) , z3.If(r23s4, 5, 0) , z3.If(r23s5, 5, 0) , z3.If(r23s6, 75, 0) , z3.If(r23s7, 5, 0) , z3.If(r23s8, 5, 0) , z3.If(r23s9, 5, 0) , z3.If(r23s10, 5, 0) , z3.If(r23s11, 5, 0) , z3.If(r23s12, 5, 0) , z3.If(r23s13, 5, 0) , z3.If(r23s14, 75, 0) , z3.If(r23s15, 5, 0) , z3.If(r23s16, 20, 0) , z3.If(r23s17, 75, 0) , z3.If(r23s19, 5, 0) , z3.If(r23s20, 5, 0) , z3.If(r23s21, 5, 0) , z3.If(r23s22, 5, 0) , z3.If(r23s23, 5, 0) , z3.If(r23s24, 5, 0) , z3.If(r23s25, 75, 0) , z3.If(r23s26, 5, 0) , z3.If(r23s27, 5, 0) , z3.If(r23s28, 5, 0) , z3.If(r23s29, 5, 0) , z3.If(r23s30, 75, 0) , z3.If(r23s31, 20, 0) , z3.If(r23s32, 5, 0) , z3.If(r23s33, 5, 0) , z3.If(r23s34, 5, 0) , z3.If(r23s35, 5, 0) , z3.If(r23s36, 5, 0) , z3.If(r23s37, 75, 0) , z3.If(r23s38, 5, 0) , z3.If(r23s39, 5, 0) , z3.If(r23s41, 5, 0) , z3.If(r23s42, 5, 0) , z3.If(r23s43, 5, 0) , z3.If(r23s44, 5, 0) , z3.If(r23s45, 5, 0) , z3.If(r23s46, 5, 0) , z3.If(r23s47, 5, 0) , z3.If(r23s48, 5, 0) , z3.If(r23s49, 5, 0) , z3.If(r23s50, 5, 0) , z3.If(r23s51, 5, 0) , z3.If(r23s52, 5, 0) , z3.If(r23s53, 5, 0) , z3.If(r24s0, 20, 0) , z3.If(r24s1, 5, 0) , z3.If(r24s2, 5, 0) , z3.If(r24s3, 5, 0) , z3.If(r24s4, 5, 0) , z3.If(r24s5, 5, 0) , z3.If(r24s6, 20, 0) , z3.If(r24s7, 5, 0) , z3.If(r24s8, 5, 0) , z3.If(r24s9, 5, 0) , z3.If(r24s10, 5, 0) , z3.If(r24s11, 5, 0) , z3.If(r24s12, 5, 0) , z3.If(r24s14, 5, 0) , z3.If(r24s15, 75, 0) , z3.If(r24s16, 20, 0) , z3.If(r24s17, 5, 0) , z3.If(r24s18, 5, 0) , z3.If(r24s19, 20, 0) , z3.If(r24s20, 75, 0) , z3.If(r24s21, 5, 0) , z3.If(r24s22, 75, 0) , z3.If(r24s23, 5, 0) , z3.If(r24s24, 5, 0) , z3.If(r24s25, 5, 0) , z3.If(r24s26, 5, 0) , z3.If(r24s27, 20, 0) , z3.If(r24s28, 5, 0) , z3.If(r24s29, 5, 0) , z3.If(r24s30, 5, 0) , z3.If(r24s31, 5, 0) , z3.If(r24s32, 5, 0) , z3.If(r24s33, 5, 0) , z3.If(r24s34, 5, 0) , z3.If(r24s35, 5, 0) , z3.If(r24s36, 5, 0) , z3.If(r24s37, 5, 0) , z3.If(r24s38, 5, 0) , z3.If(r24s39, 5, 0) , z3.If(r24s40, 5, 0) , z3.If(r24s42, 5, 0) , z3.If(r24s44, 5, 0) , z3.If(r24s45, 5, 0) , z3.If(r24s46, 20, 0) , z3.If(r24s47, 5, 0) , z3.If(r24s48, 5, 0) , z3.If(r24s50, 5, 0) , z3.If(r24s52, 5, 0) , z3.If(r24s53, 5, 0) , z3.If(r25s0, 5, 0) , z3.If(r25s1, 5, 0) , z3.If(r25s2, 20, 0) , z3.If(r25s3, 5, 0) , z3.If(r25s4, 5, 0) , z3.If(r25s5, 5, 0) , z3.If(r25s6, 75, 0) , z3.If(r25s7, 20, 0) , z3.If(r25s8, 5, 0) , z3.If(r25s9, 20, 0) , z3.If(r25s10, 5, 0) , z3.If(r25s11, 5, 0) , z3.If(r25s13, 5, 0) , z3.If(r25s14, 5, 0) , z3.If(r25s15, 5, 0) , z3.If(r25s16, 20, 0) , z3.If(r25s17, 75, 0) , z3.If(r25s19, 5, 0) , z3.If(r25s20, 5, 0) , z3.If(r25s21, 5, 0) , z3.If(r25s22, 5, 0) , z3.If(r25s23, 5, 0) , z3.If(r25s24, 5, 0) , z3.If(r25s25, 5, 0) , z3.If(r25s26, 5, 0) , z3.If(r25s27, 75, 0) , z3.If(r25s28, 5, 0) , z3.If(r25s29, 5, 0) , z3.If(r25s30, 20, 0) , z3.If(r25s31, 20, 0) , z3.If(r25s32, 20, 0) , z3.If(r25s33, 5, 0) , z3.If(r25s34, 5, 0) , z3.If(r25s35, 5, 0) , z3.If(r25s36, 20, 0) , z3.If(r25s37, 5, 0) , z3.If(r25s38, 5, 0) , z3.If(r25s39, 5, 0) , z3.If(r25s40, 20, 0) , z3.If(r25s41, 5, 0) , z3.If(r25s42, 20, 0) , z3.If(r25s43, 20, 0) , z3.If(r25s44, 5, 0) , z3.If(r25s45, 5, 0) , z3.If(r25s46, 5, 0) , z3.If(r25s47, 20, 0) , z3.If(r25s48, 5, 0) , z3.If(r25s49, 5, 0) , z3.If(r25s50, 5, 0) , z3.If(r25s51, 5, 0) , z3.If(r25s52, 5, 0) , z3.If(r25s53, 5, 0) , z3.If(r26s0, 75, 0) , z3.If(r26s1, 20, 0) , z3.If(r26s2, 5, 0) , z3.If(r26s3, 75, 0) , z3.If(r26s4, 75, 0) , z3.If(r26s5, 5, 0) , z3.If(r26s6, 5, 0) , z3.If(r26s7, 5, 0) , z3.If(r26s8, 5, 0) , z3.If(r26s9, 5, 0) , z3.If(r26s10, 5, 0) , z3.If(r26s11, 75, 0) , z3.If(r26s12, 5, 0) , z3.If(r26s13, 5, 0) , z3.If(r26s14, 5, 0) , z3.If(r26s15, 5, 0) , z3.If(r26s16, 5, 0) , z3.If(r26s17, 5, 0) , z3.If(r26s18, 5, 0) , z3.If(r26s20, 75, 0) , z3.If(r26s21, 75, 0) , z3.If(r26s22, 5, 0) , z3.If(r26s23, 5, 0) , z3.If(r26s24, 5, 0) , z3.If(r26s25, 5, 0) , z3.If(r26s26, 5, 0) , z3.If(r26s27, 5, 0) , z3.If(r26s28, 5, 0) , z3.If(r26s29, 20, 0) , z3.If(r26s30, 5, 0) , z3.If(r26s31, 5, 0) , z3.If(r26s32, 5, 0) , z3.If(r26s33, 5, 0) , z3.If(r26s34, 5, 0) , z3.If(r26s35, 5, 0) , z3.If(r26s36, 5, 0) , z3.If(r26s37, 5, 0) , z3.If(r26s38, 5, 0) , z3.If(r26s39, 5, 0) , z3.If(r26s40, 5, 0) , z3.If(r26s41, 5, 0) , z3.If(r26s42, 5, 0) , z3.If(r26s43, 75, 0) , z3.If(r26s44, 5, 0) , z3.If(r26s45, 5, 0) , z3.If(r26s46, 5, 0) , z3.If(r26s47, 75, 0) , z3.If(r26s48, 5, 0) , z3.If(r26s49, 75, 0) , z3.If(r26s50, 5, 0) , z3.If(r26s51, 75, 0) , z3.If(r26s52, 5, 0) , z3.If(r26s53, 5, 0) , z3.If(r27s0, 5, 0) , z3.If(r27s1, 75, 0) , z3.If(r27s2, 5, 0) , z3.If(r27s3, 5, 0) , z3.If(r27s4, 5, 0) , z3.If(r27s5, 5, 0) , z3.If(r27s6, 20, 0) , z3.If(r27s7, 5, 0) , z3.If(r27s8, 5, 0) , z3.If(r27s9, 75, 0) , z3.If(r27s10, 5, 0) , z3.If(r27s11, 5, 0) , z3.If(r27s12, 5, 0) , z3.If(r27s13, 5, 0) , z3.If(r27s14, 75, 0) , z3.If(r27s15, 5, 0) , z3.If(r27s16, 5, 0) , z3.If(r27s17, 5, 0) , z3.If(r27s18, 5, 0) , z3.If(r27s19, 5, 0) , z3.If(r27s20, 5, 0) , z3.If(r27s21, 5, 0) , z3.If(r27s22, 5, 0) , z3.If(r27s23, 5, 0) , z3.If(r27s24, 20, 0) , z3.If(r27s25, 5, 0) , z3.If(r27s26, 5, 0) , z3.If(r27s27, 5, 0) , z3.If(r27s28, 5, 0) , z3.If(r27s29, 5, 0) , z3.If(r27s30, 5, 0) , z3.If(r27s31, 5, 0) , z3.If(r27s32, 5, 0) , z3.If(r27s33, 5, 0) , z3.If(r27s34, 5, 0) , z3.If(r27s35, 5, 0) , z3.If(r27s36, 5, 0) , z3.If(r27s37, 75, 0) , z3.If(r27s38, 5, 0) , z3.If(r27s39, 20, 0) , z3.If(r27s40, 5, 0) , z3.If(r27s41, 5, 0) , z3.If(r27s42, 75, 0) , z3.If(r27s43, 20, 0) , z3.If(r27s44, 5, 0) , z3.If(r27s45, 5, 0) , z3.If(r27s46, 5, 0) , z3.If(r27s47, 5, 0) , z3.If(r27s48, 5, 0) , z3.If(r27s49, 5, 0) , z3.If(r27s50, 75, 0) , z3.If(r27s51, 5, 0) , z3.If(r27s52, 5, 0) , z3.If(r27s53, 5, 0) , z3.If(r28s0, 5, 0) , z3.If(r28s1, 20, 0) , z3.If(r28s2, 5, 0) , z3.If(r28s3, 5, 0) , z3.If(r28s4, 5, 0) , z3.If(r28s5, 5, 0) , z3.If(r28s6, 20, 0) , z3.If(r28s7, 5, 0) , z3.If(r28s8, 5, 0) , z3.If(r28s9, 20, 0) , z3.If(r28s10, 5, 0) , z3.If(r28s11, 5, 0) , z3.If(r28s12, 20, 0) , z3.If(r28s13, 75, 0) , z3.If(r28s14, 5, 0) , z3.If(r28s15, 5, 0) , z3.If(r28s16, 5, 0) , z3.If(r28s17, 20, 0) , z3.If(r28s18, 75, 0) , z3.If(r28s19, 5, 0) , z3.If(r28s20, 5, 0) , z3.If(r28s21, 5, 0) , z3.If(r28s23, 5, 0) , z3.If(r28s24, 75, 0) , z3.If(r28s25, 75, 0) , z3.If(r28s26, 5, 0) , z3.If(r28s29, 5, 0) , z3.If(r28s30, 20, 0) , z3.If(r28s32, 5, 0) , z3.If(r28s33, 5, 0) , z3.If(r28s35, 5, 0) , z3.If(r28s36, 5, 0) , z3.If(r28s37, 5, 0) , z3.If(r28s38, 5, 0) , z3.If(r28s39, 5, 0) , z3.If(r28s40, 75, 0) , z3.If(r28s41, 5, 0) , z3.If(r28s42, 20, 0) , z3.If(r28s43, 5, 0) , z3.If(r28s44, 5, 0) , z3.If(r28s45, 5, 0) , z3.If(r28s46, 5, 0) , z3.If(r28s47, 5, 0) , z3.If(r28s48, 5, 0) , z3.If(r28s49, 5, 0) , z3.If(r28s50, 5, 0) , z3.If(r28s51, 5, 0) , z3.If(r28s52, 5, 0) , z3.If(r28s53, 5, 0) , z3.If(r29s0, 5, 0) , z3.If(r29s1, 5, 0) , z3.If(r29s2, 5, 0) , z3.If(r29s3, 5, 0) , z3.If(r29s4, 5, 0) , z3.If(r29s5, 5, 0) , z3.If(r29s6, 75, 0) , z3.If(r29s7, 5, 0) , z3.If(r29s8, 5, 0) , z3.If(r29s9, 5, 0) , z3.If(r29s10, 5, 0) , z3.If(r29s11, 5, 0) , z3.If(r29s12, 75, 0) , z3.If(r29s13, 5, 0) , z3.If(r29s14, 5, 0) , z3.If(r29s15, 5, 0) , z3.If(r29s16, 75, 0) , z3.If(r29s17, 75, 0) , z3.If(r29s18, 5, 0) , z3.If(r29s19, 5, 0) , z3.If(r29s20, 5, 0) , z3.If(r29s21, 5, 0) , z3.If(r29s23, 5, 0) , z3.If(r29s24, 75, 0) , z3.If(r29s25, 75, 0) , z3.If(r29s26, 5, 0) , z3.If(r29s29, 5, 0) , z3.If(r29s30, 75, 0) , z3.If(r29s32, 75, 0) , z3.If(r29s33, 5, 0) , z3.If(r29s35, 5, 0) , z3.If(r29s36, 75, 0) , z3.If(r29s37, 5, 0) , z3.If(r29s38, 5, 0) , z3.If(r29s39, 5, 0) , z3.If(r29s40, 75, 0) , z3.If(r29s41, 5, 0) , z3.If(r29s42, 75, 0) , z3.If(r29s43, 5, 0) , z3.If(r29s44, 5, 0) , z3.If(r29s45, 5, 0) , z3.If(r29s46, 5, 0) , z3.If(r29s47, 5, 0) , z3.If(r29s48, 5, 0) , z3.If(r29s49, 5, 0) , z3.If(r29s50, 5, 0) , z3.If(r29s51, 5, 0) , z3.If(r29s52, 5, 0) , z3.If(r29s53, 5, 0) , z3.If(r30s0, 5, 0) , z3.If(r30s1, 5, 0) , z3.If(r30s2, 5, 0) , z3.If(r30s3, 75, 0) , z3.If(r30s4, 5, 0) , z3.If(r30s5, 20, 0) , z3.If(r30s6, 75, 0) , z3.If(r30s7, 5, 0) , z3.If(r30s8, 5, 0) , z3.If(r30s9, 5, 0) , z3.If(r30s10, 5, 0) , z3.If(r30s11, 20, 0) , z3.If(r30s12, 5, 0) , z3.If(r30s13, 75, 0) , z3.If(r30s14, 5, 0) , z3.If(r30s15, 75, 0) , z3.If(r30s16, 20, 0) , z3.If(r30s17, 5, 0) , z3.If(r30s18, 5, 0) , z3.If(r30s19, 5, 0) , z3.If(r30s20, 5, 0) , z3.If(r30s21, 20, 0) , z3.If(r30s22, 5, 0) , z3.If(r30s23, 5, 0) , z3.If(r30s24, 5, 0) , z3.If(r30s25, 75, 0) , z3.If(r30s26, 5, 0) , z3.If(r30s27, 5, 0) , z3.If(r30s28, 5, 0) , z3.If(r30s29, 20, 0) , z3.If(r30s30, 5, 0) , z3.If(r30s31, 5, 0) , z3.If(r30s32, 20, 0) , z3.If(r30s33, 5, 0) , z3.If(r30s34, 5, 0) , z3.If(r30s35, 20, 0) , z3.If(r30s37, 5, 0) , z3.If(r30s38, 5, 0) , z3.If(r30s39, 5, 0) , z3.If(r30s40, 5, 0) , z3.If(r30s41, 5, 0) , z3.If(r30s42, 5, 0) , z3.If(r30s43, 5, 0) , z3.If(r30s44, 5, 0) , z3.If(r30s45, 20, 0) , z3.If(r30s46, 20, 0) , z3.If(r30s47, 5, 0) , z3.If(r30s48, 5, 0) , z3.If(r30s49, 75, 0) , z3.If(r30s50, 5, 0) , z3.If(r30s51, 5, 0) , z3.If(r30s52, 5, 0) , z3.If(r30s53, 5, 0)]
# 85/14/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 85, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 14, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 85, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 14, 0) , z3.If(r0s17, 14, 0) , z3.If(r0s18, 14, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 85, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 85, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 85, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 14, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 14, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 14, 0) , z3.If(r0s43, 14, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 14, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 14, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 85, 0) , z3.If(r1s7, 14, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 85, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 14, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 85, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 14, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 85, 0) , z3.If(r1s25, 14, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 85, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 14, 0) , z3.If(r1s31, 14, 0) , z3.If(r1s32, 85, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 14, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 85, 0) , z3.If(r1s43, 14, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 85, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 14, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 14, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 85, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 85, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 85, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 14, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 14, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 14, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 85, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 85, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 85, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 85, 0) , z3.If(r2s43, 14, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 85, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 85, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 85, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 85, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 85, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 85, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 14, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 85, 0) , z3.If(r4s13, 85, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 85, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 85, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 85, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 14, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 14, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 85, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 14, 0) , z3.If(r4s46, 14, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 85, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 85, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 14, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 14, 0) , z3.If(r5s25, 14, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 14, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 14, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 14, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 14, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 85, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 14, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 14, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 14, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 85, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 85, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 14, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 85, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 14, 0) , z3.If(r6s50, 85, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 14, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 14, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 85, 0) , z3.If(r7s13, 14, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 85, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 85, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 14, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 14, 0) , z3.If(r7s31, 85, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 85, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 85, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 85, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 85, 0) , z3.If(r8s4, 85, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 85, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 85, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 85, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 85, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 85, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 14, 0) , z3.If(r9s6, 85, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 85, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 14, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 14, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 14, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 14, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 14, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 14, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 85, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 14, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 14, 0) , z3.If(r10s7, 14, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 14, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 14, 0) , z3.If(r10s13, 14, 0) , z3.If(r10s14, 14, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 14, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 14, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 14, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 14, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 14, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 14, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 14, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 85, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 14, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 14, 0) , z3.If(r11s14, 85, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 85, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 85, 0) , z3.If(r11s42, 85, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 14, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 14, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 14, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 14, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 14, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 14, 0) , z3.If(r12s17, 14, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 14, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 14, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 85, 0) , z3.If(r13s1, 85, 0) , z3.If(r13s2, 85, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 14, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 85, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 85, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 14, 0) , z3.If(r13s18, 85, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 85, 0) , z3.If(r13s25, 85, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 85, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 85, 0) , z3.If(r13s40, 14, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 14, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 85, 0) , z3.If(r14s7, 14, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 14, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 85, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 14, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 85, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 14, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 14, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 85, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 14, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 14, 0) , z3.If(r15s25, 14, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 85, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 14, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 14, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 14, 0) , z3.If(r16s1, 14, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 14, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 14, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 14, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 85, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 14, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 14, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 14, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 85, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 85, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 85, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 85, 0) , z3.If(r17s10, 85, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 85, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 85, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 85, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 85, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 85, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 14, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 14, 0) , z3.If(r18s18, 14, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 85, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 14, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 85, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 14, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 85, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 14, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 14, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 85, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 85, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 85, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 85, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 85, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 85, 0) , z3.If(r20s18, 85, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 85, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 85, 0) , z3.If(r20s25, 85, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 85, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 85, 0) , z3.If(r20s31, 85, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 85, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 85, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 14, 0) , z3.If(r21s7, 14, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 85, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 14, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 14, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 14, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 85, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 14, 0) , z3.If(r21s28, 85, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 85, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 85, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 85, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 14, 0) , z3.If(r22s8, 14, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 14, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 85, 0) , z3.If(r22s17, 14, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 85, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 14, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 85, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 85, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 14, 0) , z3.If(r23s17, 85, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 85, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 85, 0) , z3.If(r23s31, 14, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 85, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 14, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 14, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 85, 0) , z3.If(r24s16, 14, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 14, 0) , z3.If(r24s20, 85, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 85, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 14, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 14, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 14, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 85, 0) , z3.If(r25s7, 14, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 14, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 14, 0) , z3.If(r25s17, 85, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 85, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 14, 0) , z3.If(r25s31, 14, 0) , z3.If(r25s32, 14, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 14, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 14, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 14, 0) , z3.If(r25s43, 14, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 14, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 85, 0) , z3.If(r26s1, 14, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 85, 0) , z3.If(r26s4, 85, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 85, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 85, 0) , z3.If(r26s21, 85, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 14, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 85, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 85, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 85, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 85, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 85, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 14, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 85, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 85, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 14, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 85, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 14, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 85, 0) , z3.If(r27s43, 14, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 85, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 14, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 14, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 14, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 14, 0) , z3.If(r28s13, 85, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 14, 0) , z3.If(r28s18, 85, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 85, 0) , z3.If(r28s25, 85, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 14, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 85, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 14, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 85, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 85, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 85, 0) , z3.If(r29s17, 85, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 85, 0) , z3.If(r29s25, 85, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 85, 0) , z3.If(r29s32, 85, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 85, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 85, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 85, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 85, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 14, 0) , z3.If(r30s6, 85, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 14, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 85, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 85, 0) , z3.If(r30s16, 14, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 14, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 85, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 14, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 14, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 14, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 14, 0) , z3.If(r30s46, 14, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 85, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]
# 88/11/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 88, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 11, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 88, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 11, 0) , z3.If(r0s17, 11, 0) , z3.If(r0s18, 11, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 88, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 88, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 88, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 11, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 11, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 11, 0) , z3.If(r0s43, 11, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 11, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 11, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 88, 0) , z3.If(r1s7, 11, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 88, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 11, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 88, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 11, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 88, 0) , z3.If(r1s25, 11, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 88, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 11, 0) , z3.If(r1s31, 11, 0) , z3.If(r1s32, 88, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 11, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 88, 0) , z3.If(r1s43, 11, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 88, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 11, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 11, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 88, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 88, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 88, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 11, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 11, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 11, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 88, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 88, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 88, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 88, 0) , z3.If(r2s43, 11, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 88, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 88, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 88, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 88, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 88, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 88, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 11, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 88, 0) , z3.If(r4s13, 88, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 88, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 88, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 88, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 11, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 11, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 88, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 11, 0) , z3.If(r4s46, 11, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 88, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 88, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 11, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 11, 0) , z3.If(r5s25, 11, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 11, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 11, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 11, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 11, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 88, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 11, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 11, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 11, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 88, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 88, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 11, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 88, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 11, 0) , z3.If(r6s50, 88, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 11, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 11, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 88, 0) , z3.If(r7s13, 11, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 88, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 88, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 11, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 11, 0) , z3.If(r7s31, 88, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 88, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 88, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 88, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 88, 0) , z3.If(r8s4, 88, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 88, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 88, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 88, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 88, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 88, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 11, 0) , z3.If(r9s6, 88, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 88, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 11, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 11, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 11, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 11, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 11, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 11, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 88, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 11, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 11, 0) , z3.If(r10s7, 11, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 11, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 11, 0) , z3.If(r10s13, 11, 0) , z3.If(r10s14, 11, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 11, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 11, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 11, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 11, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 11, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 11, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 11, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 88, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 11, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 11, 0) , z3.If(r11s14, 88, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 88, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 88, 0) , z3.If(r11s42, 88, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 11, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 11, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 11, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 11, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 11, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 11, 0) , z3.If(r12s17, 11, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 11, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 11, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 88, 0) , z3.If(r13s1, 88, 0) , z3.If(r13s2, 88, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 11, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 88, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 88, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 11, 0) , z3.If(r13s18, 88, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 88, 0) , z3.If(r13s25, 88, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 88, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 88, 0) , z3.If(r13s40, 11, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 11, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 88, 0) , z3.If(r14s7, 11, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 11, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 88, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 11, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 88, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 11, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 11, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 88, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 11, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 11, 0) , z3.If(r15s25, 11, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 88, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 11, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 11, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 11, 0) , z3.If(r16s1, 11, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 11, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 11, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 11, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 88, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 11, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 11, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 11, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 88, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 88, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 88, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 88, 0) , z3.If(r17s10, 88, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 88, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 88, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 88, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 88, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 88, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 11, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 11, 0) , z3.If(r18s18, 11, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 88, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 11, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 88, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 11, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 88, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 11, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 11, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 88, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 88, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 88, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 88, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 88, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 88, 0) , z3.If(r20s18, 88, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 88, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 88, 0) , z3.If(r20s25, 88, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 88, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 88, 0) , z3.If(r20s31, 88, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 88, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 88, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 11, 0) , z3.If(r21s7, 11, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 88, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 11, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 11, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 11, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 88, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 11, 0) , z3.If(r21s28, 88, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 88, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 88, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 88, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 11, 0) , z3.If(r22s8, 11, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 11, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 88, 0) , z3.If(r22s17, 11, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 88, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 11, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 88, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 88, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 11, 0) , z3.If(r23s17, 88, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 88, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 88, 0) , z3.If(r23s31, 11, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 88, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 11, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 11, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 88, 0) , z3.If(r24s16, 11, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 11, 0) , z3.If(r24s20, 88, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 88, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 11, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 11, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 11, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 88, 0) , z3.If(r25s7, 11, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 11, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 11, 0) , z3.If(r25s17, 88, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 88, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 11, 0) , z3.If(r25s31, 11, 0) , z3.If(r25s32, 11, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 11, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 11, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 11, 0) , z3.If(r25s43, 11, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 11, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 88, 0) , z3.If(r26s1, 11, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 88, 0) , z3.If(r26s4, 88, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 88, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 88, 0) , z3.If(r26s21, 88, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 11, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 88, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 88, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 88, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 88, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 88, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 11, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 88, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 88, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 11, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 88, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 11, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 88, 0) , z3.If(r27s43, 11, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 88, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 11, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 11, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 11, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 11, 0) , z3.If(r28s13, 88, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 11, 0) , z3.If(r28s18, 88, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 88, 0) , z3.If(r28s25, 88, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 11, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 88, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 11, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 88, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 88, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 88, 0) , z3.If(r29s17, 88, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 88, 0) , z3.If(r29s25, 88, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 88, 0) , z3.If(r29s32, 88, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 88, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 88, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 88, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 88, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 11, 0) , z3.If(r30s6, 88, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 11, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 88, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 88, 0) , z3.If(r30s16, 11, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 11, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 88, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 11, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 11, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 11, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 11, 0) , z3.If(r30s46, 11, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 88, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]
# 87/12/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 87, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 12, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 87, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 12, 0) , z3.If(r0s17, 12, 0) , z3.If(r0s18, 12, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 87, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 87, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 87, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 12, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 12, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 12, 0) , z3.If(r0s43, 12, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 12, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 12, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 87, 0) , z3.If(r1s7, 12, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 87, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 12, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 87, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 12, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 87, 0) , z3.If(r1s25, 12, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 87, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 12, 0) , z3.If(r1s31, 12, 0) , z3.If(r1s32, 87, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 12, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 87, 0) , z3.If(r1s43, 12, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 87, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 12, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 12, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 87, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 87, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 87, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 12, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 12, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 12, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 87, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 87, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 87, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 87, 0) , z3.If(r2s43, 12, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 87, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 87, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 87, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 87, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 87, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 87, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 12, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 87, 0) , z3.If(r4s13, 87, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 87, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 87, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 87, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 12, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 12, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 87, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 12, 0) , z3.If(r4s46, 12, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 87, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 87, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 12, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 12, 0) , z3.If(r5s25, 12, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 12, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 12, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 12, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 12, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 87, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 12, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 12, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 12, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 87, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 87, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 12, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 87, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 12, 0) , z3.If(r6s50, 87, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 12, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 12, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 87, 0) , z3.If(r7s13, 12, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 87, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 87, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 12, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 12, 0) , z3.If(r7s31, 87, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 87, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 87, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 87, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 87, 0) , z3.If(r8s4, 87, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 87, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 87, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 87, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 87, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 87, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 12, 0) , z3.If(r9s6, 87, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 87, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 12, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 12, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 12, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 12, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 12, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 12, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 87, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 12, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 12, 0) , z3.If(r10s7, 12, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 12, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 12, 0) , z3.If(r10s13, 12, 0) , z3.If(r10s14, 12, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 12, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 12, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 12, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 12, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 12, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 12, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 12, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 87, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 12, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 12, 0) , z3.If(r11s14, 87, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 87, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 87, 0) , z3.If(r11s42, 87, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 12, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 12, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 12, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 12, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 12, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 12, 0) , z3.If(r12s17, 12, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 12, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 12, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 87, 0) , z3.If(r13s1, 87, 0) , z3.If(r13s2, 87, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 12, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 87, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 87, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 12, 0) , z3.If(r13s18, 87, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 87, 0) , z3.If(r13s25, 87, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 87, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 87, 0) , z3.If(r13s40, 12, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 12, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 87, 0) , z3.If(r14s7, 12, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 12, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 87, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 12, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 87, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 12, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 12, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 87, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 12, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 12, 0) , z3.If(r15s25, 12, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 87, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 12, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 12, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 12, 0) , z3.If(r16s1, 12, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 12, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 12, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 12, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 87, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 12, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 12, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 12, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 87, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 87, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 87, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 87, 0) , z3.If(r17s10, 87, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 87, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 87, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 87, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 87, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 87, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 12, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 12, 0) , z3.If(r18s18, 12, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 87, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 12, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 87, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 12, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 87, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 12, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 12, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 87, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 87, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 87, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 87, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 87, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 87, 0) , z3.If(r20s18, 87, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 87, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 87, 0) , z3.If(r20s25, 87, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 87, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 87, 0) , z3.If(r20s31, 87, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 87, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 87, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 12, 0) , z3.If(r21s7, 12, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 87, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 12, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 12, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 12, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 87, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 12, 0) , z3.If(r21s28, 87, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 87, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 87, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 87, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 12, 0) , z3.If(r22s8, 12, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 12, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 87, 0) , z3.If(r22s17, 12, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 87, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 12, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 87, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 87, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 12, 0) , z3.If(r23s17, 87, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 87, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 87, 0) , z3.If(r23s31, 12, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 87, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 12, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 12, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 87, 0) , z3.If(r24s16, 12, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 12, 0) , z3.If(r24s20, 87, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 87, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 12, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 12, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 12, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 87, 0) , z3.If(r25s7, 12, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 12, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 12, 0) , z3.If(r25s17, 87, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 87, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 12, 0) , z3.If(r25s31, 12, 0) , z3.If(r25s32, 12, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 12, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 12, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 12, 0) , z3.If(r25s43, 12, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 12, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 87, 0) , z3.If(r26s1, 12, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 87, 0) , z3.If(r26s4, 87, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 87, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 87, 0) , z3.If(r26s21, 87, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 12, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 87, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 87, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 87, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 87, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 87, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 12, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 87, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 87, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 12, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 87, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 12, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 87, 0) , z3.If(r27s43, 12, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 87, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 12, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 12, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 12, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 12, 0) , z3.If(r28s13, 87, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 12, 0) , z3.If(r28s18, 87, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 87, 0) , z3.If(r28s25, 87, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 12, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 87, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 12, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 87, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 87, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 87, 0) , z3.If(r29s17, 87, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 87, 0) , z3.If(r29s25, 87, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 87, 0) , z3.If(r29s32, 87, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 87, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 87, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 87, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 87, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 12, 0) , z3.If(r30s6, 87, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 12, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 87, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 87, 0) , z3.If(r30s16, 12, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 12, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 87, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 12, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 12, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 12, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 12, 0) , z3.If(r30s46, 12, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 87, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]
# 89/10/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 89, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 10, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 89, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 10, 0) , z3.If(r0s17, 10, 0) , z3.If(r0s18, 10, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 89, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 89, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 89, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 10, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 10, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 10, 0) , z3.If(r0s43, 10, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 10, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 10, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 89, 0) , z3.If(r1s7, 10, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 89, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 10, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 89, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 10, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 89, 0) , z3.If(r1s25, 10, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 89, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 10, 0) , z3.If(r1s31, 10, 0) , z3.If(r1s32, 89, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 10, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 89, 0) , z3.If(r1s43, 10, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 89, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 10, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 10, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 89, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 89, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 89, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 10, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 10, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 10, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 89, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 89, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 89, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 89, 0) , z3.If(r2s43, 10, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 89, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 89, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 89, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 89, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 89, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 89, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 10, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 89, 0) , z3.If(r4s13, 89, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 89, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 89, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 89, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 10, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 10, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 89, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 10, 0) , z3.If(r4s46, 10, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 89, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 89, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 10, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 10, 0) , z3.If(r5s25, 10, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 10, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 10, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 10, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 10, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 89, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 10, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 10, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 10, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 89, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 89, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 10, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 89, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 10, 0) , z3.If(r6s50, 89, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 10, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 10, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 89, 0) , z3.If(r7s13, 10, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 89, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 89, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 10, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 10, 0) , z3.If(r7s31, 89, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 89, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 89, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 89, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 89, 0) , z3.If(r8s4, 89, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 89, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 89, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 89, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 89, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 89, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 10, 0) , z3.If(r9s6, 89, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 89, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 10, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 10, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 10, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 10, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 10, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 10, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 89, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 10, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 10, 0) , z3.If(r10s7, 10, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 10, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 10, 0) , z3.If(r10s13, 10, 0) , z3.If(r10s14, 10, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 10, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 10, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 10, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 10, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 10, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 10, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 10, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 89, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 10, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 10, 0) , z3.If(r11s14, 89, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 89, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 89, 0) , z3.If(r11s42, 89, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 10, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 10, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 10, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 10, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 10, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 10, 0) , z3.If(r12s17, 10, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 10, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 10, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 89, 0) , z3.If(r13s1, 89, 0) , z3.If(r13s2, 89, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 10, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 89, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 89, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 10, 0) , z3.If(r13s18, 89, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 89, 0) , z3.If(r13s25, 89, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 89, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 89, 0) , z3.If(r13s40, 10, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 10, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 89, 0) , z3.If(r14s7, 10, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 10, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 89, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 10, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 89, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 10, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 10, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 89, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 10, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 10, 0) , z3.If(r15s25, 10, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 89, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 10, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 10, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 10, 0) , z3.If(r16s1, 10, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 10, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 10, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 10, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 89, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 10, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 10, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 10, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 89, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 89, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 89, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 89, 0) , z3.If(r17s10, 89, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 89, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 89, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 89, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 89, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 89, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 10, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 10, 0) , z3.If(r18s18, 10, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 89, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 10, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 89, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 10, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 89, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 10, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 10, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 89, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 89, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 89, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 89, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 89, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 89, 0) , z3.If(r20s18, 89, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 89, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 89, 0) , z3.If(r20s25, 89, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 89, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 89, 0) , z3.If(r20s31, 89, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 89, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 89, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 10, 0) , z3.If(r21s7, 10, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 89, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 10, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 10, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 10, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 89, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 10, 0) , z3.If(r21s28, 89, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 89, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 89, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 89, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 10, 0) , z3.If(r22s8, 10, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 10, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 89, 0) , z3.If(r22s17, 10, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 89, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 10, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 89, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 89, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 10, 0) , z3.If(r23s17, 89, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 89, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 89, 0) , z3.If(r23s31, 10, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 89, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 10, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 10, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 89, 0) , z3.If(r24s16, 10, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 10, 0) , z3.If(r24s20, 89, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 89, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 10, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 10, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 10, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 89, 0) , z3.If(r25s7, 10, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 10, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 10, 0) , z3.If(r25s17, 89, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 89, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 10, 0) , z3.If(r25s31, 10, 0) , z3.If(r25s32, 10, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 10, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 10, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 10, 0) , z3.If(r25s43, 10, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 10, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 89, 0) , z3.If(r26s1, 10, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 89, 0) , z3.If(r26s4, 89, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 89, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 89, 0) , z3.If(r26s21, 89, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 10, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 89, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 89, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 89, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 89, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 89, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 10, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 89, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 89, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 10, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 89, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 10, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 89, 0) , z3.If(r27s43, 10, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 89, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 10, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 10, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 10, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 10, 0) , z3.If(r28s13, 89, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 10, 0) , z3.If(r28s18, 89, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 89, 0) , z3.If(r28s25, 89, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 10, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 89, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 10, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 89, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 89, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 89, 0) , z3.If(r29s17, 89, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 89, 0) , z3.If(r29s25, 89, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 89, 0) , z3.If(r29s32, 89, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 89, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 89, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 89, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 89, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 10, 0) , z3.If(r30s6, 89, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 10, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 89, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 89, 0) , z3.If(r30s16, 10, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 10, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 89, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 10, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 10, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 10, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 10, 0) , z3.If(r30s46, 10, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 89, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]
# 91/8/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 91, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 8, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 91, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 8, 0) , z3.If(r0s17, 8, 0) , z3.If(r0s18, 8, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 91, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 91, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 91, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 8, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 8, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 8, 0) , z3.If(r0s43, 8, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 8, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 8, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 91, 0) , z3.If(r1s7, 8, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 91, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 8, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 91, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 8, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 91, 0) , z3.If(r1s25, 8, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 91, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 8, 0) , z3.If(r1s31, 8, 0) , z3.If(r1s32, 91, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 8, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 91, 0) , z3.If(r1s43, 8, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 91, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 8, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 8, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 91, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 91, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 91, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 8, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 8, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 8, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 91, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 91, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 91, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 91, 0) , z3.If(r2s43, 8, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 91, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 91, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 91, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 91, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 91, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 91, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 8, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 91, 0) , z3.If(r4s13, 91, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 91, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 91, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 91, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 8, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 8, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 91, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 8, 0) , z3.If(r4s46, 8, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 91, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 91, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 8, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 8, 0) , z3.If(r5s25, 8, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 8, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 8, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 8, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 8, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 91, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 8, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 8, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 8, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 91, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 91, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 8, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 91, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 8, 0) , z3.If(r6s50, 91, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 8, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 8, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 91, 0) , z3.If(r7s13, 8, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 91, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 91, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 8, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 8, 0) , z3.If(r7s31, 91, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 91, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 91, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 91, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 91, 0) , z3.If(r8s4, 91, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 91, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 91, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 91, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 91, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 91, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 8, 0) , z3.If(r9s6, 91, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 91, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 8, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 8, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 8, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 8, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 8, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 8, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 91, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 8, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 8, 0) , z3.If(r10s7, 8, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 8, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 8, 0) , z3.If(r10s13, 8, 0) , z3.If(r10s14, 8, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 8, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 8, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 8, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 8, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 8, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 8, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 8, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 91, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 8, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 8, 0) , z3.If(r11s14, 91, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 91, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 91, 0) , z3.If(r11s42, 91, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 8, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 8, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 8, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 8, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 8, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 8, 0) , z3.If(r12s17, 8, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 8, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 8, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 91, 0) , z3.If(r13s1, 91, 0) , z3.If(r13s2, 91, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 8, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 91, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 91, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 8, 0) , z3.If(r13s18, 91, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 91, 0) , z3.If(r13s25, 91, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 91, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 91, 0) , z3.If(r13s40, 8, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 8, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 91, 0) , z3.If(r14s7, 8, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 8, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 91, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 8, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 91, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 8, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 8, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 91, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 8, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 8, 0) , z3.If(r15s25, 8, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 91, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 8, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 8, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 8, 0) , z3.If(r16s1, 8, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 8, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 8, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 8, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 91, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 8, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 8, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 8, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 91, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 91, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 91, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 91, 0) , z3.If(r17s10, 91, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 91, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 91, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 91, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 91, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 91, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 8, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 8, 0) , z3.If(r18s18, 8, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 91, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 8, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 91, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 8, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 91, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 8, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 8, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 91, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 91, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 91, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 91, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 91, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 91, 0) , z3.If(r20s18, 91, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 91, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 91, 0) , z3.If(r20s25, 91, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 91, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 91, 0) , z3.If(r20s31, 91, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 91, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 91, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 8, 0) , z3.If(r21s7, 8, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 91, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 8, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 8, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 8, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 91, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 8, 0) , z3.If(r21s28, 91, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 91, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 91, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 91, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 8, 0) , z3.If(r22s8, 8, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 8, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 91, 0) , z3.If(r22s17, 8, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 91, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 8, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 91, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 91, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 8, 0) , z3.If(r23s17, 91, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 91, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 91, 0) , z3.If(r23s31, 8, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 91, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 8, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 8, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 91, 0) , z3.If(r24s16, 8, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 8, 0) , z3.If(r24s20, 91, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 91, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 8, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 8, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 8, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 91, 0) , z3.If(r25s7, 8, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 8, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 8, 0) , z3.If(r25s17, 91, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 91, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 8, 0) , z3.If(r25s31, 8, 0) , z3.If(r25s32, 8, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 8, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 8, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 8, 0) , z3.If(r25s43, 8, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 8, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 91, 0) , z3.If(r26s1, 8, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 91, 0) , z3.If(r26s4, 91, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 91, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 91, 0) , z3.If(r26s21, 91, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 8, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 91, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 91, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 91, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 91, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 91, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 8, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 91, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 91, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 8, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 91, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 8, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 91, 0) , z3.If(r27s43, 8, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 91, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 8, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 8, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 8, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 8, 0) , z3.If(r28s13, 91, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 8, 0) , z3.If(r28s18, 91, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 91, 0) , z3.If(r28s25, 91, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 8, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 91, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 8, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 91, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 91, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 91, 0) , z3.If(r29s17, 91, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 91, 0) , z3.If(r29s25, 91, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 91, 0) , z3.If(r29s32, 91, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 91, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 91, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 91, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 91, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 8, 0) , z3.If(r30s6, 91, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 8, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 91, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 91, 0) , z3.If(r30s16, 8, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 8, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 91, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 8, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 8, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 8, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 8, 0) , z3.If(r30s46, 8, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 91, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]
# 93/6/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 93, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 6, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 93, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 6, 0) , z3.If(r0s17, 6, 0) , z3.If(r0s18, 6, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 93, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 93, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 93, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 6, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 6, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 6, 0) , z3.If(r0s43, 6, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 6, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 6, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 93, 0) , z3.If(r1s7, 6, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 93, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 6, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 93, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 6, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 93, 0) , z3.If(r1s25, 6, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 93, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 6, 0) , z3.If(r1s31, 6, 0) , z3.If(r1s32, 93, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 6, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 93, 0) , z3.If(r1s43, 6, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 93, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 6, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 6, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 93, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 93, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 93, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 6, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 6, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 6, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 93, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 93, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 93, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 93, 0) , z3.If(r2s43, 6, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 93, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 93, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 93, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 93, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 93, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 93, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 6, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 93, 0) , z3.If(r4s13, 93, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 93, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 93, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 93, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 6, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 6, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 93, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 6, 0) , z3.If(r4s46, 6, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 93, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 93, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 6, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 6, 0) , z3.If(r5s25, 6, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 6, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 6, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 6, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 6, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 93, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 6, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 6, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 6, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 93, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 93, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 6, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 93, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 6, 0) , z3.If(r6s50, 93, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 6, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 6, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 93, 0) , z3.If(r7s13, 6, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 93, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 93, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 6, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 6, 0) , z3.If(r7s31, 93, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 93, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 93, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 93, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 93, 0) , z3.If(r8s4, 93, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 93, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 93, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 93, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 93, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 93, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 6, 0) , z3.If(r9s6, 93, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 93, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 6, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 6, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 6, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 6, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 6, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 6, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 93, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 6, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 6, 0) , z3.If(r10s7, 6, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 6, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 6, 0) , z3.If(r10s13, 6, 0) , z3.If(r10s14, 6, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 6, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 6, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 6, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 6, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 6, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 6, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 6, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 93, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 6, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 6, 0) , z3.If(r11s14, 93, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 93, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 93, 0) , z3.If(r11s42, 93, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 6, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 6, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 6, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 6, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 6, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 6, 0) , z3.If(r12s17, 6, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 6, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 6, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 93, 0) , z3.If(r13s1, 93, 0) , z3.If(r13s2, 93, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 6, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 93, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 93, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 6, 0) , z3.If(r13s18, 93, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 93, 0) , z3.If(r13s25, 93, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 93, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 93, 0) , z3.If(r13s40, 6, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 6, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 93, 0) , z3.If(r14s7, 6, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 6, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 93, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 6, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 93, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 6, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 6, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 93, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 6, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 6, 0) , z3.If(r15s25, 6, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 93, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 6, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 6, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 6, 0) , z3.If(r16s1, 6, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 6, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 6, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 6, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 93, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 6, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 6, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 6, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 93, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 93, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 93, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 93, 0) , z3.If(r17s10, 93, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 93, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 93, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 93, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 93, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 93, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 6, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 6, 0) , z3.If(r18s18, 6, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 93, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 6, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 93, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 6, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 93, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 6, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 6, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 93, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 93, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 93, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 93, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 93, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 93, 0) , z3.If(r20s18, 93, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 93, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 93, 0) , z3.If(r20s25, 93, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 93, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 93, 0) , z3.If(r20s31, 93, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 93, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 93, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 6, 0) , z3.If(r21s7, 6, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 93, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 6, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 6, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 6, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 93, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 6, 0) , z3.If(r21s28, 93, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 93, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 93, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 93, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 6, 0) , z3.If(r22s8, 6, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 6, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 93, 0) , z3.If(r22s17, 6, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 93, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 6, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 93, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 93, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 6, 0) , z3.If(r23s17, 93, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 93, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 93, 0) , z3.If(r23s31, 6, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 93, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 6, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 6, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 93, 0) , z3.If(r24s16, 6, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 6, 0) , z3.If(r24s20, 93, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 93, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 6, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 6, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 6, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 93, 0) , z3.If(r25s7, 6, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 6, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 6, 0) , z3.If(r25s17, 93, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 93, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 6, 0) , z3.If(r25s31, 6, 0) , z3.If(r25s32, 6, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 6, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 6, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 6, 0) , z3.If(r25s43, 6, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 6, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 93, 0) , z3.If(r26s1, 6, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 93, 0) , z3.If(r26s4, 93, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 93, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 93, 0) , z3.If(r26s21, 93, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 6, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 93, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 93, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 93, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 93, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 93, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 6, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 93, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 93, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 6, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 93, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 6, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 93, 0) , z3.If(r27s43, 6, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 93, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 6, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 6, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 6, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 6, 0) , z3.If(r28s13, 93, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 6, 0) , z3.If(r28s18, 93, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 93, 0) , z3.If(r28s25, 93, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 6, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 93, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 6, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 93, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 93, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 93, 0) , z3.If(r29s17, 93, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 93, 0) , z3.If(r29s25, 93, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 93, 0) , z3.If(r29s32, 93, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 93, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 93, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 93, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 93, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 6, 0) , z3.If(r30s6, 93, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 6, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 93, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 93, 0) , z3.If(r30s16, 6, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 6, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 93, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 6, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 6, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 6, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 6, 0) , z3.If(r30s46, 6, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 93, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]
# 97/2/1
# scores = [z3.If(r0s0, 1, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s6, 97, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 2, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s11, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 97, 0) , z3.If(r0s14, 1, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 2, 0) , z3.If(r0s17, 2, 0) , z3.If(r0s18, 2, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 97, 0) , z3.If(r0s23, 1, 0) , z3.If(r0s24, 97, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 1, 0) , z3.If(r0s27, 97, 0) , z3.If(r0s28, 1, 0) , z3.If(r0s29, 1, 0) , z3.If(r0s30, 1, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 2, 0) , z3.If(r0s33, 1, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 2, 0) , z3.If(r0s37, 1, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 1, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 2, 0) , z3.If(r0s43, 2, 0) , z3.If(r0s44, 1, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 2, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s51, 2, 0) , z3.If(r0s52, 1, 0) , z3.If(r0s53, 1, 0) , z3.If(r1s0, 1, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 1, 0) , z3.If(r1s3, 1, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 97, 0) , z3.If(r1s7, 2, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 1, 0) , z3.If(r1s10, 1, 0) , z3.If(r1s11, 1, 0) , z3.If(r1s12, 97, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 2, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 97, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 2, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 97, 0) , z3.If(r1s25, 2, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 97, 0) , z3.If(r1s28, 1, 0) , z3.If(r1s29, 1, 0) , z3.If(r1s30, 2, 0) , z3.If(r1s31, 2, 0) , z3.If(r1s32, 97, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s34, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 2, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 97, 0) , z3.If(r1s43, 2, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 97, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 2, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r1s52, 1, 0) , z3.If(r1s53, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 2, 0) , z3.If(r2s2, 1, 0) , z3.If(r2s3, 97, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 97, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 97, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 2, 0) , z3.If(r2s13, 1, 0) , z3.If(r2s14, 2, 0) , z3.If(r2s15, 1, 0) , z3.If(r2s16, 1, 0) , z3.If(r2s17, 2, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s21, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 97, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 97, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s32, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 1, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 97, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 97, 0) , z3.If(r2s43, 2, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 1, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 1, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 1, 0) , z3.If(r2s52, 1, 0) , z3.If(r2s53, 1, 0) , z3.If(r3s0, 97, 0) , z3.If(r3s1, 1, 0) , z3.If(r3s2, 1, 0) , z3.If(r3s3, 1, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 97, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 1, 0) , z3.If(r3s9, 1, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 1, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s13, 97, 0) , z3.If(r3s14, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s16, 1, 0) , z3.If(r3s17, 1, 0) , z3.If(r3s18, 1, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s24, 1, 0) , z3.If(r3s25, 1, 0) , z3.If(r3s26, 1, 0) , z3.If(r3s27, 1, 0) , z3.If(r3s28, 1, 0) , z3.If(r3s29, 1, 0) , z3.If(r3s30, 1, 0) , z3.If(r3s31, 1, 0) , z3.If(r3s32, 1, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 1, 0) , z3.If(r3s36, 1, 0) , z3.If(r3s37, 97, 0) , z3.If(r3s38, 1, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 97, 0) , z3.If(r3s41, 1, 0) , z3.If(r3s42, 1, 0) , z3.If(r3s43, 1, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 1, 0) , z3.If(r3s47, 1, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 1, 0) , z3.If(r3s50, 1, 0) , z3.If(r3s51, 97, 0) , z3.If(r3s52, 1, 0) , z3.If(r3s53, 1, 0) , z3.If(r4s0, 1, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 1, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 1, 0) , z3.If(r4s5, 1, 0) , z3.If(r4s6, 2, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s10, 1, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 97, 0) , z3.If(r4s13, 97, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 1, 0) , z3.If(r4s17, 97, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 1, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 97, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 97, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 1, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 2, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 2, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 97, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 2, 0) , z3.If(r4s46, 2, 0) , z3.If(r4s47, 1, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 1, 0) , z3.If(r4s52, 1, 0) , z3.If(r4s53, 1, 0) , z3.If(r5s0, 1, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 1, 0) , z3.If(r5s3, 1, 0) , z3.If(r5s4, 1, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 97, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 1, 0) , z3.If(r5s9, 1, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 97, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 2, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 1, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 1, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 2, 0) , z3.If(r5s25, 2, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 2, 0) , z3.If(r5s28, 1, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 2, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 1, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 2, 0) , z3.If(r5s37, 1, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 1, 0) , z3.If(r5s40, 2, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 97, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 2, 0) , z3.If(r5s48, 1, 0) , z3.If(r5s49, 1, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r5s52, 1, 0) , z3.If(r5s53, 1, 0) , z3.If(r6s0, 1, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 1, 0) , z3.If(r6s3, 1, 0) , z3.If(r6s4, 1, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s6, 2, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 1, 0) , z3.If(r6s10, 1, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 2, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 1, 0) , z3.If(r6s18, 1, 0) , z3.If(r6s19, 1, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 1, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 97, 0) , z3.If(r6s24, 1, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 1, 0) , z3.If(r6s29, 1, 0) , z3.If(r6s30, 1, 0) , z3.If(r6s31, 1, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s35, 1, 0) , z3.If(r6s36, 97, 0) , z3.If(r6s37, 1, 0) , z3.If(r6s38, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 2, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 1, 0) , z3.If(r6s43, 1, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s46, 1, 0) , z3.If(r6s47, 97, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 2, 0) , z3.If(r6s50, 97, 0) , z3.If(r6s51, 1, 0) , z3.If(r6s52, 1, 0) , z3.If(r6s53, 1, 0) , z3.If(r7s0, 2, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 2, 0) , z3.If(r7s3, 1, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 1, 0) , z3.If(r7s8, 1, 0) , z3.If(r7s9, 1, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s11, 1, 0) , z3.If(r7s12, 97, 0) , z3.If(r7s13, 2, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 97, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s21, 1, 0) , z3.If(r7s22, 1, 0) , z3.If(r7s23, 1, 0) , z3.If(r7s25, 97, 0) , z3.If(r7s26, 1, 0) , z3.If(r7s27, 2, 0) , z3.If(r7s28, 1, 0) , z3.If(r7s29, 1, 0) , z3.If(r7s30, 2, 0) , z3.If(r7s31, 97, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 1, 0) , z3.If(r7s37, 1, 0) , z3.If(r7s38, 1, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 1, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 97, 0) , z3.If(r7s43, 1, 0) , z3.If(r7s44, 1, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 97, 0) , z3.If(r7s48, 1, 0) , z3.If(r7s49, 1, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r7s52, 97, 0) , z3.If(r7s53, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 97, 0) , z3.If(r8s4, 97, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 97, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 1, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s13, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s16, 97, 0) , z3.If(r8s17, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 97, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 1, 0) , z3.If(r8s23, 1, 0) , z3.If(r8s24, 1, 0) , z3.If(r8s25, 1, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 1, 0) , z3.If(r8s32, 1, 0) , z3.If(r8s33, 1, 0) , z3.If(r8s34, 1, 0) , z3.If(r8s35, 1, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 1, 0) , z3.If(r8s38, 97, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 1, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 1, 0) , z3.If(r8s43, 1, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 1, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 1, 0) , z3.If(r8s50, 1, 0) , z3.If(r8s51, 1, 0) , z3.If(r8s52, 1, 0) , z3.If(r8s53, 1, 0) , z3.If(r9s0, 1, 0) , z3.If(r9s1, 97, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 2, 0) , z3.If(r9s6, 97, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 97, 0) , z3.If(r9s14, 1, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 2, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 2, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 1, 0) , z3.If(r9s25, 2, 0) , z3.If(r9s26, 1, 0) , z3.If(r9s27, 2, 0) , z3.If(r9s28, 1, 0) , z3.If(r9s29, 1, 0) , z3.If(r9s30, 2, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 1, 0) , z3.If(r9s33, 1, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 1, 0) , z3.If(r9s37, 1, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 2, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 1, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 1, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 1, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r9s52, 97, 0) , z3.If(r9s53, 1, 0) , z3.If(r10s0, 2, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 2, 0) , z3.If(r10s7, 2, 0) , z3.If(r10s8, 1, 0) , z3.If(r10s9, 2, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s11, 1, 0) , z3.If(r10s12, 2, 0) , z3.If(r10s13, 2, 0) , z3.If(r10s14, 2, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 2, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 1, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 1, 0) , z3.If(r10s22, 2, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 2, 0) , z3.If(r10s28, 1, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 1, 0) , z3.If(r10s31, 1, 0) , z3.If(r10s32, 1, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 1, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 1, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 2, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s48, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 2, 0) , z3.If(r10s51, 1, 0) , z3.If(r10s52, 1, 0) , z3.If(r10s53, 1, 0) , z3.If(r11s0, 2, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 2, 0) , z3.If(r11s4, 1, 0) , z3.If(r11s5, 1, 0) , z3.If(r11s6, 97, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 2, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 1, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 2, 0) , z3.If(r11s14, 97, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 1, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 97, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 1, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 97, 0) , z3.If(r11s42, 97, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s47, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s49, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r11s52, 1, 0) , z3.If(r11s53, 1, 0) , z3.If(r12s0, 1, 0) , z3.If(r12s1, 2, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 2, 0) , z3.If(r12s4, 1, 0) , z3.If(r12s5, 1, 0) , z3.If(r12s6, 2, 0) , z3.If(r12s8, 1, 0) , z3.If(r12s9, 2, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s11, 1, 0) , z3.If(r12s12, 2, 0) , z3.If(r12s13, 1, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 2, 0) , z3.If(r12s17, 2, 0) , z3.If(r12s18, 1, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 2, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 1, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 1, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 1, 0) , z3.If(r12s32, 1, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 1, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 1, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 1, 0) , z3.If(r12s41, 1, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 1, 0) , z3.If(r12s44, 1, 0) , z3.If(r12s45, 1, 0) , z3.If(r12s46, 2, 0) , z3.If(r12s47, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r12s52, 1, 0) , z3.If(r12s53, 1, 0) , z3.If(r13s0, 97, 0) , z3.If(r13s1, 97, 0) , z3.If(r13s2, 97, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 2, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s8, 1, 0) , z3.If(r13s9, 97, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s11, 1, 0) , z3.If(r13s12, 97, 0) , z3.If(r13s13, 1, 0) , z3.If(r13s14, 1, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s17, 2, 0) , z3.If(r13s18, 97, 0) , z3.If(r13s19, 1, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s21, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s24, 97, 0) , z3.If(r13s25, 97, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 97, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 1, 0) , z3.If(r13s36, 1, 0) , z3.If(r13s37, 1, 0) , z3.If(r13s38, 1, 0) , z3.If(r13s39, 97, 0) , z3.If(r13s40, 2, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 1, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 1, 0) , z3.If(r13s47, 1, 0) , z3.If(r13s48, 1, 0) , z3.If(r13s49, 1, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 1, 0) , z3.If(r13s52, 1, 0) , z3.If(r13s53, 1, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 2, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s3, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s6, 97, 0) , z3.If(r14s7, 2, 0) , z3.If(r14s8, 1, 0) , z3.If(r14s9, 2, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s14, 97, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s16, 1, 0) , z3.If(r14s17, 1, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 1, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s25, 1, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 2, 0) , z3.If(r14s31, 1, 0) , z3.If(r14s32, 1, 0) , z3.If(r14s33, 1, 0) , z3.If(r14s34, 1, 0) , z3.If(r14s35, 1, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s37, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 97, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 1, 0) , z3.If(r14s43, 1, 0) , z3.If(r14s44, 1, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 2, 0) , z3.If(r14s47, 1, 0) , z3.If(r14s48, 1, 0) , z3.If(r14s49, 1, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r14s52, 1, 0) , z3.If(r14s53, 1, 0) , z3.If(r15s0, 2, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 1, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 1, 0) , z3.If(r15s13, 97, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s18, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 2, 0) , z3.If(r15s23, 1, 0) , z3.If(r15s24, 2, 0) , z3.If(r15s25, 2, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 97, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 1, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 1, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 2, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s47, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 2, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 1, 0) , z3.If(r15s52, 1, 0) , z3.If(r15s53, 1, 0) , z3.If(r16s0, 2, 0) , z3.If(r16s1, 2, 0) , z3.If(r16s2, 1, 0) , z3.If(r16s3, 2, 0) , z3.If(r16s4, 1, 0) , z3.If(r16s5, 2, 0) , z3.If(r16s6, 1, 0) , z3.If(r16s7, 2, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s10, 1, 0) , z3.If(r16s11, 97, 0) , z3.If(r16s12, 1, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s18, 1, 0) , z3.If(r16s19, 2, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 2, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 1, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 1, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s37, 2, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 1, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s44, 1, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 97, 0) , z3.If(r16s47, 1, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s52, 1, 0) , z3.If(r16s53, 1, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 97, 0) , z3.If(r17s2, 1, 0) , z3.If(r17s3, 97, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s6, 1, 0) , z3.If(r17s7, 1, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s9, 97, 0) , z3.If(r17s10, 97, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 1, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 1, 0) , z3.If(r17s17, 1, 0) , z3.If(r17s18, 1, 0) , z3.If(r17s19, 1, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s24, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s27, 1, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 97, 0) , z3.If(r17s30, 1, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 1, 0) , z3.If(r17s33, 97, 0) , z3.If(r17s34, 1, 0) , z3.If(r17s35, 1, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 1, 0) , z3.If(r17s38, 1, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 1, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 1, 0) , z3.If(r17s44, 97, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r17s52, 1, 0) , z3.If(r18s0, 1, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 1, 0) , z3.If(r18s5, 97, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s9, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 97, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 2, 0) , z3.If(r18s15, 1, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 2, 0) , z3.If(r18s18, 2, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 97, 0) , z3.If(r18s23, 1, 0) , z3.If(r18s24, 1, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 1, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 2, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 1, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 1, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 97, 0) , z3.If(r18s51, 1, 0) , z3.If(r18s52, 1, 0) , z3.If(r18s53, 1, 0) , z3.If(r19s0, 2, 0) , z3.If(r19s1, 1, 0) , z3.If(r19s2, 97, 0) , z3.If(r19s3, 1, 0) , z3.If(r19s4, 1, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 1, 0) , z3.If(r19s7, 1, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 1, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 2, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 1, 0) , z3.If(r19s16, 2, 0) , z3.If(r19s17, 1, 0) , z3.If(r19s18, 1, 0) , z3.If(r19s19, 1, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 1, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 1, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 1, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 1, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 1, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 1, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 1, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 97, 0) , z3.If(r19s52, 1, 0) , z3.If(r19s53, 1, 0) , z3.If(r20s0, 97, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 97, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 97, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 97, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s13, 1, 0) , z3.If(r20s14, 1, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s16, 1, 0) , z3.If(r20s17, 97, 0) , z3.If(r20s18, 97, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 97, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 97, 0) , z3.If(r20s25, 97, 0) , z3.If(r20s26, 1, 0) , z3.If(r20s27, 97, 0) , z3.If(r20s28, 1, 0) , z3.If(r20s29, 1, 0) , z3.If(r20s30, 97, 0) , z3.If(r20s31, 97, 0) , z3.If(r20s32, 1, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 1, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 1, 0) , z3.If(r20s37, 1, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 1, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 1, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r20s52, 1, 0) , z3.If(r20s53, 1, 0) , z3.If(r21s0, 97, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 1, 0) , z3.If(r21s3, 97, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 1, 0) , z3.If(r21s6, 2, 0) , z3.If(r21s7, 2, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s9, 97, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s11, 1, 0) , z3.If(r21s12, 1, 0) , z3.If(r21s13, 2, 0) , z3.If(r21s14, 1, 0) , z3.If(r21s15, 1, 0) , z3.If(r21s16, 2, 0) , z3.If(r21s17, 1, 0) , z3.If(r21s18, 2, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s21, 1, 0) , z3.If(r21s22, 97, 0) , z3.If(r21s23, 1, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 2, 0) , z3.If(r21s28, 97, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s32, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 97, 0) , z3.If(r21s35, 1, 0) , z3.If(r21s36, 1, 0) , z3.If(r21s37, 1, 0) , z3.If(r21s38, 1, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 97, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s43, 1, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 1, 0) , z3.If(r21s51, 1, 0) , z3.If(r21s52, 1, 0) , z3.If(r21s53, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 97, 0) , z3.If(r22s4, 1, 0) , z3.If(r22s5, 1, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 2, 0) , z3.If(r22s8, 2, 0) , z3.If(r22s9, 1, 0) , z3.If(r22s10, 1, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 2, 0) , z3.If(r22s13, 1, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 97, 0) , z3.If(r22s17, 2, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 1, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 1, 0) , z3.If(r22s33, 1, 0) , z3.If(r22s34, 1, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 1, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 1, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 1, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r22s52, 1, 0) , z3.If(r22s53, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 97, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 2, 0) , z3.If(r23s4, 1, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 97, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 1, 0) , z3.If(r23s9, 1, 0) , z3.If(r23s10, 1, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 97, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 2, 0) , z3.If(r23s17, 97, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s24, 1, 0) , z3.If(r23s25, 97, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 97, 0) , z3.If(r23s31, 2, 0) , z3.If(r23s32, 1, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 1, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 97, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 1, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 1, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 1, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) , z3.If(r23s52, 1, 0) , z3.If(r23s53, 1, 0) , z3.If(r24s0, 2, 0) , z3.If(r24s1, 1, 0) , z3.If(r24s2, 1, 0) , z3.If(r24s3, 1, 0) , z3.If(r24s4, 1, 0) , z3.If(r24s5, 1, 0) , z3.If(r24s6, 2, 0) , z3.If(r24s7, 1, 0) , z3.If(r24s8, 1, 0) , z3.If(r24s9, 1, 0) , z3.If(r24s10, 1, 0) , z3.If(r24s11, 1, 0) , z3.If(r24s12, 1, 0) , z3.If(r24s14, 1, 0) , z3.If(r24s15, 97, 0) , z3.If(r24s16, 2, 0) , z3.If(r24s17, 1, 0) , z3.If(r24s18, 1, 0) , z3.If(r24s19, 2, 0) , z3.If(r24s20, 97, 0) , z3.If(r24s21, 1, 0) , z3.If(r24s22, 97, 0) , z3.If(r24s23, 1, 0) , z3.If(r24s24, 1, 0) , z3.If(r24s25, 1, 0) , z3.If(r24s26, 1, 0) , z3.If(r24s27, 2, 0) , z3.If(r24s28, 1, 0) , z3.If(r24s29, 1, 0) , z3.If(r24s30, 1, 0) , z3.If(r24s31, 1, 0) , z3.If(r24s32, 1, 0) , z3.If(r24s33, 1, 0) , z3.If(r24s34, 1, 0) , z3.If(r24s35, 1, 0) , z3.If(r24s36, 1, 0) , z3.If(r24s37, 1, 0) , z3.If(r24s38, 1, 0) , z3.If(r24s39, 1, 0) , z3.If(r24s40, 1, 0) , z3.If(r24s42, 1, 0) , z3.If(r24s44, 1, 0) , z3.If(r24s45, 1, 0) , z3.If(r24s46, 2, 0) , z3.If(r24s47, 1, 0) , z3.If(r24s48, 1, 0) , z3.If(r24s50, 1, 0) , z3.If(r24s52, 1, 0) , z3.If(r24s53, 1, 0) , z3.If(r25s0, 1, 0) , z3.If(r25s1, 1, 0) , z3.If(r25s2, 2, 0) , z3.If(r25s3, 1, 0) , z3.If(r25s4, 1, 0) , z3.If(r25s5, 1, 0) , z3.If(r25s6, 97, 0) , z3.If(r25s7, 2, 0) , z3.If(r25s8, 1, 0) , z3.If(r25s9, 2, 0) , z3.If(r25s10, 1, 0) , z3.If(r25s11, 1, 0) , z3.If(r25s13, 1, 0) , z3.If(r25s14, 1, 0) , z3.If(r25s15, 1, 0) , z3.If(r25s16, 2, 0) , z3.If(r25s17, 97, 0) , z3.If(r25s19, 1, 0) , z3.If(r25s20, 1, 0) , z3.If(r25s21, 1, 0) , z3.If(r25s22, 1, 0) , z3.If(r25s23, 1, 0) , z3.If(r25s24, 1, 0) , z3.If(r25s25, 1, 0) , z3.If(r25s26, 1, 0) , z3.If(r25s27, 97, 0) , z3.If(r25s28, 1, 0) , z3.If(r25s29, 1, 0) , z3.If(r25s30, 2, 0) , z3.If(r25s31, 2, 0) , z3.If(r25s32, 2, 0) , z3.If(r25s33, 1, 0) , z3.If(r25s34, 1, 0) , z3.If(r25s35, 1, 0) , z3.If(r25s36, 2, 0) , z3.If(r25s37, 1, 0) , z3.If(r25s38, 1, 0) , z3.If(r25s39, 1, 0) , z3.If(r25s40, 2, 0) , z3.If(r25s41, 1, 0) , z3.If(r25s42, 2, 0) , z3.If(r25s43, 2, 0) , z3.If(r25s44, 1, 0) , z3.If(r25s45, 1, 0) , z3.If(r25s46, 1, 0) , z3.If(r25s47, 2, 0) , z3.If(r25s48, 1, 0) , z3.If(r25s49, 1, 0) , z3.If(r25s50, 1, 0) , z3.If(r25s51, 1, 0) , z3.If(r25s52, 1, 0) , z3.If(r25s53, 1, 0) , z3.If(r26s0, 97, 0) , z3.If(r26s1, 2, 0) , z3.If(r26s2, 1, 0) , z3.If(r26s3, 97, 0) , z3.If(r26s4, 97, 0) , z3.If(r26s5, 1, 0) , z3.If(r26s6, 1, 0) , z3.If(r26s7, 1, 0) , z3.If(r26s8, 1, 0) , z3.If(r26s9, 1, 0) , z3.If(r26s10, 1, 0) , z3.If(r26s11, 97, 0) , z3.If(r26s12, 1, 0) , z3.If(r26s13, 1, 0) , z3.If(r26s14, 1, 0) , z3.If(r26s15, 1, 0) , z3.If(r26s16, 1, 0) , z3.If(r26s17, 1, 0) , z3.If(r26s18, 1, 0) , z3.If(r26s20, 97, 0) , z3.If(r26s21, 97, 0) , z3.If(r26s22, 1, 0) , z3.If(r26s23, 1, 0) , z3.If(r26s24, 1, 0) , z3.If(r26s25, 1, 0) , z3.If(r26s26, 1, 0) , z3.If(r26s27, 1, 0) , z3.If(r26s28, 1, 0) , z3.If(r26s29, 2, 0) , z3.If(r26s30, 1, 0) , z3.If(r26s31, 1, 0) , z3.If(r26s32, 1, 0) , z3.If(r26s33, 1, 0) , z3.If(r26s34, 1, 0) , z3.If(r26s35, 1, 0) , z3.If(r26s36, 1, 0) , z3.If(r26s37, 1, 0) , z3.If(r26s38, 1, 0) , z3.If(r26s39, 1, 0) , z3.If(r26s40, 1, 0) , z3.If(r26s41, 1, 0) , z3.If(r26s42, 1, 0) , z3.If(r26s43, 97, 0) , z3.If(r26s44, 1, 0) , z3.If(r26s45, 1, 0) , z3.If(r26s46, 1, 0) , z3.If(r26s47, 97, 0) , z3.If(r26s48, 1, 0) , z3.If(r26s49, 97, 0) , z3.If(r26s50, 1, 0) , z3.If(r26s51, 97, 0) , z3.If(r26s52, 1, 0) , z3.If(r26s53, 1, 0) , z3.If(r27s0, 1, 0) , z3.If(r27s1, 97, 0) , z3.If(r27s2, 1, 0) , z3.If(r27s3, 1, 0) , z3.If(r27s4, 1, 0) , z3.If(r27s5, 1, 0) , z3.If(r27s6, 2, 0) , z3.If(r27s7, 1, 0) , z3.If(r27s8, 1, 0) , z3.If(r27s9, 97, 0) , z3.If(r27s10, 1, 0) , z3.If(r27s11, 1, 0) , z3.If(r27s12, 1, 0) , z3.If(r27s13, 1, 0) , z3.If(r27s14, 97, 0) , z3.If(r27s15, 1, 0) , z3.If(r27s16, 1, 0) , z3.If(r27s17, 1, 0) , z3.If(r27s18, 1, 0) , z3.If(r27s19, 1, 0) , z3.If(r27s20, 1, 0) , z3.If(r27s21, 1, 0) , z3.If(r27s22, 1, 0) , z3.If(r27s23, 1, 0) , z3.If(r27s24, 2, 0) , z3.If(r27s25, 1, 0) , z3.If(r27s26, 1, 0) , z3.If(r27s27, 1, 0) , z3.If(r27s28, 1, 0) , z3.If(r27s29, 1, 0) , z3.If(r27s30, 1, 0) , z3.If(r27s31, 1, 0) , z3.If(r27s32, 1, 0) , z3.If(r27s33, 1, 0) , z3.If(r27s34, 1, 0) , z3.If(r27s35, 1, 0) , z3.If(r27s36, 1, 0) , z3.If(r27s37, 97, 0) , z3.If(r27s38, 1, 0) , z3.If(r27s39, 2, 0) , z3.If(r27s40, 1, 0) , z3.If(r27s41, 1, 0) , z3.If(r27s42, 97, 0) , z3.If(r27s43, 2, 0) , z3.If(r27s44, 1, 0) , z3.If(r27s45, 1, 0) , z3.If(r27s46, 1, 0) , z3.If(r27s47, 1, 0) , z3.If(r27s48, 1, 0) , z3.If(r27s49, 1, 0) , z3.If(r27s50, 97, 0) , z3.If(r27s51, 1, 0) , z3.If(r27s52, 1, 0) , z3.If(r27s53, 1, 0) , z3.If(r28s0, 1, 0) , z3.If(r28s1, 2, 0) , z3.If(r28s2, 1, 0) , z3.If(r28s3, 1, 0) , z3.If(r28s4, 1, 0) , z3.If(r28s5, 1, 0) , z3.If(r28s6, 2, 0) , z3.If(r28s7, 1, 0) , z3.If(r28s8, 1, 0) , z3.If(r28s9, 2, 0) , z3.If(r28s10, 1, 0) , z3.If(r28s11, 1, 0) , z3.If(r28s12, 2, 0) , z3.If(r28s13, 97, 0) , z3.If(r28s14, 1, 0) , z3.If(r28s15, 1, 0) , z3.If(r28s16, 1, 0) , z3.If(r28s17, 2, 0) , z3.If(r28s18, 97, 0) , z3.If(r28s19, 1, 0) , z3.If(r28s20, 1, 0) , z3.If(r28s21, 1, 0) , z3.If(r28s23, 1, 0) , z3.If(r28s24, 97, 0) , z3.If(r28s25, 97, 0) , z3.If(r28s26, 1, 0) , z3.If(r28s29, 1, 0) , z3.If(r28s30, 2, 0) , z3.If(r28s32, 1, 0) , z3.If(r28s33, 1, 0) , z3.If(r28s35, 1, 0) , z3.If(r28s36, 1, 0) , z3.If(r28s37, 1, 0) , z3.If(r28s38, 1, 0) , z3.If(r28s39, 1, 0) , z3.If(r28s40, 97, 0) , z3.If(r28s41, 1, 0) , z3.If(r28s42, 2, 0) , z3.If(r28s43, 1, 0) , z3.If(r28s44, 1, 0) , z3.If(r28s45, 1, 0) , z3.If(r28s46, 1, 0) , z3.If(r28s47, 1, 0) , z3.If(r28s48, 1, 0) , z3.If(r28s49, 1, 0) , z3.If(r28s50, 1, 0) , z3.If(r28s51, 1, 0) , z3.If(r28s52, 1, 0) , z3.If(r28s53, 1, 0) , z3.If(r29s0, 1, 0) , z3.If(r29s1, 1, 0) , z3.If(r29s2, 1, 0) , z3.If(r29s3, 1, 0) , z3.If(r29s4, 1, 0) , z3.If(r29s5, 1, 0) , z3.If(r29s6, 97, 0) , z3.If(r29s7, 1, 0) , z3.If(r29s8, 1, 0) , z3.If(r29s9, 1, 0) , z3.If(r29s10, 1, 0) , z3.If(r29s11, 1, 0) , z3.If(r29s12, 97, 0) , z3.If(r29s13, 1, 0) , z3.If(r29s14, 1, 0) , z3.If(r29s15, 1, 0) , z3.If(r29s16, 97, 0) , z3.If(r29s17, 97, 0) , z3.If(r29s18, 1, 0) , z3.If(r29s19, 1, 0) , z3.If(r29s20, 1, 0) , z3.If(r29s21, 1, 0) , z3.If(r29s23, 1, 0) , z3.If(r29s24, 97, 0) , z3.If(r29s25, 97, 0) , z3.If(r29s26, 1, 0) , z3.If(r29s29, 1, 0) , z3.If(r29s30, 97, 0) , z3.If(r29s32, 97, 0) , z3.If(r29s33, 1, 0) , z3.If(r29s35, 1, 0) , z3.If(r29s36, 97, 0) , z3.If(r29s37, 1, 0) , z3.If(r29s38, 1, 0) , z3.If(r29s39, 1, 0) , z3.If(r29s40, 97, 0) , z3.If(r29s41, 1, 0) , z3.If(r29s42, 97, 0) , z3.If(r29s43, 1, 0) , z3.If(r29s44, 1, 0) , z3.If(r29s45, 1, 0) , z3.If(r29s46, 1, 0) , z3.If(r29s47, 1, 0) , z3.If(r29s48, 1, 0) , z3.If(r29s49, 1, 0) , z3.If(r29s50, 1, 0) , z3.If(r29s51, 1, 0) , z3.If(r29s52, 1, 0) , z3.If(r29s53, 1, 0) , z3.If(r30s0, 1, 0) , z3.If(r30s1, 1, 0) , z3.If(r30s2, 1, 0) , z3.If(r30s3, 97, 0) , z3.If(r30s4, 1, 0) , z3.If(r30s5, 2, 0) , z3.If(r30s6, 97, 0) , z3.If(r30s7, 1, 0) , z3.If(r30s8, 1, 0) , z3.If(r30s9, 1, 0) , z3.If(r30s10, 1, 0) , z3.If(r30s11, 2, 0) , z3.If(r30s12, 1, 0) , z3.If(r30s13, 97, 0) , z3.If(r30s14, 1, 0) , z3.If(r30s15, 97, 0) , z3.If(r30s16, 2, 0) , z3.If(r30s17, 1, 0) , z3.If(r30s18, 1, 0) , z3.If(r30s19, 1, 0) , z3.If(r30s20, 1, 0) , z3.If(r30s21, 2, 0) , z3.If(r30s22, 1, 0) , z3.If(r30s23, 1, 0) , z3.If(r30s24, 1, 0) , z3.If(r30s25, 97, 0) , z3.If(r30s26, 1, 0) , z3.If(r30s27, 1, 0) , z3.If(r30s28, 1, 0) , z3.If(r30s29, 2, 0) , z3.If(r30s30, 1, 0) , z3.If(r30s31, 1, 0) , z3.If(r30s32, 2, 0) , z3.If(r30s33, 1, 0) , z3.If(r30s34, 1, 0) , z3.If(r30s35, 2, 0) , z3.If(r30s37, 1, 0) , z3.If(r30s38, 1, 0) , z3.If(r30s39, 1, 0) , z3.If(r30s40, 1, 0) , z3.If(r30s41, 1, 0) , z3.If(r30s42, 1, 0) , z3.If(r30s43, 1, 0) , z3.If(r30s44, 1, 0) , z3.If(r30s45, 2, 0) , z3.If(r30s46, 2, 0) , z3.If(r30s47, 1, 0) , z3.If(r30s48, 1, 0) , z3.If(r30s49, 97, 0) , z3.If(r30s50, 1, 0) , z3.If(r30s51, 1, 0) , z3.If(r30s52, 1, 0) , z3.If(r30s53, 1, 0)]

#### Rules (PbLe)

In [16]:
# rev_rules = []
# for l in pb_rev_split:
#     rev_rules.append("z3.PbGe(({0}), o)".format(l))
# # print(rev_rules)
# for i in rev_rules:
#     print(i.replace("'", ""), ',', end=' ')

In [17]:
# sub_rules = []
# for l in pb_sub_split: 
#     sub_rules.append("z3.PbGe(({0}), p)".format(l))
# # print(sub_rules)
# for i in sub_rules:
#     print(i.replace("'", ""), ',', end=' ')

### Setting variables and running the algorithm

In [18]:
# These values decide the maximum cardinality:
# n is the maximum number of submission per reviewer and m is the maximum number of reviewers per submission
###############################################################################################################
n = 3
m = 10
###############################################################################################################

# Run the algorithm
o.add([
z3.PbLe(([(r0s0,1), (r0s1,1), (r0s2,1), (r0s4,1), (r0s5,1), (r0s6,1), (r0s7,1), (r0s8,1), (r0s9,1), (r0s10,1), (r0s11,1), (r0s12,1), (r0s13,1), (r0s14,1), (r0s15,1), (r0s16,1), (r0s17,1), (r0s18,1), (r0s19,1), (r0s20,1), (r0s21,1), (r0s22,1), (r0s23,1), (r0s24,1), (r0s25,1), (r0s26,1), (r0s27,1), (r0s28,1), (r0s29,1), (r0s30,1), (r0s31,1), (r0s32,1), (r0s33,1), (r0s34,1), (r0s35,1), (r0s36,1), (r0s37,1), (r0s38,1), (r0s39,1), (r0s40,1), (r0s41,1), (r0s42,1), (r0s43,1), (r0s44,1), (r0s45,1), (r0s46,1), (r0s47,1), (r0s48,1), (r0s49,1), (r0s51,1), (r0s52,1), (r0s53,1), (r1s0,1), (r1s1,1)]), n) , z3.PbLe(([(r1s2,1), (r1s3,1), (r1s4,1), (r1s5,1), (r1s6,1), (r1s7,1), (r1s8,1), (r1s9,1), (r1s10,1), (r1s11,1), (r1s12,1), (r1s13,1), (r1s14,1), (r1s15,1), (r1s16,1), (r1s17,1), (r1s19,1), (r1s20,1), (r1s21,1), (r1s22,1), (r1s23,1), (r1s24,1), (r1s25,1), (r1s26,1), (r1s27,1), (r1s28,1), (r1s29,1), (r1s30,1), (r1s31,1), (r1s32,1), (r1s33,1), (r1s34,1), (r1s35,1), (r1s36,1), (r1s37,1), (r1s38,1), (r1s39,1), (r1s41,1), (r1s42,1), (r1s43,1), (r1s44,1), (r1s45,1), (r1s46,1), (r1s47,1), (r1s48,1), (r1s49,1), (r1s50,1), (r1s51,1), (r1s52,1), (r1s53,1), (r2s0,1), (r2s1,1), (r2s2,1), (r2s3,1)]), n) , z3.PbLe(([(r2s4,1), (r2s5,1), (r2s6,1), (r2s7,1), (r2s8,1), (r2s9,1), (r2s10,1), (r2s11,1), (r2s12,1), (r2s13,1), (r2s14,1), (r2s15,1), (r2s16,1), (r2s17,1), (r2s18,1), (r2s19,1), (r2s20,1), (r2s21,1), (r2s22,1), (r2s23,1), (r2s24,1), (r2s25,1), (r2s26,1), (r2s27,1), (r2s28,1), (r2s29,1), (r2s30,1), (r2s31,1), (r2s32,1), (r2s33,1), (r2s34,1), (r2s35,1), (r2s36,1), (r2s37,1), (r2s38,1), (r2s39,1), (r2s40,1), (r2s41,1), (r2s42,1), (r2s43,1), (r2s44,1), (r2s45,1), (r2s46,1), (r2s47,1), (r2s48,1), (r2s49,1), (r2s50,1), (r2s51,1), (r2s52,1), (r2s53,1), (r3s0,1), (r3s1,1), (r3s2,1), (r3s3,1)]), n) , z3.PbLe(([(r3s4,1), (r3s5,1), (r3s6,1), (r3s7,1), (r3s8,1), (r3s9,1), (r3s10,1), (r3s11,1), (r3s12,1), (r3s13,1), (r3s14,1), (r3s15,1), (r3s16,1), (r3s17,1), (r3s18,1), (r3s19,1), (r3s20,1), (r3s21,1), (r3s22,1), (r3s23,1), (r3s24,1), (r3s25,1), (r3s26,1), (r3s27,1), (r3s28,1), (r3s29,1), (r3s30,1), (r3s31,1), (r3s32,1), (r3s33,1), (r3s34,1), (r3s35,1), (r3s36,1), (r3s37,1), (r3s38,1), (r3s39,1), (r3s40,1), (r3s41,1), (r3s42,1), (r3s43,1), (r3s44,1), (r3s45,1), (r3s46,1), (r3s47,1), (r3s48,1), (r3s49,1), (r3s50,1), (r3s51,1), (r3s52,1), (r3s53,1), (r4s0,1), (r4s1,1), (r4s2,1), (r4s3,1)]), n) , z3.PbLe(([(r4s4,1), (r4s5,1), (r4s6,1), (r4s7,1), (r4s8,1), (r4s10,1), (r4s11,1), (r4s12,1), (r4s13,1), (r4s14,1), (r4s15,1), (r4s16,1), (r4s17,1), (r4s19,1), (r4s20,1), (r4s21,1), (r4s22,1), (r4s23,1), (r4s24,1), (r4s26,1), (r4s27,1), (r4s28,1), (r4s29,1), (r4s30,1), (r4s31,1), (r4s32,1), (r4s33,1), (r4s34,1), (r4s35,1), (r4s36,1), (r4s37,1), (r4s38,1), (r4s39,1), (r4s40,1), (r4s41,1), (r4s42,1), (r4s43,1), (r4s44,1), (r4s45,1), (r4s46,1), (r4s47,1), (r4s48,1), (r4s49,1), (r4s50,1), (r4s51,1), (r4s52,1), (r4s53,1), (r5s0,1), (r5s1,1), (r5s2,1), (r5s3,1), (r5s4,1), (r5s5,1), (r5s6,1)]), n) , z3.PbLe(([(r5s7,1), (r5s8,1), (r5s9,1), (r5s10,1), (r5s11,1), (r5s12,1), (r5s13,1), (r5s14,1), (r5s15,1), (r5s16,1), (r5s17,1), (r5s18,1), (r5s19,1), (r5s20,1), (r5s21,1), (r5s22,1), (r5s23,1), (r5s24,1), (r5s25,1), (r5s26,1), (r5s27,1), (r5s28,1), (r5s29,1), (r5s30,1), (r5s31,1), (r5s32,1), (r5s33,1), (r5s34,1), (r5s35,1), (r5s36,1), (r5s37,1), (r5s38,1), (r5s39,1), (r5s40,1), (r5s41,1), (r5s42,1), (r5s43,1), (r5s44,1), (r5s45,1), (r5s46,1), (r5s47,1), (r5s48,1), (r5s49,1), (r5s50,1), (r5s51,1), (r5s52,1), (r5s53,1), (r6s0,1), (r6s1,1), (r6s2,1), (r6s3,1), (r6s4,1), (r6s5,1), (r6s6,1)]), n) , z3.PbLe(([(r6s7,1), (r6s8,1), (r6s9,1), (r6s10,1), (r6s11,1), (r6s12,1), (r6s13,1), (r6s14,1), (r6s15,1), (r6s16,1), (r6s17,1), (r6s18,1), (r6s19,1), (r6s20,1), (r6s21,1), (r6s22,1), (r6s23,1), (r6s24,1), (r6s25,1), (r6s26,1), (r6s27,1), (r6s28,1), (r6s29,1), (r6s30,1), (r6s31,1), (r6s32,1), (r6s33,1), (r6s34,1), (r6s35,1), (r6s36,1), (r6s37,1), (r6s38,1), (r6s39,1), (r6s40,1), (r6s41,1), (r6s42,1), (r6s43,1), (r6s44,1), (r6s45,1), (r6s46,1), (r6s47,1), (r6s48,1), (r6s49,1), (r6s50,1), (r6s51,1), (r6s52,1), (r6s53,1), (r7s0,1), (r7s1,1), (r7s2,1), (r7s3,1), (r7s4,1), (r7s5,1), (r7s6,1)]), n) , z3.PbLe(([(r7s7,1), (r7s8,1), (r7s9,1), (r7s10,1), (r7s11,1), (r7s12,1), (r7s13,1), (r7s14,1), (r7s15,1), (r7s16,1), (r7s17,1), (r7s19,1), (r7s20,1), (r7s21,1), (r7s22,1), (r7s23,1), (r7s25,1), (r7s26,1), (r7s27,1), (r7s28,1), (r7s29,1), (r7s30,1), (r7s31,1), (r7s32,1), (r7s33,1), (r7s34,1), (r7s35,1), (r7s36,1), (r7s37,1), (r7s38,1), (r7s39,1), (r7s40,1), (r7s41,1), (r7s42,1), (r7s43,1), (r7s44,1), (r7s45,1), (r7s46,1), (r7s47,1), (r7s48,1), (r7s49,1), (r7s50,1), (r7s51,1), (r7s52,1), (r7s53,1), (r8s0,1), (r8s2,1), (r8s3,1), (r8s4,1), (r8s5,1), (r8s6,1), (r8s7,1), (r8s8,1), (r8s9,1)]), n) , z3.PbLe(([(r8s10,1), (r8s11,1), (r8s12,1), (r8s13,1), (r8s14,1), (r8s15,1), (r8s16,1), (r8s17,1), (r8s18,1), (r8s19,1), (r8s20,1), (r8s21,1), (r8s22,1), (r8s23,1), (r8s24,1), (r8s25,1), (r8s26,1), (r8s27,1), (r8s28,1), (r8s29,1), (r8s30,1), (r8s31,1), (r8s32,1), (r8s33,1), (r8s34,1), (r8s35,1), (r8s36,1), (r8s37,1), (r8s38,1), (r8s39,1), (r8s40,1), (r8s41,1), (r8s42,1), (r8s43,1), (r8s44,1), (r8s45,1), (r8s46,1), (r8s47,1), (r8s48,1), (r8s49,1), (r8s50,1), (r8s51,1), (r8s52,1), (r8s53,1), (r9s0,1), (r9s1,1), (r9s2,1), (r9s3,1), (r9s4,1), (r9s5,1), (r9s6,1), (r9s7,1), (r9s8,1), (r9s9,1)]), n) , z3.PbLe(([(r9s10,1), (r9s11,1), (r9s12,1), (r9s13,1), (r9s14,1), (r9s15,1), (r9s16,1), (r9s17,1), (r9s18,1), (r9s19,1), (r9s20,1), (r9s21,1), (r9s22,1), (r9s23,1), (r9s24,1), (r9s25,1), (r9s26,1), (r9s27,1), (r9s28,1), (r9s29,1), (r9s30,1), (r9s31,1), (r9s32,1), (r9s33,1), (r9s34,1), (r9s35,1), (r9s36,1), (r9s37,1), (r9s38,1), (r9s39,1), (r9s40,1), (r9s41,1), (r9s42,1), (r9s43,1), (r9s44,1), (r9s45,1), (r9s46,1), (r9s47,1), (r9s48,1), (r9s49,1), (r9s50,1), (r9s51,1), (r9s52,1), (r9s53,1), (r10s0,1), (r10s1,1), (r10s2,1), (r10s3,1), (r10s4,1), (r10s5,1), (r10s6,1), (r10s7,1), (r10s8,1), (r10s9,1)]), n) , z3.PbLe(([(r10s10,1), (r10s11,1), (r10s12,1), (r10s13,1), (r10s14,1), (r10s15,1), (r10s16,1), (r10s17,1), (r10s18,1), (r10s19,1), (r10s20,1), (r10s21,1), (r10s22,1), (r10s23,1), (r10s24,1), (r10s25,1), (r10s26,1), (r10s27,1), (r10s28,1), (r10s29,1), (r10s30,1), (r10s31,1), (r10s32,1), (r10s33,1), (r10s34,1), (r10s35,1), (r10s36,1), (r10s37,1), (r10s38,1), (r10s39,1), (r10s40,1), (r10s41,1), (r10s42,1), (r10s43,1), (r10s44,1), (r10s45,1), (r10s46,1), (r10s47,1), (r10s48,1), (r10s49,1), (r10s50,1), (r10s51,1), (r10s52,1), (r10s53,1), (r11s0,1), (r11s1,1), (r11s2,1), (r11s3,1), (r11s4,1), (r11s5,1), (r11s6,1), (r11s7,1), (r11s8,1), (r11s9,1)]), n) , z3.PbLe(([(r11s10,1), (r11s11,1), (r11s12,1), (r11s13,1), (r11s14,1), (r11s15,1), (r11s16,1), (r11s17,1), (r11s18,1), (r11s19,1), (r11s20,1), (r11s21,1), (r11s22,1), (r11s23,1), (r11s24,1), (r11s25,1), (r11s26,1), (r11s27,1), (r11s28,1), (r11s29,1), (r11s30,1), (r11s31,1), (r11s32,1), (r11s33,1), (r11s34,1), (r11s35,1), (r11s36,1), (r11s37,1), (r11s38,1), (r11s39,1), (r11s40,1), (r11s41,1), (r11s42,1), (r11s43,1), (r11s44,1), (r11s45,1), (r11s46,1), (r11s47,1), (r11s48,1), (r11s49,1), (r11s50,1), (r11s51,1), (r11s52,1), (r11s53,1), (r12s0,1), (r12s1,1), (r12s2,1), (r12s3,1), (r12s4,1), (r12s5,1), (r12s6,1), (r12s8,1), (r12s9,1), (r12s10,1)]), n) , z3.PbLe(([(r12s11,1), (r12s12,1), (r12s13,1), (r12s14,1), (r12s15,1), (r12s16,1), (r12s17,1), (r12s18,1), (r12s19,1), (r12s20,1), (r12s21,1), (r12s22,1), (r12s23,1), (r12s24,1), (r12s25,1), (r12s26,1), (r12s27,1), (r12s28,1), (r12s30,1), (r12s31,1), (r12s32,1), (r12s33,1), (r12s34,1), (r12s35,1), (r12s36,1), (r12s37,1), (r12s38,1), (r12s39,1), (r12s40,1), (r12s41,1), (r12s42,1), (r12s43,1), (r12s44,1), (r12s45,1), (r12s46,1), (r12s47,1), (r12s48,1), (r12s49,1), (r12s50,1), (r12s51,1), (r12s52,1), (r12s53,1), (r13s0,1), (r13s1,1), (r13s2,1), (r13s3,1), (r13s4,1), (r13s5,1), (r13s6,1), (r13s7,1), (r13s8,1), (r13s9,1), (r13s10,1), (r13s11,1)]), n) , z3.PbLe(([(r13s12,1), (r13s13,1), (r13s14,1), (r13s15,1), (r13s16,1), (r13s17,1), (r13s18,1), (r13s19,1), (r13s20,1), (r13s21,1), (r13s22,1), (r13s23,1), (r13s24,1), (r13s25,1), (r13s26,1), (r13s27,1), (r13s28,1), (r13s29,1), (r13s30,1), (r13s31,1), (r13s32,1), (r13s33,1), (r13s34,1), (r13s35,1), (r13s36,1), (r13s37,1), (r13s38,1), (r13s39,1), (r13s40,1), (r13s41,1), (r13s42,1), (r13s43,1), (r13s44,1), (r13s45,1), (r13s46,1), (r13s47,1), (r13s48,1), (r13s49,1), (r13s50,1), (r13s51,1), (r13s52,1), (r13s53,1), (r14s0,1), (r14s1,1), (r14s2,1), (r14s3,1), (r14s4,1), (r14s5,1), (r14s6,1), (r14s7,1), (r14s8,1), (r14s9,1), (r14s10,1), (r14s11,1)]), n) , z3.PbLe(([(r14s12,1), (r14s14,1), (r14s15,1), (r14s16,1), (r14s17,1), (r14s18,1), (r14s19,1), (r14s20,1), (r14s21,1), (r14s22,1), (r14s23,1), (r14s25,1), (r14s26,1), (r14s27,1), (r14s28,1), (r14s29,1), (r14s30,1), (r14s31,1), (r14s32,1), (r14s33,1), (r14s34,1), (r14s35,1), (r14s36,1), (r14s37,1), (r14s38,1), (r14s39,1), (r14s40,1), (r14s41,1), (r14s42,1), (r14s43,1), (r14s44,1), (r14s45,1), (r14s46,1), (r14s47,1), (r14s48,1), (r14s49,1), (r14s50,1), (r14s51,1), (r14s52,1), (r14s53,1), (r15s0,1), (r15s1,1), (r15s2,1), (r15s3,1), (r15s4,1), (r15s5,1), (r15s6,1), (r15s7,1), (r15s8,1), (r15s9,1), (r15s10,1), (r15s11,1), (r15s12,1), (r15s13,1)]), n) , z3.PbLe(([(r15s14,1), (r15s15,1), (r15s16,1), (r15s17,1), (r15s18,1), (r15s19,1), (r15s20,1), (r15s21,1), (r15s22,1), (r15s23,1), (r15s24,1), (r15s25,1), (r15s26,1), (r15s27,1), (r15s28,1), (r15s29,1), (r15s30,1), (r15s31,1), (r15s32,1), (r15s33,1), (r15s34,1), (r15s35,1), (r15s36,1), (r15s37,1), (r15s38,1), (r15s39,1), (r15s40,1), (r15s41,1), (r15s42,1), (r15s43,1), (r15s44,1), (r15s45,1), (r15s46,1), (r15s47,1), (r15s48,1), (r15s49,1), (r15s50,1), (r15s51,1), (r15s52,1), (r15s53,1), (r16s0,1), (r16s1,1), (r16s2,1), (r16s3,1), (r16s4,1), (r16s5,1), (r16s6,1), (r16s7,1), (r16s8,1), (r16s9,1), (r16s10,1), (r16s11,1), (r16s12,1), (r16s13,1)]), n) , z3.PbLe(([(r16s14,1), (r16s15,1), (r16s16,1), (r16s17,1), (r16s18,1), (r16s19,1), (r16s20,1), (r16s21,1), (r16s22,1), (r16s23,1), (r16s24,1), (r16s25,1), (r16s26,1), (r16s27,1), (r16s28,1), (r16s29,1), (r16s30,1), (r16s31,1), (r16s32,1), (r16s33,1), (r16s34,1), (r16s35,1), (r16s36,1), (r16s37,1), (r16s38,1), (r16s39,1), (r16s40,1), (r16s41,1), (r16s42,1), (r16s44,1), (r16s45,1), (r16s46,1), (r16s47,1), (r16s48,1), (r16s50,1), (r16s52,1), (r16s53,1), (r17s0,1), (r17s1,1), (r17s2,1), (r17s3,1), (r17s4,1), (r17s5,1), (r17s6,1), (r17s7,1), (r17s8,1), (r17s9,1), (r17s10,1), (r17s11,1), (r17s12,1), (r17s13,1), (r17s14,1), (r17s15,1), (r17s16,1)]), n) , z3.PbLe(([(r17s17,1), (r17s18,1), (r17s19,1), (r17s20,1), (r17s22,1), (r17s23,1), (r17s24,1), (r17s25,1), (r17s27,1), (r17s28,1), (r17s29,1), (r17s30,1), (r17s31,1), (r17s32,1), (r17s33,1), (r17s34,1), (r17s35,1), (r17s36,1), (r17s37,1), (r17s38,1), (r17s39,1), (r17s40,1), (r17s41,1), (r17s42,1), (r17s43,1), (r17s44,1), (r17s46,1), (r17s47,1), (r17s49,1), (r17s50,1), (r17s51,1), (r17s52,1), (r18s0,1), (r18s1,1), (r18s2,1), (r18s3,1), (r18s4,1), (r18s5,1), (r18s6,1), (r18s7,1), (r18s8,1), (r18s9,1), (r18s10,1), (r18s11,1), (r18s12,1), (r18s13,1), (r18s14,1), (r18s15,1), (r18s16,1), (r18s17,1), (r18s18,1), (r18s19,1), (r18s20,1), (r18s21,1)]), n) , z3.PbLe(([(r18s22,1), (r18s23,1), (r18s24,1), (r18s25,1), (r18s26,1), (r18s27,1), (r18s28,1), (r18s29,1), (r18s30,1), (r18s31,1), (r18s32,1), (r18s33,1), (r18s34,1), (r18s35,1), (r18s36,1), (r18s37,1), (r18s38,1), (r18s39,1), (r18s40,1), (r18s41,1), (r18s42,1), (r18s43,1), (r18s44,1), (r18s45,1), (r18s46,1), (r18s47,1), (r18s48,1), (r18s49,1), (r18s50,1), (r18s51,1), (r18s52,1), (r18s53,1), (r19s0,1), (r19s1,1), (r19s2,1), (r19s3,1), (r19s4,1), (r19s5,1), (r19s6,1), (r19s7,1), (r19s8,1), (r19s9,1), (r19s10,1), (r19s11,1), (r19s12,1), (r19s13,1), (r19s14,1), (r19s15,1), (r19s16,1), (r19s17,1), (r19s18,1), (r19s19,1), (r19s20,1), (r19s21,1)]), n) , z3.PbLe(([(r19s22,1), (r19s23,1), (r19s24,1), (r19s25,1), (r19s26,1), (r19s27,1), (r19s28,1), (r19s29,1), (r19s30,1), (r19s31,1), (r19s32,1), (r19s33,1), (r19s34,1), (r19s35,1), (r19s36,1), (r19s37,1), (r19s38,1), (r19s39,1), (r19s40,1), (r19s41,1), (r19s42,1), (r19s43,1), (r19s44,1), (r19s45,1), (r19s46,1), (r19s47,1), (r19s48,1), (r19s49,1), (r19s50,1), (r19s51,1), (r19s52,1), (r19s53,1), (r20s0,1), (r20s1,1), (r20s2,1), (r20s3,1), (r20s4,1), (r20s5,1), (r20s6,1), (r20s7,1), (r20s8,1), (r20s9,1), (r20s10,1), (r20s11,1), (r20s13,1), (r20s14,1), (r20s15,1), (r20s16,1), (r20s17,1), (r20s18,1), (r20s19,1), (r20s20,1), (r20s21,1), (r20s22,1)]), n) , z3.PbLe(([(r20s23,1), (r20s24,1), (r20s25,1), (r20s26,1), (r20s27,1), (r20s28,1), (r20s29,1), (r20s30,1), (r20s31,1), (r20s32,1), (r20s33,1), (r20s34,1), (r20s35,1), (r20s36,1), (r20s37,1), (r20s38,1), (r20s39,1), (r20s40,1), (r20s41,1), (r20s42,1), (r20s43,1), (r20s44,1), (r20s45,1), (r20s46,1), (r20s47,1), (r20s48,1), (r20s49,1), (r20s50,1), (r20s51,1), (r20s52,1), (r20s53,1), (r21s0,1), (r21s1,1), (r21s2,1), (r21s3,1), (r21s4,1), (r21s5,1), (r21s6,1), (r21s7,1), (r21s8,1), (r21s9,1), (r21s10,1), (r21s11,1), (r21s12,1), (r21s13,1), (r21s14,1), (r21s15,1), (r21s16,1), (r21s17,1), (r21s18,1), (r21s19,1), (r21s20,1), (r21s21,1), (r21s22,1)]), n) , z3.PbLe(([(r21s23,1), (r21s24,1), (r21s25,1), (r21s26,1), (r21s27,1), (r21s28,1), (r21s29,1), (r21s30,1), (r21s31,1), (r21s32,1), (r21s33,1), (r21s34,1), (r21s35,1), (r21s36,1), (r21s37,1), (r21s38,1), (r21s39,1), (r21s40,1), (r21s41,1), (r21s43,1), (r21s44,1), (r21s45,1), (r21s46,1), (r21s47,1), (r21s48,1), (r21s49,1), (r21s50,1), (r21s51,1), (r21s52,1), (r21s53,1), (r22s0,1), (r22s1,1), (r22s2,1), (r22s3,1), (r22s4,1), (r22s5,1), (r22s6,1), (r22s7,1), (r22s8,1), (r22s9,1), (r22s10,1), (r22s11,1), (r22s12,1), (r22s13,1), (r22s14,1), (r22s15,1), (r22s16,1), (r22s17,1), (r22s18,1), (r22s19,1), (r22s20,1), (r22s21,1), (r22s22,1), (r22s23,1)]), n) , z3.PbLe(([(r22s24,1), (r22s25,1), (r22s26,1), (r22s27,1), (r22s28,1), (r22s29,1), (r22s30,1), (r22s31,1), (r22s32,1), (r22s33,1), (r22s34,1), (r22s35,1), (r22s36,1), (r22s37,1), (r22s38,1), (r22s39,1), (r22s40,1), (r22s41,1), (r22s42,1), (r22s43,1), (r22s44,1), (r22s45,1), (r22s46,1), (r22s47,1), (r22s48,1), (r22s49,1), (r22s50,1), (r22s51,1), (r22s52,1), (r22s53,1), (r23s0,1), (r23s1,1), (r23s2,1), (r23s3,1), (r23s4,1), (r23s5,1), (r23s6,1), (r23s7,1), (r23s8,1), (r23s9,1), (r23s10,1), (r23s11,1), (r23s12,1), (r23s13,1), (r23s14,1), (r23s15,1), (r23s16,1), (r23s17,1), (r23s19,1), (r23s20,1), (r23s21,1), (r23s22,1), (r23s23,1), (r23s24,1)]), n) , z3.PbLe(([(r23s25,1), (r23s26,1), (r23s27,1), (r23s28,1), (r23s29,1), (r23s30,1), (r23s31,1), (r23s32,1), (r23s33,1), (r23s34,1), (r23s35,1), (r23s36,1), (r23s37,1), (r23s38,1), (r23s39,1), (r23s41,1), (r23s42,1), (r23s43,1), (r23s44,1), (r23s45,1), (r23s46,1), (r23s47,1), (r23s48,1), (r23s49,1), (r23s50,1), (r23s51,1), (r23s52,1), (r23s53,1), (r24s0,1), (r24s1,1), (r24s2,1), (r24s3,1), (r24s4,1), (r24s5,1), (r24s6,1), (r24s7,1), (r24s8,1), (r24s9,1), (r24s10,1), (r24s11,1), (r24s12,1), (r24s14,1), (r24s15,1), (r24s16,1), (r24s17,1), (r24s18,1), (r24s19,1), (r24s20,1), (r24s21,1), (r24s22,1), (r24s23,1), (r24s24,1), (r24s25,1), (r24s26,1)]), n) , z3.PbLe(([(r24s27,1), (r24s28,1), (r24s29,1), (r24s30,1), (r24s31,1), (r24s32,1), (r24s33,1), (r24s34,1), (r24s35,1), (r24s36,1), (r24s37,1), (r24s38,1), (r24s39,1), (r24s40,1), (r24s42,1), (r24s44,1), (r24s45,1), (r24s46,1), (r24s47,1), (r24s48,1), (r24s50,1), (r24s52,1), (r24s53,1), (r25s0,1), (r25s1,1), (r25s2,1), (r25s3,1), (r25s4,1), (r25s5,1), (r25s6,1), (r25s7,1), (r25s8,1), (r25s9,1), (r25s10,1), (r25s11,1), (r25s13,1), (r25s14,1), (r25s15,1), (r25s16,1), (r25s17,1), (r25s19,1), (r25s20,1), (r25s21,1), (r25s22,1), (r25s23,1), (r25s24,1), (r25s25,1), (r25s26,1), (r25s27,1), (r25s28,1), (r25s29,1), (r25s30,1), (r25s31,1), (r25s32,1)]), n) , z3.PbLe(([(r25s33,1), (r25s34,1), (r25s35,1), (r25s36,1), (r25s37,1), (r25s38,1), (r25s39,1), (r25s40,1), (r25s41,1), (r25s42,1), (r25s43,1), (r25s44,1), (r25s45,1), (r25s46,1), (r25s47,1), (r25s48,1), (r25s49,1), (r25s50,1), (r25s51,1), (r25s52,1), (r25s53,1), (r26s0,1), (r26s1,1), (r26s2,1), (r26s3,1), (r26s4,1), (r26s5,1), (r26s6,1), (r26s7,1), (r26s8,1), (r26s9,1), (r26s10,1), (r26s11,1), (r26s12,1), (r26s13,1), (r26s14,1), (r26s15,1), (r26s16,1), (r26s17,1), (r26s18,1), (r26s20,1), (r26s21,1), (r26s22,1), (r26s23,1), (r26s24,1), (r26s25,1), (r26s26,1), (r26s27,1), (r26s28,1), (r26s29,1), (r26s30,1), (r26s31,1), (r26s32,1), (r26s33,1)]), n) , z3.PbLe(([(r26s34,1), (r26s35,1), (r26s36,1), (r26s37,1), (r26s38,1), (r26s39,1), (r26s40,1), (r26s41,1), (r26s42,1), (r26s43,1), (r26s44,1), (r26s45,1), (r26s46,1), (r26s47,1), (r26s48,1), (r26s49,1), (r26s50,1), (r26s51,1), (r26s52,1), (r26s53,1), (r27s0,1), (r27s1,1), (r27s2,1), (r27s3,1), (r27s4,1), (r27s5,1), (r27s6,1), (r27s7,1), (r27s8,1), (r27s9,1), (r27s10,1), (r27s11,1), (r27s12,1), (r27s13,1), (r27s14,1), (r27s15,1), (r27s16,1), (r27s17,1), (r27s18,1), (r27s19,1), (r27s20,1), (r27s21,1), (r27s22,1), (r27s23,1), (r27s24,1), (r27s25,1), (r27s26,1), (r27s27,1), (r27s28,1), (r27s29,1), (r27s30,1), (r27s31,1), (r27s32,1), (r27s33,1)]), n) , z3.PbLe(([(r27s34,1), (r27s35,1), (r27s36,1), (r27s37,1), (r27s38,1), (r27s39,1), (r27s40,1), (r27s41,1), (r27s42,1), (r27s43,1), (r27s44,1), (r27s45,1), (r27s46,1), (r27s47,1), (r27s48,1), (r27s49,1), (r27s50,1), (r27s51,1), (r27s52,1), (r27s53,1), (r28s0,1), (r28s1,1), (r28s2,1), (r28s3,1), (r28s4,1), (r28s5,1), (r28s6,1), (r28s7,1), (r28s8,1), (r28s9,1), (r28s10,1), (r28s11,1), (r28s12,1), (r28s13,1), (r28s14,1), (r28s15,1), (r28s16,1), (r28s17,1), (r28s18,1), (r28s19,1), (r28s20,1), (r28s21,1), (r28s23,1), (r28s24,1), (r28s25,1), (r28s26,1), (r28s29,1), (r28s30,1), (r28s32,1), (r28s33,1), (r28s35,1), (r28s36,1), (r28s37,1), (r28s38,1)]), n) , z3.PbLe(([(r28s39,1), (r28s40,1), (r28s41,1), (r28s42,1), (r28s43,1), (r28s44,1), (r28s45,1), (r28s46,1), (r28s47,1), (r28s48,1), (r28s49,1), (r28s50,1), (r28s51,1), (r28s52,1), (r28s53,1), (r29s0,1), (r29s1,1), (r29s2,1), (r29s3,1), (r29s4,1), (r29s5,1), (r29s6,1), (r29s7,1), (r29s8,1), (r29s9,1), (r29s10,1), (r29s11,1), (r29s12,1), (r29s13,1), (r29s14,1), (r29s15,1), (r29s16,1), (r29s17,1), (r29s18,1), (r29s19,1), (r29s20,1), (r29s21,1), (r29s23,1), (r29s24,1), (r29s25,1), (r29s26,1), (r29s29,1), (r29s30,1), (r29s32,1), (r29s33,1), (r29s35,1), (r29s36,1), (r29s37,1), (r29s38,1), (r29s39,1), (r29s40,1), (r29s41,1), (r29s42,1), (r29s43,1)]), n) , z3.PbLe(([(r29s44,1), (r29s45,1), (r29s46,1), (r29s47,1), (r29s48,1), (r29s49,1), (r29s50,1), (r29s51,1), (r29s52,1), (r29s53,1), (r30s0,1), (r30s1,1), (r30s2,1), (r30s3,1), (r30s4,1), (r30s5,1), (r30s6,1), (r30s7,1), (r30s8,1), (r30s9,1), (r30s10,1), (r30s11,1), (r30s12,1), (r30s13,1), (r30s14,1), (r30s15,1), (r30s16,1), (r30s17,1), (r30s18,1), (r30s19,1), (r30s20,1), (r30s21,1), (r30s22,1), (r30s23,1), (r30s24,1), (r30s25,1), (r30s26,1), (r30s27,1), (r30s28,1), (r30s29,1), (r30s30,1), (r30s31,1), (r30s32,1), (r30s33,1), (r30s34,1), (r30s35,1), (r30s37,1), (r30s38,1), (r30s39,1), (r30s40,1), (r30s41,1), (r30s42,1), (r30s43,1), (r30s44,1)]), n) , z3.PbLe(([(r30s45,1), (r30s46,1), (r30s47,1), (r30s48,1), (r30s49,1), (r30s50,1), (r30s51,1), (r30s52,1), (r30s53,1)]), n),
z3.PbLe(([(r0s0,1), (r1s0,1), (r2s0,1), (r3s0,1), (r4s0,1), (r5s0,1), (r6s0,1), (r7s0,1), (r8s0,1), (r9s0,1), (r10s0,1), (r11s0,1), (r12s0,1), (r13s0,1), (r14s0,1), (r15s0,1), (r16s0,1), (r17s0,1), (r18s0,1), (r19s0,1), (r20s0,1), (r21s0,1), (r22s0,1), (r23s0,1), (r24s0,1), (r25s0,1), (r26s0,1), (r27s0,1), (r28s0,1), (r29s0,1), (r30s0,1)]), m) , z3.PbLe(([(r0s1,1), (r1s1,1), (r2s1,1), (r3s1,1), (r4s1,1), (r5s1,1), (r6s1,1), (r7s1,1), (r9s1,1), (r10s1,1), (r11s1,1), (r12s1,1), (r13s1,1), (r14s1,1), (r15s1,1), (r16s1,1), (r17s1,1), (r18s1,1), (r19s1,1), (r20s1,1), (r21s1,1), (r22s1,1), (r23s1,1), (r24s1,1), (r25s1,1), (r26s1,1), (r27s1,1), (r28s1,1), (r29s1,1), (r30s1,1), (r0s2,1)]), m) , z3.PbLe(([(r1s2,1), (r2s2,1), (r3s2,1), (r4s2,1), (r5s2,1), (r6s2,1), (r7s2,1), (r8s2,1), (r9s2,1), (r10s2,1), (r11s2,1), (r12s2,1), (r13s2,1), (r14s2,1), (r15s2,1), (r16s2,1), (r17s2,1), (r18s2,1), (r19s2,1), (r20s2,1), (r21s2,1), (r22s2,1), (r23s2,1), (r24s2,1), (r25s2,1), (r26s2,1), (r27s2,1), (r28s2,1), (r29s2,1), (r30s2,1), (r1s3,1)]), m) , z3.PbLe(([(r2s3,1), (r3s3,1), (r4s3,1), (r5s3,1), (r6s3,1), (r7s3,1), (r8s3,1), (r9s3,1), (r10s3,1), (r11s3,1), (r12s3,1), (r13s3,1), (r14s3,1), (r15s3,1), (r16s3,1), (r17s3,1), (r18s3,1), (r19s3,1), (r20s3,1), (r21s3,1), (r22s3,1), (r23s3,1), (r24s3,1), (r25s3,1), (r26s3,1), (r27s3,1), (r28s3,1), (r29s3,1), (r30s3,1), (r0s4,1), (r1s4,1)]), m) , z3.PbLe(([(r2s4,1), (r3s4,1), (r4s4,1), (r5s4,1), (r6s4,1), (r7s4,1), (r8s4,1), (r9s4,1), (r10s4,1), (r11s4,1), (r12s4,1), (r13s4,1), (r14s4,1), (r15s4,1), (r16s4,1), (r17s4,1), (r18s4,1), (r19s4,1), (r20s4,1), (r21s4,1), (r22s4,1), (r23s4,1), (r24s4,1), (r25s4,1), (r26s4,1), (r27s4,1), (r28s4,1), (r29s4,1), (r30s4,1), (r0s5,1), (r1s5,1)]), m) , z3.PbLe(([(r2s5,1), (r3s5,1), (r4s5,1), (r5s5,1), (r6s5,1), (r7s5,1), (r8s5,1), (r9s5,1), (r10s5,1), (r11s5,1), (r12s5,1), (r13s5,1), (r14s5,1), (r15s5,1), (r16s5,1), (r17s5,1), (r18s5,1), (r19s5,1), (r20s5,1), (r21s5,1), (r22s5,1), (r23s5,1), (r24s5,1), (r25s5,1), (r26s5,1), (r27s5,1), (r28s5,1), (r29s5,1), (r30s5,1), (r0s6,1), (r1s6,1)]), m) , z3.PbLe(([(r2s6,1), (r3s6,1), (r4s6,1), (r5s6,1), (r6s6,1), (r7s6,1), (r8s6,1), (r9s6,1), (r10s6,1), (r11s6,1), (r12s6,1), (r13s6,1), (r14s6,1), (r15s6,1), (r16s6,1), (r17s6,1), (r18s6,1), (r19s6,1), (r20s6,1), (r21s6,1), (r22s6,1), (r23s6,1), (r24s6,1), (r25s6,1), (r26s6,1), (r27s6,1), (r28s6,1), (r29s6,1), (r30s6,1), (r0s7,1), (r1s7,1)]), m) , z3.PbLe(([(r2s7,1), (r3s7,1), (r4s7,1), (r5s7,1), (r6s7,1), (r7s7,1), (r8s7,1), (r9s7,1), (r10s7,1), (r11s7,1), (r13s7,1), (r14s7,1), (r15s7,1), (r16s7,1), (r17s7,1), (r18s7,1), (r19s7,1), (r20s7,1), (r21s7,1), (r22s7,1), (r23s7,1), (r24s7,1), (r25s7,1), (r26s7,1), (r27s7,1), (r28s7,1), (r29s7,1), (r30s7,1), (r0s8,1), (r1s8,1), (r2s8,1)]), m) , z3.PbLe(([(r3s8,1), (r4s8,1), (r5s8,1), (r6s8,1), (r7s8,1), (r8s8,1), (r9s8,1), (r10s8,1), (r11s8,1), (r12s8,1), (r13s8,1), (r14s8,1), (r15s8,1), (r16s8,1), (r17s8,1), (r18s8,1), (r19s8,1), (r20s8,1), (r21s8,1), (r22s8,1), (r23s8,1), (r24s8,1), (r25s8,1), (r26s8,1), (r27s8,1), (r28s8,1), (r29s8,1), (r30s8,1), (r0s9,1), (r1s9,1), (r2s9,1)]), m) , z3.PbLe(([(r3s9,1), (r5s9,1), (r6s9,1), (r7s9,1), (r8s9,1), (r9s9,1), (r10s9,1), (r11s9,1), (r12s9,1), (r13s9,1), (r14s9,1), (r15s9,1), (r16s9,1), (r17s9,1), (r18s9,1), (r19s9,1), (r20s9,1), (r21s9,1), (r22s9,1), (r23s9,1), (r24s9,1), (r25s9,1), (r26s9,1), (r27s9,1), (r28s9,1), (r29s9,1), (r30s9,1), (r0s10,1), (r1s10,1), (r2s10,1), (r3s10,1)]), m) , z3.PbLe(([(r4s10,1), (r5s10,1), (r6s10,1), (r7s10,1), (r8s10,1), (r9s10,1), (r10s10,1), (r11s10,1), (r12s10,1), (r13s10,1), (r14s10,1), (r15s10,1), (r16s10,1), (r17s10,1), (r18s10,1), (r19s10,1), (r20s10,1), (r21s10,1), (r22s10,1), (r23s10,1), (r24s10,1), (r25s10,1), (r26s10,1), (r27s10,1), (r28s10,1), (r29s10,1), (r30s10,1), (r0s11,1), (r1s11,1), (r2s11,1), (r3s11,1)]), m) , z3.PbLe(([(r4s11,1), (r5s11,1), (r6s11,1), (r7s11,1), (r8s11,1), (r9s11,1), (r10s11,1), (r11s11,1), (r12s11,1), (r13s11,1), (r14s11,1), (r15s11,1), (r16s11,1), (r17s11,1), (r18s11,1), (r19s11,1), (r20s11,1), (r21s11,1), (r22s11,1), (r23s11,1), (r24s11,1), (r25s11,1), (r26s11,1), (r27s11,1), (r28s11,1), (r29s11,1), (r30s11,1), (r0s12,1), (r1s12,1), (r2s12,1), (r3s12,1)]), m) , z3.PbLe(([(r4s12,1), (r5s12,1), (r6s12,1), (r7s12,1), (r8s12,1), (r9s12,1), (r10s12,1), (r11s12,1), (r12s12,1), (r13s12,1), (r14s12,1), (r15s12,1), (r16s12,1), (r17s12,1), (r18s12,1), (r19s12,1), (r21s12,1), (r22s12,1), (r23s12,1), (r24s12,1), (r26s12,1), (r27s12,1), (r28s12,1), (r29s12,1), (r30s12,1), (r0s13,1), (r1s13,1), (r2s13,1), (r3s13,1), (r4s13,1), (r5s13,1)]), m) , z3.PbLe(([(r6s13,1), (r7s13,1), (r8s13,1), (r9s13,1), (r10s13,1), (r11s13,1), (r12s13,1), (r13s13,1), (r15s13,1), (r16s13,1), (r17s13,1), (r18s13,1), (r19s13,1), (r20s13,1), (r21s13,1), (r22s13,1), (r23s13,1), (r25s13,1), (r26s13,1), (r27s13,1), (r28s13,1), (r29s13,1), (r30s13,1), (r0s14,1), (r1s14,1), (r2s14,1), (r3s14,1), (r4s14,1), (r5s14,1), (r6s14,1), (r7s14,1)]), m) , z3.PbLe(([(r8s14,1), (r9s14,1), (r10s14,1), (r11s14,1), (r12s14,1), (r13s14,1), (r14s14,1), (r15s14,1), (r16s14,1), (r17s14,1), (r18s14,1), (r19s14,1), (r20s14,1), (r21s14,1), (r22s14,1), (r23s14,1), (r24s14,1), (r25s14,1), (r26s14,1), (r27s14,1), (r28s14,1), (r29s14,1), (r30s14,1), (r0s15,1), (r1s15,1), (r2s15,1), (r3s15,1), (r4s15,1), (r5s15,1), (r6s15,1), (r7s15,1)]), m) , z3.PbLe(([(r8s15,1), (r9s15,1), (r10s15,1), (r11s15,1), (r12s15,1), (r13s15,1), (r14s15,1), (r15s15,1), (r16s15,1), (r17s15,1), (r18s15,1), (r19s15,1), (r20s15,1), (r21s15,1), (r22s15,1), (r23s15,1), (r24s15,1), (r25s15,1), (r26s15,1), (r27s15,1), (r28s15,1), (r29s15,1), (r30s15,1), (r0s16,1), (r1s16,1), (r2s16,1), (r3s16,1), (r4s16,1), (r5s16,1), (r6s16,1), (r7s16,1)]), m) , z3.PbLe(([(r8s16,1), (r9s16,1), (r10s16,1), (r11s16,1), (r12s16,1), (r13s16,1), (r14s16,1), (r15s16,1), (r16s16,1), (r17s16,1), (r18s16,1), (r19s16,1), (r20s16,1), (r21s16,1), (r22s16,1), (r23s16,1), (r24s16,1), (r25s16,1), (r26s16,1), (r27s16,1), (r28s16,1), (r29s16,1), (r30s16,1), (r0s17,1), (r1s17,1), (r2s17,1), (r3s17,1), (r4s17,1), (r5s17,1), (r6s17,1), (r7s17,1)]), m) , z3.PbLe(([(r8s17,1), (r9s17,1), (r10s17,1), (r11s17,1), (r12s17,1), (r13s17,1), (r14s17,1), (r15s17,1), (r16s17,1), (r17s17,1), (r18s17,1), (r19s17,1), (r20s17,1), (r21s17,1), (r22s17,1), (r23s17,1), (r24s17,1), (r25s17,1), (r26s17,1), (r27s17,1), (r28s17,1), (r29s17,1), (r30s17,1), (r0s18,1), (r2s18,1), (r3s18,1), (r5s18,1), (r6s18,1), (r8s18,1), (r9s18,1), (r10s18,1)]), m) , z3.PbLe(([(r11s18,1), (r12s18,1), (r13s18,1), (r14s18,1), (r15s18,1), (r16s18,1), (r17s18,1), (r18s18,1), (r19s18,1), (r20s18,1), (r21s18,1), (r22s18,1), (r24s18,1), (r26s18,1), (r27s18,1), (r28s18,1), (r29s18,1), (r30s18,1), (r0s19,1), (r1s19,1), (r2s19,1), (r3s19,1), (r4s19,1), (r5s19,1), (r6s19,1), (r7s19,1), (r8s19,1), (r9s19,1), (r10s19,1), (r11s19,1), (r12s19,1)]), m) , z3.PbLe(([(r13s19,1), (r14s19,1), (r15s19,1), (r16s19,1), (r17s19,1), (r18s19,1), (r19s19,1), (r20s19,1), (r21s19,1), (r22s19,1), (r23s19,1), (r24s19,1), (r25s19,1), (r27s19,1), (r28s19,1), (r29s19,1), (r30s19,1), (r0s20,1), (r1s20,1), (r2s20,1), (r3s20,1), (r4s20,1), (r5s20,1), (r6s20,1), (r7s20,1), (r8s20,1), (r9s20,1), (r10s20,1), (r11s20,1), (r12s20,1), (r13s20,1)]), m) , z3.PbLe(([(r14s20,1), (r15s20,1), (r16s20,1), (r17s20,1), (r18s20,1), (r19s20,1), (r20s20,1), (r21s20,1), (r22s20,1), (r23s20,1), (r24s20,1), (r25s20,1), (r26s20,1), (r27s20,1), (r28s20,1), (r29s20,1), (r30s20,1), (r0s21,1), (r1s21,1), (r2s21,1), (r3s21,1), (r4s21,1), (r5s21,1), (r6s21,1), (r7s21,1), (r8s21,1), (r9s21,1), (r10s21,1), (r11s21,1), (r12s21,1), (r13s21,1)]), m) , z3.PbLe(([(r14s21,1), (r15s21,1), (r16s21,1), (r18s21,1), (r19s21,1), (r20s21,1), (r21s21,1), (r22s21,1), (r23s21,1), (r24s21,1), (r25s21,1), (r26s21,1), (r27s21,1), (r28s21,1), (r29s21,1), (r30s21,1), (r0s22,1), (r1s22,1), (r2s22,1), (r3s22,1), (r4s22,1), (r5s22,1), (r6s22,1), (r7s22,1), (r8s22,1), (r9s22,1), (r10s22,1), (r11s22,1), (r12s22,1), (r13s22,1), (r14s22,1)]), m) , z3.PbLe(([(r15s22,1), (r16s22,1), (r17s22,1), (r18s22,1), (r19s22,1), (r20s22,1), (r21s22,1), (r22s22,1), (r23s22,1), (r24s22,1), (r25s22,1), (r26s22,1), (r27s22,1), (r30s22,1), (r0s23,1), (r1s23,1), (r2s23,1), (r3s23,1), (r4s23,1), (r5s23,1), (r6s23,1), (r7s23,1), (r8s23,1), (r9s23,1), (r10s23,1), (r11s23,1), (r12s23,1), (r13s23,1), (r14s23,1), (r15s23,1), (r16s23,1)]), m) , z3.PbLe(([(r17s23,1), (r18s23,1), (r19s23,1), (r20s23,1), (r21s23,1), (r22s23,1), (r23s23,1), (r24s23,1), (r25s23,1), (r26s23,1), (r27s23,1), (r28s23,1), (r29s23,1), (r30s23,1), (r0s24,1), (r1s24,1), (r2s24,1), (r3s24,1), (r4s24,1), (r5s24,1), (r6s24,1), (r8s24,1), (r9s24,1), (r10s24,1), (r11s24,1), (r12s24,1), (r13s24,1), (r15s24,1), (r16s24,1), (r17s24,1), (r18s24,1)]), m) , z3.PbLe(([(r19s24,1), (r20s24,1), (r21s24,1), (r22s24,1), (r23s24,1), (r24s24,1), (r25s24,1), (r26s24,1), (r27s24,1), (r28s24,1), (r29s24,1), (r30s24,1), (r0s25,1), (r1s25,1), (r2s25,1), (r3s25,1), (r5s25,1), (r6s25,1), (r7s25,1), (r8s25,1), (r9s25,1), (r10s25,1), (r11s25,1), (r12s25,1), (r13s25,1), (r14s25,1), (r15s25,1), (r16s25,1), (r17s25,1), (r18s25,1), (r19s25,1)]), m) , z3.PbLe(([(r20s25,1), (r21s25,1), (r22s25,1), (r23s25,1), (r24s25,1), (r25s25,1), (r26s25,1), (r27s25,1), (r28s25,1), (r29s25,1), (r30s25,1), (r0s26,1), (r1s26,1), (r2s26,1), (r3s26,1), (r4s26,1), (r5s26,1), (r6s26,1), (r7s26,1), (r8s26,1), (r9s26,1), (r10s26,1), (r11s26,1), (r12s26,1), (r13s26,1), (r14s26,1), (r15s26,1), (r16s26,1), (r18s26,1), (r19s26,1), (r20s26,1)]), m) , z3.PbLe(([(r21s26,1), (r22s26,1), (r23s26,1), (r24s26,1), (r25s26,1), (r26s26,1), (r27s26,1), (r28s26,1), (r29s26,1), (r30s26,1), (r0s27,1), (r1s27,1), (r2s27,1), (r3s27,1), (r4s27,1), (r5s27,1), (r6s27,1), (r7s27,1), (r8s27,1), (r9s27,1), (r10s27,1), (r11s27,1), (r12s27,1), (r13s27,1), (r14s27,1), (r15s27,1), (r16s27,1), (r17s27,1), (r18s27,1), (r19s27,1), (r20s27,1)]), m) , z3.PbLe(([(r21s27,1), (r22s27,1), (r23s27,1), (r24s27,1), (r25s27,1), (r26s27,1), (r27s27,1), (r30s27,1), (r0s28,1), (r1s28,1), (r2s28,1), (r3s28,1), (r4s28,1), (r5s28,1), (r6s28,1), (r7s28,1), (r8s28,1), (r9s28,1), (r10s28,1), (r11s28,1), (r12s28,1), (r13s28,1), (r14s28,1), (r15s28,1), (r16s28,1), (r17s28,1), (r18s28,1), (r19s28,1), (r20s28,1), (r21s28,1), (r22s28,1)]), m) , z3.PbLe(([(r23s28,1), (r24s28,1), (r25s28,1), (r26s28,1), (r27s28,1), (r30s28,1), (r0s29,1), (r1s29,1), (r2s29,1), (r3s29,1), (r4s29,1), (r5s29,1), (r6s29,1), (r7s29,1), (r8s29,1), (r9s29,1), (r10s29,1), (r11s29,1), (r13s29,1), (r14s29,1), (r15s29,1), (r16s29,1), (r17s29,1), (r18s29,1), (r19s29,1), (r20s29,1), (r21s29,1), (r22s29,1), (r23s29,1), (r24s29,1), (r25s29,1)]), m) , z3.PbLe(([(r26s29,1), (r27s29,1), (r28s29,1), (r29s29,1), (r30s29,1), (r0s30,1), (r1s30,1), (r2s30,1), (r3s30,1), (r4s30,1), (r5s30,1), (r6s30,1), (r7s30,1), (r8s30,1), (r9s30,1), (r10s30,1), (r11s30,1), (r12s30,1), (r13s30,1), (r14s30,1), (r15s30,1), (r16s30,1), (r17s30,1), (r18s30,1), (r19s30,1), (r20s30,1), (r21s30,1), (r22s30,1), (r23s30,1), (r24s30,1), (r25s30,1)]), m) , z3.PbLe(([(r26s30,1), (r27s30,1), (r28s30,1), (r29s30,1), (r30s30,1), (r0s31,1), (r1s31,1), (r2s31,1), (r3s31,1), (r4s31,1), (r5s31,1), (r6s31,1), (r7s31,1), (r8s31,1), (r9s31,1), (r10s31,1), (r11s31,1), (r12s31,1), (r13s31,1), (r14s31,1), (r15s31,1), (r16s31,1), (r17s31,1), (r18s31,1), (r19s31,1), (r20s31,1), (r21s31,1), (r22s31,1), (r23s31,1), (r24s31,1), (r25s31,1)]), m) , z3.PbLe(([(r26s31,1), (r27s31,1), (r30s31,1), (r0s32,1), (r1s32,1), (r2s32,1), (r3s32,1), (r4s32,1), (r5s32,1), (r6s32,1), (r7s32,1), (r8s32,1), (r9s32,1), (r10s32,1), (r11s32,1), (r12s32,1), (r13s32,1), (r14s32,1), (r15s32,1), (r16s32,1), (r17s32,1), (r18s32,1), (r19s32,1), (r20s32,1), (r21s32,1), (r22s32,1), (r23s32,1), (r24s32,1), (r25s32,1), (r26s32,1), (r27s32,1)]), m) , z3.PbLe(([(r28s32,1), (r29s32,1), (r30s32,1), (r0s33,1), (r1s33,1), (r2s33,1), (r3s33,1), (r4s33,1), (r5s33,1), (r6s33,1), (r7s33,1), (r8s33,1), (r9s33,1), (r10s33,1), (r11s33,1), (r12s33,1), (r13s33,1), (r14s33,1), (r15s33,1), (r16s33,1), (r17s33,1), (r18s33,1), (r19s33,1), (r20s33,1), (r21s33,1), (r22s33,1), (r23s33,1), (r24s33,1), (r25s33,1), (r26s33,1), (r27s33,1)]), m) , z3.PbLe(([(r28s33,1), (r29s33,1), (r30s33,1), (r0s34,1), (r1s34,1), (r2s34,1), (r3s34,1), (r4s34,1), (r5s34,1), (r6s34,1), (r7s34,1), (r8s34,1), (r9s34,1), (r10s34,1), (r11s34,1), (r12s34,1), (r13s34,1), (r14s34,1), (r15s34,1), (r16s34,1), (r17s34,1), (r18s34,1), (r19s34,1), (r20s34,1), (r21s34,1), (r22s34,1), (r23s34,1), (r24s34,1), (r25s34,1), (r26s34,1), (r27s34,1)]), m) , z3.PbLe(([(r30s34,1), (r0s35,1), (r1s35,1), (r2s35,1), (r3s35,1), (r4s35,1), (r5s35,1), (r6s35,1), (r7s35,1), (r8s35,1), (r9s35,1), (r10s35,1), (r11s35,1), (r12s35,1), (r13s35,1), (r14s35,1), (r15s35,1), (r16s35,1), (r17s35,1), (r18s35,1), (r19s35,1), (r20s35,1), (r21s35,1), (r22s35,1), (r23s35,1), (r24s35,1), (r25s35,1), (r26s35,1), (r27s35,1), (r28s35,1), (r29s35,1)]), m) , z3.PbLe(([(r30s35,1), (r0s36,1), (r1s36,1), (r2s36,1), (r3s36,1), (r4s36,1), (r5s36,1), (r6s36,1), (r7s36,1), (r8s36,1), (r9s36,1), (r10s36,1), (r11s36,1), (r12s36,1), (r13s36,1), (r14s36,1), (r15s36,1), (r16s36,1), (r17s36,1), (r18s36,1), (r19s36,1), (r20s36,1), (r21s36,1), (r22s36,1), (r23s36,1), (r24s36,1), (r25s36,1), (r26s36,1), (r27s36,1), (r28s36,1), (r29s36,1)]), m) , z3.PbLe(([(r0s37,1), (r1s37,1), (r2s37,1), (r3s37,1), (r4s37,1), (r5s37,1), (r6s37,1), (r7s37,1), (r8s37,1), (r9s37,1), (r10s37,1), (r11s37,1), (r12s37,1), (r13s37,1), (r14s37,1), (r15s37,1), (r16s37,1), (r17s37,1), (r18s37,1), (r19s37,1), (r20s37,1), (r21s37,1), (r22s37,1), (r23s37,1), (r24s37,1), (r25s37,1), (r26s37,1), (r27s37,1), (r28s37,1), (r29s37,1), (r30s37,1)]), m) , z3.PbLe(([(r0s38,1), (r1s38,1), (r2s38,1), (r3s38,1), (r4s38,1), (r5s38,1), (r6s38,1), (r7s38,1), (r8s38,1), (r9s38,1), (r10s38,1), (r11s38,1), (r12s38,1), (r13s38,1), (r14s38,1), (r15s38,1), (r16s38,1), (r17s38,1), (r18s38,1), (r19s38,1), (r20s38,1), (r21s38,1), (r22s38,1), (r23s38,1), (r24s38,1), (r25s38,1), (r26s38,1), (r27s38,1), (r28s38,1), (r29s38,1), (r30s38,1)]), m) , z3.PbLe(([(r0s39,1), (r1s39,1), (r2s39,1), (r3s39,1), (r4s39,1), (r5s39,1), (r6s39,1), (r7s39,1), (r8s39,1), (r9s39,1), (r10s39,1), (r11s39,1), (r12s39,1), (r13s39,1), (r14s39,1), (r15s39,1), (r16s39,1), (r17s39,1), (r18s39,1), (r19s39,1), (r20s39,1), (r21s39,1), (r22s39,1), (r23s39,1), (r24s39,1), (r25s39,1), (r26s39,1), (r27s39,1), (r28s39,1), (r29s39,1), (r30s39,1)]), m) , z3.PbLe(([(r0s40,1), (r2s40,1), (r3s40,1), (r4s40,1), (r5s40,1), (r6s40,1), (r7s40,1), (r8s40,1), (r9s40,1), (r10s40,1), (r11s40,1), (r12s40,1), (r13s40,1), (r14s40,1), (r15s40,1), (r16s40,1), (r17s40,1), (r18s40,1), (r19s40,1), (r20s40,1), (r21s40,1), (r22s40,1), (r24s40,1), (r25s40,1), (r26s40,1), (r27s40,1), (r28s40,1), (r29s40,1), (r30s40,1), (r0s41,1), (r1s41,1)]), m) , z3.PbLe(([(r2s41,1), (r3s41,1), (r4s41,1), (r5s41,1), (r6s41,1), (r7s41,1), (r8s41,1), (r9s41,1), (r10s41,1), (r11s41,1), (r12s41,1), (r13s41,1), (r14s41,1), (r15s41,1), (r16s41,1), (r17s41,1), (r18s41,1), (r19s41,1), (r20s41,1), (r21s41,1), (r22s41,1), (r23s41,1), (r25s41,1), (r26s41,1), (r27s41,1), (r28s41,1), (r29s41,1), (r30s41,1), (r0s42,1), (r1s42,1), (r2s42,1)]), m) , z3.PbLe(([(r3s42,1), (r4s42,1), (r5s42,1), (r6s42,1), (r7s42,1), (r8s42,1), (r9s42,1), (r10s42,1), (r11s42,1), (r12s42,1), (r13s42,1), (r14s42,1), (r15s42,1), (r16s42,1), (r17s42,1), (r18s42,1), (r19s42,1), (r20s42,1), (r22s42,1), (r23s42,1), (r24s42,1), (r25s42,1), (r26s42,1), (r27s42,1), (r28s42,1), (r29s42,1), (r30s42,1), (r0s43,1), (r1s43,1), (r2s43,1), (r3s43,1)]), m) , z3.PbLe(([(r4s43,1), (r5s43,1), (r6s43,1), (r7s43,1), (r8s43,1), (r9s43,1), (r10s43,1), (r11s43,1), (r12s43,1), (r13s43,1), (r14s43,1), (r15s43,1), (r17s43,1), (r18s43,1), (r19s43,1), (r20s43,1), (r21s43,1), (r22s43,1), (r23s43,1), (r25s43,1), (r26s43,1), (r27s43,1), (r28s43,1), (r29s43,1), (r30s43,1), (r0s44,1), (r1s44,1), (r2s44,1), (r3s44,1), (r4s44,1), (r5s44,1)]), m) , z3.PbLe(([(r6s44,1), (r7s44,1), (r8s44,1), (r9s44,1), (r10s44,1), (r11s44,1), (r12s44,1), (r13s44,1), (r14s44,1), (r15s44,1), (r16s44,1), (r17s44,1), (r18s44,1), (r19s44,1), (r20s44,1), (r21s44,1), (r22s44,1), (r23s44,1), (r24s44,1), (r25s44,1), (r26s44,1), (r27s44,1), (r28s44,1), (r29s44,1), (r30s44,1), (r0s45,1), (r1s45,1), (r2s45,1), (r3s45,1), (r4s45,1), (r5s45,1)]), m) , z3.PbLe(([(r6s45,1), (r7s45,1), (r8s45,1), (r9s45,1), (r10s45,1), (r11s45,1), (r12s45,1), (r13s45,1), (r14s45,1), (r15s45,1), (r16s45,1), (r18s45,1), (r19s45,1), (r20s45,1), (r21s45,1), (r22s45,1), (r23s45,1), (r24s45,1), (r25s45,1), (r26s45,1), (r27s45,1), (r28s45,1), (r29s45,1), (r30s45,1), (r0s46,1), (r1s46,1), (r2s46,1), (r3s46,1), (r4s46,1), (r5s46,1), (r6s46,1)]), m) , z3.PbLe(([(r7s46,1), (r8s46,1), (r9s46,1), (r10s46,1), (r11s46,1), (r12s46,1), (r13s46,1), (r14s46,1), (r15s46,1), (r16s46,1), (r17s46,1), (r18s46,1), (r19s46,1), (r20s46,1), (r21s46,1), (r22s46,1), (r23s46,1), (r24s46,1), (r25s46,1), (r26s46,1), (r27s46,1), (r28s46,1), (r29s46,1), (r30s46,1), (r0s47,1), (r1s47,1), (r2s47,1), (r3s47,1), (r4s47,1), (r5s47,1), (r6s47,1)]), m) , z3.PbLe(([(r7s47,1), (r8s47,1), (r9s47,1), (r10s47,1), (r11s47,1), (r12s47,1), (r13s47,1), (r14s47,1), (r15s47,1), (r16s47,1), (r17s47,1), (r18s47,1), (r19s47,1), (r20s47,1), (r21s47,1), (r22s47,1), (r23s47,1), (r24s47,1), (r25s47,1), (r26s47,1), (r27s47,1), (r28s47,1), (r29s47,1), (r30s47,1), (r0s48,1), (r1s48,1), (r2s48,1), (r3s48,1), (r4s48,1), (r5s48,1), (r6s48,1)]), m) , z3.PbLe(([(r7s48,1), (r8s48,1), (r9s48,1), (r10s48,1), (r11s48,1), (r12s48,1), (r13s48,1), (r14s48,1), (r15s48,1), (r16s48,1), (r18s48,1), (r19s48,1), (r20s48,1), (r21s48,1), (r22s48,1), (r23s48,1), (r24s48,1), (r25s48,1), (r26s48,1), (r27s48,1), (r28s48,1), (r29s48,1), (r30s48,1), (r0s49,1), (r1s49,1), (r2s49,1), (r3s49,1), (r4s49,1), (r5s49,1), (r6s49,1), (r7s49,1)]), m) , z3.PbLe(([(r8s49,1), (r9s49,1), (r10s49,1), (r11s49,1), (r12s49,1), (r13s49,1), (r14s49,1), (r15s49,1), (r17s49,1), (r18s49,1), (r19s49,1), (r20s49,1), (r21s49,1), (r22s49,1), (r23s49,1), (r25s49,1), (r26s49,1), (r27s49,1), (r28s49,1), (r29s49,1), (r30s49,1), (r1s50,1), (r2s50,1), (r3s50,1), (r4s50,1), (r5s50,1), (r6s50,1), (r7s50,1), (r8s50,1), (r9s50,1), (r10s50,1)]), m) , z3.PbLe(([(r11s50,1), (r12s50,1), (r13s50,1), (r14s50,1), (r15s50,1), (r16s50,1), (r17s50,1), (r18s50,1), (r19s50,1), (r20s50,1), (r21s50,1), (r22s50,1), (r23s50,1), (r24s50,1), (r25s50,1), (r26s50,1), (r27s50,1), (r28s50,1), (r29s50,1), (r30s50,1), (r0s51,1), (r1s51,1), (r2s51,1), (r3s51,1), (r4s51,1), (r5s51,1), (r6s51,1), (r7s51,1), (r8s51,1), (r9s51,1), (r10s51,1)]), m) , z3.PbLe(([(r11s51,1), (r12s51,1), (r13s51,1), (r14s51,1), (r15s51,1), (r17s51,1), (r18s51,1), (r19s51,1), (r20s51,1), (r21s51,1), (r22s51,1), (r23s51,1), (r25s51,1), (r26s51,1), (r27s51,1), (r28s51,1), (r29s51,1), (r30s51,1), (r0s52,1), (r1s52,1), (r2s52,1), (r3s52,1), (r4s52,1), (r5s52,1), (r6s52,1), (r7s52,1), (r8s52,1), (r9s52,1), (r10s52,1), (r11s52,1), (r12s52,1)]), m) , z3.PbLe(([(r13s52,1), (r14s52,1), (r15s52,1), (r16s52,1), (r17s52,1), (r18s52,1), (r19s52,1), (r20s52,1), (r21s52,1), (r22s52,1), (r23s52,1), (r24s52,1), (r25s52,1), (r26s52,1), (r27s52,1), (r28s52,1), (r29s52,1), (r30s52,1), (r0s53,1), (r1s53,1), (r2s53,1), (r3s53,1), (r4s53,1), (r5s53,1), (r6s53,1), (r7s53,1), (r8s53,1), (r9s53,1), (r10s53,1), (r11s53,1), (r12s53,1)]), m) , z3.PbLe(([(r13s53,1), (r14s53,1), (r15s53,1), (r16s53,1), (r18s53,1), (r19s53,1), (r20s53,1), (r21s53,1), (r22s53,1), (r23s53,1), (r24s53,1), (r25s53,1), (r26s53,1), (r27s53,1), (r28s53,1), (r29s53,1), (r30s53,1)]), m) ,
score == z3.Sum(scores),
m_set == 0,
goal == score + m_set])

start = timeit.default_timer()
o.maximize(score)
if o.check() == z3.sat:
    model = o.model()
    stop = timeit.default_timer()
    print('Time: ', stop - start, 'sec.')  
    print([(d, model[d]) for d in model])

#     print(sorted ([(d, m[d]) for d in m], key = lambda x: str(x[0])))
    matches = [(d, model[d]) for d in model if model[d] == True]
#     print(matches)
else:
    print("not satisfiable")
performance = o.statistics()
# print(performance)

n_yes = 0 
n_maybe = 0
n_no = 0
for match in matches:
    if rs_score[str(match[0])] > maybe:
        n_yes += 1
    if no < rs_score[str(match[0])] < yes:
        n_maybe += 1
    if rs_score[str(match[0])] < maybe:
        n_no += 1
print('total: {0}, yes:{1}, maybe:{2}, no: {3}'.format(len(matches), n_yes, n_maybe, n_no))

Time:  971.1951785 sec.
[(r0s0, False), (r0s1, False), (r0s2, False), (r0s4, False), (r0s5, False), (r0s6, False), (r0s7, False), (r0s8, False), (r0s9, False), (r0s10, False), (r0s11, False), (r0s12, False), (r0s13, True), (r0s14, False), (r0s15, False), (r0s16, False), (r0s17, False), (r0s18, False), (r0s19, False), (r0s20, False), (r0s21, False), (r0s22, True), (r0s23, False), (r0s24, True), (r0s25, False), (r0s26, False), (r0s27, False), (r0s28, False), (r0s29, False), (r0s30, False), (r0s31, False), (r0s32, False), (r0s33, False), (r0s34, False), (r0s35, False), (r0s36, False), (r0s37, False), (r0s38, False), (r0s39, False), (r0s40, False), (r0s41, False), (r0s42, False), (r0s43, False), (r0s44, False), (r0s45, False), (r0s46, False), (r0s47, False), (r0s48, False), (r0s49, False), (r0s51, False), (r0s52, False), (r0s53, False), (r1s0, False), (r1s1, False), (r1s2, False), (r1s3, False), (r1s4, False), (r1s5, False), (r1s6, True), (r1s7, False), (r1s8, False), (r1s9, False), (r1s10

total: 93, yes:86, maybe:7, no: 0
